Install depedencies

In [1]:
%pip install azure-ai-documentintelligence==1.0.0b1 azure-search-documents==11.4.0 unidecode==1.3.8 nltk==3.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 557.6 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 495.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 864.5 kB/s eta 0:00:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 3.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Plus, install NLTK data

In [2]:
from nltk import download

download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/loicsteve/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Then initialize the clients to Document intelligence and AI search

In [11]:
from azure.ai.documentintelligence.aio import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.search.documents import SearchClient
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from random import randint
from typing import Dict, Tuple
from unidecode import unidecode
import asyncio
import glob
import os.path
import re

In [32]:
doc_endpoint = "https://steve-2ggjkui.cognitiveservices.azure.com"
doc_credential = AzureKeyCredential("7723a5e867f241fc8abf584b57b056ba")
doc_client = DocumentIntelligenceClient(
    endpoint=doc_endpoint,
    credential=doc_credential,
)

search_endpoint = "https://ai-search-acpr-hackathon.search.windows.net"
search_credential = AzureKeyCredential("i3uTFH3gvIBRilB13ZZLubY6SUvUaXRTdDyASV4UQQAzSeChtl4E")
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name="equipe-8-v1",
    credential=search_credential,
)

Next, transform PDF in Markdown text. We are using Document Intelligence for that.

Be warned that this step can take a few minutes, depending on the size of the PDF. From a minute for a few pages to 20 minutes for a 1000 pages PDF

In [33]:
async def pdf_to_markdown(source: str) -> Tuple[str, str]:
    if os.path.exists(source + ".md"):  # Test cache
        print(f"Skipping {source}, cached found")
        with open(source + ".md", "r") as file:
            return source, file.read()

    with open(source, "rb") as file:  # Load file content
        print(f"Starting {source}, no cache found")
        await asyncio.sleep(randint(0, 5))  # Avoid API rate limit
        doc_poller = await doc_client.begin_analyze_document(
            analyze_request=file,
            content_type="application/octet-stream",
            locale="fr-FR",  # We only have French documents in this dataset
            model_id="prebuilt-layout",
            output_content_format="markdown",
        )
        doc_result = await doc_poller.result()

        with open(source + ".md", "w") as file:  # Store result in cache
            file.write(doc_result.content)

        return source, doc_result.content



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x126707490>
/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tokenize.py:529: RuntimeWarning: coroutine 'main' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)


In [35]:
doc_results: Dict[str, str] = {}
doc_tasks = []

for source in glob.glob("/Users/loicsteve/Desktop/Tech sprint/*.pdf"):
    doc_tasks.append(asyncio.create_task(pdf_to_markdown(source)))

print("Waiting for results...")
for doc_task in asyncio.as_completed(doc_tasks):
    source = None
    try:
        source, content = await doc_task
        print(f"Ended {source}")
        doc_results[source] = content
    except Exception as e:
        print(f"Failed {source}: {e}")
print(f"{len(doc_results)} documents parsed")

Waiting for results...
Skipping /Users/loicsteve/Desktop/Tech sprint/rapport-solvabilite-2021-maif-vie.pdf, cached found
Starting /Users/loicsteve/Desktop/Tech sprint/rapport-solvabilite-2022-maif-vie.pdf, no cache found
Skipping /Users/loicsteve/Desktop/Tech sprint/2022 SFCR GPMA.pdf, cached found
Ended /Users/loicsteve/Desktop/Tech sprint/rapport-solvabilite-2021-maif-vie.pdf
Ended /Users/loicsteve/Desktop/Tech sprint/2022 SFCR GPMA.pdf
Ended /Users/loicsteve/Desktop/Tech sprint/rapport-solvabilite-2022-maif-vie.pdf
3 documents parsed


In [36]:
lang = "french"
stemmer = SnowballStemmer(lang)
sentence_r = r"[^\w\s+\-/'\",:;()@=]"


def compress_and_clean(text: str) -> str:
    """
    Compress and clean a text.

    Use the Snowball stemmer to stem words and remove all special characters, as the LLM does not give a damn.
    """
    text = text.replace("\\", "")  # Remove all backslashes
    text = re.sub(r":[a-z]*:", "", text)  # Remove all :unselected: and :selected: tags
    text = re.sub(r"<!--[^<>]*-->", "", text)  # Remove all comments
    tokenized = word_tokenize(text, language=lang)
    tokens = [stemmer.stem(token) for token in tokenized]
    prompt = " ".join(tokens)
    return prompt


def data(content: str, source_uri: str, title: str, iterator: int) -> dict[str, str]:
    """
    Generate a data object for the search index.

    Use deterministic ID to avoid duplicates after a new run. Remove all special characters from title.
    """
    return {
        "content": content,
        "id": f"{'_'.join(re.sub('[^a-z0-9]', ' ', unidecode(source_uri).lower()).split())}-{iterator}",
        "source_uri": unidecode(source_uri).lower(),
        "title": " ".join(re.sub("[^a-z0-9]", " ", unidecode(title).lower()).split()),
    }


chuncks = []
iterator = 0

for source, content in doc_results.items():
    # Skip empty documents
    if not content:
        continue

    # Split the document into sections
    for section in content.split("\n#"):
        lines = section.split("\n")
        title = lines[0].strip()
        paragraph = " ".join(lines[1:])

        # Skip empty sections
        if not section:
            continue

        # Split the paragraph into sentences and group them into groups of 300 characters
        groups = []
        group = ""
        separators = re.findall(sentence_r, paragraph)
        splits = re.split(sentence_r, paragraph)
        for i, separator in enumerate(separators):
            sentence = compress_and_clean(splits[i] + separator)
            if len(group) + len(sentence) >= 300:
                groups.append(group)
                group = ""
            group += sentence
        if group:
            groups.append(group)

        # Create a data object for each group
        for group in groups:
            chuncks.append(
                data(
                    f"# {title} {group}",
                    source,
                    title,
                    iterator,
                )
            )
            iterator += 1

print(chuncks[:5])
print(f"Created {len(chuncks)} chunks")

[{'content': "# MAIF VIE 2021 <figur >![]( figures/0 ) </figur >rapport sur la solvabl et la situat financi <figur >![]( figures/1 ) <!-- figurecontent= '' maif '' -- ></figur >assureur milit |sommair |2 / 107 ||- |- ||présent &synthes |4 ||present |5 ||synthes |6 ||activ &résultat |10 ||a activ &résultat |11 ||a .1 activ |11 |", 'id': 'users_loicsteve_desktop_tech_sprint_rapport_solvabilite_2021_maif_vie_pdf-0', 'source_uri': '/users/loicsteve/desktop/tech sprint/rapport-solvabilite-2021-maif-vie.pdf', 'title': 'maif vie 2021'}, {'content': "# MAIF VIE 2021 |a .2 résultat de souscript |15 ||a .3 résultat de invest |18 ||a .4 résultat de autr activ |19 ||a .5 autr inform |20 ||system de gouvern |21 ||b system de gouvern |22 ||b .1 inform général sur le system de gouvern |22 ||b .2 exigent de compétent et d'honor |30 |||b .", 'id': 'users_loicsteve_desktop_tech_sprint_rapport_solvabilite_2021_maif_vie_pdf-1', 'source_uri': '/users/loicsteve/desktop/tech sprint/rapport-solvabilite-2021-m

{'content': "# # Profil de risque ›aux acte opérationnel dus à de défaill de person , de system ou d'éven extern ou de don : cybercriminal ( vol de don , utilis fraudul de don ...", 'id': 'users_loicsteve_desktop_tech_sprint_rapport_solvabilite_2021_maif_vie_pdf-40', 'source_uri': '/users/loicsteve/desktop/tech sprint/rapport-solvabilite-2021-maif-vie.pdf', 'title': 'profil de risque'}
Created 2538 chunks


In [2]:
import os
import re
import requests
import sys
from num2words import num2words
import os
#import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [3]:
os.environ['OPENAI_API_KEY'] = '7e93421f46cd4680831023addcb0f42d'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2024-02-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://francecentral-openai.openai.azure.com/'

In [4]:
# Initialize the AzureOpenAI client 
client = AzureOpenAI(
  azure_endpoint = "https://francecentral-openai.openai.azure.com/openai/deployments/gpt-4-turbo/chat/completions?api-version=2024-02-15-preview", 
  api_key="7e93421f46cd4680831023addcb0f42d",  
  api_version="2024-02-15-preview"
)

In [5]:
def get_openai_response(prompt):    
    # Create the message list with the new user prompt    
    message_text = [    
        {"role": "system", "content": "You are an insurance controller helping to synthetize or summarize insurance company report provided to you."},    
        {"role": "user", "content": prompt}    
    ]    
    
    # Call the OpenAI API to get the completion    
    completion = client.chat.completions.create(    
        model="gpt-4-turbo",  # model = "deployment_name"    
        messages=message_text,    
        temperature=0.0,    
        max_tokens=1000,    
        top_p=0.95,    
        frequency_penalty=0,    
        presence_penalty=0,    
        stop=None    
    )    
    
    # Assuming the completion object has a 'choices' attribute and each choice has a 'message' attribute  
    if completion.choices:  
        first_choice = completion.choices[0]  
        # Assuming 'message' is a ChatCompletionMessage object with 'content' attribute  
        return first_choice.message.content if hasattr(first_choice.message, 'content') else None  
    else:  
        return None  
    

In [ ]:
map_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance. Tu sais bien extraire les informations à partir d'un
passage de texte.

Ces informations sont généralement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
-Placements autres que les actifs en represenation de contrats en UC ou indexe.
-Changement de la gouvernance.
-Perspectives vision de l'organisme et projets futurs.

#Règles:
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
- N'invente pas des informations qui ni fugrent pas dans le texte.

Ecris moi un résumé  pour le texte suivant:
"{text}"
Résumé:
"""

In [53]:
prompt = '''Tu es un assistant expert dans le domaine financier et en assurance. Tu sais bien extraire les informations à partir d'un
passage de texte.

Ces informations sont généralement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
-Placements autres que les actifs en represenation de contrats en UC ou indexe.
-Changement de la gouvernance.
-Perspectives vision de l'organisme et projets futurs.

#Règles:
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
- N'invente pas des informations qui ni fugrent pas dans le texte.
- Et le resume de fin devrait sous forme de six paragraphes comme suit:
1) Présentation de l'organisme.
2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
3) Provisionnement
4) Placements autres que les actifs en represenation de contrats en UC ou indexe.
5) Changement de la gouvernance.
6) Perspectives vision de l'organisme et projets futurs.

Le tyexte est le suivant:
"{
MAIF VIE 2022

<figure>

![](figures/0)

</figure>


Rapport sur la SOLVABILITÉ et la SITUATION FINANCIÈRE

<figure>

![](figures/1)

<!-- FigureContent="MAIF" -->

</figure>


assureur militant

2 / 111


# SOMMAIRE

|||||
| - | - | - | - |
| PRÉSENTATION & SYNTHÈSES ||| 4 |
| PRESENTATION ||| 5 |
| SYNTHÈSES ||| 6 |
| | ACTIVITÉ & RÉSULTATS || 10 |
| A || Activité & Résultats | 11 |
| A.1 Activité ||| 11 |
| A.2 Résultats de souscription ||| 15 |
| | A.3 Résultats des investissements || 18 |
| | A.4 Résultats des autres activités || 19 |
| | A.5 Autres informations || 20 |
| | SYSTÈME DE GOUVERNANCE || 21 |
| B || Système de Gouvernance | 22 |
| B.1 Informations générales sur le système de gouvernance ||| 22 |
|| B.2 Exigences de compétences et d'honorabilité | | 30 |
| ||| B.3 Système de gestion des risques y compris l'évaluation interne des risques et de la   |
| solvabilité ||| 32 |
|| B.4 Système de contrôle interne | | 38 |
|| B.5 Fonction d'audit interne | | 39 |
|| B.6 Fonction actuarielle | | 41 |
|| B.7 Sous-traitance | | 42 |
| || B.8 Autres informations  | 43 |
| | PROFIL DE RISQUE || 44 |
| C || Profil de Risque | 45 |
| C.1 Risque de souscription ||| 46 |
| || C.2 Risque de marché  | 49 |
| | C.3 Risque de crédit || 53 | :unselected: |
|| C.4 Risque de liquidité | 56 |
| C.5 Risque opérationnel :unselected: ||| 58 | :unselected: |
|| C.6 Autres risques importants | 59 | :unselected: |
|| C.7 Autres informations | 63 |
| | VALORISATION À DES FINS DE SOLVABILITÉ || 64 |
| D :unselected: || Valorisation à des fins de Solvabilité | 65 |
| D.1 Actifs :unselected: ||| 68 |
| D.2 Provisions techniques :unselected: ||| 72 |
| D.3 Autres passifs :unselected: ||| 82 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

SOMMAIRE
===

3 / 111

D.4 Méthodes de valorisation alternatives 85

D.5 Autres informations

85

| GESTION DU CAPITAL | 86 |
| - | - |
| E Gestion du capital | 87 |
| E.1 Fonds propres | 87 |
| E.2 Capital de solvabilité requis et Minimum de capital requis | 94 |
| E.3 Utilisation du sous-module "risque sur actions" fonde sur la durée dans le calcul du Capital ||
| de solvabilité requis | 97 |
| E.4 Différences entre la formule standard et tout modèle interne utilisé | 97 |
| E.5 Non-respect du Minimum de capital requis et non-respect du Capital de solvabilité requis ||
| | 97 |
| E.6 Autres informations | 97 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

PRÉSENTATION & SYNTHÈSES

<!-- PageHeader="PRÉSENTATION& SYNTHÈSES" -->

<!-- PageHeader="5 / 111" -->

PRESENTATION
===

Solvabilité 2 est une directive européenne réformant la réglementation prudentielle du secteur de l'assurance et entrée en vigueur le 1er janvier 2016.

Solvabilité 2 crée un cadre réglementaire harmonisé et fondé sur le risque pour les entreprises d'assurance, avec des règles quantitatives et des exigences en matière de gouvernance et d'information prudentielle. Les règles communes visent à faciliter les activités des compagnies d'assurances au sein de l'Union Européenne, à garantir une gestion saine et prudente dans des périodes difficiles et à protéger les souscripteurs d'assurances. Cette réglementation impose un niveau de détention de fonds propres et des règles en matière de gestion et de surveillance.

Les exigences sont structurées en trois piliers :

· premier pilier : exigences quantitatives, notamment en matière de fonds propres et de calculs des provisions techniques,

· deuxième pilier : exigences en matière d'organisation et de gouvernance des organismes,

· troisième pilier : exigences en matière d'informations prudentielles et de publication.

Le présent rapport s'inscrit dans le cadre de ce troisième pilier.

Le rapport sur la solvabilité et la situation financière (Solvency and Financial Conditions Report en anglais - SFCR), décrit l'activité de l'entreprise, son système de gouvernance, son profil de risque et donne des informations sur les méthodes de valorisation utilisées et la gestion du capital.

Le rapport sur la solvabilité et la situation financière (SFCR) comprend en annexe différents états quantitatifs.

Les données financières présentées dans ce rapport sont, sauf exceptions dûment précisées, exprimées en milliers d'euros.

Après avis du comité des risques, d'audit et des comptes du 21 mars 2023, le présent rapport a été approuvé par le conseil d'administration du 5 avril 2023.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageNumber="6 / 111" -->

<!-- PageHeader="PRÉSENTATION& SYNTHÈSES" -->


## SYNTHÈSES

MAIF VIE est une société anonyme régie par le Code des assurances, intervenant principalement pour les risques vie. Elle est intégralement détenue par la MAIF.

Constituée le 14 septembre 1984, MAIF VIE s'est développée en restant fidèle à ses valeurs fondatrices : solidarité, responsabilité et respect de la personne.

Depuis 2021, MAIF VIE a adopté le statut de société à mission, inscrivant et affirmant ainsi sa mission d'intérêt collectif, fondée sur une attention sincère portée à toutes ses parties prenantes.


### Activité et résultats

L'année 2022 se positionne comme la dernière du Plan stratégique Groupe 2019-2022 « Engagés pour demain » dans lequel MAIF VIE s'inscrit. Les indicateurs de performance à la fin de ce Plan stratégique sont en ligne avec les objectifs cibles définis.

L'activité de l'exercice 2022 est principalement caractérisée par :

› des encaissements bruts de réassurance de 834 368 milliers d'euros, en hausse de 3% par rapport à 2021, avec, en épargne, une collecte nette positive aussi bien sur les supports euros que sur ceux en unités de compte ;

› des taux de rémunération servis dans la moyenne du marché (2,10 % net) ;

› une provision pour participation aux bénéfices portée à 5,7 % des provisions mathématiques en euros de nos contrats d'épargne ;

› un résultat net comptable de 51 214 milliers d'euros permettant de renforcer les fonds propres tout en préservant des marges de manœuvre pour l'avenir.

La forte inflation et la remontée brutale des taux constatées au cours de l'exercice 2022 en lien avec le conflit entre la Russie et l'Ukraine n'ont pas eu d'impacts significatifs sur les actifs financiers évalués en normes françaises de MAIF VIE étant donné l'absence d'expositions directes et l'exposition marginale indirecte sur ces pays. Le portefeuille d'actifs a fait objet d'un suivi régulier tout au long de l'exercice.


### Système de gouvernance

MAIF VIE place le conseil d'administration au sommet de la gouvernance et distingue, depuis le conseil d'administration du 17 décembre 2015, les fonctions de président du conseil d'administration et de directeur général. Le règlement intérieur du conseil d'administration, adopté en juin 2017, complète les statuts de MAIF VIE notamment sur les engagements des administrateurs.

Yves Pellicier a été nommé président du conseil d'administration en juin 2022.Hélène N'Diaye, directrice générale et Marina Sellari, directrice générale déléguée sont les dirigeants effectifs de MAIF VIE. À ce titre, elles disposent d'un domaine de compétences et de pouvoirs très larges sur les activités et les risques de MAIF VIE et sont systématiquement impliquées dans les décisions ayant un impact important. Le système de gouvernance comprend un système de gestion des risques et de contrôle interne, organisé autour de quatre fonctions clés : gestion des risques, actuarielle, audit interne et vérification de la conformité.

Les quatre fonctions clés accompagnent le conseil d'administration dans sa mission d'orientation et de contrôle et disposent d'un libre accès aux instances de gouvernance.

La gouvernance décisionnelle est structurée autour des quatre thèmes suivants : souscription et produits, pilotage économique et prudentiel, gestion des placements et pilotage des risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

PRÉSENTATION& SYNTHÈSES
===

<!-- PageNumber="7 / 111" -->

Le président du conseil d'administration, la directrice générale, la directrice générale déléguée, les responsables des fonctions cles et les administrateurs répondent aux exigences de compétences et d'honorabilité fixées par la réglementation.

MAIF VIE considère que son système de gouvernance est en adéquation avec sa stratégie, la nature et l'ampleur des risques inhérents à son activité d'assureur.

Elle veille en permanence à l'efficacité et à l'adéquation de son système de gouvernance avec la stratégie, la nature et l'ampleur des risques inhérents à ses activités.


## Profil de risque

MAIF VIE est une société d'assurance dont l'activité principale est de couvrir les besoins des adhérents en matière d'assurance vie : protéger sa famille, assurer l'avenir de ses enfants, se constituer par l'épargne un complément de retraite, favoriser la transmission d'un capital. À ce titre, elle n'est pas exposée à des risques spécifiques par rapport au marché et ne couvre pas de risques de nature exceptionnelle.

Dans un contexte géopolitique tendu, ayant des conséquences économiques et financières majeures, MAIF VIE reste attentive à toute évolution de son profil de risque. En ce début d'année 2023, les marchés financiers se montrent toujours volatils. Des tensions sur les matières premières et des pressions inflationnistes persistent dans un environnement de forte remontée des taux. Ce contexte fait l'objet d'un suivi régulier.

Les principaux risques auxquels MAIF VIE est exposée résultent de son activité d'assureur vie, de son organisation et de l'environnement, notamment économique. Ils sont liés :

› à la souscription : une forte hausse des rachats sur les produits d'épargne et une dérive de la performance économique de plusieurs produits comme conséquence de leur distribution et de leur tarification ;

Pour limiter les risques de souscription, MAIF VIE entretient une relation privilégiée avec ses adhérents et soigne son accessibilité et la qualité de ses prestations. Le haut niveau de fidélité des adhérents et assurés confirme la réalité d'une relation exceptionnelle sur le marché de l'assurance vie ;

› aux marchés financiers : une hausse brutale et pérenne des taux d'intérêt associée à une chute des marchés actions (sans retour à la situation initiale avant plusieurs années) ou bien des taux durablement bas mettant sous pression la rentabilité des assureurs vie et leur capacité à garantir le capital ;

Pour faire face à ces risques de marché, MAIF VIE s'inscrit dans une démarche de gestion saine, prudente et raisonnée de ses investissements financiers.

› aux actes opérationnels dus à des défaillances de personnes, de systèmes ou d'évènements externes ou de données : cybercriminalité (vol de données, utilisation frauduleuse de données ... ), indisponibilité des systèmes d'information, défaut d'un sous- traitant important ou critique, non-respect aux obligations règlementaires (lutte contre le blanchiment et le financement du terrorisme, contrats non réglés, protection de la clientèle dont le devoir de conseil, la loi Informatique et liberté ou encore détection des personnes politiquement exposées et gel des avoirs ... ) ;

Pour gérer les risques opérationnels, MAIF VIE s'inscrit pleinement dans le respect de la réglementation et procède régulièrement à des audits.

› à la complexité et la précision des outils de modélisation développés par l'entreprise pour mesurer les engagements, les risques et la solvabilité ;

L'encadrement des modèles mis en place par MAIF VIE limite la portée de ce risque : existence d'une gouvernance stricte concernant l'utilisation et les évolutions des modèles. Une documentation robuste et structurée vient compléter ces dispositions pour assurer une accessibilité et une compréhension du fonctionnement des modèles.

› aux enjeux sociaux et environnementaux (durabilité), notamment en matière d'investissements financiers. Le groupe MAIF est sensible par ses valeurs à son empreinte sociale et environnementale, et souhaite renforcer l'impact positif de ses activités sur son environnement. Il veille également à limiter les impacts négatifs en termes de durabilité de ses actifs financiers. A ce titre, le groupe MAIF a défini sa stratégie Climat des MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

PRÉSENTATION& SYNTHÈSES
===

<!-- PageNumber="8 / 111" -->

investissements depuis 2020, visant à s'inscrire dans la trajectoire de l'Accord de Paris. La mise en œuvre de cette stratégie se traduit notamment par un renforcement régulier de politiques d'exclusion (charbon et autres énergies fossiles) ainsi que par des actions d'engagement auprès de l'ensemble des parties prenantes ;

›

› aux activités émergentes : une montée en puissance des "robot-conseiller" générant une fuite de l'activité d'épargne de MAIF VIE ;

La gestion des risques émergents fait partie intégrante du processus de gestion des risques au sein de MAIF VIE et du groupe MAIF.

MAIF VIE dispose d'un ensemble de dispositifs de gestion des risques qu'elle juge maîtrisé et adapté. Le dispositif en place est réajusté régulièrement via les travaux conduits au sein du système de gestion des risques et contribue à la performance et à la pérennité de la société.

La diversification des risques, le niveau et la qualité des fonds propres, la stratégie d'investissement prudente sont autant d'éléments qui ont assuré à MAIF VIE de fortes capacités de résistance alors même que les incertitudes sont nombreuses et les risques accrus. MAIF VIE s'assure d'un impact positif de ses activités ainsi que de leur durabilité.


## Valorisation

L'établissement du bilan en normes Solvabilité 2 requiert de déterminer la valeur économique de l'ensemble des actifs et passifs. Les principales différences entre le bilan en normes françaises et le bilan en normes Solvabilité 2 s'expliquent par :

› la valorisation des actifs à leur valeur de marché conduisant à inclure des plus ou moins- values latentes dans les postes placements de l'actif du bilan en normes Solvabilité 2 alors que le bilan en normes françaises retient essentiellement le coût historique (prix d'achat) net de dépréciations ;

› le remplacement d'une estimation suffisante des engagements envers les assurés (dans le bilan en normes françaises) par la meilleure estimation des flux futurs actualisés des règlements auxquels s'ajoute une marge de risque (dans le bilan en normes Solvabilité 2). MAIF VIE utilise la correction pour volatilité de la courbe des taux prévue par la Directive Solvabilité 2 ;

› le reclassement de 67,6% de la Provision pour Participation aux Bénéfices (PPB) en fonds excédentaires, conformément à l'arrêté du 24 décembre 2019 et en application des préconisations de l'ACPR portant sur les modalités de calcul à retenir par les entreprises d'assurance ;

› la comptabilisation au bilan prudentiel d'impôts différés qui correspondent à des supplements ou des deductions futurs d'impôts, lies notamment aux évolutions précédentes.

La réglementation a mis en place des mesures transitoires pour faciliter la transition vers le nouveau dispositif prudentiel. S'agissant du calcul des engagements envers les assurés en normes Solvabilité 2, MAIF VIE a fait le choix de ne pas avoir recours aux mesures transitoires et répond d'ores et déjà aux exigences.


### Gestion du capital

MAIF VIE doit couvrir les exigences réglementaires (notamment Capital de solvabilité requis - SCR - et Minimum de capital requis - MCR) par des fonds propres qui remplissent les critères de qualité attendus pour être éligibles, et en quantité suffisante.

MAIF VIE détermine les exigences de capital à l'aide de la formule standard qui reflète le profil de risque moyen des entreprises d'assurance européennes. MAIF VIE considère que le calibrage de cette formule est adapté à la mesure de son profil de risque.

La réglementation a mis en place des mesures transitoires pour faciliter la transition vers le nouveau dispositif prudentiel. S'agissant du calcul des fonds propres et du SCR, MAIF VIE a MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

à l'évolution du cadre sociétal, règlementaire et fiscal ;

PRÉSENTATION& SYNTHÈSES
===

9 / 111

fait le choix de ne pas avoir recours aux mesures transitoires et répond d'ores et déjà aux exigences.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->
<figure>

![](figures/2)

<!-- FigureContent="A" -->

</figure>


ACTIVITÉ & RÉSULTATS

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

11 / 111


## A Activité & Résultats

Ce paragraphe est consacré à la présentation :

› de MAIF VIE et de son activité ;

› des résultats de souscription, des investissements et des autres activités.


### A.1 Activité


#### A.1.1 Présentation de la société

MAIF VIE est une société anonyme au capital de 122 000 000 euros constituée le 14 septembre 1984 , régie par le Code des assurances, située au 50 avenue Salvador Allende - « Le Pavois >> CS90000 79029 NIORT Cedex 9 (Téléphone : 05.49.04.49.04 - Site internet : www.maif.fr).

MAIF VIE est également societe a mission au sens de l'article L210-10 du code de commerce.

Son superviseur est l'Autorité de Contrôle Prudentiel et de Résolution (ACPR), 4 Place de Budapest CS 92459 75436 Paris Cedex 09 (Téléphone : 01.49.95.40.00 - Site internet : www.acpr.banque-france.fr).

Ses comptes en normes françaises sont audités par les cabinets de commissaires aux comptes suivants :

› PricewaterhouseCoopers Audit, 63 rue de Villiers 92208 Neuilly-sur-seine Cedex Téléphone : 01.56.57.58.59 - Site internet : www.pwc.fr Représenté par Frédéric Trouillard-Mignen.

› Mazars, Tour Exaltis 61 rue Henri Regnault 92400 Courbevoie Téléphone : 01.49.97.60.00 - Site internet : www.mazars.fr Représenté par Pierre De Latude.

Créée en 1984, MAIF VIE compte aujourd'hui plus de 614 500 adhérents pour un encaissement brut de reassurance de 834 368 milliers d'euros réalisé sur le territoire français et un effectif en équivalent temps plein à la clôture de 222 (212 en 2021).

MAIF VIE assure le développement de l'offre d'assurance vie du groupe MAIF en apportant une réponse adaptée aux besoins des sociétaires MAIF :

› protéger sa famille ;

› assurer l'avenir de ses enfants ;

› se constituer, par l'épargne, un complément de retraite et de revenu ;

› favoriser la transmission d'un capital, ...

MAIF VIE est présente dans les domaines de :

› l'épargne et l'épargne retraite (contrat d'assurance vie en euros Nouveau Cap, contrats multisupports Assurance vie Responsable et Solidaire et PER Individuel MAIF) ;

› la prévoyance (contrats d'assurance deces toutes causes Rassurcap et Rassurcap Solutions) ;

› la prise en charge des obsèques avec Sollicitudes ;

› l'assurance en couverture de prêts (Assurance Emprunteur MAIF, Avantage Emprunteur).

MAIF VIE gère ses produits principalement sous forme de contrats collectifs à adhésion facultative relevant des branches 20 et 22 du Code des assurances.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> ACTIVITÉ & RÉSULTATS
===

<!-- PageHeader="12 / 111" -->

La correspondance entre les lignes d'activité (segmentation Solvabilité 2 par groupe de risques homogènes) et les principales garanties des contrats est la suivante :

| Catégorie de produits | Produits | Ligne d'activité (LOB) |
| - | - | - |
| Épargne | Assurance vie monosupport - Cap'Epargne, Cap'Etudes - Nouveau Cap (y compris PEP), Capiplan | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance vie multisupports Compartiment euros - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DS Cap | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance vie multisupports Compartiments UC - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DS Cap | Assurance indexée ou en unités de comptes (UC) |
| Retraite ou assimilés | Épargne retraite - PER Responsable et Solidaire - Compartiment euros | Contrats d'assurance vie avec participation aux bénéfices |
|| Épargne retraite - PER Responsable et Solidaire - Compartiments UC | Assurance indexée ou en unités de comptes (UC) |
|| Assurance retraite - IFC | Contrats d'assurance vie avec participation aux bénéfices |
| Prévoyance | Obsèques vie entière - Sollicitudes | Contrats d'assurance vie avec participation aux bénéfices |
|| Temporaire décès - Rassurcap - Rassurcap Solutions - Assurance Prévoyance Professionnelle | Autres activités d'assurance vie |
|| Assurance en couverture de prêts - Assurance Emprunteur MAIF, Avantage Emprunteur - Autres Prêts immobiliers - Prêts à la consommation | Autres activités d'assurance vie |
|| Rentes | Contrats d'assurance vie avec participation aux bénéfices |

Au 31 décembre 2022, le capital social de MAIF VIE est réparti de la manière suivante :

› MAIF : 99,99 % ;

› 8 personnes physiques : 0,01 %.

MAIF VIE entre dans le périmètre de consolidation de la société MAIF et est intégrée globalement.

En tant que mutuelle, la MAIF n'a ni capital social ni actionnaire à rémunérer. Elle est la société mère du Groupe dont elle consolide l'ensemble des sociétés d'importances significatives.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="13 / 111" -->

<!-- PageHeader="\> ACTIVITÉ & RÉSULTATS" -->

MAIF

<figure>

![](figures/3)

<!-- FigureContent="100 % 100 % 100 % 20 % MAIF AVENIR Socram Banque MAIF VIE aLTima 67 % Groupe Delta 33 % 84 % camifR Immo 56 % Groupe Delta Loisirs Evasion 44 % 64 % Groupe Immo 36 % 86 % SMACL ASSURANCES :selected: 0,3 % INTER MIUTUELLES 29 % Pan Europe ASSISTANCE Autres activités Assurance vie Assurance non vie Intégration globale Mise en équivalence" -->

</figure>


Indication du pourcentage de contrôle

Groupe Delta Immo : Delta Immo, Delta Falguière, Pommerim, Delta Rue Monsieur, Paludate, Delta Galilée, Delta Réaumur et Delta Trévise Groupe Immo Pan Europe : Immo Pan Europe, Immo Pan 1, IPE Fort Neipperg, Immo Pan 2, IPE Froissart, Immo Pan 3, IPE Llacuna, IPE Vizcaya, Immo Pan 4, IPE Keizersgracht

Groupe Delta Loisirs Evasion : OPCI Delta Loisirs Evasion et SAS Delta Loisirs Evasion


## A.1.2 Objectifs généraux de l'entreprise et principales tendances


### › Objectifs généraux de l'entreprise

L'année 2022 marque la dernière année de mise en œuvre du plan stratégique Groupe « Engagés pour demain >> 2019-2022 au sein duquel MAIF VIE s'inscrit pleinement. Ce plan est articule autour de 2 axes stratégiques majeurs que sont la singularité et le pivot :

› le premier axe precite vise à approfondir les caractéristiques de notre fonctionnement afin de proposer un modèle d'entreprise différent, créant plus de satisfaction chez nos sociétaires, plus d'épanouissement chez nos acteurs internes et un impact toujours plus positif sur notre environnement. L'adoption par l'Assemblée Générale du 23 juin 2021 du statut de Société à mission s'inscrit naturellement dans cette démarche ;

› le second, quant à lui, va nous permettre de continuer à évoluer, en intégrant au mieux les transformations de notre métier induites par le développement du numérique, dans l'expérience que l'on propose à nos sociétaires, dans le bon usage des données collectées. Il s'agit également de nous ouvrir progressivement à de nouvelles activités et/ou de nouveaux marchés, capables de compléter notre activité traditionnelle.

Clair dans nos objectifs et dans notre vision à 4 ans, ce plan stratégique nous a permis de garder une capacité d'adaptation dans sa déclinaison opérationnelle, indispensable pour répondre aux transformations et évolutions du marché, toujours plus profondes et rapides. 2022 a marqué l'année de l'élaboration du nouveau plan stratégique Groupe 2023- 2026 « Régénération ».

MAIF VIE a décliné son propre plan stratégique s'inscrivant dans les ambitions du Groupe mais adapté à ses enjeux propres. Celui-ci a été validé par le Conseil d'Administration MAIF VIE du 2 mars 2023.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="14 / 111" -->

› Principales tendances

Au 31 décembre 2022, MAIF VIE compte 614 556 adhérents soit une augmentation de 4,6 % (+ 5,2 % en 2021).

Les taux de rémunération servis sur les principaux contrats d'épargne restent dans la moyenne du marché (2,10 % nets servis en 2022).

Les encaissements totaux bruts de réassurance sont en progression par rapport à 2021 et s'élèvent à 834 368 milliers d'euros soit + 3 % (+ 10 % en 2021).

Le montant des prestations augmente de 5,9 % (+ 3,9% en 2021), l'ensemble du secteur présente une hausse moins marquée avec + 3 % (+ 7 % en 2021).

En épargne, MAIF VIE enregistre un léger retrait de la collecte nette (- 3,1%), celle-ci s'établissant à + 211 M€. Le marché de l'assurance vie conserve également une situation de collecte nette positive (+ 14,3 milliards d'euros) mais avec une collecte nette négative sur les supports euros (-20,3 milliards d'euros) là où MAIF VIE observe une collecte nette positive aussi bien sur les supports euros que sur ceux en unités de compte.

Les provisions techniques brutes de réassurance progressent de 2,7 % par rapport à 2021 (+ 4,5 % en 2021) et portent principalement sur les contrats d'assurance en cas de vie.

L'exercice 2022 se traduit par un bénéfice de 51 214 milliers d'euros (50 332 milliers d'euros en 2021).

Les frais généraux, avec un montant total de 105 995 milliers d'euros augmentent de 8 140 milliers d'euros sous l'effet principalement de l'activité croissante de MAIF VIE aussi bien sur l'épargne que sur la prévoyance générant une hausse des commissions versées à nos partenaires distributeurs et de la politique d'investissement informatique et sociale.

Néanmoins, cette augmentation s'accompagne d'une maîtrise de l'évolution des frais généraux (hors charges de distribution) qui représentent 0,49 % des provisions mathématiques.


## Principaux résultats de l'exercice ›

Pour l'exercice 2022, le résultat global de l'entreprise fait l'objet d'une présentation économique distinguant les résultats de souscription, financier et exceptionnel.

Il s'établit ainsi :

| En milliers d'euros | 2022 | 2021 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Résultat de souscription | 21 404 | 23 603 | - 2 199 | - 9,3 % |
| Ressources de gestion | 99 634 | 94 071 | 5 563 | + 5,9 % |
| Frais généraux | - 105 995 | - 97 854 | 8 140 - | + 8,3 % |
| Résultat technique | 27 765 | 27 387 | 378 | + 1,4 % |
| Résultat financier | 47 251 | 52 013 | 4 762 - | - 9,2 % |
| Résultat exceptionnel | 1 167 - | 459 - | 708 - | NS |
| Charges (-) et Produits (+) non techniques 1 | - 541 | 953 | 1 494 - | NS |
| Impôt sur les sociétés, participation et intéressement | - 15 732 | - 25 778 | 10 045 | - 39,0 % |
| RESULTAT NET COMPTABLE | 51 214 | 50 332 | 882 | 1,8% |

NA : non applicable NS : non significatif

' Produits non techniques d'un montant equivalent à l'impôt sur les sociétés relatif aux dotations nettes de reprises sur la réserve de capitalisation (Arrêté du 30 décembre 2010 relatif aux modalités de calcul des mouvements sur la réserve de capitalisation et modifiant le Code des assurances A 333-3).

Le résultat de souscription est en recul de 2 199 milliers d'euros, soit -9,3 % par rapport à 2021, sous les effets combines de ses différentes composantes détaillées ci-après.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="15 / 111" -->

Le résultat financier diminue de 4 762 milliers d'euros, soit -9,2% par rapport à 2021. Il intègre les préconisations de la Direction générale validées par le conseil d'administration liées à la politique de taux de rendement servis, au niveau de la provision pour participation aux bénéfices et aux éventuelles incidences du contexte financier.

Le conseil d'administration de MAIF VIE, lors de sa réunion du 14 décembre 2022, a décidé de servir un taux de rémunération dans la moyenne du marché tout en renforçant les fonds propres de l'entreprise et en consommant prudemment la provision pour participation aux bénéfices ayant pour conséquences :

› une reprise nette de la provision pour participation aux bénéfices pour un montant de 17 754 milliers d'euros ;

› des différents taux servis nets à 2,10 % sur la majorité des fonds euros.

En tant qu'entreprise engagée, MAIF VIE souhaite que sa performance puisse avoir encore plus d'impact positif sur l'environnement. Ainsi, l'entreprise a décidé de consacrer une partie de son résultat pour contribuer à une plus grande justice climatique et pour préserver la biodiversité en s'inscrivant dans le dispositif de dividende écologique porté par le Groupe et pour lequel MAIF VIE contribue.

Les paragraphes suivants présentent les résultats de souscription, des investissements et des autres activités tels qu'ils apparaissent dans les états financiers établis en normes comptables françaises avec leur évolution par rapport à la période précédente. En complément, les résultats de souscription sont présentés par ligne d'activité Solvabilité 2 et les résultats des investissements par catégorie de placements Solvabilité 2.


## A.2 Résultats de souscription

Cette partie présente notamment les informations contenues dans l'état quantitatif "Primes, sinistres et dépenses par ligne d'activité". Il s'agit d'un état réglementaire en normes Solvabilité 2 qui a la particularité de présenter par ligne d'activité en normes Solvabilité 2 (cf. tableau de correspondance paragraphe A.1.1), des montants de produits et de charges valorisés selon les normes comptables françaises. Cet état n'intègre pas les revenus financiers issus de l'activité d'assurance.

Une ligne d'activité est un regroupement de garanties d'assurance similaires. Pour MAIF VIE, il existe 3 lignes d'activités (cf. paragraphe A.1.1. tableau de correspondance entre les lignes d'activité et les principales garanties des contrats MAIF VIE).


### A.2.1 Résultats au niveau agrégé et comparaison avec N-1

Le résultat global de souscription de MAIF VIE se présente ainsi au 31/12/2022 :

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="16 / 111" -->

<!-- PageHeader="\> ACTIVITÉ & RÉSULTATS" -->

| En milliers d'euros Ressources de gestion | 2022 | 2021 | Variation ||
|||| (en montant) | (en %) |
|| 99 634 | 94 071 | 5 563 | + 5,9 % |
| - | - | - | - | - |
| Frais sur versement | 21 947 | 20 933 | 1 014 | + 4,8 % |
| Frais sur épargne gérée | 62 876 | 60 501 | 2 375 | + 3,9 % |
| Autres ressources de gestion | 14 811 | 12 637 | 2 175 | + 17,2 % |
| Frais généraux | - 105 995 | 97 854 - | 8 140 | + 8,3 % |
| Résultat technique | 27 765 | 27 387 | 378 | + 1,4 % |
| Rassurcap et Rassurcap Solutions | 21 993 | 23 690 | 1697 - | - 7,2 % |
| Assurance en couverture de prêts | 3 503 | 1740 | 1764 | NS |
| Sollicitudes | 1 432 | 1 218 | 214 | + 17,6 % |
| Rentes | 56 - | 83 | 139 - | NS |
| Autres (Epargne, PGG (1)) | 893 | 657 | 236 | + 35,9 % |
| RESULTAT DE SOUSCRIPTION | 21 404 | 23 604 | - 2 199 | -9,3% |

(1) Provision Globale de Gestion

Les frais sur versements sont en hausse en lien avec la croissance du chiffre d'affaires épargne et prévoyance.

Les frais sur épargne gérée augmentent par rapport à l'année précédente, en lien avec les encours des provisions mathématiques et les frais prélevés de la gestion sous mandat sur une année pleine.

Les autres produits de gestion sont également en hausse. Ils sont principalement composés des :

› frais de prorogation du contrat Cap'Épargne ;

› commissions reçues ou à recevoir des reassureurs relatives aux contrats de prévoyance (Rassurcap, Rassurcap Solutions et Sollicitudes) et à l'assurance en couverture de prêts ; › produits de gestion perçus des partenaires rétrocessionnaires en lien direct avec la gestion des Unités de Comptes (UC).

› commissions d'apport et de distribution nettes de reassurance.

Les frais généraux, avec un montant total de 105 995 milliers d'euros (97 854 milliers d'euros en 2021), augmentent de 8 141 milliers d'euros (+ 8,3 %). Cette hausse reflète le développement de l'activité de MAIF VIE et l'impact de sa politique sociale. Elle s'explique principalement par :

› l'activité croissante de MAIF VIE aussi bien sur l'épargne que sur la prévoyance générant une hausse des commissions versées de 3 317 milliers d'euros ;

› la stratégie d'investissements de MAIF VIE générant de moindres projets immobilisables et de moindres refacturations impactant les frais generaux à la hausse de 2 336 milliers d'euros ;

› la prise en compte de l'accord «< Reconnaissance mutuelle >> signé en octobre 2022 et générant une hausse des frais généraux de 1 658 milliers d'euros au titre du provisionnement de la retraite progressive .

Le résultat technique net de réassurance correspond à l'excédent ou au déficit constaté lorsque l'on met en regard d'une part, les primes pures (cotisations après déduction des frais) et, d'autre part, les prestations versées et les provisions techniques constituées pour faire face aux engagements futurs. Il s'établit, pour 2022, à 27 765 milliers d'euros, en progression de 1,4 % par rapport à 2021 (27 387 milliers d'euros) :

› les résultats techniques de Rassurcap et Rassurcap Solutions s'élèvent globalement à 21 993 milliers d'euros contre 23 690 milliers d'euros en 2021 sous l'effet principalement de la revue du traité de réassurance et de l'augmentation de la charge de sinistres ;

› l'assurance en couverture de prêts génère un résultat technique net de reassurance de 3

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

503 milliers d'euros (1 740 milliers en 2021), en progression du fait principalement des effets de rattrapage sur les variations des provisions techniques pour AEM et de l'augmentation de production pour le produit Avantage Emprunteur ; le résultat technique Sollicitudes s'établit à 1 432 milliers d'euros (1 218 milliers en 2021), en augmentation du fait d'un recul de la charge de provision ;

›

› les autres éléments du résultat technique atteignent 893 milliers d'euros (657 milliers en 2021), ils proviennent de l'activité Epargne et des variations de la Provision Globale de Gestion.


### A.2.2 Analyse de la performance de souscription par ligne d'activité (ou Lob «Line Of Business >>)

MAIF VIE est exclusivement concernée par 3 lignes d'activité au sens des normes Solvabilité 2 :

› Lob 1 : contrats d'assurance vie avec participation aux bénéfices ;

› Lob 2 : assurance indexée ou en unités de compte (UC) ;

›

Lob 3 : autres activités d'assurance vie.

En 2022, les ressources de gestion sont en hausse de 6 % et principalement issues des contrats d'assurance vie avec participation aux bénéfices représentant plus de 76 % de l'encaissement total. Les contrats d'assurance en unités de compte et de prévoyance génèrent respectivement 16 % et 8 % des encaissements.

| En milliers d'euros | 2022 2021 ||||||||
|| Lob 1 | Lob 2 | Lob 3 | Total | Lob 1 | Lob 2 | Lob 3 | Total |
| - | - | - | - | - | - | - | - | - |
| Ressources de gestion | 69 804 | 10 867 | 18 963 | 99 634 | 67 787 | 10 014 | 16 271 | 94 070 |
| Frais sur versements | 9 867 | 633 | 11 447 | 21 947 | 9 991 | 561 | 10 381 | 20 933 |
| Frais sur épargne gérée | 57 640 | 5 236 | - | 62 876 | 56 648 | 4 529 | - | 61 177 |
| Autres produits de gestion | 2 297 | 4 998 | 7 516 | 14 811 | 1 148 | 4 924 | 5 890 | 11 960 |
| Frais généraux | 69 724 | 6 718 | 29 552 | 105 994 | 63 671 | 5 550 | 28 634 | 97 854 |
| Resultat technique | 1700 | 561 | 25 504 | 27 765 | 2 031 | 82 - | 25 438 | 27 387 |
| Rassurcap et Rassurcap Solutions | | | 21 993 | 21 993 | | | 23 690 | 23 690 |
| Assurance en couverture de prêts | | | 3 503 | 3 503 | | | 1 740 | 1740 |
| Sollicitudes | 1 432 | | | 1 432 | 1 218 | | | 1 218 |
| Rentes | 56 - | | | 56 - | 83 | | | 83 |
| Autres (Epargne, PGG (1)) | 324 | 561 | 8 | 893 | 730 | 82 - | 8 | 656 |
| RESULTAT GLOBAL DE SOUSCRIPTION | 1 779 | 4 710 | 14 915 | 21 405 | 6 147 | 4 382 | 13 075 | 23 603 |

(1) Provision Globale de Gestion

Le résultat global de souscription de MAIF VIE s'établit à 21 405 milliers d'euros et diminue de 9 % entre 2021 et 2022. Les contrats d'assurance vie avec participation aux bénéfices et autres activités d'assurance vie représentent 78 % du résultat global de souscription 2022 de l'entreprise.


## Contrats d'assurance vie avec participation aux bénéfices ›

Cette ligne d'activité regroupe les contrats d'assurance vie monosupport, les compartiments euros des contrats multisupports, le contrat obsèques Sollicitudes et les produits de retraite hors compartiment UC du PERI.

Le résultat global de souscription est bénéficiaire et s'explique par des ressources de gestion supérieures aux frais généraux.


## Contrats d'assurance indexée ou en unités de compte (UC) ›

Cette activité concerne l'ensemble des compartiments UC des contrats d'assurance vie multisupports et du PERI.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageNumber="17 / 111" -->

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->


## › Autres activités d'assurance vie

Le résultat de souscription d'un montant de 14 915 milliers d'euros est constitué du résultat technique de Rassurcap et Rassurcap Solutions ainsi que celui de l'assurance en couverture de prêt.


## A.3 Résultats des investissements


### A.3.1 Les revenus et les dépenses générés par les activités d'investissement

Ils sont constitués par les produits financiers acquis à l'entreprise et qui sont, pour l'essentiel, ceux générés par les placements représentatifs des capitaux propres.

De manière simplifiée, le résultat financier peut être présenté de la manière suivante :

| En milliers d'euros | 2022 | 2021 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Produits financiers | 356 688 | 337 632 | 19 056 | + 5,6 % |
| Charges financières | 64 450 - | 46 693 - | - 17 757 | + 38,0 % |
| Produits financiers nets | 292 238 | 290 939 | 1 299 | + 0,4 % |
| Produits financiers affectes aux contrats aux taux servis | - 262 741 | - 183 223 | - 79 519 | + 43,4 % |
| Reprise (+) ou Dotation (-) de Provision pour Participation aux Bénéfices | 17 754 | - 55 703 | 73 457 | NS |
| RESULTAT FINANCIER | 47 251 | 52 013 | - 4 762 | -9,2% |

Hors frais de gestion, les produits nets des placements s'élèvent à 292 238 milliers d'euros, en hausse de 1 299 milliers d'euros par rapport à 2021. Les produits ont, en 2022, augmenté de 19 056 milliers d'euros et les charges de 17 757 milliers d'euros.

La comparaison avec l'exercice précédent fait apparaître les évolutions suivantes :

| Actif / En milliers d'euros | 2022 || 2021 || Variation ||
|| Produits | Charges | Produits | Charges | Produits | Charges |
| - | - | - | - | - | - | - |
| Placements (autres que les actifs en | | | | | | |
| representation de contrats en UC ou | 356 560 | 64 450 | 337 384 | 46 693 | 23 452 | 17 757 |
| indexés) ||| | | | | 
| Immobilier (autre que pour usage propre) | | | | | | |
| Participations | 34 690 | 157 | 34 330 | 4 | 360 | 153 |
| Actions | 11 639 | 2 672 | 4 033 | 6 323 | 7 606 | - 3 651 |
| Obligations | 215 567 | 39 606 | 209 099 | 31 401 | 6 468 | 8 205 |
| Fonds d'investissement | 96 802 | 22 015 | 88 266 | 8 965 | 8 536 | 13 050 |
| Produits dérivés | | | | | | - |
| Dépôts autres que ceux assimilables à de la | 2 138 | | 1 656 | | 482 | |
| trésorerie | | | | | | |
| Autres placements | | | | | | |
| Prêts et prêts hypothécaires | 102 | | 88 | | 14 | |
| Trésorerie et équivalent trésorerie | 26 | | 160 | | 134 - | |
| TOTAL | 356 688 | 64 450 | 337 632 | 46 693 | 19 056 | 17 757 |
| Produits nets des placements | 292 238 || 290 939 || 1 299 ||

La hausse des produits de placements de 19 056 milliers d'euros provient notamment de :

› la réalisation de plus-values suite à la cession de sociétés et fonds immobiliers plus MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

<!-- PageHeader="18 / 111" -->

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="19 / 111" -->

importantes qu'en 2021 ;

› la hausse des revenus sur les obligations liés à la croissance du portefeuille.

La hausse des charges de placements de 17 757 milliers d'euros provient plus particulièrement :

› des moins-values réalisées sur des cessions d'obligations et des fonds d'investissement actions plus importantes qu'en 2021 en raison de la hausse des taux d'intérêt et de la baisse des marchés boursiers.

Les flux concernant les ajustements ACAV portant spécifiquement sur les placements en représentation des contrats en UC sont intégrés au niveau du résultat technique des UC.


## A.4 Résultats des autres activités


### A.4.1 Évolution des autres charges et produits

| Rubriques (En milliers d'euros) | 2022 | 2021 | Variation |
| - | - | - | - |
| Résultat exceptionnel | 1 167 - | - 459 | - 708 |
| Charges (-) et Produits (+) non techniques 1 | 541 - | 953 | - 1494 |
| Participation et intéressement | 779 - | 5 361 - | 4 582 |
| Participation | 250 | 4 854 | - 4 604 |
| Intéressement | 530 | 507 | 23 |
| Supplément d'intéressement | | - | - |
| Impôt sur les sociétés | - 14 953 | - 20 417 | 5 464 |
| Autres charges et produits | 17 441 - | - 25 284 | 7 843 |


## \> Résultat exceptionnel

Le résultat exceptionnel de l'exercice est principalement constitue de dotations pour provisions pour risques et charges et contentieux.


## › Charges et produits non techniques

Des produits non techniques d'un montant équivalent à l'impôt sur les sociétés relatif aux dotations nettes de reprises sur la réserve de capitalisation ont été constatés pour un montant de - 540 milliers d'euros (contre + 953 milliers d'euros en 2021).


### ›


### Participation et intéressement

Un accord de participation Groupe, signé le 11 juin 2015, repose conformément aux dispositions des articles L 3324-1 à L 3324-4 du Code du travail sur le montant le plus élevé déterminé entre la formule de droit commun et la formule dérogatoire.

Une charge de participation a été comptabilisée à hauteur de 250 milliers d'euros en 2022 contre 4 854 milliers d'euros en 2021.

L'intéressement des salariés, au titre de l'exercice 2022, sur la base de l'accord signé le 30 juin 2018 s'élève à 530 milliers d'euros (507 milliers d'euros en 2021). Aucun supplément d'intéressement n'a été versé sur l'année.


### ›


## Impôt sur les sociétés

La détermination de l'impôt sur les sociétés, conformément à la réglementation fiscale, s'élève à 14 953 milliers d'euros (contre 20 417 milliers d'euros en 2021).

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> ACTIVITÉ & RÉSULTATS
===

<!-- PageHeader="20 / 111" -->


## A.5 Autres informations

MAIF VIE n'identifie pas d'autre information importante à communiquer sur l'activité et les résultats en dehors de ce qui est décrit dans ce rapport.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->
<figure>

![](figures/4)

<!-- FigureContent="B" -->

</figure>


SYSTÈME DE GOUVERNANCE

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageHeader="22 / 111" -->


## B Système de Gouvernance

Cette partie est consacrée à la présentation d'informations qualitatives relatives :

› aux caractéristiques générales du système de gouvernance de MAIF VIE ;

› aux exigences de compétence et d'honorabilité en place, notamment pour ce qui concerne les dirigeants effectifs et les responsables des fonctions clés gestion des risques, vérification de la conformité, audit interne et actuarielle ;

› au système de gestion des risques, avec le rôle de la fonction clé gestion des risques ;

› au système de contrôle interne, avec le rôle de la fonction cle vérification de la conformité

›

; à la fonction clé d'audit interne ;

› à la fonction clé actuarielle ;

›

à la sous-traitance.


## B.1 Informations générales sur le système de gouvernance

Pour renforcer son engagement et sa singularité, MAIF VIE est société à mission depuis 2021. Un comité de mission est chargé du suivi des objectifs inscrits dans ses statuts déclinant sa raison d'être libellée dans les termes suivants : « Convaincus que seule une attention sincère portée à l'autre et au monde permet de garantir un réel mieux commun, nous, MAIF VIE, plaçons cette attention au cœur de chacun de nos engagements et de chacune de nos actions. ».

Le comité de mission valide la feuille de route et suit son exécution tout au long de l'année. Il établit un rapport annuel transmis à l'assemblée générale. La réalisation des objectifs est en outre contrôlée périodiquement par un Organisme tiers Indépendant (OTI) qui émet un rapport mis à disposition du public.


### B.1.1 Évaluation du système de gouvernance

MAIF VIE considère avoir mis en place un système de gouvernance permettant de garantir une gestion saine, prudente et efficace de l'activité, conforme aux exigences de la réglementation. Son système de gouvernance est proportionné à la nature, à l'ampleur et à la complexité de ses opérations.


### B.1.2 Structure de gouvernance et fonctions clés

La description ci-après vise à communiquer des informations relatives à l'organisation des fonctions de gouvernance au sein de MAIF VIE.


#### B.1.2.1 Conseil d'administration


##### Composition du conseil d'administration ›

Les membres du conseil d'administration sont élus par l'assemblée générale des actionnaires pour six ans et rééligibles. Le conseil d'administration se renouvelle partiellement suivant le nombre de membres en fonction afin que le renouvellement soit complet dans chaque période de six ans. Le conseil d'administration de MAIF VIE au 31 décembre 2022 est composé de huit membres.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageHeader="23 / 111" -->


###### ›


## Attributions du conseil d'administration

Le conseil d'administration détermine les orientations de l'activité de la société et veille à leur mise en œuvre. Sous réserve des pouvoirs expressément attribués par la loi aux assemblées d'actionnaires et dans la limite de l'objet social, il se saisit de toute question intéressant la bonne marche de la société et règle par ses délibérations les affaires la concernant. Il procède à tout moment aux contrôles et vérifications qu'il estime opportuns.

Le conseil d'administration valide les orientations en matière d'allocation stratégique d'actifs, de gestion actif/passif et de politique des investissements financiers et des placements. Il se prononce en particulier sur les modalités de choix des intermédiaires financiers et sur les opérations des instruments financiers à terme.

Le conseil d'administration dispose également depuis l'exercice 2015 du pouvoir de choisir, nommer et révoquer le directeur général, comme il en dispose pour le président.


## › Président du conseil d'administration

Le président est désigné par le conseil d'administration.

Yves Pellicier a été désigné président lors de la réunion du conseil d'administration du 22 juin 2022.

En lien avec la stratégie (construction des orientations et suivi), il prend part aux travaux de construction et de suivi du Plan stratégique, à son adaptation/réorientation.

Le président est présent dans l'organisation, a accès aux informations et est un interlocuteur privilégié sur des sujets majeurs. Il dispose des compétences et de l'expérience requises sur les activités et les risques de l'entreprise, dispose d'une vue assez complète et approfondie de l'ensemble de l'activité, ainsi que de réels pouvoirs de décision et de contrôle.

Le président est également chargé d'assurer les missions règlementaires dévolues au président du conseil d'administration. A ce titre, il convoque et anime le conseil d'administration qu'il préside. Il fixe les ordres du jour et facilite l'exercice par le conseil d'administration de sa capacité d'initiative, de décision et de contrôle. Il s'assure que les administrateurs disposent des éléments nécessaires pour exercer leurs missions, s'assure du bon fonctionnement et de la cohésion du collectif. Il dirige la fonction clé d'audit interne.

Le président rend compte à l'assemblée générale et exécute ses décisions.

Il présente au conseil d'administration, avec le directeur général, les orientations de l'activité de la société (notamment en matière de stratégie et d'opérations financières) et veille après décision à leur mise en œuvre, il se saisit de toute question intéressant la bonne marche de la société pour la soumettre au conseil d'administration et lui présente tout contrôle et vérification qu'il a fait mettre en œuvre. Il assure le rayonnement de la société.


## Vice-president et administrateurs ›

Le président est secondé par le vice-président du conseil d'administration. Le vice-président est désigné par les membres du conseil d'administration.

Christian Ponsolle est vice-président depuis le 22 juin 2022.

Le conseil d'administration comporte également 6 administrateurs : Arnaud Courdesses, Luc Delage, Marie-Pierre Elie, Christine Mathe-Cathala, Nadine Naas et Laurent Pertusa.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTEME DE GOUVERNANCE
===

24 / 111

› Activités du conseil d'administration en 2022

Au cours de l'exercice écoulé, le conseil d'administration s'est réuni 5 fois. Sur l'ensemble de ces réunions qui se sont déroulées au cours de l'exercice 2022, le taux de présence des administrateurs était en moyenne de 93 % et le taux de représentés de 3 %.

Les décisions rendues par le conseil d'administration et / ou les informations portées à sa connaissance ont notamment porté sur :

› l'arrêté des comptes en normes françaises et proposition d'affectation du résultat ;

› l'arrêté des comptes annuels prudentiels en normes Solvabilité 2 ;

› le suivi du budget 2022 et la présentation du budget 2023 ;

› les orientations relatives aux décisions de gestion et fixation des taux de rémunération des contrats et des taux garantis 2023 ;

› l'allocation stratégique d'actifs ;

› la feuille de route MAIF VIE ;

› la feuille de route mission 2022, le rapport du comité mission et la désignation de l'organisme tiers indépendant ;

› le rapport annuel d'activité 2021 du comité des risques, d'audit et des comptes ;

› les rapports individuels RSR et SFCR ;

› le rapport de la fonction clé actuarielle ;

\> le rapport de la fonction clé audit interne ;

\> le rapport de la fonction clé vérification de la conformité ;

\> le rapport de la fonction clé gestion des risques ;

› le rapport sur le contrôle interne du dispositif LCB/FT et le rapport sur les procédures d'élaboration et de vérification de l'information financière et comptable ;

› le suivi de l'activité conformité, contrôle interne et continuité ;

› le plan d'audit interne 2023 ;

› la politique de la société en matière d'égalité professionnelle et salariale ;

› la revue des conventions réglementées.


## B.1.2.2 Direction générale

Le directeur général est désigné par le conseil d'administration.

Hélène N'Diaye occupe cette fonction depuis la réunion du conseil d'administration du 24 mai 2018.

En cette qualité, elle est dirigeant effectif de droit et assume la direction générale et effective de l'entreprise, sous le contrôle du conseil d'administration et dans le cadre des orientations arrêtées par ce dernier.

Elle participe à l'élaboration de la stratégie, assure la mise en œuvre de la stratégie validée par le conseil d'administration, rend compte devant le conseil d'administration, préside les Comités de direction générale de la société et représente la société aux Comités financiers.

Elle est investie des pouvoirs les plus étendus pour agir en toutes circonstances au nom de la société dans la limite de l'objet social et sous réserve des pouvoirs que la loi attribue expressément à l'assemblée générale et au conseil d'administration.

Elle représente la société dans ses rapports avec les tiers et dirige les domaines d'activités et équipes opérationnelles de l'entreprise.

Les quatre fonctions clés lui sont rattachées et lui rendent compte de leurs travaux.

Elle décide avec le président, des orientations budgétaires, de l'arrêté des comptes et du résultat de l'entreprise, de la fabrication ou de l'évolution d'un produit, de la politique de MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

AAA

> SYSTEME DE GOUVERNANCE
===

<!-- PageNumber="25 / 111" -->

gestion tactique des placements et de l'allocation stratégique d'actifs et tous les sujets règlementaires relevant de Solvabilité 2, soumis à décision du conseil d'administration.

Sur proposition de la directrice générale, le conseil d'administration a désigné Marina Sellari, responsable du pôle performance économique, solvabilité et gouvernance, en tant que directrice générale déléguée.

Le directeur général délégué est dirigeant effectif de droit et à ce titre, il dispose d'un domaine de compétences et de pouvoirs suffisamment large sur les activités et les risques de l'entreprise, et est impliqué dans les décisions ayant un impact important sur la société, notamment en matière de stratégie, de budget ou de questions financières. Il fait preuve d'une disponibilité suffisante pour exercer ce rôle.


## B.1.2.3 Comités spécialisés du conseil d'administration

Depuis 2009, le conseil d'administration de MAIF VIE s'est prononcé pour ne pas constituer en son sein son propre comité d'audit et pour déléguer les missions lui incombant au comité d'audit et des comptes MAIF, conformément aux dispositions de l'article L.823-20 5° du code de commerce. Ses principales missions sont décrites ci-après :

| Comités du conseil d'administration | Principales missions et activités en 2022 |
| - | - |
| Comité des risques d'audit et des comptes de MAIF Six administrateurs MAIF et deux personnes externes ‡ l'entreprise. | Le Comité des risques, d'audit et des comptes est chargé, pour les sociétés incluses dans le périmètre de ses interventions précisé dans son règlement d'application pratique, à savoir le groupe MAIF, MAIF, et MAIF VIE, d'éclairer et d'alerter les conseils d'administration concernés dans le domaine des risques, des comptes, du contrôle interne et du contrôle externe, et des politiques écrites. |
|| Sans préjudice des compétences du conseil d'administration et de la direction générale, il est notamment chargé de : › suivre le processus d'élaboration de l'information financière et, le cas échéant, de formuler des recommandations pour en garantir l'intégrité ; › suivre l'efficacité du système de contrôle interne ; › suivre l'efficacité de l'audit interne, en ce qui concerne les procédures relatives à l'élaboration et au traitement de l'information comptable et financière, sans qu'il soit porté atteinte à l'indépendance de ce dernier ; › suivre la politique, les procédures et l'efficacité du système de gestion des risques ; › piloter la procédure de sélection des commissaires aux comptes ; › émettre une recommandation sur les commissaires aux comptes proposés à la désignation et au renouvellement par l'assemblée générale ; › veiller à l'indépendance des commissaires aux comptes ; › suivre la réalisation du contrôle legal des comptes sociaux par les Commissaires aux comptes ; › auditionner les responsables des fonctions clés sur demande du conseil d'administration. › établir son rapport annuel. |
|| En outre, le Comité des risques d'audit et des comptes approuve préalablement les services autres que la certification des comptes fournis par les commissaires aux comptes ou les membres du réseau auquel ils appartiennent, après analyse des risques pesant sur l'indépendance des Commissaires aux comptes et des mesures de sauvegarde appliquées par ces derniers. |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageNumber="26 / 111" -->

Il doit rendre compte régulièrement au conseil d'administration de l'exercice de ses missions et l'informer sans délai de toute difficulté rencontrée. Il doit notamment rendre compte des résultats de la mission de certification des comptes, de la manière dont cette mission a contribué à l'intégrité de l'information financière et du rôle qu'il a joué dans ce processus.

Par ailleurs, le règlement d'application pratique en vigueur définit les modalités de l'accès aux réunions du comité des fonctions clés Groupe et/ou solo.

Ce comité exerce son activité sous la responsabilité du conseil d'administration, seul habilité à prendre les décisions sauf exception prévue par la réglementation, et organise librement ses travaux dont il rend compte au moins annuellement dans un rapport remis au conseil d'administration.


## B.1.2.4 Fonctions clés

Le système de gouvernance de MAIF VIE comporte quatre fonctions clés règlementaires de contrôle, chacune dotée de responsables clairement identifiés :

› la fonction gestion des risques, confiée à un chef de projets Actuariat ;

› la fonction de vérification de la conformité, confiée au responsable de la conformité et des contrôles ;

› la fonction audit interne confiée au responsable de l'audit interne MAIF ;

› la fonction actuarielle, confiée à un chef de projets Actuariat.

Trois des quatre fonctions clés (gestion des risques, actuarielle et vérification de la conformité) et leurs travaux sont rattachées hiérarchiquement au directeur général de MAIF VIE.

Le responsable de la fonction d'audit interne a été désigné au sein du groupe MAIF (salarié MAIF), l'organisation de sa mission et les modalités d'exercice de l'autorité du directeur général sur le responsable de cette fonction clé étant précisées dans une convention conclue entre MAIF VIE et MAIF.

Les rôles et missions de ces fonctions sont précisés en paragraphes B.3 pour la fonction gestion des risques, B.4.3 pour la fonction de vérification de la conformité, B.5 pour la fonction audit interne et B.6 pour la fonction actuarielle.

Les responsables des fonctions clés ont été formellement désignés par le directeur général et présentés au conseil d'administration.

Les fonctions clés disposent de l'autorité, des ressources et de l'indépendance opérationnelle nécessaires afin de mener leurs tâches à bien, de rendre des comptes au conseil d'administration et de le conseiller.

Les responsables des fonctions clés se tiennent en relation permanente avec les dirigeants effectifs, notamment dans le cadre des instances relatives à la gouvernance décisionnelle du groupe MAIF.

Lors de sa séance du 21 juin 2018, le conseil d'administration de Parnasse-MAIF (devenue depuis MAIF VIE) a validé une procédure d'accès au conseil d'administration des fonctions clés et un contrat de service relatif à la fonction d'audit interne de MAIF VIE.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageNumber="27 / 111" -->

<!-- PageHeader="\> SYSTEME DE GOUVERNANCE" -->

<figure>

![](figures/5)

<!-- FigureContent="Conseil d'administration de MAIF VIE Yves Pellicier - Président Hélène N'Diaye Directrice générale Dirigeant effectif Responsable Fonction clé Audit Interne Vianney Dumont Responsable Fonction clé Vérification de la conformité Fabrice Couturier Responsable Fonction clé Gestion des risques François Lair Responsable Fonction clé Actuarielle Christophe Lateulère Marina Sellari Directrice générale déléguée Dirigeant effectif" -->

</figure>



### B.1.3 Changement important du système de gouvernance intervenu au cours de la période

Dans le cadre des exigences renforcées par la réglementation, en particulier la Directive Solvabilité 2, visant à répartir clairement les rôles et responsabilités entre les fonctions exécutives et les fonctions de surveillance au sein de la gouvernance des sociétés d'assurance, en particulier dans les sociétés de grande taille pratiquant des opérations complexes, la direction effective de MAIF VIE est désormais confiée au directeur général et au directeur général délégué, le nouveau président nommé par le conseil d'administration n'étant plus lui- même nommé dirigeant effectif.


#### B.1.4 Information sur la politique et les pratiques de rémunération et droits à rémunération sur la période

MAIF VIE considère que sa pratique de rémunération la préserve de toute prise de risques excessive par ses dirigeants et salariés afin de se centrer sur le seul intérêt qui prime, celui de l'adhérent.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageNumber="28 / 111" -->

La politique de rémunération est réexaminée chaque année dans le cadre des dispositifs prévus par la loi pour les classes 1 à 7 (NAO, participation, intéressement) et/ou en cas de changement significatif de la gouvernance ayant un impact sur les responsabilités qui seraient confiées.

La rémunération du président du conseil d'administration, et celles du directeur général et du directeur général délégué sont fixées par le conseil d'administration, dans le respect de la politique de rémunération du groupe MAIF et de celle arrêtée par le conseil d'administration MAIF VIE


## B.1.4.1 Administrateurs

Conformément aux statuts de la société, les fonctions d'administrateurs ne sont pas rémunérées. Seuls peuvent être remboursés les frais engagés pour l'exercice de leur fonction et dans l'intérêt de la société.

B.1.4.2 Président, Dirigeants effectifs et cadres hors classe


## › Critères de rémunération

Le président ne perçoit aucune rémunération, ni variable, ni primes d'intéressement ou de participation au titre de cette fonction. Par ailleurs, il n'y a aucun salarié cadre hors classe au sein de l'entreprise. Le directeur général est salarié MAIF, ainsi que le directeur général délégué.


## Avantages sociaux ›

Le président, le directeur général et le directeur général délégué, relevant d'un statut de mandataire social ou de salarié MAIF hors classe, possèdent les avantages associés à ces statuts qui sont ceux accordés par MAIF. Ces avantages correspondent notamment, depuis le 01/01/2020, a un PER Obligatoire qui s'ajoute au régime à cotisations définies (1% de branche) dont bénéficie l'ensemble des salariés.

Le président et le directeur général bénéficient d'un régime à cotisations définies (article 82 du Code Général des Impôts).

B.1.4.3 Salariés relevant des classes 1 à 7 de la convention collective des sociétés d'assurance


## ›


## Critères de rémunération

L'évolution du salaire de base des classes 1 à 7 est encadrée par un accord et par le dispositif des négociations annuelles obligatoires portant sur les salaires.

En 2022, la masse salariale a évolué de 2,71 % sous l'effet de l'ancienneté, des progressions et augmentations individuelles et des promotions.

La direction a souhaité le versement d'une prime exceptionnelle de pouvoir d'achat à l'ensemble des salariés liés par un contrat de travail au 31 mars 2022, calculée en fonction de la rémunération perçue et correspondant à 1,18 % de la masse salariale.

Par principe, il n'y a pas de bonus individuel pour les salariés des classes 1 à 7. Les salariés bénéficient en outre d'un accord de participation qui vise à redistribuer une partie des résultats réalisés par le groupe ainsi que d'un accord d'intéressement spécifique conclu pour une période triennale.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> SYSTEME DE GOUVERNANCE" -->

<!-- PageHeader="29 / 111" -->

L'intéressement et la participation versés en 2022 au titre de l'année 2021 ont représenté un avantage équivalent à 9,44 % de la masse salariale brute.

Cette rémunération variable collective est complétée d'un plan d'épargne entreprise et d'un plan d'épargne pour la retraite collective au bénéfice de l'ensemble des collaborateurs.


## Avantages sociaux ›

Les avantages sociaux constituent une part de rémunération supplémentaire qui s'ajoute au salaire de base. Ces avantages sociaux sont déconnectés de toute notion de performance individuelle et/ou collective. Ils sont attribués à tous les salariés bénéficiaires tels que prévus par la loi, la convention collective nationale ou les accords collectifs de MAIF VIE.

Sans être exhaustif, les avantages sociaux se retrouvent essentiellement dans les domaines suivants :

› santé, Prévoyance ;

› retraite ;

\----

transports ;

aides familiales ;

\> aides au logement ;

\> restauration d'entreprise ;

› activités sociales et culturelles du comité social et économique.

En 2022, le poids des avantages sociaux représentait 10,25 % de la masse salariale brute pour les classes 1 à 7.


### B.1.4.4 Rémunération des fonctions-clés

La rémunération des fonctions clés est déterminée dans la politique de rémunération.

Pour la fonction clé Audit interne, le principe et la pratique de rémunération relèvent de la catégorie des cadres hors classe. La formalisation des objectifs et le taux du variable applicable pour cette fonction ne sont pas de nature à conduire à une prise de risque excessive.

Pour les autres fonctions : Actuarielle, Gestion des Risques et Vérification de la Conformité, le principe et la pratique de rémunération relèvent de la catégorie des salariés de la classe 1 à 7 de la Convention Collective des Sociétés d'Assurance. Il n'existe donc pas de rémunération variable pour ces fonctions clés.


#### B.1.5 Transactions importantes conclues avec des personnes exerçant une influence notable sur l'entreprise ou des membres du conseil d'administration ou de la Direction générale

Les contrats souscrits auprès de MAIF VIE par ses administrateurs, dirigeants salariés, leurs conjoints, ascendants et descendants ne comportent aucune condition préférentielle par rapport aux conditions pratiquées par les autres sociétaires.

Il n'a pas été accordé d'avance et de crédit à l'ensemble des membres des organes d'administration et de direction ni conclu de transactions importantes avec des personnes exerçant une influence notable sur l'entreprise ou des membres du conseil d'administration ou de la Direction générale.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="30 / 111" -->


## B.2 Exigences de compétences et d'honorabilité

MAIF VIE accorde une attention particulière aux exigences de compétence et d'honorabilité.


### B.2.1 Personnes concernées

Les membres du conseil d'administration, les dirigeants effectifs, les membres du Comité de direction générale ainsi que les responsables des fonctions clés répondent aux conditions d'honorabilité, d'expérience et de compétences qui leur sont applicables.

En outre, conformément à la directive sur la distribution d'assurances, les personnels MAIF prenant directement part à la distribution des produits d'assurance MAIF VIE et leurs responsables répondent, sous la responsabilité de la direction des richesses humaines groupe, à un profil de compétence et d'honorabilité en adéquation avec leurs missions et obligations.

B.2.2 Exigences spécifiques de compétences appliquées aux membres du conseil d'administration, aux dirigeants effectifs et aux responsables des fonctions clés


#### B.2.2.1 Membres du conseil d'administration

Le conseil d'administration veille à ce que ses membres disposent collectivement des connaissances et de l'expérience appropriée dans les marchés de l'assurance et les marchés financiers, la stratégie de la société et son modèle économique, le système de gouvernance de la société, l'analyse financière et actuarielle et les exigences législatives et règlementaires applicables à la société.

La compétence collective du conseil d'administration de MAIF VIE est assurée par les mandats alloués en son sein aux membres du conseil d'administration de MAIF VIE. Ces derniers répondent aux exigences de compétences et d'honorabilité fixées dans le cadre de la politique de compétences et d'honorabilité du groupe MAIF.

Par ailleurs, chaque administrateur est tenu de respecter les exigences d'honorabilité fixées par le Code des assurances et s'engage formellement à défendre et respecter les principes et obligations à caractère déontologique des administrateurs des mutuelles de l'association des assureurs mutualistes (AAM).


#### B.2.2.2 Dirigeants effectifs

Afin de garantir une gestion saine, prudente et avisée de la Société, dans le respect des intérêts des assurés, les Dirigeants effectifs nommés par le Conseil d'administration répondent aux conditions d'honorabilité, de compétence et d'expérience nécessaires à leurs fonctions.

Le directeur général et le(s) directeur(s) général(aux) délégué(s) sont dirigeants effectifs de droit.


#### B.2.2.3 Responsables des fonctions clés

Le directeur général arrête le profil individuel de compétences des responsables des fonctions clés. Ce profil est réévalué afin de tenir compte des évolutions législatives et règlementaires propres à la fonction.

Les responsables de fonctions clés sont nommés par le directeur général.

S'agissant de la nomination du responsable de fonction clé de l'audit interne, un avis préalable du comité des risques, d'audit et des comptes est sollicité.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageNumber="31 / 111" -->

Une fiche de poste est rédigée pour chaque responsable de fonction clé correspondant aux exigences de compétences et missions fixées par la réglementation pour la fonction clé concernée.

Les responsables des fonctions clés à MAIF VIE (François Lair - fonction clé « Gestion des risques >>, Fabrice Couturier - fonction cle << Verification de la conformité >>, Christophe Lateulere - fonction cle << Actuarielle > et Vianney Dumont - fonction cle << Audit interne >> par convention conclue entre MAIF et MAIF VIE) possèdent l'honorabilité, les compétences (diplômes, qualifications professionnelles et connaissances) ainsi que l'expérience professionnelle nécessaires à leurs fonctions.

B.2.3 Processus d'appréciation des compétences et de l'honorabilité des membres du conseil d'administration, des dirigeants effectifs et des responsables des fonctions clés à MAIF VIE


##### B.2.3.1 Membres du conseil d'administration

L'assemblée générale des actionnaires élit les administrateurs en s'appuyant sur l'information délivrée préalablement par le conseil d'administration.

Le dossier de candidature des postulants à un mandat d'administrateur élu par l'assemblée générale comprend une copie d'une pièce d'identité, leur curriculum vitae afin d'évaluer leurs diplômes, qualifications professionnelles, connaissances et expériences pertinentes, ainsi qu'un extrait de casier judiciaire national datant de moins de trois mois (non conservé) et une attestation sur l'honneur manuscrite de non-incapacité a exercer un mandat d'administrateur conformément aux articles L. 322-2, R. 322-11-6 et R. 322-167 du Code des assurances et 258 et 273 du Règlement délégué du 10 octobre 2014.

Les membres du conseil d'administration, à titre personnel ou en qualité de représentants permanents des actionnaires personnes morales proposés par la MAIF, bénéficient du dispositif de formation prévu dans le cadre des politiques respectives de compétences et d'honorabilité de ces sociétés. Le conseil d'administration consacre une fois par an un point de son ordre du jour à la formation des administrateurs.


### B.2.3.2 Dirigeants effectifs

En tant que de besoin, les dirigeants effectifs bénéficient de formations spécifiques, appropriées à l'exercice de leurs responsabilités. Avant leur nomination, ils produisent un extrait de leur casier judiciaire datant de moins de trois mois (non conservé) ainsi qu'une attestation sur l'honneur de non-incapacité à exercer la fonction conforme à l'art. L. 322-2 du Code des assurances.


### B.2.3.3 Responsables des fonctions clés

Les responsables des fonctions clés bénéficient d'une formation ad hoc, en particulier sur les évolutions législatives et règlementaires impactant leur activité.

Avant leur nomination, ils produisent un extrait de leur casier judiciaire datant de moins de trois mois (non conservé) ainsi qu'une attestation sur l'honneur de non-incapacité à exercer la fonction conforme à l'art. L. 322-2 du Code des assurances

Le directeur général s'assure par tout moyen de l'adéquation des fonctions clés, de leur indépendance et de la qualité de leur fonctionnement.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageNumber="32 / 111" -->


### B.2.4 Évaluation sur une base continue

Le conseil d'administration procède régulièrement, par tout moyen à sa convenance, à sa propre évaluation. Souhaitant incarner la vision de l'entreprise « Engagés pour demain », il a mené en son sein des travaux visant à renforcer sa cohésion, sa diversité et son efficacité afin de favoriser le plein exercice des responsabilités d'administrateur, l'efficacité du processus de décision et la valorisation du collectif.

Le conseil d'administration s'assure en outre que les dirigeants effectifs continuent à satisfaire aux exigences mentionnées précédemment. Ces derniers sont également tenus de porter sans délai à la connaissance du conseil d'administration tout élément nouveau susceptible d'exercer une influence sur leur situation d'honorabilité.

Enfin, le directeur général s'assure en ce qui le concerne que les responsables des fonctions clés continuent à satisfaire auxdites exigences de compétences et d'honorabilité. Les responsables des fonctions clés sont tenus de porter sans délai à la connaissance du directeur général tout élément nouveau pouvant exercer une influence sur leur situation d'honorabilité professionnelle.


## B.3 Système de gestion des risques y compris l'évaluation interne des risques et de la solvabilité

La gestion des risques s'attache à identifier, évaluer et hiérarchiser les risques relatifs aux activités d'une organisation, quelles que soient la nature ou l'origine de ces risques, pour les traiter méthodiquement de manière coordonnée et économique, afin de diminuer et contrôler la probabilité des événements redoutés, et réduire l'impact éventuel de ces événements. Le risque est l'effet de l'incertitude sur les objectifs (normes ISO31000), il peut être vu comme une menace ou une opportunité.

À ce titre, il s'agit d'une composante de la stratégie d'entreprise qui vise à connaitre et réduire la probabilité d'échec ou d'incertitude de tous les facteurs pouvant affecter les ambitions de l'entreprise. Le système de gestion des risques repose sur une approche systémique contenant de nombreux livrables. Une démarche globale est engagée pour disposer d'une vision holistique des risques.

A la suite des évolutions réglementaires (directive Solvabilité 2 et loi Pacte), la Gestion des risques couvre également les risques environnementaux et les risques sociétaux intégrés dans la notion de "durabilité".


### B.3.1 Description du système de gestion des risques


#### B.3.1.1 Organisation et gouvernance du système de gestion des risques

Le système de gestion des risques repose sur le partage d'informations et la coordination de nombreux acteurs. Les premiers intervenants sont les propriétaires de risques (directions métiers) qui gèrent couramment leurs risques liés à leurs activités. La coordination d'ensemble, soit par famille de risque, soit dans une vision globale entreprise, est réalisée à travers des comités décisionnels de l'entreprise.

Cette gouvernance décisionnelle a pour objectif de répondre aux exigences de la réglementation prudentielle Solvabilité 2 en termes de gestion par familles de risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="33 / 111" -->

Les principaux comités sont présentés sur le schéma ci-dessous et les domaines couverts et missions des comités sont décrits tels qu'ils figurent dans les politiques associées en vigueur.

<figure>

![](figures/6)

<!-- FigureContent="Gouvernance décisionnelle Solvabilité 2 Groupe\\* MAIF Groupe et sociétés : \\* Toutes les fonctions clés solos et groupes, sauf pour la gestion des placements d'ALTIMA- ASSURANCES et SMACL Assurances SA qui ont leurs propres gouvernances, articulées avec les équipes de MAIF Instance avec les dirigeants effectifs et le directeur général adjoint - DPEF MAIF A NOTER : - l'absence d'un dirigeant effectif doit rester occasionnelle, a défaut de pouvoir participer, il doit communiquer par écrit son accord sur les décisions proposées ou prises, \\* . seuls les DE et FC Groupe sont requis pour les séances traitant des sujets spécifiques Groupe (notamment la validation des comptes consolidés), les DE des filiales sont invités mais peuvent se faire représenter, Fonction Gestion des Risques Conseil d'administration et/ou comité spécialisé :unselected: :selected: Fonction Actuarielle Fonction Vérification de la Conformité Fonction Audit Interne :selected: :unselected: Membre permanent Membre invité Souscription et produits Pilotage économique et prudentiel Gestion des placements Pilotage des risques \\* \\* Réunion Produits et Politique Souscription Réunion Pilotage Economique et Prudentiel \\* + Comité politiques d'investissements et gestion d'actifs Comité actif/passif Comité pilotage risques audit et conformité Sessions Souscription et produits Sessions Pilotage économique et prudentiel (intégrant les investissements stratégiques) . Sessions reassurance" -->

</figure>


Dans le cadre du système de gestion des risques, le conseil d'administration s'assure de la mise en place d'un système de gestion des risques efficace. Il approuve notamment la stratégie de gestion des risques, définit l'appétence et la préférence au risque.

L'appétence au risque est le niveau de risque agrégé maximum qu'une entité est prête à prendre en vue de la poursuite de son activité et afin d'atteindre ses objectifs stratégiques à l'horizon de son Plan stratégique. La préférence aux risques définit des lignes directrices qui fixent des acceptations ou des refus en termes de prise de risque pour l'entreprise.

Quant au Comité des risques d'audit et des comptes (comité spécialisé du conseil d'administration MAIF), sa mission dans le cadre du système de gestion des risques consiste plus particulièrement à examiner et suivre le dispositif global de maîtrise des risques et son application.


## › Domaine Souscription et produits

La réunion produits et politique souscription :

Cette réunion est une instance de validation par les deux dirigeants effectifs de livrables à soumettre au conseil d'administration, relevant notamment du domaine souscription et produits. À titre d'illustration : la politique globale de souscription et le suivi des business plans.

› Domaine pilotage économique et prudentiel

La réunion pilotage économique et prudentiel (RPEP) :

Cette réunion est une instance de validation par les deux dirigeants effectifs de livrables à soumettre au conseil d'administration relevant notamment du domaine pilotage économique et prudentiel comme les reporting annuels (Normes Françaises et Solvabilité 2) et les rapports Solvabilité 2 (SFCR, RSR).


### › Domaine gestion des placements

Le Comité actif/passif :

› est une instance decisionnaire sur les résultats de l'allocation stratégique a moyen-long terme et la politique de gestion actif/passif, à soumettre au conseil d'administration ou en cas d'alerte du responsable de la fonction clé gestion des risques ;

› est une instance de reporting pour les autres sujets : évolution et paramétrage des modèles.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" --> :selected: :selected: :selected: :unselected: :selected: :selected: :selected: :selected: :unselected: :unselected: :unselected: :unselected: :unselected:

> SYSTÈME DE GOUVERNANCE
===

34 / 111

Le Comité politiques d'investissements et gestion d'actifs :

› est une instance decisionnaire pour les points à présenter au conseil d'administration et les alertes du responsable de la fonction clé gestion des risques : politique de gestion des risques des investissements (y compris aménagements en cours d'exercice), politique gestion des risques de liquidité et concentration, politique d'allocation et de gestion tactique des placements ;

› est une instance de reporting pour les autres sujets : perspectives économiques à court et moyen terme, exécution de la politique de gestion des risques des investissements (respect, performance, etc), anticipations et perspectives en matière de gestion financière, contrôle interne et maîtrise des risques, expertises externes (en cas de besoin).

› Domaine pilotage des risques

Le Comité pilotage des risques, audit et conformité :

\-AAAA

\> évalue le profil de risque ;

\> propose le niveau d'appétence et une déclinaison en tolérances par type de risques ;

fournit un cadre de gestion des risques ;

définit les scénarii de stress contre lesquels la société entend se prémunir ;

supervise les travaux ORSA (régulier et ponctuel) et soumet au conseil d'administration des préconisations issues de l'analyse des résultats ;

› prépare notamment la mise à jour des politiques globales de gestion des risques, ORSA, de conformité, de contrôle interne, de sous-traitance, de continuité d'activité et d'audit interne ;

› étudie le tableau de bord des risques ;

› est informe de l'actualisation de la charte de gouvernance de la sécurité des SI

B.3.1.2 Missions de la fonction clé gestion des risques, principaux travaux, compétences

La fonction gestion des risques, au travers du système de gestion des risques qu'elle anime, vise à déceler, mesurer, contrôler, gérer et reporter en permanence les risques auxquels est soumis l'organisme conformément au cadre arrêté par le conseil d'administration MAIF et la Direction générale MAIF.

Les principales missions de la fonction clé gestion des risques groupe sont :

› animer et veiller au système de gestion des risques sur l'ensemble des risques, y-compris émergents, et le processus ORSA ;

› rendre compte des expositions aux risques de manière détaillée et conseiller le conseil d'administration et la Direction générale MAIF sur les questions de gestion des risques, y compris celles d'ordre stratégique ;

› assurer la relation avec les autorités de tutelle sur son domaine. Elle est l'interlocuteur privilégié des organisations professionnelles sur les thèmes relevant de sa mission.

Le titulaire de la fonction clé gestion des risques, actuaire membre certifié de l'Institut des Actuaires, s'appuie sur les dispositifs clés suivants :

› une politique globale de gestion des risques qui établit annuellement la stratégie en matière de gestion des risques qui est ensuite déclinée dans des politiques « métiers » ;

› une « cartographie tous risques >> intégrant une section spécifique pour les risques opérationnels y compris ceux de non-conformité ;

› un outil de modélisation prospective permettant d'évaluer des risques, notamment dans le cadre des travaux de l'ORSA ;

› un rapport ORSA et les reportings de suivi des risques (appétence, tolérances et indicateurs).

Il s'appuie pour l'exécution de ses missions :

› sur les équipes du Pôle Performance Economique Solvabilité et Gouvernance. Notamment

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTEME DE GOUVERNANCE
===

<!-- PageHeader="35 / 111" -->

l'Unité Actuariat et Pilotage des Risques de MAIF VIE, unité composée de 10 collaborateurs dont 5 actuaires membres associés, qualifiés ou certifiés de l'Institut des actuaires,

› sur les équipes du Pôle Conformité et Contrôles de MAIF VIE pour la gestion des risques opérationnels (9 personnes).

B.3.1.3 Identification, stratégies, prises de décisions et communication sur les risques


## \> Cartographie tous risques

La cartographie tous risques regroupe toute l'information sur les risques auxquels est exposée MAIF VIE. Elle porte aussi bien sur les risques supportés (acceptés, garantis, ... ) dans le cadre du métier d'assureur que sur ceux découlant de leur mise en œuvre opérationnelle.

Elle a plusieurs objectifs :

› décrire les risques et leur évaluation à un instant donné en vue d'alimenter l'ORSA ;

› hiérarchiser les risques à l'aide de différentes mesures et permettre de se concentrer sur certains risques ;

› comparer la perception des risques, à un instant donné, entre le SCR et ses composantes par risque à l'évaluation ORSA ;

› identifier et valider/faire évoluer la stratégie de gestion des risques et les politiques « métiers » ;

› définir les responsabilités et les rôles/contributions des propriétaires de risques et des contributeurs à la maîtrise des risques ;

› piloter et suivre le système de gestion des risques ;

› organiser un reporting interne aux instances décisionnelles ;

› revoir annuellement les risques identifies, compléter en fonction des évolutions internes ou externes.

L'approche retenue par MAIF VIE, en cohérence avec le groupe MAIF, propose de définir les natures des risques selon trois niveaux de risques :

› niveau 1 : les familles de risques (souscription, de marché, opérationnels, ... ) ;

› niveau 2 : les catégories de risques au sein d'une même famille (fraude interne par exemple) ;

› niveau 3 : les risques. Ce niveau correspond aux risques identifiés dans chacune des catégories de risques, pour lesquels une description détaillée et un scénario d'évaluation sont précisés. Il s'agit d'une déclinaison opérationnelle du risque (compréhension et maîtrise du risque à un niveau opérationnel).

Les familles et catégories de risques ont été construites en cohérence avec celles définies dans le cadre de la formule standard réglementaire utilisée pour calculer le Capital de solvabilité requis. Le schéma synoptique présentant la structure de la cartographie tous risques du groupe MAIF (grandes familles et catégories de risques) est présenté en partie C.


### Stratégie de gestion des risques ›

La stratégie en matière de gestion des risques est formalisée dans la politique globale de gestion des risques, et ensuite déclinée opérationnellement au niveau de chacune des politiques « métiers ».

Les principales orientations de la gestion des risques sont définies chaque fin année, elles s'inscrivent dans la stratégie de l'entreprise afin de conserver le profil de risque à un niveau compatible avec l'appétence au risque et sa préférence au risque.

Toutes les décisions prises dans la gestion quotidienne s'inscrivent dans cette stratégie de risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageHeader="36 / 111" -->

›


## Considération des risques dans la prise de décisions

La gouvernance décisionnelle mise en place permet de structurer un reporting adéquat à la fois sur l'activité, la performance ou la vision risque.

La gestion des risques relève de la responsabilité de tous à travers une coordination de toutes les actions quotidiennes. L'approche coordonnée et intégrée de la gestion des risques est animée par le responsable de la fonction. Le responsable de la fonction clé gestion des risques s'assure du partage de l'information entre les acteurs, à la fois sur les activités récurrentes et stratégiques de l'entreprise.

Le responsable de la fonction clé gestion des risques de l'entreprise est membre permanent ou invité de ces comités permettant ainsi la mise sous tension et l'efficience du système de gestion des risques. Ce fonctionnement permet également de s'assurer d'une bonne prise en compte des risques dans la chaîne décisionnelle et d'une bonne information des responsables des fonctions clés, essentielles pour la réalisation de leurs missions dans le système de gestion des risques.

Le responsable de la fonction vérification de la conformité vérifie la conformité des informations produites aux exigences réglementaires en s'appuyant sur le dispositif de contrôle interne selon le périmètre et les modalités définis dans les politiques de conformité et de contrôle interne.

La fonction actuarielle collabore notamment sur la qualité des données et les méthodes d'évaluation des risques (mesure quantitative et modèle prospectif).

L'audit interne réalise des missions d'audit du système de gestion des risques.

Des analyses de sensibilité sur le taux SCR sont réalisées dans certains dossiers pour aider à la construction des tests de résistance pour l'ORSA régulier ou ponctuel.


## › Communication sur les risques

Afin d'assurer en permanence l'efficacité du dispositif de maîtrise des risques et permettre, le cas échéant, l'adoption de mesures correctrices, la fonction clé gestion des risques rend compte au Comité pilotage des risques, audit et conformité, au Comité des risques d'audit et des comptes MAIF et au conseil d'administration du déroulement de ses missions.

Elle informe a minima annuellement ces instances sur le degré du risque et des mesures correctrices mises en œuvre selon les modalités prévues par ces instances.


### B.3.2 Évaluation interne des risques et de la Solvabilité (ORSA)

L'ORSA (Own Risk and Solvency Assessment) est l'un des outils du système de gestion des risques, qui formalise et accompagne la prise de décision.

Pour la réalisation de l'ORSA régulier, MAIF VIE a retenu une fréquence annuelle.

Certaines circonstances pourraient rendre l'ORSA nécessaire en dehors de l'échéance régulière annuelle. Cet ORSA dit « ponctuel >> sera réalisé chaque fois que le profil de risque de l'entreprise évolue ou serait amené à évoluer de manière significative, sur décision des dirigeants effectifs ou bien encore pour répondre à une demande émanant de l'autorité de contrôle (ACPR). Le processus de réflexion pour le lancement d'un ORSA ponctuel a été enrichi de critères quantitatifs au niveau du groupe MAIF.

L'évaluation du Besoin Global de Solvabilité (BGS) est réalisée :

› sur la base d'une évaluation à moyen/long terme des mesures de risques fondées sur les critères d'appétence ;

› en étudiant la résistance de l'entreprise, sur ses critères d'appétence, face à des situations dégradées ;

› sur l'analyse des risques matériels ou jugés les plus importants pour l'entreprise ou de la déviation aux hypothèses sous-jacentes du SCR ;

› et enfin, en décrivant les actions du management envisageables et les autres moyens mis

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

<!-- PageNumber="37 / 111" -->

<!-- PageHeader="\> SYSTÈME DE GOUVERNANCE" -->

en place pour surveiller et agir sur les risques.


#### B.3.2.1 Processus ORSA régulier

Le responsable de la fonction clé gestion des risques supervise la réalisation des évaluations internes des risques et de la solvabilité dites « ORSA >> (Own Risk and Solvency Assessment). Une politique ORSA a été arrêtée par le conseil d'administration. L'ORSA est réalisé via un travail collaboratif des salariés, des dirigeants effectifs et des administrateurs autour d'une vision intégrée des risques. Cette vision rendement/risque se déploie progressivement au sein de l'entreprise.

Les résultats du processus ORSA permettent au conseil d'administration de formaliser :

› le profil de risque de l'entreprise et les principaux facteurs de risque ;

› le respect de l'appétence au risque sur l'horizon temporel defini ainsi que la disponibilité et la suffisance des fonds propres ;

› les changements significatifs du profil de risque sur la période écoulée et les principales sensibilités sur les indicateurs financiers (sur le bilan en particulier) ;

› le périmètre des risques non couverts par le SCR (ainsi que la portée de ces risques) et ceux devant faire l'objet d'un contrôle renforce dit risque juge important1 qu'il soit quantifiable ou non ;

Les éléments ci-après décrivent les principales étapes clés du processus ORSA régulier telles que mises en place au sein de MAIF VIE, en lien fonctionnel avec le groupe MAIF, à savoir :

› la construction des tests de résistances ;

› les chiffrages Solvabilité 2 dans plusieurs dossiers décisionnels ;

› l'évaluation des tests de résistance ORSA ;

› la détermination d'éventuelles actions de management ;

› la rédaction du rapport ORSA ;

› et la validation du rapport.

La fonction cle gestion des risques coordonne l'exécution de l'ORSA. Ces travaux sont supervisés par le Comité pilotage des risques audit et conformité qui soumet le rapport et ses conclusions au conseil d'administration. Après avis du Comité des risques d'audit et des comptes de la MAIF, le conseil d'administration valide le rapport ORSA.

Le rapport ORSA est examiné au moins une fois par an par le conseil d'administration et à chaque fois que nécessaire en cohérence avec les éléments déclenchants énumérés dans la politique ORSA.


#### B.3.2.2 Lien entre la gestion du capital et le profil de risques

MAIF VIE a identifié ses principaux risques. Elle immobilise des capitaux pour couvrir les pertes potentielles qui pourraient subvenir. À ce jour, le profil de risque de MAIF VIE est en adéquation avec les hypothèses sous-jacentes au calcul du Capital de solvabilité requis (SCR) selon la formule standard prévue par la réglementation. Par conséquent, MAIF VIE évalue ses risques « quantifiables >> par la formule standard. Ces mêmes risques peuvent faire l'objet de scénarios de stress permettant d'évaluer l'évolution des taux de solvabilité en cas de réalisation de tels événements. Des risques « non quantifiables >> sont évalués à dire d'expert de manière qualitative.

<!-- Footnote="1 Dans Solvabilité 2, un élément « important » est, par définition, susceptible d'influencer le jugement et la décision. C'est ainsi qu'il faut comprendre la notion de risque important (source ACPR). Pour MAIF VIE, les risques jugés importants peuvent être définis comme des risques pouvant nuire de manière brutale et significative à la société. MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="38 / 111" -->


## B.4 Système de contrôle interne


### B.4.1 Description du système de contrôle interne

L'organisation du contrôle interne est structurée selon trois lignes de maîtrise et repose sur deux notions : le contrôle permanent regroupant les contrôles de premier et deuxième niveaux (les deux premières lignes de maîtrise) et le contrôle périodique avec le contrôle de troisième niveau (la troisième et dernière ligne de maîtrise).

Le contrôle permanent de premier niveau encadre la première prise de risque qui se situe au niveau des opérationnels et de la hiérarchie sur le terrain.

Le contrôle permanent de deuxième niveau anime et coordonne le système de contrôle interne.

Le contrôle périodique est indépendant des deux premiers niveaux et a pour vocation d'aider l'entreprise à atteindre ses objectifs en évaluant ses processus de gestion des risques, de contrôle, et plus globalement de gouvernance d'entreprise. Il est assuré par l'audit interne (voir § B.5).

Le pôle conformité et contrôles (PCC) pilote par subdélégation de la directrice générale la mise en œuvre du système de contrôle interne et rend compte de ses travaux au Comité de pilotage des risques, d'audit et de conformité, instance regroupant les dirigeants effectifs de MAIF, de ses principales filiales et les responsables des fonctions clés. Il informe le Comité des risques d'audit et des comptes et le conseil d'administration. Il rend compte de la mise en œuvre des pratiques définies au sein du groupe MAIF et pour ce faire, participe aux liens fonctionnels réalisés par la Direction Maîtrise des Risques et Contrôle Interne de la MAIF.

Le pôle conformité et contrôles pilote la mise en œuvre de la gestion des risques opérationnels et anime le système de contrôle interne avec le concours d'un réseau de correspondants métiers. Les procédures clés du système de contrôle interne reposent sur :

› la cartographie des risques opérationnels ;

› un dispositif de contrôle permanent de 2ème niveau organisé autour d'autoévaluations de vérifications de déclaration, de vérifications de procédure et de vérification d'efficacité. Il est traduit dans un plan annuel de vérifications validé par la directrice générale en Comité conformité, contrôle interne et continuité d'activité MAIF VIE ;

› l'animation et le support au contrôle permanent de 1er niveau ;

› un dispositif de suivi des incidents opérationnels ;

› un suivi des indicateurs des risques opérationnels ;

› un reporting du dispositif.

Le pôle conformité et contrôles assure également la coordination du plan de continuité des activités.


#### B.4.2 Procédures clés du système de contrôle interne


## › Lien entre le système de gestion des risques et le système de contrôle interne

Le système de gestion des risques et le système de contrôle interne interagissent de manière complémentaire.

Le système de gestion des risques pilote tous les risques en fixant des limites et un cadre de gestion des risques. Le système de contrôle interne veille à son application et au respect des normes de la société. Le système de contrôle interne s'appuie donc sur le système de gestion des risques pour positionner les contrôles à effectuer. Ainsi, les travaux des deux systèmes interagissent et assurent une séparation des rôles et responsabilités.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTEME DE GOUVERNANCE
===

<!-- PageHeader="39 / 111" -->

À compter de 2022, des échanges réguliers ont été mis en place entre le contrôle interne et la fonction clé Gestion des risques afin de donner à cette dernière plus de visibilité sur la cartographie des risques opérationnels.


## B.4.3 Fonction de vérification de la conformité


### B.4.3.1 Missions

Le pôle conformité et contrôles s'assure, sur son périmètre d'activité, de la sécurisation des opérations conformément aux dispositions legales, règlementaires et aux normes professionnelles ou déontologiques. Il contribue à limiter les risques de sanctions, de pertes financières et/ou d'image qui seraient liés à une non-conformité.

Son périmètre d'activité concerne principalement :

› la protection de la clientèle, y compris dans la gestion des capitaux non reclames ;

› la lutte contre le blanchiment d'argent et le financement du terrorisme (LCB-FT) ;

› la conformité des nouveaux produits et services ainsi que la conformité des modifications substantielles de produits et services existants ;

› la protection des données personnelles ;

› la déontologie ;

› la sécurisation des sous-traitances.


## B.4.3.2 Organisation de la fonction de vérification de la conformité et principaux travaux

La fonction de vérification de la conformité reste indépendante dans l'exercice de sa mission et n'effectue pas d'acte de gestion opérationnelle.

Le pôle conformité et contrôles réalise également les contrôles de vérification de la conformité à partir du Plan de contrôle conformité.


## B.5 Fonction d'audit interne


### B.5.1 Missions

La finalité première de l'audit interne est d'évaluer l'adéquation et l'efficacité du système de gouvernance de l'entreprise.

Au travers de cette évaluation, l'audit interne est la dernière « ligne de maîtrise >>, qui donne à l'entreprise une assurance sur le degré de maîtrise de ses opérations.

Au-delà de l'assurance donnée, l'audit interne apporte aussi, par ses recommandations, des conseils d'amélioration.

Une valeur ajoutée complémentaire de l'audit interne est de réaliser, grâce à son indépendance et sa qualification, des investigations ou états des lieux sur des sujets pouvant être sensibles, pour le compte de la Direction générale.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="40 / 111" -->


## B.5.2 Organisation de la fonction audit interne et principaux travaux

La Direction de l'Audit interne de MAIF assure la fonction clé d'audit interne de la société MAIF VIE solo. Le responsable de la fonction clé audit interne MAIF VIE est le directeur de l'Audit interne de MAIF.

La Direction de l'Audit interne de MAIF est rattachée hiérarchiquement au directeur général de MAIF, qui supervise le management de l'entité.

En outre, une fois par an, le directeur de l'Audit interne rencontre le président de MAIF.

Pour ce qui relève de sa responsabilité de fonction clé d'audit interne MAIF VIE solo, il rend compte fonctionnellement :

› aux dirigeants effectifs de MAIF VIE, sur les domaines de co-decision des dirigeants effectifs (politique d'audit interne à proposer au CA, plan d'audit) et leur exécution,

› au Comité des risques, d'audit et des comptes de MAIF (dont le champ de compétences est étendu à MAIF VIE), instance dont la mission est d'éclairer et d'alerter le conseil d'administration en matière d'examen et d'appréciation des risques encourus par l'entreprise.

Au 01/01/2022, le groupe ayant designe le directeur de l'Audit interne MAIF comme responsable de la fonction clé audit interne groupe, celui-ci assure aussi la coordination des activités d'audit interne MAIF VIE avec celles du groupe et leur conformité par rapport aux exigences d'audit interne au niveau du groupe.

Ce rattachement et ce reporting au plus haut niveau, contribuent à l'indépendance de l'audit interne.

La Direction Audit interne de MAIF est composée à fin décembre 2022 d'une équipe pluridisciplinaire de 17 postes, dimensionnée pour mener à bien ses missions. Les acteurs sont formés à la méthodologie de l'audit interne directement issue de celle enseignée par l'IFACI. En fonction du thème des missions confiées, des formations spécialisées complémentaires peuvent être assurées.

La Direction de l'Audit interne de MAIF conduit ses activités en conformité aux normes internationales de la pratique professionnelle de l'audit interne émises par l'IIA (Institute of Internal Auditors). Elle a obtenu le 4 mai 2022 le renouvellement de son label de certification 'The IIA / IFACI', label qu'elle détient depuis le 14 février 2018.

L'audit interne réalise principalement des missions d'assurance portant sur un processus, un thème ou une entité. En complément, il mène aussi ponctuellement des missions de conseil. Les conclusions d'audit font l'objet d'un rapport final remis notamment aux dirigeants effectifs de MAIF VIE, aux membres du Comité des risques d'audit et des comptes, aux responsables des fonctions clés MAIF VIE et GROUPE. Il contient les constats, les risques identifiés, les recommandations et le plan d'action des structures auditées (le cas échéant, celui-ci a pu faire l'objet au préalable d'un arbitrage par la Direction Générale).

Annuellement, l'audit interne communique à la Direction générale, au conseil d'administration et au Comité des risques d'audit et des comptes un rapport comprenant les résultats des activités d'audit interne de l'année et le suivi de mise en œuvre des principaux plans d'actions des missions antérieures.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

41 / 111

B.6 Fonction actuarielle
===


## B.6.1 Missions

Les missions de la fonction actuarielle peuvent être regroupées selon quatre thématiques :

› les provisions techniques :

o coordonner le calcul des provisions techniques voire le superviser pour certains cas prévus par la directive Solvabilité 2 ;

o garantir le caractère approprié des méthodologies, des modèles sous-jacents et des hypothèses utilisés pour le calcul des provisions techniques ;

o apprécier la suffisance et la qualité des données utilisées dans le calcul des provisions techniques ;

o comparer les meilleures estimations aux observations empiriques ;

o informer le conseil d'administration de la fiabilité et du caractère adéquat du calcul des provisions techniques.

› la souscription : émettre un avis sur la politique globale de souscription ;

› la reassurance : emettre un avis sur l'adéquation des dispositions prises en matière de réassurance ;

› le système de gestion des risques : contribuer à la mise en œuvre effective du système de gestion des risques, en particulier pour ce qui concerne la modélisation des risques sous- tendant le calcul du Capital de solvabilité requis (SCR) et du Minimum de capital requis (MCR) et pour ce qui concerne l'évaluation interne des risques et de la solvabilité (ORSA).


## B.6.2 Organisation de la fonction actuarielle et principaux travaux

Le système de gouvernance de MAIF VIE comporte quatre fonctions clés de contrôle dont la fonction actuarielle.

La fonction actuarielle est exercée par un actuaire certifié par l'institut des actuaires, ayant une connaissance des mathématiques actuarielles et financières à la mesure de la nature, de l'ampleur et de la complexité des risques inhérents à l'activité de l'entreprise d'assurance ou de réassurance et pouvant démontrer une expérience pertinente à la lumière des normes professionnelles et autres normes applicables.

La production des calculs S2 est confiée à l'Unité Actuariat et Pilotage des Risques (UAPR).

La fonction actuarielle peut s'appuyer sur l'UAPR et d'autres services de MAIF VIE et de la MAIF (notamment la GAPEF) pour réaliser des sensibilités ou des calculs complémentaires.

La fonction actuarielle rédige notamment un rapport annuel permettant de motiver les différentes opinions exprimées par le responsable de la fonction actuarielle sur les provisions techniques, la politique de souscription et la politique de réassurance.

Dans ce rapport, la fonction actuarielle documente les travaux réalisés et les résultats, identifie les éventuelles lacunes et émet des recommandations.

Ce rapport, ainsi que les opinions, sont notamment partagés en amont de la présentation aux instances au sein du Comité groupe de la fonction actuarielle qui associe les différentes directions impliquées dans les travaux.

Cependant, si l'opinion de la fonction actuarielle faisait émerger une problématique significative sur un des sujets évoqués précédemment, l'opinion serait alors transmise au conseil d'administration au moment de sa validation.

Les productions de la fonction actuarielle, qui a en charge également le pilotage et la réalisation de l'évaluation quantitative des risques dont l'ORSA, sont notamment présentées au responsable de la fonction gestion des risques et aux différentes instances.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022 :unselected: :unselected: :unselected: :unselected: :unselected:

<!-- PageNumber="42 / 111" -->

<!-- PageHeader="\> SYSTÈME DE GOUVERNANCE" -->


## B.7 Sous-traitance

MAIF VIE a conclu des accords de sous-traitance en vue d'externaliser diverses activités. Ce recours à la sous-traitance est justifié par la nature des activités externalisées, les décisions stratégiques de l'entreprise et la volonté de recourir à des niveaux d'expertise, de savoir-faire ou de connaissances adaptés.

La Directive Européenne 2009/138/CE du 25 novembre 2009 sur l'accès aux activités de l'assurance et de la réassurance et leur exercice dite « Directive Solvabilité II >> impose aux entreprises d'assurance la mise en place d'un dispositif de maîtrise des activités externalisées et plus particulièrement de ses sous-traitants dits << Importants ou Critiques >> (STIC) selon des critères définis par les articles 49 et 50 de la Directive.

MAIF VIE restant responsable des activités sous traitées, son dispositif de maîtrise de la sous- traitance a pour objectif principal la protection des adhérents et la maîtrise des risques inhérents à l'intervention d'un acteur tiers (risques opérationnels avec impacts financier et/ou de réputation dans une hypothèse de défaillance desdits prestataires de services).

Dans le cadre de la définition et de la mise en œuvre de son dispositif STIC, MAIF VIE a rédigé sa Politique de sous-traitance, laquelle est validée par le conseil d'administration de l'entreprise et révisée annuellement. Le 22/06/2022 le CA de MAIF VIE a ainsi validé la mise à jour 2022 de la Politique de sous-traitance de MAIF VIE. Cette politique précise le cadre et les principales règles en matière de sous-traitance à MAIF VIE, dont les critères de qualification d'une sous-traitance importante ou critique, le dispositif de suivi des activités externalisées, ainsi que les responsabilités des différents acteurs. Elle vient également en déclinaison de la Politique de Sous-traitance Groupe MAIF.

En 2022, en concertation avec le groupe MAIF, MAIF VIE a poursuivi les travaux d'animation et de renforcement de son dispositif de supervision des sous-traitances importantes ou critiques.

Le Comité STIC de MAIF VIE a pour mission de suivre les risques liés aux STIC avec l'ensemble des acteurs concernés, et notamment de valider la mise à jour de la cartographie STIC, l'entrée et la poursuite de la relation d'affaires avec les partenaires identifiés comme étant critiques ou importants. Il a vocation à se réunir, à minima, une fois par an.


## La sécurisation de la sous-traitance de MAIF VIE ›

En 2022, MAIF VIE s'est attaché à optimiser la sécurisation de ces relations partenariales au travers de deux processus :

› le processus d'identification, de sélection et de sécurisation du partenariat avec le sous- traitant important ou critique ;

› le processus de suivi du sous-traitant important ou critique.

› La selection du sous-traitant important ou critique

MAIF VIE détecte les situations de sous-traitance et qualifie ses sous-traitances d'importantes ou critiques (STIC) en s'appuyant sur un certain nombre de définitions et de critères combinés dans un arbre de décision et qui permettent de définir la criticité de la sous-traitance.

Ces critères sont :

› la qualification de l'activité :

o une activité standard au sens de l'article R354-7 Il a du Code des assurances,

o une activité relevant de la sous-traitance et/ou de l'externalisation au sens de l'article L310-3 13 du Code des assurances,

o une fonction clé au sens de Solvabilité 2,

o une activité cœur de métier,

o une fonction/activité support sensible,

o une sous-traitance d'un risque opérationnel dont l'arrêt brutal engendrerait une MAIF VIE - Rapport sur la solvabilité et la situation financière 2022 :unselected: :unselected: :unselected: :unselected: :unselected: :unselected:

> SYSTEME DE GOUVERNANCE
===

<!-- PageHeader="43 / 111" -->

perte significative ou le non-respect des engagements envers ses sociétaires,

› des seuils économiques définis en interne,

› le caractère substituable ou non de l'activité dans un délai limité,

› le caractère durable ou non de l'activité.

A cela s'ajoute une étude du sous-traitant sur un ensemble d'éléments qui peuvent s'ajuster selon le type d'activité sous-traitée :

› leur dispositif de continuité d'activité,

› leur sécurité du système d'information,

› leur dispositif de contrôle interne,

› leur viabilité financière,

› leurs éléments contractuels : attestation d'assurance, attestation RC ...

Enfin, la sécurisation de la relation s'effectue également au travers d'une convention écrite signée entre le représentant de l'entité délégante et le représentant du sous-traitant qui intègre des clauses spécifiques telles que la continuité des activités, la protection et la qualité des données utilisées pour l'activité ou la fonction confiée par le délégant, le respect de la réglementation, l'honorabilité du prestataire, ....

› Le suivi regulier du sous-traitant important ou critique

Pour maîtriser les risques liés aux sous-traitants importants ou critiques, MAIF VIE a mis en place un certain nombre de contrôles :

› Des verifications annuelles : questionnaire annuel de suivi a destination des sous-traitants importants ou critiques, renseigné sur une base déclarative ((suivi du Plan de continuité et de reprise d'activité, suivi du dispositif de contrôle interne, évaluation du système d'information ... )

Un suivi régulier dans le cadre des comités de suivi du partenariat. ›

Parallèlement, le plan de contrôle interne de MAIF VIE intègre la vérification de l'effectivité, de la qualité et de l'exhaustivité des vérifications annuelles réalisées dans le cadre du dispositif STIC.


## B.8 Autres informations

MAIF VIE n'a pas identifié d'autre information significative à mentionner dans ce paragraphe.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->
<figure>

![](figures/7)

<!-- FigureContent="C" -->

</figure>


PROFIL DE RISQUE

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

C Profil de Risque
===

Le profil de risque regroupe l'ensemble des risques auxquels la société d'assurance est exposée, la quantification de ces expositions et l'ensemble des mesures de protection face à ces risques.

Ces éléments sont présentés de manière détaillée dans les sections ci-après.

MAIF VIE est une société d'assurance dont l'activité principale est de couvrir les besoins des adhérents en matière d'assurance vie : protéger sa famille, assurer l'avenir de ses enfants, se constituer par l'épargne un complément de retraite, favoriser la transmission d'un capital ... A ce titre, elle n'est pas exposée à des risques spécifiques par rapport au marché et ne couvre pas de risques de nature exceptionnelle.

Les principaux risques auxquels MAIF VIE est exposée résultent de son activité d'assureur vie, de son organisation et de son environnement, notamment économique. Le profil de risque reflète les conclusions de la cartographie tous risques, outil essentiel du système de gestion des risques dont la structure est présentée en § B.3 de ce rapport.

Le schéma synoptique suivant présente la cartographie tous risques qui s'applique aux principales entités du groupe MAIF :

C.1. Risque de souscription

C.2. Risque de marché

C.3. Risque de crédit

C.5. Risque opérationnel C.6. Autres risques importants

<figure>

![](figures/8)

<!-- FigureContent="Risque de Risque de Risque de Risque opérationnel souscription vie souscription non marché Risque de défaut y compris de non-conformité Risque vie et santé d'entreprise Risque émergent Mortalité Prime et Frais Action Créance du passif Stratégique Economique Longévité Catastrophe par scénario Immobilier Créance de l'actif Business Environnemental Invalidité / Catastrophe Morbidité Taux engendrée par Image et Sociétal l'homme réputation et Change engagement Catastrophe Réserve sociétal Technologique Cessation Spread Résiliation Politique Concentration Dépense Règlementaire Liquidité Révision C.4. Risque de liquidité Tarification" -->

</figure>


Risque quantifié dans le SCR

Risque non quantifié dans le SCR

Le profil de risque est notamment évalué de manière quantitative selon le calcul du SCR déterminé par la formule standard, pour la plupart des risques quantifiables (SCR pour Solvency Capital Requirement, représentant le capital cible requis dans le cadre de la réglementation européenne Solvabilité 2). Le SCR correspond au montant de fonds propres estimé comme nécessaire pour absorber des chocs de forte intensité susceptibles de se produire environ une fois tous les 200 ans. Les résultats de ce calcul sont présentés dans la suite de ce rapport au § E.2.

Conformément à la révision de la directive Solvabilité 2, les fonctions clés étendent progressivement leurs analyses à la durabilité ou soutenabilité (enjeux sociaux et environnementaux).

Ainsi, les risques les plus importants sont compris dans le calcul du SCR. Pour ces risques, la formule standard définie dans la directive Solvabilité 2 prévoit une méthode d'évaluation normalisée qui facilite la comparaison des sociétés d'assurance sur leur exposition aux risques de natures différentes. Le calibrage de la formule standard est adapté pour la mesure du profil de risque MAIF VIE. Ces risques sont couverts par la mobilisation de fonds propres en quantité MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

et en qualité suffisante (présentés dans la partie E) et par divers dispositifs de gestion des risques.

Dans le calcul du SCR, une réduction du besoin en capital est prévue pour tenir compte des effets de diversification entre les risques. Les principales catégories qui contribuent au SCR et génèrent de la diversification entre elles sont le risque de souscription vie et le risque de marché. Les autres catégories de risques, ont des expositions unitaires plus faibles.

D'autres risques significatifs pour MAIF VIE - risques d'entreprise, émergents (décrits au § C.6.6), ou de liquidité (§ C.4) - sont également détaillés. Ces derniers ne sont pas intégrés dans le calcul du SCR selon la formule standard, et sont couverts par des dispositifs de gestion des risques spécifiques.

Les traits saillants de l'exposition MAIF VIE à chacun de ces principaux risques sont présentés ci-dessous, après un rappel de la nature du risque.

Les principales techniques de gestion ou d'atténuation de ces risques mises en œuvre par MAIF VIE sont également décrites ci-après. Ces techniques propres à chaque risque s'inscrivent dans le dispositif général de gestion des risques détaillé auparavant au § B.3.

Afin de faciliter la lecture, les principaux risques auxquels MAIF VIE est exposée sont regroupés au sein des modules suivants :

› Risque de souscription,

› Risque de marché,

› Risque de crédit,

› Risque de liquidité,

› Risque opérationnel,

› Autres risques importants, incluant les risques d'entreprise et les risques émergents.

Ces modules rappellent la structure de la cartographie tous risques. Les risques qui n'entrent pas dans le champ du SCR sont distingués, permettant ainsi de faire le lien avec la structure de la cartographie tous risques de MAIF VIE.

Le profil de risque MAIF VIE a évolué en lien avec les conséquences économiques des phénomènes d'ampleur vécus sur 2022 : guerre entre la Russie et l'Ukraine, crise énergétique, remontée des taux et résurgence de phénomènes inflationnistes notamment. Les conséquences de cette remontée brutale des taux sont pour MAIF VIE une exposition plus importante au risque d'un rachat de masse de ses encours du fait des moins-values latentes de son portefeuille. MAIF VIE n'a pas observé en 2022 d'augmentation notable de ses rachats, en ayant même une collecte nette positive sur le fonds en euro. La diversification des risques, le niveau et la qualité des fonds propres, la stratégie d'investissement sont autant d'éléments qui assurent à MAIF VIE, de fortes capacités de résistance dans des scénarios adverses.


## C.1 Risque de souscription

Les risques auxquels est exposée MAIF VIE diffèrent selon la nature et l'objet du produit :

› les produits d'épargne, pour leur composante libellée en euros, induisent principalement des risques de marché, des risques de rachats et des risques de dérive des frais (dépenses). La composante en unités de compte des produits d'épargne, sur laquelle les risques de marché sont supportés par l'assure, est exposée à une diminution de l'encours (décès, rachats, arbitrages avec le compartiment libellé en euros ... ) ayant pour incidence une dégradation du résultat sur ce type de produit ;

› les produits d'épargne retraite (non assimilés à des produits d'épargne) et les rentes viagères (issues des options de règlements d'autres produits d'épargne ou prévoyance de MAIF VIE) engendrent essentiellement des risques financiers, mais aussi des risques techniques de souscription (longévité, ... ) ;

› les produits de prévoyance génèrent des risques techniques de souscription (décès, invalidité, anti-sélection, ... ) pouvant entraîner, en cas de dérive des indicateurs techniques,

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022 :selected:

une dégradation du résultat sur ces produits.

Au regard de son profil d'activité, majoritairement concentrée sur les produits d'épargne, MAIF VIE est intrinsèquement exposée à l'environnement des aléas assurantiels, engendrant des risques de souscription, et plus particulièrement :

› risque de distribution et de tarification ;

› risque d'une forte hausse des rachats en épargne.

Au regard de la nature des produits, MAIF VIE n'est pas exposée aux risques de révision.

Le SCR souscription vie de MAIF VIE se décompose en risque de mortalité, de longévité, d'invalidité, de cessation, de dépenses, de révision et catastrophe.


### C.1.1 Risque de distribution et de tarification

En tant que filiale assurance vie de la MAIF, MAIF VIE s'attache à fournir une réponse adaptée aux besoins des sociétaires MAIF en matière d'assurance vie et prévoyance ; ainsi, MAIF VIE jouit du réseau de distribution MAIF pour diffuser son offre produit, mais également de l'expertise des conseillers-vendeurs de MAIF Solutions Financières (filiale détenue à 100 % par MAIF) pour la souscription de contrats d'assurance vie avec des versements élevés.

La non-atteinte des objectifs de distribution sur un ou plusieurs produits peut impliquer une dérive de la performance économique attendue sur ce ou ces produits. L'élaboration et le suivi d'un plan de développement annuel (définition des objectifs de développement) permet de piloter la stratégie commerciale. Le CPAD suit mensuellement l'atteinte des objectifs de distribution.

Plusieurs travaux permettent de vérifier l'exposition aux risques de distribution et de tarification :

› l'analyse annuelle du positionnement de l'offre permet de comparer par produit l'offre MAIF par rapport au marché ;

› l'adéquation des tarifs, et notamment le suivi annuel de la mortalité, le suivi annuel de la tarification des contrats, l'analyse empirique de la sinistralité et des rachats, ...

Par ailleurs, l'existence d'une politique Gouvernance et Surveillance des Produits (GSP) et d'un Comité souscription et produits renforce le pilotage de la stratégie produits de MAIF VIE.

Le risque potentiel sur de nouveaux produits est encadré par un processus spécifique dédié à la conception ou la modification d'un produit.


### C.1.2 Risque de cessation

Au regard de son profil d'activité, majoritairement concentrée sur les produits d'épargne, MAIF VIE est exposée à un risque de forte hausse des rachats en épargne.

Pour la plupart, des contrats d'assurance vie d'épargne peuvent être rachetés à tout moment par les assurés. Les rachats peuvent être motivés par des besoins de liquidité, des facteurs liés à la fiscalité avantageuse des produits d'assurance, des repositionnements vers des concurrents offrant des produits plus remunerateurs/performants ...

Notamment, dans certaines situations financières :

› des rachats importants de contrats d'épargne, pour leur composante libellée en euros, pourraient avoir des impacts significatifs sur la situation financière et la solvabilité de l'entreprise ; ce serait notamment le cas dans une situation de forte et rapide hausse des taux d'intérêt, occasionnant la vente de titres en moins-values latentes et affectant directement le résultat financier et donc les fonds propres de l'entreprise (voir section suivante C.2 sur l'exposition au risque de taux d'intérêt). MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

› dans une situation persistante de taux d'intérêt long terme à des niveaux extrêmement bas, des rachats importants pourraient améliorer ponctuellement la solvabilité de l'entreprise mais nuiraient à son équilibre économique et à sa pérennité.

Les études actif/passif, le ratio de PPB/PM (provision pour participation aux bénéfices sur provision mathématique) et la situation de richesse latente sont des dispositifs existants de maîtrise du risque de rachat. Voir également les « facultés contractuelles et règlementaires face au risque de liquidité >> (ci-après, partie C.4) qui permettent également de faire face au risque de cessation.

Au 31/12/2022, la situation du portefeuille d'actif de MAIF VIE affiche des moins-values latentes faisant augmenter le risque associé à une hausse des rachats. Toutefois cette situation de rachats plus importants observée par le marché en 2022 ne s'est pas matérialisée pour MAIF VIE qui a observé une collecte nette positive sur l'année 2022.


### C.1.3 Risque de dépenses

Le risque de dépense peut être d'origine endogène, comme une mauvaise maîtrise des charges internes (masse salariale, dérive des coûts des projets, ... ) ou d'origine exogène comme l'inflation, des cotisations sociales supplémentaires, des nouvelles règlementations engendrant des coûts projets complémentaires ...

Le système de commissionnement mis en place auprès des réseaux de distribution a été conçu de manière à ce que les ressources de gestion prélevées sur les contrats d'épargne soient toujours supérieures aux commissions versées.

Les frais et leur dérive potentielle sont par ailleurs régulièrement mesurés de manière prospective permettant ainsi d'anticiper et de piloter les coûts en adéquation avec les ressources de gestion.

.


### C.1.4 Risque de mortalité, de longévité, d'invalidité et catastrophe

Les risques de mortalité, de longévité et d'invalidité correspondent respectivement à une dérive de la mortalité, de la longévité ou de l'invalidité par rapport aux lois comportementales construites et utilisées par MAIF VIE. Le risque de catastrophe résulte d'évènements extrêmes ou irréguliers ayant comme conséquence un effet de surmortalité ponctuelle et immédiate. A ce titre, l'éventualité de la survenance d'une épidémie ou d'un phénomène climatique (canicule ... ) est un risque de sursinistralité (décès / incapacité / invalidité) impactant l'équilibre économique de nos produits. La structure du portefeuille conjuguée à la couverture de reassurance actuelle permet une bonne maitrise de ce risque.

Des méthodes statistiques et actuarielles sont utilisées pour concevoir les produits et suivre leur adéquation et équilibre technique. La mesure des engagements et des flux futurs des produits d'assurance vie et de prévoyance (et des risques de souscription sous-jacents) s'appuie sur des hypothèses d'évolution prospective de facteurs d'origine économique, démographique, ou liés à la mortalité, l'invalidité ou le comportement des assurés. MAIF VIE utilise son expérience, ses données et celles de la profession afin d'élaborer des estimations au plus juste.

Des risques émergents, tels qu'un nouveau cadre fiscal, pourraient entraîner des pertes incohérentes par rapport aux hypothèses retenues ; avec un éventuel impact significatif de la situation financière.

Les lois comportementales relatives aux risques de mortalité, de longévité, d'invalidité/morbidité et de rachat sont mises à jour annuellement et permettent d'appréhender d'éventuelles modifications de tendances, qui pourraient entraîner une modification de la structure et de la cession des risques. Il s'agit également de s'assurer qu'aucune déformation du portefeuille dans ses caractéristiques ne nécessite une modification de la tarification. Par ailleurs, le système de suivi interne construit autour de comparaisons

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

mensuelles et annuelles permet d'alerter en cas de modification importante et durable de la sinistralité.

À noter que MAIF VIE utilise une table de mortalité d'expérience, certifiée par un actuaire agree à cet effet, conformément à l'article A132-18 du Code des assurances, pour la tarification de garanties en cas de décès des contrats de prévoyance individuelle temporaire et de prêts qu'elle assure.

De plus, les échanges entretenus au cours de l'année avec nos réassureurs nous permettent de comparer le coût de nos traités à ceux du marché. Les propositions tarifaires pour l'année suivante sont également évaluées en regard de l'évolution des portefeuilles.

Programme de reassurance pour limiter les risques techniques des produits de prévoyance

Les produits de prévoyance génèrent des risques techniques de souscription (décès, invalidité, antisélection, ... ).

La politique de réassurance de MAIF VIE s'inscrit en cohérence avec celle des autres entités du groupe.


### C.2 Risque de marché

Le risque de marché est le risque de perte qui peut résulter des fluctuations des prix des actifs financiers qui composent un portefeuille d'actifs.

Le risque de marché de MAIF VIE se décompose en risques actions, immobilier, de taux d'intérêt, de spread, de change et de concentration.

Le risque de spread de crédit est décrit au § C.3 consacré au risque de crédit.

MAIF VIE a délégué à MAIF la mise en œuvre de sa gestion financière et de sa gestion actif/passif ainsi que la comptabilité de ses placements

MAIF VIE applique l'approche par transparence, qui consiste à calculer le capital de solvabilité requis sur la base de chaque actif sous-jacent des organismes de placement collectif et autres investissements sous forme de fonds.

Les principaux risques identifiés sont les risques financiers (risques de marché, risque de concentration, risque de liquidité et risque de défaut) et les risques de durabilité (mauvaises pratiques ESG des entités financées et risques de non-adaptation à la transition énergétique et écologique).

À noter que pour le risque de marché (§ C.2), le risque crédit (§ C.3), le risque de liquidité (§ C.4) et les risques de durabilité (§ C.6.4), le principe de la personne prudente s'applique.

Les investissements dans des instruments financiers sont effectués après une analyse pour identifier, mesurer, suivre, gérer, contrôler et déclarer les risques de manière adéquate et selon le « principe de la personne prudente ».

MAIF VIE détermine, en collaboration avec les équipes Investissements et placements et gestion actif-passif de MAIF, chaque année, son allocation d'actifs et ses règles de gestion présentées dans la Politique de Gestion Tactique des Placements.

La Politique de gestion des risques des investissements, la Politique de gestion tactique des placements et la Politique de gestion des risques de liquidité et concentration de MAIF VIE sont définies de façon à garantir la sécurité, la qualité, la liquidité et la rentabilité de l'ensemble du portefeuille. De même, les actifs sont majoritairement domiciliés dans les pays membres de L'OCDE. Cela permet de garantir leur disponibilité. Les investissements directs sont effectués selon un processus de sélection rigoureux des émetteurs et de limites portant sur MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

le respect d'un haut niveau de qualité de portefeuille (exclusions, notations financières et extra-financières) et sur des stratégies de diversification par pays, secteur, et émetteur.

Par ailleurs, l'investissement dans une nouvelle classe d'actifs est conditionné à l'approbation du comité d'investissements puis du comité politiques d'investissements et gestion d'actifs avant d'être validé par le conseil d'administration.

Le respect de l'ensemble des allocations et règles de gestion définies dans [les][la] politique[s] annuelle[s] de gestion tactique des placements ou validées lors des différents comités est vérifié quotidiennement.

Les équipes de gestion spécialisées par classes d'actifs effectuent quotidiennement un suivi des risques de marché. Elles réalisent leurs propres analyses des actifs financiers dans lesquels MAIF VIE investit, en amont de l'investissement et durant la période de détention.

Ces principes s'appliquent également aux actifs détenus en représentation des contrats d'assurance vie multisupports pour leur composante en unités de compte. Les supports en unités de compte sont sélectionnés dans le meilleur intérêt des assurés (et leurs bénéficiaires) et en cohérence avec la politique d'investissements du groupe MAIF.


#### C.2.1 Risque actions

Le risque actions correspond au risque de variation du cours des actions (cotées ou non cotées).

Après un net rebond à 6% en 2021, la croissance mondiale devrait faiblir en 2022 pour atterrir autour de 3,2%. Au cours de cette année, les différentes économies ont été confrontées à de nombreux vents contraires rendant les investisseurs averses aux risques. En effet, le retour de la guerre en Europe avec le conflit russo-ukrainien a accentué la flambée des prix des matières premières et énergiques, venant s'accumuler aux tensions sur les chaines d'approvisionnements et pénuries engendrées par la crise de la Covid-19. Ces différents facteurs ont cause une forte inflation, forçant les banques centrales à procéder à un resserrement drastique des politiques monétaires.

Aux Etats-unis, la croissance annuelle en 2022 devrait approcher les 2,1 %. L'inflation a continué de progresser tout au long du premier semestre pour atteindre un pic à 9,1%, avant de décroitre grâce à des effets de base favorables sur les prix de l'énergie et s'établir en décembre à 6,5%. Afin de lutter contre cette inflation, la Réserve Fédérale a fortement resserré ses conditions monétaires avec la mise en place d'un quantitative tightening et le rehaussement, entre mars et décembre, de ses taux directeurs de 425 bp. Dans ce contexte, les indices actions ont recule, notamment les valeurs de croissances et les entreprises technologiques.

De même, en Zone euro, la croissance annuelle diminue pour finir autour de 3,3%. L'inflation s'établit en décembre à 9,2% après avoir touché 10,6% en octobre. Elle a été d'autant plus importante et disparate entre les membres de la zone euro en raison de la proximité de la guerre en Ukraine et de la forte dépendance de nombreux pays au pétrole et gaz Russe. Dans l'objectif de ramener l'inflation à la cible long terme de 2%, la BCE a procédé à un arrêt de son programme de soutien quantitatif et a augmenté ses taux directeurs qui s'établissent à 2,5% fin 2022.

Dans ce contexte, les actifs risqués ont été délaissés par les investisseurs. En zone euro la quasi-totalité des places financières se sont depreciees telles que le CAC 40 ou l'Eurostoxx 50, respectivement en recul de -10% et -12%. Seul le FTSE 100 finit sur une note stable à +0,9%. Outre-Atlantique, le constat est similaire, avec le S&P500 qui perd -20 % et le Nasdaq, indice des valeurs technologique, qui baisse de -29%. L'indice des pays émergents termine l'année à -15 %, impacté en plus, par la résurgence de la crise Covid-19 en Chine.

L'exposition de MAIF VIE aux marchés actions est modérée et a fait l'objet d'un pilotage renforcé durant cette période.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

MAIF VIE est exposée à un fort réajustement à la baisse des marchés actions (pas de retour à la situation initiale avant plusieurs années) engendrant, si elle se produisait :

› une chute de la valeur du portefeuille d'actifs et des produits financiers futurs sur les fonds propres en normes Solvabilité 2 ;

› et un risque de provisionnement pour risque d'exigibilité (PRE) et pour dépréciation durable (PDD) impactant directement le résultat de l'entreprise en normes comptables françaises.

Dans des conditions normales de marché et de volatilité, le risque actions est maîtrisé.

Ce risque est évalué selon la formule standard.

Des mesures d'impacts des variations des performances des marchés actions sur le taux de couverture sont réalisées régulièrement au cours de l'année.


### C.2.2 Risque immobilier

Le risque immobilier correspond au risque de baisse du prix des actifs immobiliers ou de la valeur des participations détenues dans des fonds/sociétés du fait de la baisse du prix des actifs immobiliers sous-jacents.

Dans un contexte géopolitique et économique fortement dégradé, le marché de l'investissement en immobilier a globalement bien résisté en 2022 avec neuf premiers mois dynamiques et une fin d'année beaucoup plus calme en raison de la forte remontée des taux d'intérêt.

Sur un plan sectoriel, les transactions en bureau, secteur le plus liquide, ont continué de baisser. Dans le même temps, les poids de la logistique et des locaux d'activités ont progressé sous l'effet tendanciel de la hausse de l'e-commerce. La même tendance a été observée en commerce qui représente désormais près de 20 % des transactions en immobilier d'Entreprise.

Dans ce contexte, la valorisation de l'allocation immobilière de MAIF VIE s'est montrée résiliente, conséquence de plusieurs orientations de gestion :

› Baisse depuis quelques années du poids des bureaux dans l'allocation ;

› Diversification dans le secteur de la santé depuis 10 ans ;

› Réalisation d'investissements et de co-investissements sur des stratégies value-added pour livrer au marché des actifs répondant aux nouveaux usages ;

› Amélioration de la valorisation des immeubles de placement qui prennent en compte les enjeux Environnementaux, Sociaux et de Gouvernance (ESG);

› Les ventes de certaines participations et d'actifs opportunement.

Ce risque est évalué selon la formule standard.

Des mesures d'impacts des variations des performances des marchés immobiliers sur le taux de couverture sont réalisées régulièrement au cours de l'année.


### C.2.3 Risque de taux d'intérêt

Le risque de taux d'intérêt est le risque de variation du prix ou de la valorisation d'un actif ou d'un passif financier résultant d'une variation défavorable des taux d'intérêt sans risque. Ceci inclut l'asymétrie entre l'actif et le passif.

En période de baisse des taux d'intérêt :

› la valorisation au bilan des provisions techniques S2 augmente,

› la valorisation du portefeuille d'obligations à taux fixe augmente, MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

› en revanche, les nouveaux investissements réalisés sur des taux d'intérêt plus faibles font baisser le rendement du portefeuille.

À l'inverse, en période de hausse des taux :

› la valorisation au bilan des provisions techniques S2 diminue,

› la valeur du portefeuille d'obligations a taux fixe diminue, ce qui peut avoir un impact significatif sur la solvabilité,

› en revanche, les nouveaux investissements réalisés sur des taux d'intérêt plus élevés augmenteront le rendement du portefeuille obligataire.

De par son activité d'épargne (domaine d'activité prédominante de l'entreprise) et la structure de son portefeuille d'actifs (majoritairement positionné sur des placements obligataires), MAIF VIE est exposée aux risques financiers (dégradation des marchés financiers) et plus particulièrement à une situation de hausse brutale et pérenne des taux d'intérêts, ce qui conduirait à une chute de la valeur de marche du portefeuille d'actifs et exposerait MAIF VIE à un risque de rachats.

Plusieurs dispositifs de gestion de ce risque sont mis en œuvre :

› la duration du portefeuille obligataire géré en direct est strictement encadrée par une fourchette

› la constitution de réserve (provision pour participation aux bénéfices) offrant la capacité d'augmenter la rémunération des contrats d'épargne. La provision pour participation aux bénéfices s'élève à 5,7 % des encours des produits d'épargne (supports en euros) au 31 décembre 2022.

Une situation durable de taux d'intérêt négatifs pousserait à repenser le business model de l'activité assurance vie de MAIF VIE et exposerait plus fortement MAIF VIE aux autres risques financiers (chute des marchés actions, immobiliers, ... ).

La provision pour participation aux bénéfices (réserve) offre la capacité, sur quelques années, de fournir un niveau de rémunération compétitif aux contrats d'épargne dans une situation de taux bas ou négatifs.

Les anticipations de marché de fin 2021 s'accordaient sur des niveaux de taux faibles en raison d'une inflation jugée transitoire par les banques centrales, de nature à retrouver son équilibre en 2022 grâce à la fin progressive de leurs programmes d'achats de titres. La guerre en Ukraine, choc inflationniste exogène entraînant une hausse des prix des matières premières, a ravive les pressions inflationnistes déjà à l'œuvre en 2021.

Après avoir admis que l'inflation n'était pas transitoire, les banques centrales ont réagi de manière brutale et rapide afin de contenir la progression de l'inflation et éviter tout déclenchement d'une boucle prix-salaire. En neuf mois, la FED a augmenté ses taux de 375 bps, cycle de resserrement le plus rapide depuis l'après-guerre. La BCE a augmenté ses taux de 250 bps en six mois, le mouvement de resserrement monétaire ayant été initié en juin 2022. Ce resserrement brutal des conditions de financement a entraîné un choc obligataire majeur poussant les taux à des niveaux historiquement élevés, l'OAT 10 ans est passée de 0,2% début janvier 2022 à 3% fin 2022. Les taux n'ont cessé d'augmenter à mesure que les publications des chiffres d'inflation démontraient une accélération de la dynamique inflationniste et une robustesse notable de l'économie.

Ce risque est évalué selon la formule standard.

Des mesures d'impacts des variations des taux sur le taux de couverture sont réalisées régulièrement au cours de l'année.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

### C.2.4 Risque de change

Le risque de change correspond au risque de variation du prix ou de la valorisation d'un actif ou d'un passif financier libellé en devises étrangères résultant d'une évolution défavorable du cours de conversion avec la devise de référence (monnaie nationale).

Les investissements sont réalisés exclusivement sur des actifs libellés en euro. Néanmoins, de manière limitée, MAIF VIE pourrait être exposée aux risques de change dans le cadre de fonds d'investissements en gestion déléguée.

Ce risque est évalué selon la formule standard.


## C.3 Risque de crédit

Le risque de crédit est le risque de perte du principal (somme investie ou prêtée au départ) ou perte de rémunération financière résultant de fluctuations affectant la qualité de crédit d'émetteurs de valeurs mobilières, de contreparties ou de tout autre débiteur et auquel les sociétés d'assurance sont exposées sous forme de risque de spread, risque de concentration et risque de défaut de contrepartie.

Trois types de risque de crédit font l'objet d'un suivi spécifique à MAIF VIE :

› les placements financiers, et plus particulièrement le portefeuille de produits de taux : risque de spread et risque de concentration ;

› les créances sur les réassureurs, nées des opérations de cession en reassurance : risque de défaut ;

›

les créances sur les banques, nées des dépôts de trésorerie : risque de défaut.

Le principe de la personne prudente décrit au §C.2 (risque de marché) s'applique au risque de crédit.


### C.3.1 Risque de spread

Le risque de spread de crédit est le risque de variation du prix ou de la valorisation d'un actif résultant d'une évolution défavorable de l'écart entre le taux de rentabilité actuariel d'une obligation et le taux de rentabilité actuariel d'un emprunt sans risque de durée identique. L'origine du risque est une dégradation de la qualité de crédit de l'émetteur obligataire (hors obligations souveraines zone euro pour le calcul réglementaire).

Le spread est également couramment appelé prime de risque.

Le risque de spread est maîtrisé par la mise en place d'un processus de sélection rigoureux des émetteurs et de limites portant sur le respect d'un haut niveau de qualité de portefeuille (limites par notation) et sur des stratégies de diversification par pays, secteur, et émetteurs.

À fin 2022, le portefeuille obligataire, ventilé par notation, est présenté ci-après.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="Portefeuille obligataire par notation en valeur de marché - 2022" -->

<figure>

![](figures/9)

<!-- FigureContent="Non noté 1% BBB AAA 13% 11% A 20% AA 55% AA :selected: BBB
AAA
 «" --> Non noté

</figure>



En comparaison à 2021, les spreads des obligations de crédit ont été volatils en 2022. Le resserrement des politiques monétaires ainsi que les incertitudes liées à la guerre en Ukraine et le spectre de la récession ont entraîné un écartement des spreads sur les indices High Yield et sur l'Investment Grade dans une moindre proportion. Les spreads se sont sensiblement resserrés durant l'été 2022, principalement en raison de la conviction du marché d'un certain « pivot » des banques centrales (anticipation du passage de politiques monétaires restrictives à une potentielle baisse des taux). Mais la réaffirmation par les banquiers centraux de la nécessité de prolonger le resserrement monétaire, au prix d'une croissance plus faible, a conduit à de nouveaux écartements des spreads. Le portefeuille crédit MAIF VIE est essentiellement exposé à des émetteurs de bonne qualité (catégorie investissements).

Ce risque est évalué selon la formule standard.

Des mesures d'impacts des variations des spreads sur le taux de couverture sont réalisées régulièrement au cours de l'année.


### C.3.2 Risque de concentration

Le risque de concentration se définit comme le risque découlant de toutes les expositions au risque qui sont assorties d'un potentiel de perte suffisamment important pour menacer la solvabilité ou la situation financière de MAIF VIE. Ce risque découle ainsi, soit d'un manque de diversification du portefeuille d'actifs, soit d'une exposition importante à un seul et unique émetteur de valeurs mobilières ou d'un groupe d'émetteurs liés.

Le risque de concentration de marché est limité à MAIF VIE par la mise en place de différents niveaux de diversification dans le cadre de la politique de gestion tactique des placements :

› le premier niveau de diversification est établi a partir d'une matrice d'allocation des actifs qui fixe des proportions minimales et maximales pour chacune des grandes classes d'actifs de l'univers d'investissements,

› de plus, des règles de dispersion existent pour la gestion directe,

› enfin, des ratios d'emprise sont imposés par la politique de gestion tactique des placements pour la gestion déléguée, hors fonds dédiés aux sociétés du groupe MAIF.

Le respect de ces limites est vérifié à chaque souscription sur un émetteur.

À fin 2022, le portefeuille obligataire en direct, ventilé par secteur, est présenté ci-après. Le secteur Agences & Supranationales regroupe les institutions gouvernementales supranationales comme l'Union Européenne ou la Banque Internationale pour la Reconstruction et le Développement (BIRD).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="Portefeuille obligataire par secteur- 2022" -->

<figure>

![](figures/10)

<!-- FigureContent="Non financières 21% Etats 42% Financières Agences & 30% Supranationales 7%" -->

</figure>


L'exposition de MAIF VIE aux titres souverains émis par les Etats et agences & supranationales est présentée ci-après.

Exposition de MAIF VIE aux titres souverains- 2022

<figure>

![](figures/11)

<!-- FigureContent="Espagne Allemagne 2% 2% Finlande Pays Bas 1% Irlande Autriche Danemark 0% 4% 2% 0% Belgique 4% Supranational 14% France 71%" -->

</figure>


Dans le cadre de la gestion déléguée, MAIF VIE a défini plusieurs critères de sélection des sociétés de gestion (ou de toute autre entité assurant ou conseillant la gestion.

Le risque de concentration est évalué selon la formule standard (SCR Marché).


#### C.3.3 Risque de défaut de contrepartie pour les réassureurs

Le risque de défaut de contrepartie est le risque qu'un débiteur ou qu'une contrepartie de MAIF VIE n'honore pas ses engagements dans les conditions initialement prévues (livraison, paiement, remboursement, etc.).

Le risque de défaut de contrepartie pour les réassureurs est géré en amont de la souscription des traités de réassurance, notamment en définissant des critères de sélection des réassureurs, essentiellement sur la notation des réassureurs et sur la dispersion des réassureurs, de manière à éviter une exposition excessive à un réassureur en particulier.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

Au cours de l'année, un suivi régulier est réalisé sur les traités de réassurance. Ce suivi comprend notamment :

› la constitution des comptes de trésorerie,

› la réalisation des comptes techniques annuels, par traite et par type de risques couverts,

› l'évolution des notations des réassureurs,

› la communication des informations aux réassureurs,

› le recouvrement et le suivi des soldes à régler.

Le risque de défaut de contrepartie est évalué selon la formule standard.


## C.4 Risque de liquidité

Le risque de liquidité se décompose en deux risques :

› le risque d'illiquidité, qui se définit comme le risque de ne pas pouvoir céder des actifs en vue d'honorer les engagements financiers de l'assureur au moment où ceux-ci deviennent exigibles. MAIF VIE considère également comme risque d'illiquidité, le risque de céder nécessairement des actifs dans des conditions dégradées pour honorer ses engagements financiers au moment où ceux-ci deviennent exigibles (risque de devoir vendre des actifs en moins-values et donc de générer des pertes pour faire face aux engagements : décaissements pour régler les sinistres, les frais généraux ... ) ;

› le risque de trésorerie (gap de trésorerie), qui correspond à une impossibilité de faire face aux engagements de court terme (décaissements pour régler les prestations, les frais généraux ... ).

Le principe de la personne prudente décrit au § C.2 (risque de marché) s'applique au risque de liquidité.

MAIF VIE investit majoritairement sur des supports considérés comme liquides (placements monétaires, emprunts d'État de la zone euro, obligations dites « Investment grade » émises par des agences, des entreprises et des banques, fonds d'actifs cotés ... ).

La part importante de titres liquides dans le portefeuille MAIF VIE devrait permettre de faire face à un besoin imprévu de trésorerie sans avoir à céder des titres difficilement cessibles. Les revenus récurrents générés par les obligations, les dividendes et les loyers assurent des entrées régulières en trésorerie.


## Processus de gestion de la liquidité ›

Des travaux actif/passif sont réalisés chaque année, en amont de la rédaction des politiques de gestion des risques des investissements et de gestion tactique des placements. Ces travaux intègrent notamment la définition de l'allocation stratégique d'actifs et l'analyse du risque de liquidité avec une vision moyen/long terme, ainsi il en découle la détermination de limites opérationnelles sur les investissements.

L'analyse du risque de liquidité sur une vision moyen/long terme a été enrichie en 2022 via le calcul d'un ratio de liquidité. Ce ratio est inspiré de la réglementation Bâle III, en déterminant le rapport entre la liquidité disponible et le besoin en liquidité sur une période donnée. En fonction de la classe d'actifs, une décote est appliquée pour tenir compte de la liquidité de marché de ces derniers. Compte tenu du cycle d'activité de MAIF VIE, le besoin en liquidité est défini sur un horizon d'un an.

Les résultats montrent que même dans un environnement stressé, MAIF VIE couvre largement ses besoins en liquidité.

De plus, l'actualisation du prévisionnel de trésorerie sur l'année en cours ainsi que le suivi des règles de gestion (limites) sont réalisés quotidiennement.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

Des calculs de projections de flux sont réalisés mensuellement permettant ainsi de déceler les éventuelles insuffisances de trésorerie à court/moyen terme. Ces projections permettent chaque mois de définir quelles sont les enveloppes d'investissement à réaliser par la Direction Investissements et Placements sur le mois.

Des calculs de projections de flux sont réalisés trimestriellement par MAIF VIE, permettant ainsi de déceler les éventuelles insuffisances de trésorerie à court/moyen terme.

Au moins une fois par an, une analyse des risques de liquidité est réalisée pour le portefeuille de chaque entité du groupe MAIF.

Depuis 2020, un suivi du risque de liquidité des obligations au quotidien est réalisé à partir de BarraOne, l'outil de suivi du risque de marché utilisé par la Direction Investissements et Placements. Ce suivi repose sur un indicateur de marché très courant : le bid-ask spread qui correspond à l'écart entre le prix maximum qu'un acheteur est prêt à payer pour acheter un titre et le prix minimum auquel un vendeur est prêt à vendre.

Logiquement, plus cet écart est important, plus les acheteurs auront du mal à s'entendre pour faire des transactions et donc plus la liquidité sera dégradée.

C'est un indicateur suivi globalement sur le portefeuille pour donner une « tendance >> de l'évolution de la liquidité du marché.

Par ailleurs, comme évoqué en partie D du présent rapport, MAIF VIE a recours à la correction pour volatilité dans l'évaluation de sa solvabilité. A ce titre, conformément à l'article R354-2 du Code des assurances, elle doit établir un plan de liquidité comportant une prévision des flux de trésorerie entrants et sortants au regard des actifs et des passifs faisant l'objet de cette correction. Le prévisionnel de trésorerie contrôlé quotidiennement par la DIP s'inscrit dans ce cadre pour l'année en cours, les travaux actifs passifs menés annuellement par la GAPEF couvrant la partie prospective.


## › Facultés contractuelles et règlementaires face au risque de liquidité en assurance vie

La survenance d'une impasse de trésorerie liée à un risque non anticipé au passif deviendrait problématique en cas d'absence de liquidité des actifs.

Or, le portefeuille d'actifs de MAIF VIE est principalement positionné sur des actifs dont la cession dans des conditions normales s'effectue dans un délai inférieur ou égal à 3 jours.

Conformément aux conditions contractuelles, MAIF VIE, dispose, pour la grande majorité des produits, d'un délai d'un mois pour régler les prestations décès et rachats, après réception de l'ensemble des pièces exigées.

Par ailleurs, l'article L132-21 du Code des assurances permet à l'entreprise d'assurance de verser la valeur de rachat dans un délai n'excédant pas deux mois à compter de la date de réception de l'ensemble des pièces exigées. Au-delà de ce délai, les sommes non versées produisent de plein droit intérêt au taux légal majoré de moitié durant deux mois, puis, à l'expiration de ce délai de deux mois, au double du taux légal.

En dernier recours, dans un contexte financier dégradé, trois facultés règlementaires pourraient être activées :

› L'article L.131-4 du code des assurance, dans le cas où la valeur liquidative d'une UC ne serait pas disponible, permet à MAIF VIE de (i) proposer de remettre à l'adhérent ou bénéficiaires les parts de l'OPC ou (ii) suspendre les rachats, arbitrages, versements, prestations, transferts et conversions en rentes sur ces UC.

› L'article L.612-33 du Code monétaire et financier, permet à l'ACPR de suspendre, restreindre ou interdire temporairement la libre disposition de tout ou partie des actifs et de suspendre ou limiter le paiement des valeurs de rachat, la faculté d'arbitrages, le versement d'avances sur contrat ou la faculté de renonciation.

› La loi Sapin 2 permet, depuis 2016, au haut conseil de stabilité financière (HCSF) de suspendre ou limiter le rachat des contrats d'assurance vie, si le secteur de l'assurance vie se trouvait, dans son ensemble, en difficulté.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

MAIF VIE n'a pas eu recours à des cessions de titres pour faire face à ses engagements et n'a pas été confronté à des besoins exceptionnels de liquidité. Les indicateurs de liquidité sont restés stables en 2022, sur des niveaux n'entrainant ni d'alertes sur la liquidité des portefeuilles ni de problèmes pour que les gérants réalisent leurs operations habituelles.


## C.5 Risque opérationnel

La définition du risque opérationnel telle que rédigée dans les textes Solvabilité 2, est celle retenue par MAIF VIE et le groupe MAIF : « Le risque opérationnel est un risque de perte provenant de processus internes inadéquats ou défaillants, de personnes, de systèmes ou d'événements externes >>. Cette définition recouvre notamment les erreurs humaines, les fraudes et malveillances, les défaillances des systèmes d'information, les problèmes liés à la gestion du personnel, les litiges commerciaux, les accidents-incendies-inondations sur le bâtiment occupé par MAIF VIE.

Ce risque est évalué selon la formule standard, via une approche forfaitaire globale et indépendante des risques sous-jacents.

Les principaux risques opérationnels de MAIF VIE identifiés sont décrits ci-après.


### C.5.1 Risque de sous-traitance

Le risque et le dispositif de MAIF VIE sont présentés au § B.7.


### C.5.2 Risque de non continuité ou indisponibilité des données ou applicatifs

Le système d'information de MAIF VIE n'est pas exposé directement aux intrusions d'origine externe dans la mesure où il est protégé par le système d'information de la MAIF, et donc dépendant du niveau de sécurité du système d'information de cette dernière.

MAIF dispose d'une équipe dédiée à la sécurité des systèmes d'information (RSSI Groupe). Au sein de MAIF VIE, un collaborateur a désormais en charge la mission de RSSI, incluant la déclinaison de la politique SI groupe.

Les équipes informatiques ont élaboré et tiennent à jour un plan de reprise des activités SI. Concernant le risque d'indisponibilité du système d'information, un processus spécifique existe sur la réalisation de tests et de validation des applicatifs.


### C.5.3 Risque de non-conformité aux obligations réglementaires

Afin de sécuriser la mise en conformité aux nombreuses obligations règlementaires qui s'imposent aux sociétés d'assurance vie, MAIF VIE s'inscrit dans une démarche d'amélioration continue tant sur la réglementation existante que sur la déclinaison des règlementations nouvelles.

En amont, les mises en conformité au regard des évolutions législatives et réglementaires impliquent tous les métiers concernés et sont accompagnées par les différentes entités juridiques et la Conformité.

En aval, la maîtrise des risques de non-conformité, et le suivi des dispositifs mis en œuvre, sont sécurisés par le contrôle permanent et périodique.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

## C.6 Autres risques importants

En complément des risques décrits ci-avant, MAIF VIE a identifié plusieurs autres risques d'entreprise ou risques émergents significatifs.

Pour ces risques, l'impact n'est pas quantifiable avec précision. Cependant, des processus de gestion des risques et de contrôle interne ont été déployés de manière à fournir une maîtrise attentive des risques et une sécurité raisonnable de l'activité, de la solvabilité et de la situation financière de MAIF.


### C.6.1 Risque de construction, maintenance ou usages inadaptés ou inadéquats des modélisations

MAIF VIE utilise des modèles pour mesurer ses engagements (calcul de provisions), évaluer sa solvabilité (mesure des SCR et MCR), concevoir ses tarifications, estimer les impacts potentiels face à des situations extrêmes futures, réaliser les études la gestion actif/passif et definir l'allocation stratégique d'actif.

La complexité et la précision des modèles développés par MAIF VIE peuvent engendrer un risque, dit de modèle, se manifestant par l'écart entre les phénomènes que l'on cherche à représenter et leur réalisation.

L'encadrement des modèles mis en place par MAIF VIE limite la portée de ce risque : existence d'une gouvernance stricte concernant l'utilisation (une charte d'utilisation et recueil de bonnes pratiques) et les évolutions des modèles (environnement dédié de développement et processus de validation). Une documentation robuste et structurée vient compléter ces dispositions pour assurer une accessibilité et une compréhension du fonctionnement des modèles.

L'interprétation des résultats issus des modélisations est réalisée par confrontation avec le passé (back testing), partagée avec différents intervenants en dehors des développeurs et complétée par des analyses de sensibilités (sur les hypothèses).


### C.6.2 Risque de réputation et gestion de crise

Le groupe MAIF procède à une veille régulière sur son image et sa réputation. Cette veille est à la fois vis-à-vis de la presse, internet par exemple blog, réseaux sociaux ou publications judiciaires.

La gestion de crise peut conduire à activer notamment le Plan de continuité des activités de MAIF VIE qui est présenté au §B4.


### C.6.3 Risque d'évolution économique ou juridique

MAIF VIE, de par son secteur d'activité, est exposée à un risque d'évolution économique, réglementaire ou juridique qui engendrerait une perte de compétitivité des produits d'assurance vie par rapport à d'autres produits d'épargne (bancaires ou autres), et pourrait ainsi conduire, non seulement à des rachats massifs par arbitrage, mais aussi à une baisse significative du chiffre d'affaires. Cette situation conduirait nécessairement MAIF VIE à repenser son offre produits ainsi que son business model.

MAIF VIE est attentive à toute évolution juridique, règlementaire et économique au travers des différents processus de veille auxquels elle participe soit directement, soit sous couvert du groupe et réalise des chiffrages le cas échéant.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

C.6.4 Risque de durabilité
===

Le principe de la personne prudente décrit au § C.2 (risque de marché) s'applique en matière de gestion d'actifs, au risque de durabilité.

C.6.4.1 Risque de durabilité en matière de gestion d'actifs

Les risques de durabilité intègrent :

› Les risques de non-adaptation des actifs financés au changement climatique et à la transition énergétique et écologique (risques physiques, risques de transition et risques de responsabilité).

› Les « risques sociaux >>, c'est-à-dire les risques issus de sujets sociaux comme les droits, le bien-être et les intérêts des peuples et communautés,

› Et, plus largement, tous risques liés aux mauvaises pratiques ESG (environnement, social, gouvernance) des actifs finances.

Ces risques peuvent se matérialiser par un impact financier négatif et/ou des externalités négatives sur la société ou l'environnement pouvant générer un risque d'image.

En cohérence avec sa raison d'être et ses valeurs, le groupe MAIF s'engage dans la prise en compte des risques extra-financiers dans ses politiques d'investissements (politique de gestion des risques des investissements et politique de gestion tactique des placements), et dans une démarche d'investisseur responsable.

Cette démarche d'investisseur responsable consiste à évaluer systématiquement et en préalable à tout investissement les caractéristiques environnementales, sociales et de gouvernance des entreprises et des Etats dans lesquels elle investit directement, ainsi que la démarche de prise en compte des sujets extra-financiers par les fonds d'investissement qu'elle sélectionne.

Ainsi, les gérants d'actifs financiers du groupe MAIF ne peuvent investir directement que dans des Etats qui ont aboli la peine de mort et présentent des niveaux de sécurité, de développement humain retraité des inégalités et de maîtrise de la corruption satisfaisants. Dans le même sens, les entreprises qui ne répondraient pas aux attentes du groupe MAIF en matière d'orientation de leur activité ou de bonnes pratiques de responsabilité sociétale des entreprises (RSE) ne peuvent pas être financées directement (grâce à l'analyse ESG).

Le groupe MAIF s'engage ainsi à ne pas financer directement les entreprises qui ne respecteraient pas les 10 principes du Pacte Mondial des Nations Unies, les entreprises qui commercialisent des armes, les producteurs de tabac, les entreprises liées au charbon et à l'industrie pétrolière. En complément, un suivi quotidien des actualités concernant les entreprises dans le portefeuille géré en direct est effectué. En cas de controverse sur une société, une analyse des expositions sur l'ensemble du portefeuille (gestion directe et gestion déléguée) via la transparisation est réalisée. En gestion déléguée, les gérants font leurs meilleurs efforts pour intégrer l'ISR et le financement de la transition énergétique et écologique ainsi que les enjeux sociaux dans chaque classe d'actifs.

Pour 2022, l'objectif d'allocation ISR (investissement socialement responsable) pour MAIF VIE était d'investir au moins 93% des actifs financiers (hors participations stratégiques et immobilier d'exploitation) dans des supports qui prennent en compte les sujets environnementaux, sociaux et de gouvernance. A fin 2022, MAIF VIE a atteint une part ISR de 93 %.

De plus, MAIF VIE avait un objectif de part sociale de ses investissements (investissements qui soutiennent une activité dont l'objectif est de résoudre directement une problématique sociétale, ou une activité soutenant la transformation sociale des entreprises) de 5%. A fin 2022, la part sociale s'élève à 5,8%.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

## C.6.4.2 Risque lié à la transition énergétique et écologique

Le groupe MAIF s'inscrit dans la transition énergétique et écologique en proposant notamment une politique d'investissements en adéquation avec les enjeux liés au changement climatique. Cette politique a pour objectif de contribuer au financement de la transition énergétique et écologique et de prendre en compte les risques de non-adaptation des actifs financés au changement climatique et à la transition énergétique et écologique (risque physique, risque de transition et risque de responsabilité).

Une mauvaise prise en compte de la transition climatique et énergétique dans les investissements constituerait un double risque pour le groupe MAIF :

› une perte de contrôle sur les externalités négatives provoquées par les entreprises, Etats et projets financés par le groupe,

› une perte de valeur ou de capital associée aux conséquences physiques du changement climatique, à une mauvaise adaptation des entreprises ou politiques publiques à la transition énergétique, et aux risques d'image ou juridiques induits par la lutte contre les émissions de substances polluantes et de gaz à effet de serre.

En 2020, le groupe MAIF a défini sa stratégie Climat des investissements, visant à s'inscrire dans la trajectoire de l'Accord de Paris afin de contribuer à l'objectif mondial de neutralité carbone à horizon 2050. Le groupe MAIF s'inscrit avec volontarisme dans cet objectif en visant d'ici à 2030 un alignement de l'ensemble de son portefeuille d'investissements sur une trajectoire de réchauffement climatique au plus proche de 1,5°C. La stratégie Climat des investissements s'inscrit désormais dans la stratégie globale Climat du groupe MAIF.

Pour atteindre les objectifs qu'il s'est fixé, le groupe MAIF a pris les engagements suivants :

› Arrêter le financement de nouveaux projets de pétrole et de gaz via ses investissements en direct

› Sortir totalement du charbon thermique et des énergies fossiles non conventionnelles d'ici à 2030

› Sortir de l'industrie pétrolière d'ici à 2040

Le groupe MAIF a également rejoint en octobre 2021 la Net Zero Asset Owner Alliance (NZAOA) et a défini de nouveaux objectifs pour le groupe, à horizon 2025 (entre le 31/12/2020 et le 31/12/2024), à savoir :

› Réduire l'empreinte carbone des actifs financiers de 25%

o Le périmètre concerné regroupe le portefeuille obligataire d'entreprises, en gestion directe et en gestion déléguée, le portefeuille actions et les fonds dédiés immobiliers (applicables au groupe MAIF et à MAIF VIE)

o La réduction de l'empreinte carbone concerne les scopes 1, 2 et 3 amont en intensité carbone (tCO2/M€ investis)

› Atteindre à minima 15% de part verte pour le groupe MAIF et 13% pour MAIF VIE
 :unselected:
o Cet engagement couvre 100% des actifs financiers
 :unselected:
o La part verte est calculée selon la méthodologie MAIF (référentiel du label Greenfin)

› Dialoguer avec les 20 entreprises les plus émettrices du portefeuille (gestion directe) et les 20 principales sociétés de gestion, afin de les inciter à s'inscrire dans une trajectoire de réduction de leurs émissions de gaz à effet de serre.

L'adoption de cette stratégie est également l'opportunité pour MAIF VIE de démontrer sa singularité et de contribuer à une transition climatique inclusive, c'est-à-dire attentive aux enjeux sociétaux liés à la transition tels que son impact sur l'emploi, les territoires ou l'accès à l'énergie.

La mise en œuvre de cette stratégie se traduit par un renforcement régulier de notre politique d'exclusion ainsi que par des actions d'engagement auprès de l'ensemble de nos parties prenantes.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022 :unselected: :unselected:

## Politique d'exclusion du charbon MAIF VIE ne finance plus en direct et via ses fonds dédiés :

› les entreprises minières qui produisent du charbon thermique,

› les entreprises qui développent de nouveaux projets de mines, centrales et infrastructures charbon,

les entreprises qui réalisent plus de 10 % de leur chiffre d'affaires grâce au charbon,

\--- les entreprises qui produisent plus de 10 % de leur électricité à partir du charbon,

les entreprises dont les capacités installées de production d'électricité à partir de charbon sont supérieures à 5 GW.


## Politique d'exclusion des autres énergies fossiles

Sont exclus des investissements en direct et des fonds dédiés :

› les entreprises développant de nouveaux projets d'exploration ou de production de pétrole ou de gaz naturel,

› les entreprises dont plus de 5 % de la production d'hydrocarbures est issue d'énergies fossiles non conventionnelles (pétrole et gaz de schiste, sables bitumineux, forage en Arctique, forage en eaux très profondes, l'extraction de pétrole extra-lourd et l'exploitation du gaz de houille),

› les entreprises de l'industrie pétrolière ou produisant de l'électricité à partir de pétrole pour plus de 5 % de leur chiffre d'affaires.

En complément de ces exclusions, la mutuelle entend mener une démarche d'engagement auprès des entreprises et faire évoluer favorablement les sociétés de gestion en s'engageant auprès de celles à travers lesquelles elle investit. Elle ciblera dans un premier temps les 20 entreprises les plus émettrices de gaz à effet de serre du portefeuille géré en direct, les 20 principales sociétés de gestion auxquelles MAIF VIE confie des encours, et les sociétés de gestion qui n'ont pas défini de trajectoire de sortie du charbon pour leurs investissements.

Pour 2022, MAIF VIE s'était par ailleurs engagé à investir au moins 9 % des actifs financiers en faveur de la transition énergétique et écologique (« part verte >>). A fin 2022, la part verte s'élève à 12,3%.

En ligne avec sa raison d'être, qui vise à garantir un mieux commun, MAIF VIE cherche à contribuer par ses investissements à une transition climatique inclusive, créatrice d'emplois durables, attentive aux inégalités sociales et au développement des territoires et promotrice de modes de vie plus durables.

La société d'assurance mutuelle rendra compte régulièrement de la politique mise en œuvre et de l'exposition résiduelle de ses investissements liés au charbon thermique et aux énergies fossiles.

En parallèle, le groupe MAIF évalue les émissions de gaz à effet de serre induites par ses placements financiers, ainsi que l'alignement des activités et des engagements pris par les entreprises et les États qu'elle finance avec un scénario de réchauffement climatique limité à 2ºC.

Depuis 2020, elle réalise également une mesure des risques physiques et des risques de transition liés à ses investissements.

Par ailleurs, une veille active est menée sur les évolutions règlementaires sur le climat, les nouveaux outils et les nouvelles méthodologies d'analyse des risques climatiques des portefeuilles d'investissements financiers.

Le groupe MAIF procède également à une veille sur les impacts physiques du réchauffement climatique en particulier pour des évènements extrêmes ou l'offre d'assurance.

Fin 2020, le groupe MAIF a participé au premier exercice de stress test climatique proposé par L'ACPR.

L'approche RSE tâche de sensibiliser les sociétaires sur l'urgence climatique au regard des usages quotidiens. Plusieurs solutions éducatives sur le thème du développement durable sont proposées par le groupe MAIF et mis à la disposition de tous.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

Le groupe MAIF mène des actions pour réduire l'impact environnemental lié à ses immeubles d'exploitation et moyens généraux. Sa stratégie repose sur les objectifs de diminution des sources de pollution et la préservation des ressources. Elle se traduit par des actions concrètes : la réduction de la consommation énergétique, l'intégration des principes d'économie circulaire dans ses achats de mobilier ou encore le respect des référentiels de Haute Qualité Environnementale (NF HQE). Ses démarches sont certifiées et, en particulier, MAIF est ISO 50001 pour son management de l'énergie.


## C.6.5 Risques émergents

MAIF VIE est en veille sur les risques émergents. Ce sont les risques non encore connus, qui peuvent se développer ou qui existent déjà et évoluent constamment. Les risques émergents sont ainsi caractérisés par un haut degré d'incertitude, puisque certains ne se manifesteront jamais. Ces risques font l'objet d'un processus spécifique d'identification et d'analyse. La société réalise une veille de marché (publications scientifiques, règlementation, décisions de justice, économie ... ) pour identifier et élaborer des réponses de manière à anticiper la survenance de ces risques.


## › Risque de montée en puissance des robot-advisors (« robots-conseillers »)

Le risque concerne les produits d'épargne, il se matérialiserait ainsi : les sociétaires confient à des agrégateurs, la gestion de leur multi-bancarisation.

Ces autres acteurs se positionnent ainsi en intermédiaire sur les offres épargne dont l'assurance vie et font développer un comportement opportuniste des sociétaires.

Une conséquence serait une chute du portefeuille d'assurance : les assurés se repositionnant sur des offres digitales, plus conformes à leurs attentes.

Ce risque émergent est suivi notamment à travers les risques de dépense et de hausse des rachats.


## C.7 Autres informations

MAIF VIE n'a pas eu recours à des véhicules de titrisation.

La titrisation est une technique financière qui vise à transférer à des investisseurs des actifs financiers en les transformant en titres financiers émis sur le marché des capitaux, le tout via une société ad hoc.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->
<figure>

![](figures/12)

<!-- FigureContent="D" -->

</figure>


VALORISATION À DES FINS DE SOLVABILITÉ

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="65 / 111" -->

D Valorisation à des fins de Solvabilité
===

La valorisation des actifs et des passifs de MAIF VIE concourt à la détermination du SCR, du MCR et des fonds propres éligibles pour les couvrir, présentes dans la suite de ce rapport au paragraphe E.

Le bilan repris ci-après présente le bilan prudentiel, établi selon les règles définies dans la directive Solvabilité 2 et ses textes d'application. Le montant des actifs et passifs valorisé selon les normes comptables françaises y figure également à titre de comparaison. L'actif net correspond à la différence entre le total de l'actif et le total du passif. Le bilan est arrêté au 31 décembre 2022.

Les chiffres sont présentés en milliers d'euros.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="66 / 111" -->

| ACTIF - 31/12/2022 | Solvabilité 2 | Normes comptables francaises | Ecart |
| - | - | - | - |
| Écarts d'acquisitions | - | - | - |
| Frais d'acquisition reportés | - | - | - |
| Actifs incorporels | - | 18 463 | -18 463 |
| Impôts différés actifs | - | - | - |
| Excédent de régime de retraite | - | - | - |
| Immobilisations corporelles pour usage propre | 2 189 | 4 783 | -2 594 |
| Placements (autres que les actifs en représentation de contrats en UC ou indexés) | 10 560 376 | 11 423 153 | -862 777 |
| Immobilier (autre que pour usage propre) | - | - | - |
| Participations | 939 457 | 808 889 | 130 568 |
| Actions | 64 828 | 60 597 | 4 231 |
| Actions cotées | - | - | - |
| Actions non cotées | 64 828 | 60 597 | 4 231 |
| Obligations | 7 506 141 | 8 636 087 | -1 129 946 |
| Obligations d'Etat | 3 634 630 | 4 164 899 | -530 269 |
| Obligations de sociétés | 3 871 510 | 4 471 188 | -599 677 |
| Obligations structurées | - | - | - |
| Titres garantis | - | - | - |
| Fonds d'investissement | 1999 689 | 1 867 318 | 132 371 |
| Produits dérivés | - | - | - |
| Dépôts autres que ceux assimilables à de la trésorerie | 50 262 | 50 262 | - |
| Autres placements | - | - | - |
| Placements en représentation de contrats en UC ou indexé: | 778 385 | 778 385 | - |
| Prêts et prêts hypothécaires | 2 885 | 2 885 | - |
| Avances sur polices | 2 885 | 2 885 | - |
| Prêts et prêts hypothécaires aux particuliers | - | - | - |
| Autres prêts et prêts hypothécaires | - | - | - |
| Provisions techniques cédées | 56 845 | 84 985 | -28 139 |
| Non vie et sante similaire à la non-vie | - | - | - |
| Non vie hors santé | - | - | - |
| Santé similaire à la non vie | - | - | - |
| Vie et santé similaire à la vie, hors UC ou indexés | 56 845 | 84 985 | -28 139 |
| Santé similaire à la vie | - | - | - |
| Vie hors santé, UC ou indexés | 56 845 | 84 985 | -28 139 |
| UC ou indexés | - | - | - |
| Dépôts auprès des cédantes | - | - | - |
| Créances nees d'opérations d'assurance | 11 750 | 11 750 | - |
| Créances nées d'opérations de reassurance | - | - | - |
| Autres créances (hors assurance) | 24 878 | 24 878 | - |
| Actions auto-détenues | - | - | - |
| Instruments de fonds propres appelés et non payés | - | - | - |
| Trésorerie et équivalent trésorerie | 49 968 | 49 968 | - |
| Autres actifs non mentionnés dans les postes ci-dessus | - | - | - |
| TOTAL DE L'ACTIF | 11 487 276 | 12 399 250 | -911 973 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

67 / 111

\> VALORISATION À DES FINS DE SOLVABILITÉ
===

| PASSIF - 31/12/2022 | Solvabilité 2 | Normes comptables francaises | Ecart |
| - | - | - | - |
| Provisions techniques - non-vie | - | - | - |
| Provisions techniques non-vie (hors santé) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques santé (similaire à la non-vie) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques - vie (hors UC ou indexes) | 9 451 048 | 10 798 056 | -1 347 008 |
| Provisions techniques santé (similaire à la vie) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques vie (hors santé, UC ou indexés) | 9 451 048 | 10 798 056 | -1 347 008 |
| Provisions techniques calculées comme un tout | - | - | - |
| Meilleure estimation | 9 166 708 | | 9 166 708 |
| Marge de risque | 284 341 | | 284 341 |
| Provisions techniques UC ou indexes | 777 106 | 778 385 | -1 279 |
| Provisions techniques calculées comme un tout | - | - | - |
| Meilleure estimation | 766 056 | | 766 056 |
| Marge de risque | 11 050 | | 11 050 |
| Autres provisions techniques | - | - | - |
| Passifs éventuels | 2 619 | - | 2 619 |
| Provisions autres que les provisions techniques | 2 790 | 2 790 | - |
| Provision pour retraite et autres avantages | 1 396 | 1708 | -312 |
| Dettes pour dépôts espèces des réassureurs | 83 794 | 83 794 | - |
| Impôts différés passifs | 13 907 | - | 13 907 |
| Produits dérivés | - | - | - |
| Dettes envers les établissements de crédit | - | - | - |
| Dettes fin. autres que celles envers les établis. de crédit | - | - | - |
| Dettes nées d'opérations d'assurance | 944 | 944 | - |
| Dettes nées d'opérations de reassurance | 3 332 | 3 332 | - |
| Autres dettes (non liées aux opérations d'assurance) | 54 594 | 54 594 | - |
| Dettes subordonnées | 34 500 | 34 500 | - |
| Dettes subordonnées exclues des fonds propres de base | - | - | - |
| Dettes subordonnées incluses dans les fonds propres de base | 34 500 | 34 500 | - |
| Autres dettes non mentionnées dans les postes ci-dessus | - | - | - |
| TOTAL DU PASSIF | 10 426 030 | 11 758 103 | -1 332 073 |
| | | | |
| ACTIF NET | 1 061 246 | 641 147 | 420 100 |

L'établissement du bilan, selon les règles en normes Solvabilité 2 et selon les normes comptables françaises, s'appuie sur certaines hypothèses et estimations.

› Principes generaux de valorisation des actifs et passifs dans le bilan en normes Solvabilité 2

Conformément à l'article 75 de la directive 2009/138/CE :

› les actifs sont valorisés au montant pour lequel ils pourraient être échanges dans le cadre d'une transaction conclue, dans des conditions de concurrence normale, entre des parties informées et consentantes ;

› les passifs sont valorisés au montant pour lequel ils pourraient être transférés ou réglés dans le cadre d'une transaction conclue, dans des conditions de concurrence normales, entre des parties informées et consentantes. Aucun ajustement visant à tenir compte de la qualité de crédit propre à l'entreprise d'assurance n'est effectué.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="68 / 111" -->

Ces valorisations doivent être réalisées en se fondant sur l'hypothèse d'une continuité d'exploitation de l'entreprise et respecter les principes et la hiérarchie de valorisation définis dans les articles 9, 10 et suivants du règlement délégué 2015/35 de la commission européenne du 10 octobre 2014.


## ›


## Principales différences avec les normes comptables françaises

Les principales différences de valorisation entre le bilan en normes Solvabilité 2 et le bilan établi selon les normes comptables françaises portent sur les postes suivants :

› valorisation nulle des actifs incorporels ;

› revalorisation des placements à la juste valeur, intégrant les plus et moins-values latentes à la valeur historique corrigée des amortissements et dépréciations retenue dans le bilan normes comptables françaises ;

› revalorisation en valeur économique des provisions techniques ;

› comptabilisation d'impôts différés, non reconnus dans le bilan en normes comptables françaises ;

› reclassement de 67,6% de la Provision pour Participation aux Bénéfices (PPB) en fonds excédentaires.

Par ailleurs, l'évaluation des actifs et des passifs a été réalisée sur la base de l'hypothèse de la continuité de l'exploitation.

Les paragraphes D1, D2 et D3 ci-après développent le contenu des postes les plus significatifs du bilan et détaillent les principes généraux de valorisation décrits ci-dessus pour le bilan en normes Solvabilité 2 en les comparant aux normes comptables françaises, respectivement pour les actifs, les provisions techniques et les autres passifs (hors provisions techniques). Le paragraphe D4 revient sur les principales natures d'actifs pour lesquelles des méthodes de valorisation alternatives à l'utilisation directe ou indirecte d'un marché actif sont utilisées.

MAIF VIE n'a pas identifié d'autre information significative à mentionner dans ce paragraphe en dehors de ce qui est décrit ci-dessous.


### D.1 Actifs

Les différents postes du bilan actif de MAIF VIE sont présentés et détaillés dans cette section.


#### D.1.1 Actifs incorporels

Ce poste comprend notamment, en normes comptables françaises, des progiciels et des immobilisations incorporelles liées à des livraisons à soi-même de solutions informatiques qui sont amortis linéairement sur une durée de 3 à 8 ans.

En normes Solvabilité 2, faute de marché actif permettant de déterminer leur juste valeur, ces actifs ont une valeur nulle.


#### D.1.2 Immobilisations corporelles pour usage propre

Ce poste regroupe l'ensemble du mobilier, matériel et les aménagements s'y rapportant.

Les actifs corporels d'exploitation sont inscrits, en normes comptables françaises, à l'actif pour leur coût d'acquisition, sous déduction, le cas échéant, d'une valeur résiduelle à la condition qu'elle soit mesurable de manière fiable et significative et font l'objet d'amortissements annuels selon les modalités suivantes :

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="69 / 111" -->

› Agencements, aménagements, installations :

› Matériel de transport :

Linéaire sur 5 ou 10 ans

Linéaire sur 5 ans

› Matériel informatique et bureautique : Linéaire sur 3 ou 4 ans Linéaire sur 5 ou 10 ans

› Mobilier et matériel de bureau :

› Autres matériels : Linéaire sur 5 ans

En normes Solvabilité 2, la juste valeur retenue pour ces actifs autres que les agencements et aménagements est égale à leur valeur nette comptable établie pour les comptes en normes comptables françaises.

En l'absence de marché permettant de déterminer leur juste valeur, les agencements et aménagements ont une valeur nulle.


## D.1.3 Placements

Ce paragraphe concerne les placements autres que les actifs en représentation de contrats en unités de compte ou indexés.

La grande majorité des titres détenus par MAIF VIE est valorisée par référence au prix de transactions sur un marché actif pour le même actif ou pour des actifs similaires. Compte tenu de la qualité de son portefeuille d'instruments financiers, MAIF VIE considère que la valorisation de l'essentiel de ses placements ne présente ainsi pas d'incertitudes significatives.

La juste valeur des instruments financiers négociés sur des marchés actifs correspond au prix de transactions constatées sur ce marché, lorsqu'ils sont disponibles. Le marché est considéré comme actif si des cours présentant une dispersion limitée peuvent être obtenus aisément et régulièrement auprès d'une bourse, d'un courtier, d'un service de cotation ou d'autres sources externes et qu'ils reflètent des transactions réelles. Comme le nombre et le volume de transactions peuvent parfois difficilement être appréciés, MAIF VIE prend également en compte la fréquence d'alimentation des cotations comme critère d'activité du marché. En pratique, le dernier cours coté est retenu.

En l'absence de transactions constatées sur un marché actif, l'évaluation est confiée à des prestataires externes et indépendants ou repose sur des techniques de valorisation. Ces techniques intègrent une part importante de jugement, liée au choix des méthodes de valorisation, des hypothèses retenues, des paramètres et des données utilisés. Elles s'appuient autant que possible sur des données de marché observables. Pour les participations, des méthodes de valorisation spécifiques sont prévues dans les textes d'application de la directive Solvabilité 2. Ces méthodes sont précisées ci-après.


## D.1.3.1 Participations

Les participations correspondent aux entreprises liées à MAIF VIE dans lesquelles MAIF VIE ou le groupe MAIF détient au moins 20 % du capital.

MAIF VIE ne détient pas de titres d'entreprises liées qui sont cotés sur un marché actif ou à valoriser à zéro.

Les principales participations de MAIF VIE sont des fonds d'investissements valorisés au dernier prix de rachat calculé et publié par la société de gestion.

En normes comptables françaises, les participations sont comptabilisées sur la base du coût historique, diminué d'éventuelles dépréciations.


## D.1.3.2 Actions non cotées

Sont classées dans la rubrique « actions non cotées >> les parts des sociétés civiles immobilières qui sont valorisées au dernier prix de rachat calculé et publié par la société de gestion.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

70 / 111

En normes comptables françaises, les parts de SCI sont comptabilisées sur la base du coût historique, diminué d'éventuelles dépréciations.

Les actions sont valorisées selon la méthode retenue dans les comptes annuels publiés en normes comptables françaises correspondant au montant pour lequel elles pourraient être échangées dans le cadre d'une transaction conclue, dans des conditions de concurrence normales, entre des parties informées et consentantes.

Ainsi, ces actions sont valorisées selon la méthode retenue en normes comptables françaises.


## D.1.3.3 Obligations

Les obligations d'Etat regroupent les obligations et titres de créances négociables émis ou garantis par :

› des autorités publiques ;

› des administrations centrales ;

› des institutions publiques supranationales ;

› des administrations régionales ou municipales.

Les obligations de sociétés sont constituées des obligations et titres de créances négociables ainsi que des titres participatifs et subordonnés émis par des entreprises.

Les obligations sont valorisées à la juste valeur dans le bilan en normes Solvabilité 2. Toutefois, pour un nombre limité d'obligations ou autres titres de sociétés, le prix de revient est conservé.

En normes comptables françaises, les obligations sont comptabilisées sur la base du prix d'achat, ajusté des surcotes et décotes (différence entre le coût d'acquisition et le prix de remboursement) amorties sur la durée du titre et d'éventuelles dépréciations en cas de difficultés financières de l'émetteur.


## D.1.3.4 Fonds d'investissement

Les fonds d'investissement détenus à moins de 20 % par le groupe regroupent l'ensemble des organismes dont l'objectif est le placement collectif en valeurs mobilières et/ou dans d'autres actifs.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

71/ 111

Le graphique ci-dessous présente la répartition de la valeur des parts de fonds acquises par MAIF VIE selon la nature des actifs détenus par les fonds :

Répartition des fonds d'investissements détenus en fonction de la nature des actifs détenus par les fonds - 2022

<figure>

![](figures/13)

<!-- FigureContent="Fonds Monétaires 6% Fonds Actions 34% Fonds Obligataires 35% Fonds Immobiliers 25%" -->

</figure>


m Fonds Actions Fonds Immobiliers Fonds Obligataires Fonds Monétaires

La valeur des fonds retenue dans le bilan en normes Solvabilité 2 correspond au dernier prix de rachat calculé et publié par la société de gestion ou aux valeurs d'expertise pour les sociétés immobilières.

En normes comptables françaises, les parts de fonds sont comptabilisées au prix d'achat, ajusté des dépréciations éventuelles.


## D.1.3.5 Dépôts autres que ceux assimilables à de la trésorerie

Il s'agit des dépôts non transferables c'est-à-dire des dépôts non utilisables à tout moment pour effectuer des paiements, non échangeables contre des liquidités et non transferables sans restriction ou pénalité significative.

MAIF VIE classe sur cette ligne les dépôts et comptes à terme et les valorise en normes Solvabilité 2 comme en normes comptables françaises, à leur prix de revient, compte tenu de leur montant non significatif.


### D.1.4 Placements en représentation de contrats en unités de compte ou indexés

Cette ligne correspond aux placements représentant les provisions techniques afférentes aux compartiments en UC des produits d'assurance vie multisupports.

Elle est composée de fonds d'investissement valorisés au dernier prix de rachat calculé et publié par la société de gestion comme dans les comptes en normes comptables françaises.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="72 / 111" -->


## D.1.5 Avances sur polices

Cette ligne correspond aux prêts accordés aux assurés et garantis par les contrats.

Le montant des avances sur polices en normes comptables françaises est comptabilisé sur la base des montants prêtés et intérêts à recevoir.

En normes Solvabilité 2, le montant des avances est identique à celui en normes comptables françaises, compte tenu de leur montant non significatif.


## D.1.6 Provisions techniques cédées

Le montant des provisions techniques cédées aux réassureurs en normes comptables françaises est comptabilisé sur la base des résultats de l'année et correspond aux données communiquées aux réassureurs.

En normes Solvabilité 2, le montant des provisions techniques cédées correspond à la meilleure estimation des provisions techniques de réassurance, ajustée du risque de défaut des contreparties. Les précisions sur leur détermination sont apportées dans le paragraphe consacré aux provisions techniques.


### D.1.7 Autres actifs

Les autres actifs comprennent les créances nées d'opérations d'assurance directe, les créances nées d'opérations de réassurance, les autres créances (hors assurance) et la trésorerie. Ces postes sont enregistrés à leur valeur nominale en normes comptables françaises.

En normes Solvabilité 2, la juste valeur retenue pour ces actifs est égale à leur valeur nette comptable établie pour les comptes en normes comptables françaises.

MAIF VIE considère en effet que la différence entre le prix de revient et la valeur économique n'est pas significative pour ces actifs.


## D.2 Provisions techniques

Les provisions techniques constatent au passif du bilan les engagements de MAIF VIE vis-à-vis de ses assurés ou des bénéficiaires des contrats d'assurance.


### D.2.1 Généralités


#### D.2.1.1 Principes de valorisation des provisions techniques

En normes comptables françaises, l'évaluation des provisions techniques suit une méthodologie qui a pour objectif de respecter l'article R 343-3 du Code des assurances.

En normes Solvabilité 2, l'évaluation des provisions techniques suit les principes définis par la règlementation Solvabilité 2 pour l'évaluation des provisions techniques qui diffèrent en fonction de la nature réplicable ou non des passifs d'assurance. Est considéré comme réplicable, tout contrat d'assurance dont les flux peuvent être parfaitement répliques par un portefeuille d'instruments financiers, à condition que le marché soit suffisamment actif, liquide et transparent.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

73 / 111

Les passifs d'assurance de MAIF VIE étant non réplicables, les provisions techniques brutes sont égales à la somme d'une provision de meilleure estimation dite « Best Estimate >> et d'une marge de risque :

› la provision de meilleure estimation correspond à la meilleure estimation de la valeur actualisée probable des flux futurs de trésorerie estimés de façon la plus exacte possible et projetés jusqu'à l'extinction des contrats. L'actualisation des flux consiste à déterminer la valeur d'aujourd'hui de flux qui se produiront dans le futur.

› la marge de risque est calculée de manière à garantir que la valeur des provisions techniques soit équivalente au montant dont les entreprises d'assurance et de reassurance auraient besoin pour reprendre et honorer les engagements de la compagnie. Elle est calculée en déterminant le coût que représente la mobilisation d'un montant de fonds propres éligibles égal au SCR nécessaire pour faire face aux engagements d'assurance et de réassurance sur toute leur durée.


## D.2.1.2 Mesures transitoires

MAIF VIE a fait le choix de ne pas appliquer de mesures transitoires pour le calcul des provisions techniques.


## D.2.1.3 Courbe de taux

Les flux futurs sont actualises avec la courbe des taux sans risque fournie par l'Autorité Européenne des Assurances et des Pensions Professionnelles (EIOPA, selon son sigle anglais).


## Courbes des taux

<figure>

![](figures/14)

<!-- FigureContent="4.0% 3.5% 3.0% 2.5% 2.0% 1.5% 1.0% 0.5% 0.0% 1. 5 9 13 17 21 25 29 33 37 41 45 49 53 57 61 65 69 73 77 81 85 89 93 97 101 105 109 113 117 121 125 129 133 -0.5% -1.0% · · · · courbe des taux 2021 (avec VA) courbe des taux 2022 (avec VA)" -->

</figure>



### D.2.1.4 Classification

Les provisions techniques sont regroupées dans le bilan en normes Solvabilité 2 en cinq catégories.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

74 / 111

<figure>

![](figures/15)

<!-- FigureContent="Non-vie Vie hors UC Non-vie hors santé Vie hors santé Unités de comptes Santé similaire à la non-vie Santé similaire à la vie" -->

</figure>


Elles sont calculées au niveau des lignes d'activités (« lines of business > - LoB) ou au niveau de portefeuilles homogènes de risques - par contrat/garantie - au sein d'une même ligne d'activité.


## D.2.2 Provisions techniques vie


### D.2.2.1 Périmètre et outils

D.2.2.1.1

Périmètre produits

› Classification par ligne d'activité

En normes Solvabilité 2, la maille retenue pour compléter les états réglementaires est la «ligne d'activité». Conformément à la segmentation imposée par les normes Solvabilité 2, les engagements de MAIF VIE sont affectés aux lignes d'activité selon le tableau présenté en § A1.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

75 / 111

| Catégorie de bilan | Produits | (LOB)Ligne d'activité |
| - | - | - |
| Provisions techniques vie (hors santé, UC ou indexés) | Assurance vie monosupport | Contrats d'assurance vie avec participation aux bénéfices |
|| - Cap'Epargne, Cap'Etudes - Nouveau Cap (y compris PEP), Capiplan 
|| || Assurance vie multisupports Compartiment euros - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DSCap | Contrats d'assurance vie avec participation aux bénéfices |
|| Épargne retraite - PER Responsable et Solidaire Compartiment euros | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance retraite - IFC | Contrats d'assurance vie avec participation aux bénéfices |
|| Obsèques vie entière - Sollicitudes | Contrats d'assurance vie avec participation aux bénéfices |
|| Temporaire décès - Rassurcap - Rassurcap Solutions - Assurance Prévoyance Professionnelle | Autres activités d'assurance vie |
|| Assurance en couverture de prêts - Assurance Emprunteur MAIF, Avantage Emprunteur - Autres Prêts immobiliers - Prêts à la consommation | Autres activités d'assurance vie |
|| Rentes | Contrats d'assurance vie avec participation aux bénéfices |
| Provisions| techniques UC Compartiments UC ou indexés  Assurance vie multisupports - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DSCap | Assurance indexée ou en unités de comptes (UC) |
|| Épargne retraite - PER Responsable et Solidaire Compartiments UC | Assurance indexée ou en unités de comptes (UC) |

L'assignation d'un engagement d'assurance à une ligne d'activité reflète la nature des risques relatifs à cet engagement. Un produit est susceptible de couvrir plusieurs natures de risques et peut ainsi relever de plusieurs lignes d'activité. La forme juridique de l'engagement n'est pas nécessairement déterminante pour caractériser la nature du risque.


## › Affaires nouvelles et primes futures

La valorisation des actifs et des passifs de l'entreprise doit être réalisée sous l'hypothèse d'une continuité d'exploitation, conformément à l'article 7 du Règlement délégué (UE) 2015/35.

Au regard de la frontière des contrats, aucune prime future pouvant provenir d'affaires nouvelles ou contrats en cours n'est intégrée aux calculs, sauf pour les produits suivants :


## › Obsèques vie entière

Le paiement sous forme de primes temporaires est contractuellement prévu sous peine de réduction du contrat.


## › Temporaire décès

Les primes dues au titre de l'année en cours doivent être prises en considération en cohérence avec le caractère annuel des produits Rassurcap et Rassurcap Solutions qui implique que les engagements de MAIF VIE cessent la veille de la prochaine échéance annuelle de chaque contrat. Les adhésions Rassurcap et Rassurcap Solutions à échéance en janvier seront prises en compte dans la meilleure estimation pour une année supplémentaire dans la limite du taux de reconduction observé.


## › Assurance en couverture de prêts

Les cotisations futures sont prises en compte, conformément aux conditions contractuelles et notamment aux caractéristiques de l'amortissement du prêt correspondant.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="76 / 111" -->


## D.2.2.1.2 Outils de modélisation utilisés

›


### Diffusion des trajectoires économiques

L'outil édité par Moody's Analytics est utilisé pour générer les trajectoires économiques en univers risque neutre. Cet éditeur est reconnu et est un leader du marché. Cet outil est également utilisé pour la génération des trajectoires économiques en univers réel utilisées dans le cadre des travaux actif passif et d'allocation stratégique d'actifs.

Cet outil permet le paramétrage et la diffusion de trajectoires économiques nécessaires à la projection de toutes les classes génériques d'actifs définies par MAIF VIE.

Moody's Analytics met à disposition des utilisateurs les éléments nécessaires à la bonne compréhension des modèles et la bonne utilisation de l'outil, à savoir :

› documentation de l'outil ;

› documentation des modèles utilisés dans l'outil ;

› fichiers de calibrage standard ;

› outils de (re)calibrage.

Par ailleurs, l'outil réalise en sortie des tests nécessaires à la validation des trajectoires risque neutre générées (test de martingalité notamment).

› Projection des flux de trésorerie

Des outils de modélisation spécifiques aux calculs Solvabilité 2 ont été développés pour répondre aux calculs règlementaires de la meilleure estimation des provisions techniques et des SCR .

Les évaluations de la meilleure estimation des provisions techniques des produits de prévoyance sont réalisées selon une approche déterministe et celles des produits d'épargne euro selon une approche stochastique.

En raison de l'absence de coûts et d'options (notamment liées à la non-modélisation des arbitrages), les évaluations de la meilleure estimation des provisions techniques des produits d'épargne en unités de compte sont réalisées selon une approche déterministe.


## › Modélisation des actions futures du management (produits d'épargne en euros)

Certaines actions futures du management sont implémentées dans les outils de modélisation Solvabilité 2 pour les produits d'épargne en euros. Il s'agit des hypothèses suivantes :

› la decision des taux a servir en fin d'année sur les compartiments euros des contrats d'assurance vie est implémentée dans les outils de projection comme une action du management : le taux de rémunération des contrats d'assurance vie à servir correspond à la stratégie de rémunération détaillée ci-après ;

› la gestion d'actifs ou stratégie financière (retranscription de la politique annuelle de gestion des risques d'investissements) définit les règles appliquées sur les actifs en cas d'investissement ou de désinvestissement. Elle précise notamment :

o l'Allocation Stratégique d'Actifs cible (ASA) : cette cible est figée sur la base d'études actif-passif (réalisées par la MAIF), validee par le Conseil d'administration de MAIF VIE et utilisée pour toutes les simulations stochastiques;

o la manière d'atteindre l'ASA.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" --> :unselected: :unselected:

> VALORISATION À DES FINS DE SOLVABILITÉ
===

77 / 111

› Modélisation de la réassurance

La réassurance est modélisée sur la base du programme de réassurance en place.

›


## Determination de la meilleure estimation des provisions techniques

Le montant de la meilleure estimation des provisions techniques est déterminé par la somme du montant de la meilleure estimation des provisions techniques issu de l'outil de projection des flux de trésorerie, de 85 % des plus-values latentes résiduelles actualisées issues de l'outil de projection des flux de trésorerie et du montant des provisions techniques complémentaires constaté en normes comptables françaises (provisions pour sinistres à payer réelles, provision pour sinistres en cours, ... ) ainsi qu'éventuellement du montant de l'écart de convergence. Ce montant est diminué du montant de provision pour participation aux bénéfices mis en fonds excédentaires conformément à l'arrêté du 24 décembre 2019 (application de la méthode 1 proposée par l'ACPR). Il est à noter une évolution quant au choix de la méthode de détermination du montant de provision pour participation aux bénéfices mis en fonds excédentaires, la méthode 2 étant celle appliquée auparavant.


### D.2.2.2 Données, hypothèses et paramètres

D.2.2.2.1 Données

› Données contrats

Pour les besoins de l'inventaire, les données contrats sont mises à jour en date de clôture et contrôlées dans les bases du suivi du portefeuille vie. Ces données sont utilisées pour les calculs sous forme de models-points (agrégation par regroupement de contrats homogènes, c'est-à-dire présentant des caractéristiques identiques).


## › Données de placement

Les données du portefeuille d'actifs (type d'actif, valeur comptable, valeur de marché, coupon couru, ... ) en date d'inventaire sont utilisées pour les calculs des produits d'épargne. Les actifs sont regroupés en 3 catégories :

› obligations (souveraines, d'entreprise, sécurisées) ;

› actions et assimilés : fonds ou sociétés de capital investissement, fonds actions et d'obligations convertibles, fonds obligataires, fonds immobiliers et d'infrastructures ; › trésorerie : fonds monétaires, ...

Les obligations sont prises en compte ligne à ligne et les autres actifs sont agrégés en différentes sous-catégories d'actifs.

Les nominaux et coupons des obligations sont recalculés selon l'approche dite de neutralisation du risque de crédit avec comme références la courbe de taux d'actualisation et les valeurs de marché.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

78 / 111

D.2.2.2.2 Hypothèses


## › Hypothèses financières


### › Actualisation

MAIF VIE a retenu la courbe de taux publiée par EIOPA au 31 décembre 2022, conforme aux principes des actes délégués de la directive Solvabilité 2 (articles 43 à 48) et avec application de la correction pour volatilité visée à l'article 77 quinquies de la directive 2009/138/CE.

Dans la mesure où les contrats de MAIF VIE relèvent de la réglementation française et sont vendus en France, seule la courbe en monnaie euros est utilisée.


#### › Scénarii économiques

Les hypothèses retenues sont fournies par Moody's Analytics en calibrage standard réalisé sur la base d'une courbe de taux swap de marché. Un nouveau calibrage est effectué pour les taux nominaux afin de prendre en compte la courbe officielle fournie par l'EIOPA avec et sans correction pour volatilité et sur les taux réels afin de calibrer sur les OATi françaises (données Reuters).


##### › Stratégie financière

La stratégie financière utilisée pour les calculs stochastiques sur les produits d'épargne est fondée sur l'Allocation Stratégique d'Actifs (ASA) validée par le conseil d'administration de MAIF VIE en fin d'année.

Elle définit également les règles appliquées sur les actifs en cas d'investissement ou de désinvestissement et précise notamment les ordres de priorité de cessions par catégorie de titres.


## › Hypothèses comportementales


### › Table de mortalité

Les tables de mortalité utilisées pour traduire la mortalité réelle observée sur le portefeuille sont les tables d'expérience MAIF VIE et les tables réglementaires.


### › Table de chute

Les rachats structurels concernent les produits d'épargne, le produit retraite PER et les contrats de prévoyance obsèques et assurance en couverture de prêt.

Les hypothèses retenues correspondent à l'observation des portefeuilles en date d'inventaire de l'exercice précédent. Ces rachats sont appliqués en pourcentage de la provision mathématique en épargne (rachats partiels) et en pourcentage des adhésions en stock en épargne (rachats totaux) et prévoyance.


## › Table de rachats conjoncturels

Les rachats conjoncturels ne concernent que les produits d'épargne en euros (monosupports et compartiment euros des multisupports) et sont liés à la concurrence sur le marché de l'assurance vie.


## › Tables d'incapacité et d'invalidité

Les tables d'incapacité et d'invalidité utilisées pour traduire l'entrée et le maintien en incapacité et en invalidité réelle observés sur le portefeuille sont des tables d'expérience fournies par des prestataires externes.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> VALORISATION À DES FINS DE SOLVABILITÉ
===

79 / 111

› Taux de turn-over

Le taux de turn-over utilisé pour les engagements sociaux est basé sur des données réelles communiquées par chaque employeur.

› Hypothèses de gestion

Les hypothèses ci-dessous correspondent uniquement à celles utilisées dans le cadre du calcul des provisions techniques.

› Stratégie de rémunération

La stratégie de rémunération concerne les produits « Epargne euros »> et le produit « obsèques vie entière » (Sollicitudes).

Pour l'épargne euros, elle a pour objectif d'optimiser l'utilisation de la production financière nette des frais financiers et tenant compte des variations de la PRE, de la PPB et de la richesse de l'assureur sous différentes contraintes :

o réglementaires : Participation aux Bénéfices minimum, Ecoulement de la PPB en 8 ans ;

o :unselected: contractuelles : Taux technique, Taux Minimum Garanti ;
 :unselected:
o de revalorisation : définition d'un taux cible, qui est fonction du taux de la concurrence.

La contrainte de TMG s'applique uniquement sur la première année de projection, puisque l'engagement contractuel de MAIF VIE au 31/12/22 ne porte que sur l'année suivante (2023).

› Charges de gestion

Les données utilisées relatives aux coûts administratifs, financiers et de sinistres reposent sur :

o le calcul moyen sur trois exercices des coûts ABC (Activity Based Costing = coûts analytiques basés sur la cartographie d'activités de l'entreprise à la date d'inventaire de l'exercice) exprimés par famille de produits hors commissions de gestion sur l'assurance en couverture de prêts et hors coûts des projets exceptionnels significatifs et déclinés en charges fixes / charges variables et par destination (acquisition, administration des contrats, gestion des sinistres, gestion des placements et autres charges techniques) ;

o et sur une masse complémentaire de frais de gestion correspondant aux coûts exceptionnels de projets significatifs non compris dans le calcul des coûts de base ABC (cycles d'investissement SI), exprimés par famille (épargne, prévoyance, obsèques, assurance en couverture de prêts et engagements sociaux et autres).


## D.2.2.2.3 Paramètres

Les paramètres concernent des éléments structurels des produits (tarification, ... ) ou de contexte (impôts, ... ). Ils sont distingués des hypothèses dans la mesure où leur utilisation pour les calculs ne génère pas de risque de modélisation (application d'une règle contractuelle, juridique, réglementaire, ... ).


## Tarification ›

Les ressources de gestion prises en compte dans la modélisation correspondent aux frais sur versements des cotisations contractuellement prévues (Sollicitudes, Assurance Emprunteur MAIF, Rassurcap et Rassurcap Solutions) ainsi qu'aux chargements effectués sur les provisions

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" --> :unselected:

> VALORISATION À DES FINS DE SOLVABILITÉ
===

80 / 111

mathématiques des contrats en cours. Les valeurs retenues sont celles utilisées dans la tarification en vigueur à la date d'inventaire sans aucune simplification.


## › Commissions et frais de distribution et de gestion

Seules les commissions relatives aux cotisations contractuellement prévues ainsi que celles correspondant au commissionnement sur les frais sur épargne gérée sont à considérer.

Les modes et les niveaux de commissionnement sont ceux en vigueur à la date d'inventaire.


## › Impôts et prélèvements sociaux

Le taux d'impôt sur les sociétés retenu pour le calcul de la réserve de capitalisation tient compte des évolutions attendues.

Le taux de prélèvements sociaux est fixé à 17,2 %, conformément à la réglementation en vigueur au 31 décembre 2022 mais sans tenir compte des spécificités type épargne handicap et domiciliation à l'étranger.


### D.2.2.2.4 Informations sur les niveaux d'incertitude

L'incertitude la plus élevée se concentre sur la ligne d'activité «contrats d'assurance vie avec participation aux bénéfices».


### D.2.2.3 Valeur des provisions techniques

D.2.2.3.1 Résultats au 31 décembre 2022

Une ventilation exprimée en milliers d'euros par ligne d'activité de la meilleure estimation des provisions techniques vie brutes de réassurance montre une prépondérance des contrats d'assurance vie avec participation aux bénéfices à hauteur de 92 %.

| Lignes d'activité (LoB) | Meilleure estimation des provisions techniques vie brute 31/12/2022 | Meilleure estimation des provisions techniques vie cédées 31/12/2022 |
| - | - | - |
| Contrats d'assurance vie avec participation aux bénéfices | 9 187 163 | |
| Assurance indexée ou en UC | 766 056 | 56 845 |
| Autres activités d'assurance vie | -20 455 | |
| TOTAL PROVISIONS TECHNIQUES | 9 932 764 | 56 845 |

L'annulation des effets de la correction pour volatilité représente un impact de + 45 084 milliers d'euros sur la meilleure estimation brute de reassurance.

| 31/12/2022 | Meilleure estimation des provisions techniques vie brute | Meilleure estimation des provisions techniques vie cédées |
| - | - | - |
| Valeur retenue | 9 932 764 | 56 845 |
| Valeur sans l'effet de correction pour volatilité | 9 977 848 | 59 124 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="81 / 111" -->

D.2.2.3.2 Focus entre la meilleure estimation des provisions et les provisions en normes comptables françaises

La meilleure estimation, égale à la valeur actuelle probable des potentiels flux futurs de trésorerie, ne doit comprendre que les flux de trésorerie futurs associés aux contrats d'assurance en vigueur à la date d'évaluation, sous l'hypothèse d'une continuité d'exploitation (run-off avec maintien d'activité). La projection de flux de trésorerie intervenant dans ce calcul doit tenir compte de toutes les entrées (ressources) et sorties (prestations et dépenses) de fonds nécessaires pour honorer les engagements d'assurance sur leur durée de vie.

La meilleure estimation, évaluée en brut de réassurance, doit reposer sur des informations actuelles et crédibles et à partir d'hypothèses réalistes et spécifiques à l'entreprise en faisant appel à des méthodes actuarielles et techniques statistiques adéquates.

Compte tenu des éléments précédents, on observe de nombreuses différences dans les hypothèses et méthodes utilisées entre les états financiers et les évaluations à des fins de solvabilité, comme le montre le tableau ci-dessous :

| Hypothèses principales || Normes Solvabilité 2 | Normes comptables françaises |
| Actif | Trajectoires économiques | Barrie-Hibbert | Sans objet |
| - | - | - | - |
| | Stratégie financière | Allocation Stratégique d'actifs (ASA) | Sans objet |
| Passif | Mortalité | Table d'expérience ou table réglementaire | Table d'expérience certifiée ou table réglementaire |
| | Rachats structurels | Table d'expérience | Sans objet |
| | Rachats conjoncturels | Formule ONC (épargne uniquement) | Sans objet |
| Dépenses | Frais | Coûts par activités (approche ABC) | Coûts (approche Provision Globale de Gestion) |
| Actualisation | Taux | Courbe de taux EIOPA avec correction pour volatilité | Taux technique ou courbes de taux (Provision Globale de Gestion et Provisions pour Aléas Financiers) |


## D.2.3 Provisions techniques non-vie

MAIF VIE n'est pas concernée par les provisions techniques non-vie.


### D.2.4 Marge de risque


#### D.2.4.1 Méthodologie

La marge de risque est une composante des provisions techniques qui complète la meilleure estimation des provisions.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="82 / 111" -->

La marge de risque doit être calculée en determinant le coût d'immobilisation d'un montant de fonds propres nécessaires pour atteindre le SCR afin d'honorer les engagements d'assurance et de réassurance sur leur durée de vie.

Le calcul de la marge de risque repose en effet sur un scénario de transfert du portefeuille vers une autre société d'assurance qui, n'ayant aucun fond propre avant le transfert, doit lever les fonds propres nécessaires pour couvrir le SCR.

Pour des raisons de proportionnalité, MAIF VIE a fait le choix de retenir une méthode simplifiée pour le calcul de la marge de risque. Cette méthode consiste à déterminer les SCR futurs proportionnellement aux meilleures estimations des provisions futures.


## D.2.4.2 Valeur de la marge de risque

D.2.4.2.1 Résultat au 31 décembre 2022

Le montant de la marge de risque par ligne d'activités au 31 décembre 2022 exprimé en milliers d'euros est décomposé ci-dessous :

| Lignes d'activité (LoB) | Marge de risque |
| - | - |
| Contrats d'assurance vie avec participation aux bénéfices | 272 163 |
| Assurance indexée ou en UC | 11 050 |
| Autres activités d'assurance vie | 12 177 |
| TOTAL MARGE DE RISQUE | 295 391 |


## D.2.5 Niveau de confiance sur le niveau des provisions techniques

Des analyses de mouvements ont été effectuées de manière à expliquer les variations de la meilleure estimation des provisions techniques entre la clôture de l'année 2021 et celle de l'année 2022, ainsi qu'entre les provisions techniques calculées en normes comptables françaises et en normes Solvabilité 2.

L'ensemble des écarts peut être justifié. Dans le cadre de l'analyse expliquant le passage des provisions techniques normes comptables françaises aux provisions techniques en normes Solvabilité 2 sur le périmètre épargne, un écart résiduel subsiste. Celui-ci s'explique principalement par l'écart de convergence du modèle et est jugé non significatif : il est de l'ordre de 0,08 % de la meilleure estimation des provisions techniques au 31 décembre 2022.


## D.3 Autres passifs

Les principaux postes du passif, en dehors des provisions techniques, sont décrits ci-après :

› provisions, autres que les provisions techniques ;

› impôts différés passifs ;

› dettes autres que les dettes subordonnées ;

› dettes subordonnées incluses dans les fonds propres de base.


### D.3.1 Provisions autres que les provisions techniques

La provision pour risques et charges correspond à l'estimation des coûts liés à la conclusion d'un accord de gestion des fins de carrière au sein de l'entreprise, à l'indemnité de fin de

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="83 / 111" -->

contrat due aux titulaires de Contrats à Durée Déterminée au terme de ceux-ci et à une provision pour contentieux. En normes Solvabilité 2, l'évaluation est identique à celle comptabilisée en normes comptables françaises.


## D.3.2 Impôts différés

En normes comptables françaises, seuls les impôts exigibles sont comptabilisés dans les comptes sociaux. L'impôt exigible est le montant des impôts sur le bénéfice payable (ou récupérable) au titre du bénéfice imposable (ou perte fiscale) d'un exercice.

En normes Solvabilité 2, les entreprises d'assurance valorisent et comptabilisent des impôts différés actifs et passifs qui répondent aux définitions et principes des normes comptables internationales, notamment :

› les impôts differes sont les montants d'impôts payables ou récupérables au cours de périodes futures afférents :

o sauf exceptions, aux différences temporelles (déductibles et imposables) entre la valeur des actifs et passifs au bilan et leur base fiscale ;

o le cas échéant, au report en avant de pertes fiscales non utilisées et de crédits d'impôts non utilisés.

› les actifs d'impôts differes au titre des différences temporelles déductibles ne sont comptabilisés que s'il est probable qu'ils puissent être imputés sur des bénéfices imposables futurs.

› les passifs (actifs) d'impôts differes sont évalués au montant que l'on s'attend à payer aux (recouvrer auprès des) administrations fiscales en utilisant les taux d'impôt qui ont été adoptés ou quasi-adoptés à la date de clôture. Ils ne font pas l'objet d'une actualisation.

MAIF VIE retient, pour les impôts différés passifs ou actifs, la valeur égale à la somme :

› des impôts différés issus de la fiscalité différée (décalages temporaires entre résultat fiscal et résultat comptable) et latente (taxation éventuelle) ;

› des impôts différés liés à la revalorisation des actifs et des passifs. La valorisation des impôts differes au bilan a été réalisée poste à poste, en appliquant un taux d'imposition de 25,83 % à l'écart entre la valeur en normes comptables françaises et la valeur en normes Solvabilité 2. Toutefois, un taux d'imposition de 15 % a été appliqué pour certains actifs détenus à long terme.

Le montant inscrit au bilan correspond au solde entre les impôts différés passifs et les impôts différés actifs compensables.

Le tableau suivant détaille les impôts différés générés, d'une part par la différence d'évaluation des actifs et passifs entre le bilan en normes Solvabilité 2 et le bilan en normes comptables françaises et, d'autre part, entre l'évaluation des actifs et des passifs entre le bilan en normes françaises et le bilan fiscal.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" --> :unselected:

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="84 / 111" -->

| | 2022 |||
|| Différence d'évaluation | IDA | IDP |
| - | - | - | - |
| Différence d'évaluation des placements (Bilan S2 - Bilan Normes Françaises) | -862 777 | 224 095 | |
| Différence d'évaluation des autres actifs (Bilan S2 - Bilan Normes Françaises) | -49 197 | 12 707 | 0 |
| Actifs incorporels | -18 463 | 4 769 | |
| Immobilisations corporelles pour usage propre | -2 594 | 670 | |
| Provisions techniques cédées | -28 139 | 7 268 | |
| Autres actifs | | | |
| Différence d'évaluation des provisions techniques (Bilan S2 - Bilan Normes Françaises) | -981 156 | | 253 433 |
| Différence d'évaluation des autres postes du passif (Bilan S2 - Bilan Normes Françaises) | 2 307 | 676 | 80 |
| Provisions autres que les provisions techniques | | | |
| Provisions pour retraite et autres avantages | -312 | | 80 |
| Autres passifs | 2 619 | 676 | |
| Total des différences entre le Bilan S2 et le Bilan normes françaises Différence d'évaluation entre le Bilan normes françaises et le Bilan fiscal | -1 890 822 | 237 478 | 253 513 |
|| 8 240 | 2 128 | |
| Total des Impôts Différés | | 239 606 | 253 513 |
| Impôts Différés - Situation Nette | | | 13 907 |


## D.3.3 Dettes autres que les dettes subordonnées

Les autres dettes recouvrent les dettes nées d'opérations d'assurance directe, les dettes nées d'opérations de réassurance, les dettes envers des établissements de crédit, les dettes sociales et fiscales et les autres dettes (charges à payer comptabilisées au profit des sociétés du groupe dans le cadre des accords de distribution et de prestations intra-groupe). Ces postes comptables sont enregistrés, en normes comptables françaises, à leur valeur nominale.

En normes Solvabilité 2, ces dettes sont évaluées à leur valeur comptable sans retraitement, MAIF VIE considérant que cette valeur ne présente pas de différences significatives avec une évaluation économique.


## D.3.4 Dettes subordonnées incluses dans les fonds propres de base

Lors de ses réunions des 7 octobre et 14 décembre 2004, le conseil d'administration de MAIF VIE, conformément aux orientations prises pour le renforcement des fonds propres de la société, a décidé une émission de titres subordonnés remboursables (TSR) pour un montant nominal total de 34 500 milliers d'euros, représenté par 345 000 titres de 100 euros de nominal.

L'emprunt a été émis le 12 janvier 2005, date correspondant également à la souscription et au règlement des titres.

Les titres subordonnés remboursables ont été émis au nominal.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

85 / 111

Les droits des titulaires sont représentés par une inscription en compte à leur nom dans le registre de l'émetteur.

Les titres sont amortissables in fine et seront amortis en totalité le 11 janvier 2025 par remboursement au pair.

Au 31 décembre 2022, l'endettement s'élève à 34 500 milliers d'euros. Les dettes subordonnées sont incluses, en normes comptables françaises, sur la ligne « passifs subordonnés >> proche des fonds propres.

En normes Solvabilité 2, la valorisation est réalisée à la hauteur du montant nominal des titres.


## D.4 Méthodes de valorisation alternatives

Les méthodes de valorisation alternatives correspondent à des méthodes autres que celles qui utilisent uniquement des prix cotes sur un marché actif, pour des actifs ou des passifs identiques ou analogues. Elles ne concernent pas les provisions techniques.

Les méthodes et modalités de valorisation retenues par MAIF VIE sont précisées dans les paragraphes précédents consacrés à chaque catégorie d'actifs et d'autres passifs.

Compte tenu de la structure de son portefeuille, MAIF VIE a recours à des méthodes alternatives de valorisation pour une part limitée de ses actifs à l'image des fonds d'investissement « non cotés >> pour lesquels la valeur liquidative publiée par la société de gestion n'est pas issue de cotations sur un marché réglementé.


## D.5 Autres informations

MAIF VIE n'identifie pas d'autres informations importantes à communiquer sur ces méthodes, compte tenu du caractère non significatif de son exposition.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->
<figure>

![](figures/16)

<!-- FigureContent="E" -->

</figure>


GESTION DU CAPITAL

> GESTION DU CAPITAL
===

<!-- PageHeader="87/ 111" -->


## E Gestion du capital

Ce paragraphe est consacré à la présentation des informations qualitatives et quantitatives relatives :

› aux fonds propres et à leur gestion ;

› aux exigences de capital et à leur couverture.

Les fonds propres en normes Solvabilité 2 sont :

› essentiellement constitues par l'excédent des actifs sur les passifs du Bilan en normes Solvabilité 2 (presente dans la partie D) incluant les fonds propres en normes comptables françaises ;

› classés par niveau de qualité en fonction de leur capacité à absorber des pertes.

MAIF VIE dispose principalement de fonds propres de base de niveau 1 (les plus disponibles pour absorber des pertes et de plus grande qualité, reconnus par la réglementation Solvabilité 2).

L'exigence de capital se décline sur deux niveaux :

› Minimum de capital requis - MCR : niveau en deçà duquel l'entreprise d'assurance présente un risque inacceptable de ne pas pouvoir faire face à ses engagements ;

› Capital de solvabilité requis - SCR : niveau permettant à l'entreprise d'assurance de faire face à des pertes imprévues.

MAIF VIE utilise la formule standard définie par la réglementation pour déterminer l'exigence en capital, son calibrage étant adapté pour la mesure de son profil de risque.

MAIF VIE n'a pas recouru aux mesures temporaires optionnelles proposées par la réglementation pour faciliter la transition vers le nouveau dispositif prudentiel.

Le calcul de la couverture du besoin en capital consiste à comparer les fonds propres éligibles à l'exigence de capital.

MAIF VIE couvre de manière satisfaisante le Minimum de capital requis et le Capital de solvabilité requis, avec les ratios suivants :

› 1,6 fois le SCR - Capital de solvabilité requis pour un montant de 675 091 milliers d'euros (avec un taux de couverture de 162 %) ;

› 5,4 fois le MCR - Minimum de capital requis pour un montant de 198 212 milliers d'euros (avec un taux de couverture de 535 %).


### E.1 Fonds propres

Les fonds propres en normes Solvabilité 2 sont composés de fonds propres de base et de fonds propres auxiliaires :

› les fonds propres de base sont constitués de l'excédent des actifs sur les passifs (issus du bilan en normes Solvabilité 2) auquel est soustrait le montant des actions détenues en propre et est ajouté le montant des passifs subordonnés ;

› les fonds propres auxiliaires, qui doivent être approuvés par l'ACPR, comprennent des éléments pouvant être appelés et utilisés pour absorber des pertes (fraction de capital non appelée et/ou non versée, ... ).

Les fonds propres doivent également être classés selon leur niveau de qualité. Ce classement se fait en fonction de leur disponibilité pour absorber les pertes, de leur degré de subordination

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> GESTION DU CAPITAL
===

<!-- PageHeader="88 / 111" -->

et de leur durée ou permanence. Il existe trois niveaux et les fonds propres de niveau 1 sont ceux qui présentent la plus grande capacité d'absorption des pertes et donc la meilleure qualité.

Enfin, parmi l'ensemble de ces fonds propres disponibles, les éléments éligibles à la couverture du Capital de solvabilité requis (SCR) et du Capital minimum requis (MCR) doivent respecter des limites quantitatives assurant que les exigences de solvabilité soient couvertes majoritairement par des fonds propres de la meilleure qualité. En particulier, les fonds propres de niveau 1 doivent couvrir au moins 50 % du SCR et 80 % du MCR.

Les paragraphes suivants présentent, pour MAIF VIE :

› les objectifs, politiques et procédures appliqués pour la gestion des fonds propres ;

› la composition, les montants et la qualité des fonds propres en normes Solvabilité 2 à la fin de l'exercice ainsi qu'un comparatif avec l'exercice précédent ;

› les explications des différences entre les fonds propres en normes Solvabilité 2 et les fonds propres issus des états financiers établis en normes comptables françaises. Ces derniers font l'objet d'une présentation préalable.


## E.1.1 Informations sur les objectifs, politiques et procédures appliques par l'entreprise pour la gestion de ses fonds propres

MAIF VIE doit couvrir a minima les exigences réglementaires en fonds propres (notamment Capital de solvabilité requis - SCR - et Minimum de capital requis - MCR dans l'environnement Solvabilité 2) par des fonds propres :

› qui remplissent les critères de qualité attendus pour être éligibles ;

› en quantité suffisante.

La solidité financière et prudentielle de MAIF VIE se mesure à travers ses fonds propres. MAIF VIE gère ses ressources pour disposer en permanence d'un niveau de fonds propres qui garantisse la pérennité de l'entreprise.


### E.1.1.1 Objectifs de gestion du capital

Dans le cadre de son système de gestion des risques, MAIF VIE a défini deux indicateurs d'appétence :

› un niveau de fonds propres en normes comptables françaises, hors dettes subordonnées dont elle doit disposer à l'horizon de son Plan stratégique ;

› un taux minimum de couverture du SCR par les fonds propres éligibles du Bilan en normes Solvabilité 2.

Ces fonds propres comptables sont constitués principalement de :

› capitaux propres ;

› réserves dont la réserve de capitalisation ;

› résultat de l'exercice et report à nouveau.

L'évolution des fonds propres comptables résultera principalement de la réalisation de bénéfices (ou pertes) dans les années à venir et des mouvements sur la réserve de capitalisation suite à des cessions d'obligations.

Dans le bilan en normes Solvabilité 2, les fonds propres intègrent, en plus des fonds propres comptables :

› les fonds excedentaires dont la fraction de Provision pour Participation aux Bénéfices (décret du 24 décembre 2019) ;

› les plus ou moins-values sur les actifs et les passifs comprises dans la réserve de réconciliation ;

MAIF VIE - Rapport sur la solvabilité et la situation financière 2022

> GESTION DU CAPITAL
===

<!-- PageHeader="89 / 111" -->

› les titres subordonnés remboursables.

Aucun changement important dans la gestion des fonds propres n'est survenu au cours de la période de référence.


## E.1.1.2 Processus et procédures de gestion du capital

Une présentation annuelle du niveau de fonds propres et de la nature des fonds propres disponibles, au sens des normes Solvabilité 2, est dispensée au Comité d'audit, des risques et des comptes MAIF et au conseil d'administration de MAIF VIE.

Les fonds propres font l'objet d'une classification selon les critères définis par la réglementation Solvabilité 2 (caractéristiques, qualités, disponibilités, durées, ... ).


### E.1.2 Capitaux propres en normes comptables françaises

Les fonds propres issus des états financiers établis en normes comptables françaises font l'objet d'une présentation préalable à celle des fonds propres en normes Solvabilité 2.

| Rubriques En milliers d'euros | 2022 | 2021 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Capital social | 122 000 | 122 000 | - | 0,0% |
| Autres réserves | 367 933 | 319 153 | 48 780 | 15,3% |
| Réserve de capitalisation | 153 300 | 154 852 | -1 552 | -1,0% |
| Réserve pour le développement des activités | 212 014 | 161 782 | 50 232 | 31,0% |
| Réserve pour fonds de garantie | 2 619 | 2 519 | 100 | 4,0% |
| Report à nouveau | 100 000 | 100 000 | - | 0,0% |
| Résultat de l'exercice | 51 214 | 50 332 | 882 | 1,8% |
| TOTAL | 641 147 | 591 485 | 49 662 | 8,4% |

Le montant des fonds propres au passif du bilan en normes comptables françaises s'élève, au 31 décembre 2022, à 641 147 milliers d'euros, en progression de 49 662 milliers d'euros sous l'effet exclusivement du résultat net comptable de l'exercice 2022 et des mouvements annuels de la réserve de capitalisation.

Le conseil d'administration propose à l'assemblée générale des actionnaires :

› d'approuver les comptes sociaux et le résultat net comptable (+ 51 214 milliers d'euros), tels qu'ils ont été arrêtés par le conseil d'administration ;

› de prélever sur le résultat la somme de 173 milliers d'euros à titre de dotation de la réserve pour fonds de garantie, en application de la loi du 25 juin 1999 ;

› de verser des dividendes pour un montant de 907 milliers d'euros ;

› d'affecter le solde du résultat bénéficiaire de 50 134 milliers d'euros en réserve pour développement des activités.

La réserve pour fonds de garantie s'élèvera ainsi à 2 792 milliers d'euros et la réserve pour développement des activités à 262 148 milliers d'euros.

La proposition de versement de dividendes à MAIF correspond à une partie de la quote-part 2022 de MAIF VIE au titre du dividende écologique Groupe. Le reste de la contribution de l'entreprise se fera au travers d'un dispositif distinct.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="90 / 111" -->


## E.1.3 Fonds propres en normes Solvabilité 2

Le montant des fonds propres en normes Solvabilité 2 de MAIF VIE a diminué sur l'exercice 2022 de 126 717 milliers d'euros pour atteindre ainsi 1 095 746 milliers d'euros, dont :

› 367 130 milliers d'euros de fonds excédentaires,

› 694 116 milliers d'euros d'excédent de l'actif sur le passif hors fonds excédentaires,

› et 34 500 milliers d'euros de dettes subordonnées.


### E.1.3.1 Les fonds propres de base

Le tableau ci-dessous présente la méthodologie et les méthodes de classification des fonds propres de base de MAIF VIE :

| Eléments de Fonds propres | Bilan Normes françaises | Bilan Prudentiel (Solvabilité 2) | Fonds propres de base |||
|||||| Niveau 1 Niveau 2 Niveau 3   |
| - | - | - | - | - | - |
| Capital social | Capital social | Méthode retenue Valorisation au montant du capital versé | X :selected: | :unselected: | :unselected: |
| Fonds excédentaires | | Méthode retenue (méthode 1) 67,58% du montant de la PPB diminué du montant de la PPB de 8 ans d'ancienneté | X :selected: | :unselected: | :unselected: |
| | Réserve de capitalisation Autres réserves Report à nouveau Résultats de l'exercice | Méthode retenue Valorisation au montant comptabilisé en NF | :unselected: | :unselected: | :unselected: |
| Réserve de réconciliation | Ajustements | Somme des différences d'évaluation entre NF et normes Solvabilité 2 suivantes : 1- les actifs 2- les provisions techniques 3- les autres passifs | X :selected: | :unselected: | :unselected: |
| Passifs subordonnés | Passifs subordonnés | Evaluation à la valeur de transfert (sans prise en compte de la qualité de crédit des prêteurs). Simplification retenue : Valorisation à la hauteur du montant nominal des titres. | :unselected: | :unselected: | X :selected: |

À l'exception des TSR, l'ensemble des fonds propres de base de MAIF VIE est de niveau 1.


### E.1.3.2 Les fonds propres auxiliaires

MAIF VIE n'a pas d'éléments qui sont ou pourraient être admis comme fonds propres auxiliaires.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="91/ 111" -->


## E.1.3.3 Information sur toute restriction importante affectant la disponibilité des fonds propres de l'entreprise

Après analyse décrite ci-après, les fonds propres de MAIF VIE ne sont pas soumis à des restrictions affectant leurs disponibilités ou leurs durées.

Par ailleurs, MAIF VIE ne dépend pas d'éléments de fonds propres faisant l'objet de mesures transitoires au sens des normes Solvabilité 2.


### E.1.3.3.1 Réserve de capitalisation

Le montant de la réserve de capitalisation de 153 300 milliers d'euros, fonds propre de base classé en niveau 1, ne peut être utilisé qu'en cas de réalisation de moins-values suite à des cessions d'obligations.

Toutefois, au regard de l'article R. 351-2-1 du Code des assurances, la valeur actuelle attendue du montant de la réserve de capitalisation subsistant à l'issue de la durée des engagements ( 34 364 milliers d'euros au 31 décembre 2022) est intégrée à la réserve de réconciliation et fait partie des éléments de fonds propres de base, classée au niveau 1. Elle ne fait donc ainsi l'objet d'aucune restriction importante.

E.1.3.3.2 Titres subordonnés remboursables

MAIF VIE a émis le 12 janvier 2005 des titres subordonnés remboursables à hauteur de 34 500 milliers d'euros, exclusivement réservés aux personnes morales actionnaires, la MAIF et la CASDEN Banque Populaire.

Ces TSR portent intérêt calculé sur leur montant nominal non remboursé au taux d'intérêt. Le taux d'intérêt applicable pour chacune des périodes d'intérêts comprises entre le 1er janvier 2015 et le 11 janvier 2025 est variable et est égal à l'EURIBOR 3 mois auquel est ajoutée une marge de 280 points. Ils seront amortis en totalité le 11 janvier 2025 par remboursement au pair.

MAIF VIE peut rembourser, par anticipation, tout ou partie de ces TSR à tout moment conformément à l'article A 334-3 du Code des assurances, sous réserve de l'accord préalable de l'ACPR, à chaque échéance de période d'intérêts à partir du 31 décembre 2014. Il convient de noter que les souscripteurs ne peuvent pas demander de remboursement par anticipation.

Dans l'hypothèse d'une liquidation de MAIF VIE avant le terme convenu, ces titres seront remboursables à un prix égal au pair et uniquement après le desintéressement complet de tous les créanciers privilégiés et chirographaires et au même rang que tous les autres emprunts subordonnés qui pourraient être emis ou contractes par MAIF VIE.

MAIF VIE se réserve également la possibilité d'émettre de la dette subordonnée auprès de tiers.


#### E.1.3.3.3 Fonds cantonnés

MAIF VIE réalise chaque année un examen de l'importance des fonds cantonnés. Cet examen porte sur l'ensemble des points d'analyse décrits dans le § 5.6 de la notice << Solvabilité II >> de l'ACPR relative aux fonds propres (traduction de l'orientation EIOPA-BoS-14/169 FR sur les fonds cantonnés).

Au 31 décembre 2022, la tenue d'une comptabilité auxiliaire d'affectation des nouveaux PER est devenue obligatoire d'un point de vue réglementaire. MAIF VIE répond à cette obligation à la clôture, les opérations du PER Responsable et Solidaire ayant fait l'objet d'une comptabilité auxiliaire d'affectation à des fins de gestion dès septembre 2020, au moment de la transformation du Perp-MAIF.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="92 / 111" -->

MAIF VIE considère que le PER Responsable et Solidaire n'est pas important au regard de cet examen, dont les résultats sont résumés ci-après :

› la non-complexité des risques découlant du fonds cantonne ;

› l'approche sécurisée et la liquidité des actifs inclus dans le fonds cantonné ;

› l'absence de capitaux propres dans le Bilan du fonds cantonné (et donc pas de restriction sur les fonds propres disponibles en normes Solvabilité 2) ;

› la non-significativité du fonds cantonné au regard de l'activité globale de MAIF VIE ;

› et la très faible différence sur la mesure des exigences réglementaires (calcul du SCR).

E.1.3.3.4 Politique de dividende

MAIF VIE a vocation à autofinancer son développement, ce que permettent les résultats issus de son Plan stratégique sans versement de dividendes à l'actionnaire.

Cependant l'actionnaire peut étudier l'opportunité d'un tel versement.


## E.1.4 Tableau de passage des fonds propres

Le tableau de passage des fonds propres en normes comptables françaises aux fonds propres en normes Solvabilité 2, présente une explication quantitative des différences importantes entre les capitaux propres selon les états financiers de l'entreprise et l'excédent de l'actif sur le passif du Bilan en normes Solvabilité 2 :

| En milliers d'euros | Fonds propres des comptes individuels | Passage de Bilan NF à S2 |||| Fonds propres S2 |
||| Evaluation en valeur de marché des placements | Evaluation des provisions techniques en valeur actuelle plus marge de risque | Autres ajustements de valeurs | Reconnaissance des impôts différés ||
| - | - | - | - | - | - | - |
| Postes de fonds propres communs NF - S2 | | | | | | |
| Capital social ou Fonds d'établissement | 122 000 | | | | | 122 000 |
| Postes de fonds propres NF | | | | | | |
| Autres réserves | 365 314 | | | | | 365 314 |
| Report à nouveau | 100 000 | | | | | 100 000 |
| Resultat de l'exercice | 51 214 | | | | | 51 214 |
| Fonds de garantie | 2 619 | | | | | 2 619 |
| Postes de fonds propres S2 | | | | | | |
| Fonds excédentaires (art 91) | | | 367 130 | | | 367 130 |
| Réserve de réconciliation, hors éléments provenant des comptes sociaux | | -862 777 | 981 156 | -51 504 | -13 907 | 52 969 |
|| Total des fonds propres hors dettes subordonnée: 641 147  | -862 777 | 1 348 287 | -51 504 | -13 907 | 1 061 246 |
| Dettes subordonnées | 34 500 | | | | | 34 500 |
| Total fonds propres | 675 647 | -862 777 | 1 348 287 | -51 504 | -13 907 | 1 095 746 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===


## E.1.5 Analyse des variations significatives de fonds propres sur la période

Le montant des fonds propres de base niveau 1 s'élève au 31 décembre 2022 à 1 061 246 milliers d'euros, en retrait de 126 717 milliers d'euros par rapport au 31 décembre 2021.

| En milliers d'euros | 2022 | 2021 | Variation ||
| - | - | - | - | - |
| Fonds propres de base - Niveau 1 | 1 061 246 | 1 187 963 | -126 717 | -11% |
| Capitaux propres | 122 000 | 122 000 | - | 0% |
| Fonds excédentaires | 367 130 | 478 766 | -111 636 | -23% |
| Réserve de réconciliation | 572 116 | 587 197 | -15 081 | -3% |
| - Eléments provenant des comptes sociaux | 519 147 | 469 485 | 49 662 | 11% |
| Réserves | 367 933 | 319 153 | 48 780 | 15% |
| Report à nouveau et resultat de l'exercice | 151 214 | 150 332 | 882 | 1% |
| - Ajustements liés aux retraitements S2 | 52 969 | 117 712 | -64 743 | -55% |
| Différence d'évaluation des placements | -862 777 | 1 223 324 | -2 086 101 | NS |
| Différence d'évaluation des autres actifs | -49 197 | -23 494 | -25 703 | NS |
| Différence d'évaluation des provisions techniques | 981 156 | -1 045 373 | 2 026 529 | NS |
| Différence d'évaluation des autres postes du passif | -16 214 | -36 744 | 20 530 | -56% |

Cette évolution s'explique principalement par :

› une augmentation du montant des fonds propres établis en normes comptables françaises de 49 662 milliers d'euros sur la période, en raison notamment du résultat net 2022 de 51 214 milliers d'euros et des mouvements sur la réserve de capitalisation réalisés en 2022 pour un montant de -1 552 milliers d'euros ;

› une diminution des fonds excédentaires, liée au reclassement d'une partie du montant de la provision pour participation aux bénéfices (PPB) conformément à l'arrêté du 24 décembre 2019 et en application des préconisations de l'ACPR portant sur les modalités de calcul à retenir par les entreprises d'assurance, de 111 636 milliers d'euros. Cette baisse s'explique par le passage du taux de reclassement de 85,4% en 2021 à 67,5% en 2022, en raison notamment de l'évolution de la courbe de taux ;

› une diminution du montant de la réserve de réconciliation, hors éléments provenant des comptes en normes comptables françaises, de -64 743 milliers d'euros, conséquence :

o d'une différence d'évaluation des placements baissant de 2 086 101 milliers d'euros en raison de la situation de moins-value latente globale à la clôture ;

o des différences d'évaluation des autres actifs en baisse de 25 703 milliers d'euros, en raison principalement d'une diminution de la différence d'évaluation des provisions techniques cédées ;

o des écarts de provisionnements techniques entre les normes comptables françaises et les normes Solvabilité 2 en évolution de 2 026 529 milliers d'euros ;

o d'une variation de l'évaluation des autres postes de passif de 20 530 milliers d'euros, en raison notamment de l'évolution de la reconnaissance des impôts différés passifs.

Le montant des fonds propres niveau 3 concernant exclusivement les TSR est stable à 34 500 milliers d'euros.

L'annulation des effets de la correction pour volatilité représente un impact de -29 758 milliers d'euros sur les fonds propres de niveau 1.

| | Fonds propres de base - Niveau 1 | Fonds propres de base - Niveau 3 |
| - | - | - |
| Valeur retenue | 1 061 246 | 34 500 |
| Valeur sans l'effet de correction pour volatilité | 1 031 488 | 34 500 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="93 / 111" --> :unselected: :unselected: :unselected: :unselected:

> GESTION DU CAPITAL
===

<!-- PageHeader="94/ 111" -->


## E.2 Capital de solvabilité requis et Minimum de capital requis

L'exigence de capital se décline sur deux niveaux :

› Minimum de capital requis - MCR ;

› Capital de solvabilité requis - SCR.


### E.2.1 Capital de solvabilité requis (SCR)

Le SCR correspond au montant de fonds propres estimé comme nécessaire pour absorber des chocs de forte intensité susceptibles de se produire environ 1 fois tous les 200 ans.

Il se fonde sur l'hypothèse d'une continuité de l'exploitation de l'entreprise et est calibré de manière à garantir que tous les risques quantifiables auxquels l'entreprise est exposée soient pris en considération. Il couvre le portefeuille en cours, ainsi que le nouveau portefeuille dont la souscription est attendue dans les douze mois à venir.

Les principaux risques auxquels est exposée MAIF VIE sont décrits dans le paragraphe C de ce rapport.

Le SCR peut être calculé soit par la formule standard qui reflète le profil de risque moyen des entreprises d'assurances sur le marché européen, soit par un modèle interne après autorisation de l'ACPR, soit par une combinaison de ces deux méthodes.

MAIF VIE determine le Capital de solvabilité requis (SCR) à l'aide de la formule standard qui reflète le profil de risque moyen des entreprises d'assurance et de son activité.

Le SCR formule standard est calculé selon une approche modulaire. L'assureur doit calculer la perte potentielle subie en cas d'événement défavorable lié à une trentaine de facteurs de risque (regroupés en six modules). Les méthodes, hypothèses et paramètres à utiliser pour chacun des modules de risque ou sous-modules sont définis par la réglementation.

Pour tenir compte de la probabilité faible de réalisation simultanée de tous ces événements, la formule standard comprend des correlations entre ces facteurs de risques et permet ainsi de calculer l'effet positif de la diversification.

Enfin, le SCR est encore atténué par des ajustements pour obtenir le SCR final. Les entités vie sont notamment concernées par l'ajustement « capacité d'absorption des pertes par les impôts différés >> (capacité d'imputer des pertes au résultat fiscal permettant de payer moins d'impôts dans le futur) et par l'ajustement « capacité d'absorption des pertes par les provisions techniques ».

Au 31 décembre 2022, le montant du SCR est de 675 092 milliers d'euros, en hausse de 182 154 milliers d'euros par rapport au 31 décembre 2021. Hors capacité d'absorption des pertes par les provisions techniques, le montant du SCR de base s'élève à 1 930 828 milliers d'euros.

Le tableau présente le détail du SCR par module :

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="95 / 111" -->

\> GESTION DU CAPITAL
===

| En milliers d'euros | 2022 || 2021 | Variation SCR net ||
|| SCR net (y compris capacité d'absorption des pertes PT | SCR brut (hors capacité d'absorption des| pertes PT / ID) des pertes PT /  SCR net (y compris capacité d'absorption | En valeur | En % |
|| / ID) | | ID) | | |
| - | - | - | - | - | - |
| Risque de marché | 414 618 | 1 126 034 | 431 706 | -17 088 | -4% |
| Risque de défaut de contrepartie | 17 955 | 17 955 | 18 562 | -607 | -3% |
| Risque de souscription vie | 394 422 | 1 305 074 | 105 767 | 288 655 | NS |
| Risque de souscription santé | | | | | |
| Risque de souscription non-vie | 1 | | | | |
| Diversification entre modules | -181 371 | -518 235 | -80 935 | -100 436 | NS |
| Risque lié aux actifs incorporels | | | | | |
| SCR de base | 645 624 | 1 930 828 | 475 100 | 170 524 | 36% |
| Capacité d'absorption des pertes par les provisions techniques | | -1 285 204 | | | |
| Capacité d'absorption des pertes par les impôts différés | -13 907 | -13 907 | -34 207 | 20 300 | -59% |
| Risque opérationnel | 43 375 | 43 375 | 52 045 | -8 670 | -17% |
| SCR notionnel pour les fonds cantonnés | :unselected: | | | | |
| Diversification entre fonds cantonnés | | | 1 | | |
| SCR (hors exigences de capital supplémentaires) | 675 092 | 675 092 | 492 938 | 182 154 | 37% |
| Exigences de capital supplémentaires déjà imposées | | | | | |
| SCR | 675 092 | 675 092 | 492 938 | 182 154 | 37% |

Les principaux éléments explicatifs du montant du Capital de solvabilité requis sont :

› le SCR relatif au risque de marche, net de capacité d'absorption par les provisions techniques à 414 618 milliers d'euros (431 706 milliers d'euros au 31 décembre 2021), qui représente 64 % du SCR de base transitoire net de capacité d'absorption par les provisions techniques ;

› le SCR relatif au risque de souscription vie, net de capacité d'absorption par les provisions techniques à 394 422 milliers d'euros (105 767 milliers d'euros au 31 décembre 2021), qui pèse 61 % ;

› le SCR relatif au risque de défaut de contrepartie, net de capacité d'absorption par les provisions techniques, à 17 955 milliers d'euros (18 562 milliers d'euros au 31 décembre 2021), qui représente 3 % ;

› le SCR net relatif au risque opérationnel qui tient compte de la composante « provisions techniques >> pour atteindre 43 375 milliers d'euros.

L'annulation des effets de la correction pour volatilité représente un impact de + 2 419 milliers d'euros sur le SCR.

| | SCR |
| - | - |
| Valeur retenue | 675 092 |
| Valeur sans l'effet de correction pour volatilité | 677 511 |


## E.2.2 Minimum de capital requis

Le MCR est le niveau minimal de fonds propres que l'organisme doit détenir en permanence. Le calcul du MCR s'effectue en trois étapes :

› application d'une formule linéaire simple en fonction des primes et des provisions techniques pour obtenir le MCR linéaire ;

› retraitement éventuel du MCR linéaire pour qu'il soit compris entre 25 % et 45 % du SCR afin d'aboutir au MCR combiné ;

› application d'un plancher absolu dépendant de l'activité pour obtenir le MCR final.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="96 / 111" -->

Au 31 décembre 2022, le montant du MCR s'élève à 198 212 milliers d'euros ( 221 822 milliers d'euros au 31 décembre 2021).

La composante linéaire du MCR, basée notamment sur les provisions techniques et les capitaux sous-risques, s'élève à 198 212 milliers d'euros, en baisse de 72 251 milliers d'euros par rapport à son niveau à fin 2021.

| | 2022 | 2021 | Variation ||
| - | - | - | - | - |
| MCR | 198 212 | 221 822 | -23 610 | -10,6% |
| MCR - Formule linéaire | 198 212 | 270 463 | -72 251 | -26,7% |
| Plancher (25% SCR) | 168 773 | 123 235 | 45 538 | 37,0% |
| Plafond (45% SCR) | 303 791 | 221 822 | 81 969 | 37,0% |
| SCR | 675 092 | 492 938 | 182 154 | 37,0% |

L'annulation des effets de la correction pour volatilité représente un impact de + 3 835 milliers d'euros sur le MCR.

| | MCR |
| - | - |
| Valeur retenue | 198 212 |
| Valeur sans l'effet de correction pour volatilité | 202 047 |


## E.2.3 Informations sur la couverture des exigences de marge

Au 31 décembre 2022, MAIF VIE couvre de manière satisfaisante le Minimum de capital de solvabilité requis et le Capital de solvabilité requis.

| En milliers d'euros | 31/12/2022 | 31/12/2021 | Variation ||
|||| En valeur | En % |
| Fonds propres de base de niveau 1 | 1 061 246 | 1 187 963 | -126 717 | -10,7% |
| - | - | - | - | - |
| Capitaux propres | 122 000 | 122 000 | | 0,0% |
| Fonds excédentaires | 367 130 | 478 766 | -111 636 | -23,3% |
| Réserve de réconciliation | 572 116 | 587 197 | -15 081 | -2,6% |
| - Eléments provenant des comptes sociaux | 519 147 | 469 485 | 49 662 | 10,6% |
| - Ajustements liés aux retraitements S2 | 52 969 | 117 712 | -64 743 | -55,0% |
| Fonds propres de base de niveau 3 | 34 500 | 34 500 | | 0,0% |
| Dettes subordonnées Impôts différés actifs | 34 500 | 34 500 1 | . | 0,0% |
||||| |
| Eléments disponibles pour le SCR Eléments disponibles pour le MCR | 1 095 746 | 1 222 463 1 187 963 | -126 717 -126 717 | -10,4% -10,7% |
|| 1 061 246 ||||
| | | | | |
| Capital de solvabilité requis | 675 092 | 492 938 | 182 154 | 37,0% |
| Eléments éligibles en couverture du SCR | 1 095 746 | 1 222 463 | -126 717 | -10,4% |
| Couverture par l'ensemble des éléments éligibles sous respect des limites quantitatives | 162,3% | 248,0% | -86 pts | |
| Minimum de capital requis | 198 212 | 221 822 | -23 610 | -10,6% |
| Eléments éligibles en couverture du MCR | 1 061 246 | 1 187 963 | -126 717 | -10,7% |
| Couverture par l'ensemble des éléments éligibles sous respect des limites quantitatives | 535,4% | 535,6% | 0 pts | |


## Capital de solvabilité requis ›

Au 31 décembre 2022, le ratio de couverture du SCR est de 162,3 %, en baisse de 86 points par rapport au 31 décembre 2021, en lien avec la baisse des éléments éligibles en couverture du SCR de 126 717milliers d'euros et la hausse du SCR de 182M€.

L'annulation des effets de la correction pour volatilité représente un impact de - 5 points sur le ratio de couverture du SCR.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="97 / 111" -->

| | Ratio de couverture du |
|| SCR |
| - | - |
| Valeur retenue | 162,3% |
| Valeur sans l'effet de correction pour volatilité | 157,3% |


## › Minimum de capital de solvabilité requis

Au 31 décembre 2022, le ratio de couverture du MCR est de 535,4 %, en baisse de moins de 1 point par rapport au 31 décembre 2021.

A noter : parmi l'ensemble des fonds propres « disponibles », les éléments dits « éligibles » à la couverture du Capital de solvabilité requis (SCR) et du Capital minimum requis (MCR) doivent respecter des limites quantitatives assurant que les exigences de solvabilité sont couvertes majoritairement par des fonds propres de la meilleure qualité.


## E.3 Utilisation du sous-module "risque sur actions" fondé sur la durée dans le calcul du Capital de solvabilité requis

MAIF VIE n'utilise pas cette option.

Cette option concerne le SCR des assureurs vie qui peuvent, sous certaines conditions et après autorisation de l'ACPR, appliquer aux actifs cantonnés de certaines activités de retraite un choc sur actions réduit par rapport au niveau imposé par la formule standard.


## E.4 Différences entre la formule standard et tout modèle interne utilisé

MAIF VIE n'utilise pas de modèle interne et de paramètres spécifiques.


## E.5 Non-respect du Minimum de capital requis et non-respect du Capital de solvabilité requis

MAIF VIE respecte les exigences réglementaires.


## E.6 Autres informations

MAIF VIE n'identifie pas d'autre information notable à communiquer sur la gestion du capital.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> ANNEXES
===

<!-- PageHeader="98 / 111" -->

Les états quantitatifs Solvabilité 2 suivants sont annexés au rapport sur la solvabilité et la situation financière :

• S.02.01.02 Bilan

• S.05.01.02 Primes, sinistres et frais par ligne d'activité

S.12.01.02 Provisions techniques vie et santé similaire à la vie

· S.22.01.21

• Impact des mesures relatives aux garanties de long terme et des mesures transitoires Fonds propres •

S.23.01.01

S.25.01.21 • SCR Formule standard

· S.28.01.01 Minimum de capital requis (MCR)

À noter, l'état S.05.02.01 - Primes, sinistres et frais par pays n'est pas publié dans la mesure où il n'est pas obligatoire compte tenu du seuil de chiffre d'affaires réalisé en France.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageNumber="99 / 111" -->


## \> ANNEXES


## S.02.01.02 - BILAN

| ACTIFS | Valeur Solvabilité |
| - | - |
| Goodwill | |
| Frais d'acquisition différés | |
| Immobilisations incorporelles | - |
| Actifs d'impôts différés | - |
| Excédent du régime de retraite | - |
| Immobilisations corporelles détenues pour usage propre | 2 189 |
| Investissements (autres qu'actifs en représentation de contrats en unités de compte et indexés) | 10 560 376 |
| Biens immobiliers (autres que détenus pour usage propre) | - |
| Détentions dans des entreprises liées, y compris participations | 939 457 |
| Actions | 64 828 |
| Actions - cotées | - |
| Actions - non cotées | 64 828 |
| Obligations | 7 506 141 |
| Obligations d'État | 3 634 630 |
| Obligations d'entreprise | 3 871 510 |
| Titres structures | - |
| Titres garantis | - |
| Organismes de placement collectif | 1999 689 |
| Produits dérivés | - |
| Dépôts autres que les équivalents de trésorerie | 50 262 |
| Autres investissements | - |
| Actifs en représentation de contrats en unités de compte et indexés | 778 385 |
| Prêts et prêts hypothécaires | 2 885 |
| Avances sur police | 2 885 |
| Prêts et prêts hypothécaires aux particuliers | - |
| Autres prêts et prêts hypothécaires | - |
| Montants recouvrables au titre des contrats de reassurance | 56 845 |
| Non-vie et santé similaire à la non-vie | - |
| Non-vie hors santé | - |
| Santé similaire à la non-vie | - |
| Vie et santé similaire à la vie, hors santé, UC et indexés | 56 845 |
| Santé similaire à la vie | - |
| Vie hors santé, UC et indexés | 56 845 |
| Vie UC et indexés | - |
| Dépôts auprès des cédantes | - |
| Créances nees d'opérations d'assurance et montants à recevoir d'intermédiaires | 11 750 |
| Créances nées d'opérations de reassurance | - |
| Autres créances (hors assurance) | 24 878 |
| Actions propres auto-détenues (directement) | - |
| Éléments de fonds propres ou fonds initial appelé(s), mais non encore payé(s) | - |
| Trésorerie et équivalents de trésorerie | 49 968 |
| Autres actifs non mentionnés dans les postes ci-dessus | - |
| TOTAL DE L'ACTIF | 11 487 276 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="100 / 111" -->

\> ANNEXES
===

| PASSIFS | Valeur Solvabilité |
| - | - |
| Provisions techniques non-vie | - |
| Provisions techniques non-vie (hors santé) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | - |
| Marge de risque | - |
| Provisions techniques santé (similaire à la non-vie) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | - |
| Marge de risque | - |
| Provisions techniques vie (hors UC et indexés) | 9 451 048 |
| Provisions techniques santé (similaire à la vie) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation :selected: | - |
| Marge de risque :selected: | - |
| Provisions techniques vie (hors santé, UC et indexés) | 9 451 048 |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation :selected: | 9 166 708 |
| Marge de risque :selected: | 284 341 |
| Provisions techniques UC et indexés | 777 106 |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation :selected: | 766 056 |
| Marge de risque :selected: | 11 050 |
| Autres provisions techniques | |
| Passifs éventuels | 2 619 |
| Provisions autres que les provisions techniques | 2 790 |
| Provisions pour retraite | 1 396 |
| Dépôts des réassureurs :unselected: | 83 794 |
| Passifs d'impôts différés | 13 907 |
| Produits dérivés | - |
| Dettes envers des établissements de crédit | - |
| Dettes financières autres que celles envers les établissements de crédit | - |
| Dettes nées d'opérations d'assurance et montants dus aux intermédiaires | 944 |
| Dettes nées d'opérations de réassurance | 3 332 |
| Autres dettes (hors assurance) | 54 594 |
| Passifs subordonnés | 34 500 |
| Passifs subordonnés non inclus dans les fonds propres de base | - |
| Passifs subordonnés inclus dans les fonds propres de base | 34 500 |
| Autres dettes non mentionnées dans les postes ci-dessus | - |
| TOTAL DU PASSIF | 10 426 030 |
| ||
| EXCEDENT D'ACTIF SUR PASSIF | 1 061 246 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

## ANNEXES


## S.05.01.02 - PRIMES, SINISTRES ET DEPENSES PAR LIGNE D'ACTIVITE

| | Ligne d'activité pour: engagements d'assurance et de réassurance non-vie (assurance directe et réassurance proportionnelle acceptée) |||||||||
|| médicaux Assurance des frais | Assurance de protection du revenu | Assurance travailleurs d'indemnisation des | automobile Assurance de responsabilité civile | véhicules à moteur Autre assurance des | Assurance maritime, aérienne et transport | aux biens Assurance incendie et autres dommages | générale Assurance de responsabilité civile | cautionnement Assurance crédit et |
| - | - | - | - | - | - | - | - | - | - |
| Primes émises | | | | | | | | | |
| Brut - assurance directe | | | | | | | | | |
| Brut - Reassurance proportionnelle acceptée | :unselected: 
||| | | | | | | | |||| | | | | | | 
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Primes acquises | :unselected: | | | | | | | | |
| Brut - assurance directe | | | | | | | | | |
| Brut - Reassurance proportionnelle acceptée | :unselected: 
||| | | | | | | | |||| | | | | | | 
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Charge des sinistres | :unselected: | | | | | | | | |
| Brut - assurance directe | | | | | | | | | |
| Brut - Reassurance proportionnelle acceptée | :unselected: | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Variation des autres provisions techniques | :unselected: | | | | | | | | |
| Brut - assurance directe Brut - Reassurance proportionnelle acceptée 
|| | | | | | | | | || :unselected: | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | | | | | | | | | |
| Part des réassureurs | | | | | | | | | |
| Net | | | | | | | | | |
| Dépenses engagées | | | | | | | | | |
| Autres dépenses | | | | | | | | | |
| Total des dépenses | | | | | | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="101 / 111" -->

## ANNEXES

| | Ligne d'activité pour: engagements d'assurance et de réassurance non-vie (assurance directe et réassurance proportionnelle acceptée) ||| Ligne d'activité pour: réassurance non proportionnelle acceptée ||| | Total 
|| || juridique protection Assurance de | Assistance | Pertes diverses pécuniaires | Santé | Accidents | transport maritime, Assurance aérienne et | Biens | | |
| - | - | - | - | - | - | - | - | - | - |
| Primes émises | | | | | | | | | |
| Brut - assurance directe | :unselected: | | | | | | | | |
| Brut - Réassurance proportionnelle | | | | | | | | | |
| acceptée | | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | | | | | | | | | |
| Net | :unselected: | | | | | | | ||
| Primes acquises | :unselected: | | | | | | | | |
| Brut - assurance directe | :unselected: | | | | | | | | |
| Brut - Reassurance proportionnelle | | | | | | | | | |
| acceptée | | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | ||
| Part des réassureurs | | | | | | | | | |
| Net | :unselected: | | | | | | | ||
| Charge des sinistres | | | | | | | | ||
| Brut - assurance directe | :unselected: | | | | | | | | |
| Brut - Reassurance proportionnelle | | | | | | | | | |
| acceptée | | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | ||
| Part des réassureurs | | | | | | | | | |
| Net | :unselected: | | | | | | | ||
| Variation des autres provisions techniques | :unselected: | | | | | | | | |
| Brut - assurance directe Brut - Reassurance proportionnelle | :unselected: 
|| || | || || || | | || | | | | |
| acceptée | | | | | | | | | |
| Brut - Réassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | ||
| Net | | | | | | | | | |
| Dépenses engagées | | | | | | | | | |
| Autres dépenses | | | | | | | | ||
| Total des dépenses | | | | | | | | ||

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<figure>

![](figures/17)

</figure>


<!-- PageHeader="102 / 111" -->

### ANNEXES

| | Ligne d'activité pour: engagements d'assurance vie ||||||| | | | Engagements de réassurance vie |
|
||| ||| | Rentes découlant des contrats   |||| || d'assurance non-vie et liées    ||||||| d'assurance santé  | Total || santé ||| | | 
| Primes émises | | | | | | | | | | | | | |
| Brut | | 636 185 | 131 456 | 66 872 | | | | | | | | | 834 513 |
| Part des réassureurs | | 9 267 | - | 16 233 | | | | | | | | | 25 500 |
| Net | | 626 919 | 131 456 | 50 639 | | | | | | | | | 809 014 |
| Primes acquises | | | | | | | | | | | | | |
| Brut | | 636 301 | 131 456 | 66 611 | | | | | | | | | 834 368 |
| Part des réassureurs | | 9 351 | - | 16 228 | | | | | | | | | 25 579 |
| Net | | 626 950 | 131 456 | 50 383 | | | | | | | | | 808 789 |
| Charge des sinistres | | | | | | | | | | | | | |
| Brut | | 546 290 | 22 774 | 17 938 | | | | | | | | | 587 003 |
| Part des réassureurs | | 2 342 | - | 4 720 | | | | | | | | | 7 062 |
| Net | | 543 948 | 22 774 | 13 218 | | | | | | | | | 579 941 |
| Variation des autres provisions techniques | | | | | | | | | | | | | |
| Brut | | 16 978 | 102 251 | 1 464 - | | | | | | | | | 117 764 |
| Part des réassureurs | | 4 455 | - | 951 - | | | | | | | | | 3 504 |
| Net | | 12 523 | 102 251 | - 513 | | | | | | | | | 114 260 |
| Dépenses engagées | - | 68 700 | 1 721 | 22 763 | | | - | | | - | - | - | 93 184 |
| Autres dépenses | | | | | | | | | | | | | |
| Total des dépenses | | | | | | | | | | | | | 93 184 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="103 / 111" -->
| | | || | || | | | | | |
| || | | | | || Assurance maladie  
||
|| | | || ||| Assurance avec participation  aux bénéfices  || | | | | |
| | | || unités de compte  
||||||
| - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| | | | || Autres assurances vie   |||| || | | Assurance indexée et en   
||||| | || | ||| aux engagements d'assurance   || |||| | Rentes découlant des contrats | | d'assurance non-vie et liées    ||| aux engagements d'assurance|| | | | autres que les engagements    |||| Réassurance maladie  | | || Réassurance vie  
ANNEXES

S.12.01.02 - PROVISIONS TECHNIQUES ASSIMILABLES A LA VIE PAR CONTRAT

| | Assurance avec participation aux bénéfices | Assurance indexée et en unités de compte ||| Autres assurances vie ||| Rentes découlant des | | Assurance santé (assurance directe) ||| Rentes découlant des contrats d'assurance non- vie et liées aux engagements | Réassurance santé (réassurance acceptée) | Total (santé similaire à la vie) |
||| | Contrats sans options | Contrats avec options | | Contrats sans options ni | Contrats avec options | contrats d'assurance non- vie et liées aux engagements d'assurance autres que | Réassurance acceptée | Total (vie hors santé, y compris UC) | Contrats sans options ni | Contrats avec options ou ||||
|| | | ni garanties | ou garanties | | garanties | ou garanties | les engagements d'assurance santé | | | garanties | garanties | d'assurance santé | | |
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| Provisions techniques calculées comme un tout Total des montants recouvrables au titre de la réassurance/des véhicules de titrisation et de la réassurance finite, après l'ajustement pour pertes probables pour défaut de la contrepartie, correspondant aux provisions techniques calculées comme un tout 
|
|| | | | | | | | | || | | | | | || | | | | | | | | | | | | | || | | | | | | | | | | | | | | |
| Provisions techniques calculées comme la somme de la meilleure estimation et de la marge de risque Meilleure estimation | | | | | | | | | | | | | | | |
| Meilleure estimation brute | 9 187 163 | | 766 056 | | | 20 455 - | | | | 9 932 764 | | | | | |
| Total des montants recouvrables au titre de la | | | | | | | | | | | | | | | |
| réassurance/des véhicules de titrisation et de la réassurance finite, après ajustement pour pertes probables pour défaut de la contrepartie | 75 643 | | - | | | - 18 797 | | | | 56 845 | | | | | |
| Meilleure estimation nette des montants recouvrables au titre de la réassurance/des véhicules de titrisation et de la réassurance finite - total | 9 111 520 | | 766 056 | | | 1658 - | | | | 9 875 918 | | | | | |
| Marge de risque | 272 163 | 11 050 | | | 12 177 | | | | | 295 391 | | | | | |
| Montant de la déduction transitoire sur les provisions techniques | | | | | | | | | | | | | | | |
| Provisions techniques calculées comme un tout Meilleure estimation 
|| | | | | | | | | | | | | | | || | | | | | | | | | | | | | | |
| Marge de risque | | | | | | | | | | | | | | | |
| Provisions techniques - Total | 9 459 326 | 777 106 | | - | 8 278 | | | | | 10 228 155 | | | | | |

<!-- PageHeader="104 / 111" -->

<figure>

![](figures/18)

</figure>


<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="105 / 111" -->

\> ANNEXES
===

S.22.01.21 - IMPACT DES MESURES RELATIVES AUX GARANTIES DE LONG TERME ET DES MESURES TRANSITOIRES
===

| | Montant avec mesures relatives aux garanties de long terme et mesures transitoires | Impact des mesures| transitoires sur transitoires sur les provisions techniques  Impact des mesures les taux d'intérêt | Impact d'une correction pour volatilité fixée à zéro | Impact d'un ajustement égalisateur fixé à zéro |
| - | - | - | - | - | - |
| Provisions techniques | 10 228 155 | | | 45 084 | |
| Fonds propres de base | 1 095 746 | | | - 29 758 | |
| Fonds propres éligibles pour couvrir le SCR | 1 095 746 | | | - 29 758 | |
| Capital de solvabilité requis | 675 091 | | | 2 419 | |
| Fonds propres éligibles pour couvrir le minimum de capital requis | 1 061 246 | | | - 29 758 | |
| Minimum de capital requis | 198 212 | | | 3 835 | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="106 / 111" -->

\> ANNEXES
===

S.23.01.01 - FONDS PROPRES

| | Total | Niveau 1 - non restreint | Niveau 1 - restreint | Niveau 2 | Niveau 3 |
| - | - | - | - | - | - |
| Fonds propres de base avant déduction pour participations dans d'autres secteurs financiers, comme prévu à l'article 68 du règlement délégué 2015/35 | | | | | |
| Capital en actions ordinaires (brut des actions propres) | 122 000 | 122 000 | | - | |
| Compte de primes d'émission lié au capital en actions ordinaires | - | - | | - | |
| Fonds initial, cotisations des membres ou élément de fonds propres de base équivalent pour les mutuelles et les entreprises de type mutuel | - | - | | - | |
| Comptes mutualistes subordonnés | - | | - | - | - |
| Fonds excédentaires | 367 130 | 367 130 | | | |
| Actions de préférence | - | | - | - | - |
| Compte de primes d'émission lié aux actions de préférence | - | | - | - | - |
| Réserve de réconciliation | 572 116 | 572 116 | | | |
| Passifs subordonnés | 34 500 | | - | - | 34 500 |
| Montant égal à la valeur des actifs d'impôts différés nets | - | | : | | - |
| Autres éléments de fonds propres approuvés par L'autorité de contrôle en tant que fonds propres de base non spécifiés supra | - | - | - | - | - |
| Fonds propres issus des états financiers qui ne devraient pas être inclus dans la réserve de réconciliation et qui ne respectent pas les critères de fonds propres de Solvabilité II | : | | ! ! | | -- |
| Fonds propres issus des états financiers qui ne devraient pas être inclus dans la réserve de réconciliation et qui ne respectent pas les critères de fonds propres de Solvabilité II | - ||| | |
| Deductions | | | ! ! | | |
| Déductions pour participations dans des établissements de crédit et des établissements financiers | 1 095 746 | 1 061 246 | - | - | 34 500 |
| Total fonds propres de base après deductions ||||||
| Fonds propres auxiliaires | | | | | |
| Capital en actions ordinaires non libéré et non appelé, appelable sur demande Fonds initial, cotisations des membres ou élément de fonds propres de base équivalents, non libérés, non appelés et appelables sur demande, pour les mutuelles et les entreprises de type mutuel Actions de préférence non libérées et non appelées, appelables sur demande | --| | ! --  ! | | |
| Engagements juridiquement contraignants de souscrire et de payer des passifs subordonnés sur demande Lettres de crédit et garanties relevant de l'article 96, paragraphe 2, de la directive 2009/138/CE Lettres de crédit et garanties ne relevant pas de l'article 96, paragraphe 2, de la directive 2009/138/CE Rappels de cotisations en vertu de l'article 96, point 3, de la directive 2009/138/CE ||||||
| Rappels de cotisations ne relevant pas de l'article 96, paragraphe 3, de la directive 2009/138/CE Autres fonds propres auxiliaires | |||||

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageHeader="107 / 111" -->

\> ANNEXES
===

| | Total | Niveau 1 - non restreint | Niveau 1 - restreint | Niveau 2 | Niveau 3 |
| - | - | - | - | - | - |
| Total fonds propres auxiliaires | | | | | |
| Fonds propres éligibles et disponibles | | | | | |
| Total des fonds propres disponibles pour couvrir le capital de solvabilité requis | 1 095 746 | 1 061 246 | - | - | 34 500 |
| Total des fonds propres disponibles pour couvrir le minimum de capital requis | 1 061 246 | 1 061 246 | - | - | |
| Total des fonds propres éligibles pour couvrir le capital de solvabilité requis | 1 095 746 | 1 061 246 | - | | 34 500 |
| Total des fonds propres éligibles pour couvrir le minimum de capital requis | 1 061 246 | 1 061 246 | - | | |
| Capital de solvabilité requis | 675 091 | | | | |
| Minimum de capital requis | 198 212 | | | | |
| Ratio fonds propres éligibles sur capital de solvabilité requis | 162,31% | | | | |
| Ratio fonds propres éligibles sur minimum de capital requis | 535,41% | | | | |
| Reserve de réconciliation | | | | | |
| Excédent d'actif sur passif | 1 061 246 | | | | |
| Actions propres (détenues directement et indirectement) | | | | | |
| Dividendes, distributions et charges prévisibles Autres éléments de fonds propres de base | 489 130 | | | | |
| Ajustement pour les éléments de fonds propres restreints relatifs aux portefeuilles sous ajustement | - | | | | |
| égalisateur et aux fonds cantonnés Reserve de reconciliation | 572 116 | | | | |
| Bénéfices attendus | | | | | |
| Bénéfices attendus inclus dans les primes futures (EPIFP) - activités vie | 60 771 | | | | |
| Bénéfices attendus inclus dans les primes futures (EPIFP) - activités non-vie | | | | | |
| Total bénéfices attendus inclus dans les primes futures (EPIFP) | 60 771 | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> ANNEXES
===

<!-- PageHeader="108 / 111" -->

S.25.01.21 - CAPITAL DE SOLVABILITÉ REQUIS - POUR LES ENTREPRISES QUI UTILISENT LA FORMULE STANDARD

Article 112

Capital de solvabilité PPE Simplifications

requis brut

Risque de marché

1 126 034

17 955

Risque de défaut de la contrepartie

Risque de souscription en vie

1 305 074

Risque de souscription en santé

0

Risque de souscription en non-vie

0

Diversification

\-518 235

Risque lié aux immobilisations incorporelles

0

Capital de solvabilité requis de base

1 930 828

Calcul du capital de solvabilité requis

Risque opérationnel

43 375

Capacité d'absorption des pertes des provisions

\-1 285 205

techniques

Capacité d'absorption de pertes des impôts différés

\-13 907

Capital requis pour les activités exercées

0

conformément à l'article 4 de la directive 2003/41/CE

Capital de solvabilité requis à l'exclusion des exigences de capital supplémentaire

Exigences de capital supplémentaire déjà définies

Capital de solvabilité requis

675 091

Autres informations sur le SCR

675 091

0

Capital requis pour le sous-module risque sur actions fondé sur la durée Total du capital de solvabilité requis notionnel pour la part restante Total du capital de solvabilité requis notionnel pour les fonds cantonnés Total du capital de solvabilité requis notionnel pour les portefeuilles sous ajustement égalisateur Effets de diversification dus à l'agrégation des nSCR des FC selon l'article 304


## Approche basée sur le taux d'imposition

Approche basée sur le taux d'imposition moyen

|||
| - | - |
|| Calcul de la capacité d'absorption de pertes des impôts différés LAC DT  -13 907 |
| LAC DT justifiée par la reprise de passifs d'impôts différés | -13 907 |
| LAC DT justifiée au regard de probables bénéfices économiques imposables futurs | |
| LAC DT justifiée par le report en arrière, exercice en cours | |
| LAC DT justifiée par le report en arrière, exercices futurs | |
| LAC DT maximale | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

> ANNEXES
===

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

<!-- PageNumber="109 / 111" -->

> ANNEXES

<!-- PageHeader="110 / 111" -->

S.28.01.01 - MINIMUM DE CAPITAL REQUIS (MCR) - ACTIVITE D'ASSURANCE OU DE REASSURANCE VIE UNIQUEMENT OU ACTIVITE D'ASSURANCE OU DE REASSURANCE NON-VIE UNIQUEMENT


## Terme de la formule lineaire pour les engagements d'assurance et de reassurance non-vie

RésultatMCRNL

| | Meilleure estimation et PT calculées comme un tout, nettes (de la réassurance / des véhicules de titrisation) | Primes émises au cours des 12 derniers mois, nettes (de la réassurance) |
| - | - | - |
| Assurance frais médicaux et reassurance proportionnelle y afférente | | |
| Assurance de protection du revenu, y compris reassurance proportionnelle y afférente | | |
| Assurance indemnisation des travailleurs et reassurance proportionnelle y afférente | | |
| Assurance de responsabilité civile automobile et reassurance proportionnelle y afférente | | |
| Autre assurance des véhicules à moteur et réassurance proportionnelle y afférente | | |
| Assurance maritime, aérienne et transport et réassurance proportionnelle y afférente | | |
| Assurance incendie et autres dommages aux biens et réassurance proportionnelle y afférente | | |
| Assurance de responsabilité civile générale et réassurance proportionnelle y afférente | | |
| Assurance crédit et cautionnement et réassurance proportionnelle y afférente | | |
| Assurance de protection juridique et reassurance proportionnelle y afférente | | |
| Assurance assistance et reassurance proportionnelle y afférente | | |
| Assurance pertes pécuniaires diverses et réassurance proportionnelle y afférente | | |
| Reassurance santé non proportionnelle | | |
| Reassurance accidents non proportionnelle | | |
| Reassurance maritime, aérienne et transport non proportionnelle | | |
| Reassurance dommages non proportionnelle | | |


## Terme de la formule linéaire pour les engagements d'assurance et de réassurance vie

RésultatMCRL

198 212

| | Meilleure estimation et PT calculées comme un tout, nettes (de la réassurance / des véhicules de titrisation) | Montant total du capital sous risque net (de la réassurance/ des véhicules de titrisation) |
| - | - | - |
| Engagements avec participation aux bénéfices - Prestations garanties | 7 400 672 | |
| Engagements avec participation aux bénéfices - Prestations discretionnaires futures | 1 710 848 | |
| Engagements d'assurance avec prestations indexées et en unités de compte | 766 056 | |
| Autres engagements de (ré)assurance vie et de (ré)assurance santé | - | |
| Montant total du capital sous risque pour tous les engagements de (ré)assurance vie | | 11 412 618 |


### Calcul du MCR global

|||
| - | - |
| MCR linéaire :selected: | 198 212 |
| Capital de solvabilité requis | 675 091 |
| Plafond du MCR :unselected: | 303 791 |
| Plancher du MCR | 168 773 |
| MCR combiné :selected: | 198 212 |
| Seuil plancher absolu du MCR | 4 000 |
| Minimum de capital requis | 198 212 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2022" -->

MAIF.FR Retrouvez-nous aussi sur

MAIF VIE - société anonyme au capital de 122 000 000 € - RCS Niort 330 432 782 - CS 90000 - 79029 Niort cedex 9. Entreprise régie par le Code des assurances.

<figure>

![](figures/19)

<!-- FigureContent="MAIF" -->

</figure>
'''

response = get_openai_response(prompt)    
print(response) 



1) Présentation de l'organisme.
MAIF VIE est une société anonyme d'assurance vie, filiale du groupe MAIF, créée le 14 septembre 1984 et régie par le Code des assurances. Elle est également une société à mission depuis 2021, inscrivant sa mission d'intérêt collectif dans ses statuts. MAIF VIE est supervisée par l'Autorité de Contrôle Prudentiel et de Résolution (ACPR) et ses comptes sont audités par PricewaterhouseCoopers Audit et Mazars.

2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres, Résultat technique, Résultat financier, Résultat net après impôts)
En 2022, MAIF VIE a enregistré des encaissements bruts de réassurance de 834 368 milliers d'euros, en hausse de 3% par rapport à 2021. La société a servi des taux de rémunération dans la moyenne du marché (2,10 % net) et a porté sa provision pour participation aux bénéfices à 5,7 % des provisions mathématiques en euros de ses contrats d'épargne. Le résultat net comptable s'élève à 51 214 millier

In [6]:
prompt = '''Tu es un assistant expert dans le domaine financier et en assurance. Tu sais bien extraire les informations à partir d'un
passage de texte.

Ces informations sont généralement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
-Placements autres que les actifs en represenation de contrats en UC ou indexe.
-Changement de la gouvernance.
-Perspectives vision de l'organisme et projets futurs.

#Règles:
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
- N'invente pas des informations qui ni fugrent pas dans le texte.
- Et le resume de fin devrait sous forme de six paragraphes comme suit:
1) Présentation de l'organisme.
2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
3) Provisionnement
4) Placements autres que les actifs en represenation de contrats en UC ou indexe.
5) Changement de la gouvernance.
6) Perspectives vision de l'organisme et projets futurs.

Le tyexte est le suivant:
'{
    MAIF VIE 2021

<figure>

![](figures/0)

</figure>


Rapport sur la SOLVABILITÉ et la SITUATION FINANCIÈRE

<figure>

![](figures/1)

<!-- FigureContent="MAIF" -->

</figure>


assureur militant

| SOMMAIRE | 2 / 107 |
| - | - |
| PRÉSENTATION & SYNTHÈSES | 4 |
| PRESENTATION | 5 |
| SYNTHÈSES | 6 |
| ACTIVITÉ & RÉSULTATS | 10 |
| A Activité & Résultats | 11 |
| A.1 Activité | 11 |
| A.2 Résultats de souscription | 15 |
| A.3 Résultats des investissements | 18 |
| A.4 Résultats des autres activités | 19 |
| A.5 Autres informations | 20 |
| SYSTÈME DE GOUVERNANCE | 21 |
| B Système de Gouvernance | 22 |
| B.1 Informations générales sur le système de gouvernance | 22 |
| B.2 Exigences de compétences et d'honorabilité | 30 |
|| B.3 Système de gestion des risques y compris l'évaluation interne des risques et de la  |
| solvabilité | 32 |
| B.4 Système de contrôle interne | 38 |
| B.5 Fonction d'audit interne | 39 |
| B.6 Fonction actuarielle | 40 |
| B.7 Sous-traitance | 41 |
| B.8 Autres informations | 43 |
| PROFIL DE RISQUE | 44 |
| C Profil de Risque | 45 |
| C.1 Risque de souscription | 46 |
| C.2 Risque de marché | 49 |
| C.3 Risque de crédit | 52 |
| C.4 Risque de liquidité :unselected: | 55 |
| C.5 Risque opérationnel :unselected: | 57 |
| C.6 Autres risques importants :unselected: | 58 |
| C.7 Autres informations :unselected: | 61 |
| VALORISATION À DES FINS DE SOLVABILITÉ | 62 |
| D Valorisation à des fins de Solvabilité :unselected: | 63 |
| D.1 Actifs :unselected: | 66 |
| D.2 Provisions techniques :unselected: | 70 |
| D.3 Autres passifs :unselected: | 79 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

| SOMMAIRE 3 / 107 ||
| - | - |
| D.4 Méthodes de valorisation alternatives | 82 |
| D.5 Autres informations | 82 |
| GESTION DU CAPITAL | 83 |
| E Gestion du capital | 84 |
| E.1 Fonds propres | 84 |
| E.2 Capital de solvabilité requis et Minimum de capital requis | 90 |
| E.3 Utilisation du sous-module "risque sur actions" fonde sur la durée dans le calcul du Capital ||
| de solvabilité requis | 94 |
| E.4 Différences entre la formule standard et tout modèle interne utilisé | 94 |
| E.5 Non-respect du Minimum de capital requis et non-respect du Capital de solvabilité requis ||
| | 94 |
| E.6 Autres informations | 94 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

PRÉSENTATION & SYNTHÈSES

PRÉSENTATION& SYNTHÈSES
===

<!-- PageHeader="5 / 107" -->

PRESENTATION
===

Solvabilité 2 est une directive européenne réformant la réglementation prudentielle du secteur de l'assurance et entrée en vigueur le 1er janvier 2016.

Solvabilité 2 crée un cadre réglementaire harmonisé et fondé sur le risque pour les entreprises d'assurance, avec des règles quantitatives et des exigences en matière de gouvernance et d'information prudentielle. Les règles communes visent à faciliter les activités des compagnies d'assurances au sein de l'Union Européenne, à garantir une gestion saine et prudente dans des périodes difficiles et à protéger les souscripteurs d'assurances. Cette réglementation impose un niveau de détention de fonds propres et des règles en matière de gestion et de surveillance.

Les exigences sont structurées en trois piliers :

· premier pilier : exigences quantitatives, notamment en matière de fonds propres et de calculs des provisions techniques,

· deuxième pilier : exigences en matière d'organisation et de gouvernance des organismes,

· troisième pilier : exigences en matière d'informations prudentielles et de publication.

Le présent rapport s'inscrit dans le cadre de ce troisième pilier.

Le rapport sur la solvabilité et la situation financière (Solvency and Financial Conditions Report en anglais - SFCR), décrit l'activité de l'entreprise, son système de gouvernance, son profil de risque et donne des informations sur les méthodes de valorisation utilisées et la gestion du capital.

Le rapport sur la solvabilité et la situation financière (SFCR) comprend en annexe différents états quantitatifs.

Les données financières présentées dans ce rapport sont, sauf exceptions dûment précisées, exprimées en milliers d'euros.

Après avis du comité des risques, d'audit et des comptes du 22 mars 2022, le présent rapport a été approuvé par le conseil d'administration du 30 mars 2022.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="PRÉSENTATION& SYNTHÈSES" -->

<!-- PageHeader="6 / 107" -->


## SYNTHÈSES

MAIF VIE est une société anonyme régie par le Code des assurances, intervenant principalement pour les risques vie. Elle est intégralement détenue par la MAIF.

Constituée le 14 septembre 1984, MAIF VIE s'est développée en restant fidèle à ses valeurs fondatrices : solidarité, responsabilité et respect de la personne.

Depuis 2021, MAIF VIE a adopté le statut de société à mission, inscrivant et affirmant ainsi sa mission d'intérêt collectif, fondée sur une attention sincère portée à toutes ses parties prenantes.


### Activité et résultats

L'année 2021 se positionne comme la troisième du Plan stratégique Groupe 2019-2022 « Engagés pour demain » dans lequel MAIF VIE s'inscrit. Les indicateurs de performance à fin d'année sont en ligne avec les objectifs cibles définis.

L'activité de l'exercice 2021 est principalement caractérisée par :

› des encaissements bruts de réassurance de 810 412 milliers d'euros, en hausse de 10% par rapport à 2020 ;

› des taux de rémunération servis dans la bonne moyenne du marché (1,30 % net) ;

› une provision pour participation aux bénéfices portée à 6,04 % des provisions mathématiques en euros de nos contrats d'épargne en augmentation de 0,43 point par rapport à fin 2020 ;

› un résultat net comptable de 50 332 milliers d'euros permettant de renforcer les fonds propres tout en préservant des marges de manœuvre pour l'avenir.

Les événements constatés en ce début d'année 2022 relatifs au conflit entre la Russie et l'Ukraine n'ont pas d'impacts significatifs sur les actifs financiers du Groupe MAIF étant donné l'absence d'expositions directes et l'exposition marginale indirecte sur ces pays. Le portefeuille d'actifs continue de faire l'objet d'un suivi régulier dans le cadre de l'évolution incertaine, à date, du conflit. Par ailleurs, il n'a pas été identifié de contrat d'assurance touché par les mesures de gel des avoirs.


### Système de gouvernance

MAIF VIE place le conseil d'administration au sommet de la gouvernance et distingue, depuis le conseil d'administration du 17 décembre 2015, les fonctions de président du conseil d'administration et de directeur général. Le règlement intérieur du conseil d'administration, adopté en juin 2017, complète les statuts de MAIF VIE notamment sur les engagements des administrateurs.

Dominique Mahé, président du conseil d'administration, et Hélène N'Diaye, directrice générale, sont les dirigeants effectifs de MAIF VIE. A ce titre, ils disposent d'un domaine de compétences et de pouvoirs très larges sur les activités et les risques de MAIF VIE et sont systématiquement impliqués dans les décisions ayant un impact important. Le système de gouvernance comprend un système de gestion des risques et de contrôle interne, organisé autour de quatre fonctions clés : gestion des risques, actuarielle, audit interne et vérification de la conformité.

Les quatre fonctions clés accompagnent le conseil d'administration dans sa mission d'orientation et de contrôle et disposent d'un libre accès aux instances de gouvernance.

La gouvernance décisionnelle est structurée autour des quatre thèmes suivants : souscription et produits, pilotage économique et prudentiel, gestion des placements et pilotage des risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

PRÉSENTATION& SYNTHÈSES
===

<!-- PageNumber="7 / 107" -->

Le président du conseil d'administration, la directrice générale et les responsables des fonctions clés répondent aux exigences de compétences et d'honorabilité fixées par la réglementation.

MAIF VIE considère que son système de gouvernance est en adéquation avec sa stratégie, la nature et l'ampleur des risques inhérents à son activité d'assureur.

Elle veille en permanence à l'efficacité et à l'adéquation de son système de gouvernance avec la stratégie, la nature et l'ampleur des risques inhérents à ses activités.

Ainsi, tout au long de l'exercice 2021 marqué comme en 2020 par la pandémie de Covid-19, les dirigeants effectifs et les Conseils d'administration de MAIF et des filiales ont été régulièrement informés de la situation et ont pu prendre les décisions qui s'imposaient dans le cadre de la gouvernance du groupe conformément à la politique de continuité des activités du groupe ..

Comme en 2020, l'entreprise a utilisé la possibilité de recourir aux moyens de visioconférence et de télécommunication, y compris pour les réunions relatives à l'arrêté ou à l'examen des comptes annuels.

L'entreprise a continué de respecter les consignes et mesures gouvernementales prises depuis le début de l'épidémie visant à limiter la propagation de l'épidémie du Covid-19. Des dispositions ont été ainsi prises pour protéger et accompagner au mieux les collaborateurs tout en permettant d'assurer la continuité de la mission de service grâce au recours massif au travail à distance.


## Profil de risque

MAIF VIE est une société d'assurance dont l'activité principale est de couvrir les besoins des adhérents en matière d'assurance vie : protéger sa famille, assurer l'avenir de ses enfants, se constituer par l'épargne un complément de retraite, favoriser la transmission d'un capital ... À ce titre, elle n'est pas exposée à des risques spécifiques par rapport au marché et ne couvre pas de risques de nature exceptionnelle.

Les principaux risques auxquels MAIF VIE est exposée résultent de son activité d'assureur vie, de son organisation et de l'environnement, notamment économique. Ils sont liés :

› à la souscription : une forte hausse des rachats sur les produits d'épargne et une dérive de la performance économique de plusieurs produits comme conséquence de leur distribution et de leur tarification ;

Pour limiter les risques de souscription, MAIF VIE entretient une relation privilégiée avec ses adhérents et soigne son accessibilité et la qualité de ses prestations. Le haut niveau de fidélité des adhérents et assurés confirme la réalité d'une relation exceptionnelle sur le marché de l'assurance vie ;

› aux marchés financiers : une hausse brutale et perenne des taux d'intérêt associée à une chute des marchés actions (sans retour à la situation initiale avant plusieurs années) ou bien des taux durablement bas mettant sous pression la rentabilité des assureurs vie et leur capacité à garantir le capital ;

Pour faire face à ces risques de marché, MAIF VIE s'inscrit dans une démarche de gestion saine, prudente et raisonnée de ses investissements financiers.

› aux actes opérationnels dus à des défaillances de personnes, de systèmes ou d'évènements externes ou de données : cybercriminalité (vol de données, utilisation frauduleuse de données ... ), indisponibilité des systèmes d'information, défaut d'un sous- traitant important ou critique, non-respect aux obligations règlementaires (lutte contre le blanchiment et le financement du terrorisme, contrats non réglés, protection de la clientèle dont le devoir de conseil, la loi Informatique et liberté, ... ) ;

Pour gérer les risques opérationnels, MAIF VIE s'inscrit pleinement dans le respect de la réglementation et procède régulièrement à des audits.

› à la complexité et la précision des outils de modélisation développés par l'entreprise pour mesurer les engagements, les risques et la solvabilité ;

L'encadrement des modèles mis en place par MAIF VIE limite la portée de ce risque : MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

PRÉSENTATION& SYNTHÈSES
===

<!-- PageNumber="8 / 107" -->

existence d'une gouvernance stricte concernant l'utilisation et les évolutions des modèles. Une documentation robuste et structurée vient compléter ces dispositions pour assurer une accessibilité et une compréhension du fonctionnement des modèles.

› aux attentes sociales et environnementales, notamment en matière d'investissements financiers. Le groupe MAIF est sensible par ses valeurs à son empreinte sociale et environnementale, et souhaite renforcer l'impact positif de ses activités sur son environnement.

› à l'évolution du cadre sociétal, règlementaire et fiscal ;

› aux activités emergentes : une montée en puissance des "robot-conseiller" générant une fuite de l'activité d'épargne de MAIF VIE ; La gestion des risques émergents fait partie intégrante du processus de gestion des risques au sein de MAIF VIE et du groupe MAIF.

MAIF VIE dispose d'un ensemble de dispositifs de gestion des risques qu'elle juge maîtrisé et adapté. Le dispositif en place est réajusté régulièrement via les travaux conduits au sein du système de gestion des risques et contribue à la performance et à la pérennité de la société.

La diversification des risques, le niveau et la qualité des fonds propres, la stratégie d'investissement prudente sont autant d'éléments qui ont assuré à MAIF VIE de fortes capacités de résistance alors même que les incertitudes sont nombreuses et les risques accrus. MAIF VIE s'assure d'un impact positif de ses activités ainsi que de leur durabilité.

Dans cette période de crise liée à la pandémie COVID-19 qui se prolonge, MAIF VIE a su assurer la continuité de ses activités et continue de rester attentive à toute évolution de son profil de risque.

En ce début d'année 2022, les marchés financiers se montrent volatiles. Des tensions sur les matières premières et des pressions inflationnistes se manifestent, renforcées par un cadre géopolitique tendu. Le groupe MAIF procède à une veille active sur ce domaine.


## Valorisation

L'établissement du bilan en normes Solvabilité 2 requiert de déterminer la valeur économique de l'ensemble des actifs et passifs. Les principales différences entre le bilan en normes françaises et le bilan en normes Solvabilité 2 s'expliquent par :

› la valorisation des actifs à leur valeur de marché conduisant à inclure des plus ou moins- values latentes dans les postes placements de l'actif du bilan en normes Solvabilité 2 alors que le bilan en normes françaises retient essentiellement le coût historique (prix d'achat) net de dépréciations ;

› le remplacement d'une estimation suffisante des engagements envers les assurés (dans le bilan en normes françaises) par la meilleure estimation des flux futurs actualisés des règlements auxquels s'ajoute une marge de risque (dans le bilan en normes Solvabilité 2). MAIF VIE utilise la correction pour volatilité de la courbe des taux prévue par la Directive Solvabilité 2 ;

› le reclassement de 85,4% de la Provision pour Participation aux Bénéfices (PPB) en fonds excédentaires, conformément à l'arrêté du 24 décembre 2019 et en application des préconisations de l'ACPR portant sur les modalités de calcul à retenir par les entreprises d'assurance ;

› la comptabilisation au bilan prudentiel d'impôts différés qui correspondent à des suppléments ou des deductions futurs d'impôts, lies notamment aux évolutions précédentes.

Compte tenu de la qualité de son portefeuille d'instruments financiers, MAIF VIE considère que la valorisation en normes Solvabilité 2 de l'essentiel de ses placements ne présente pas d'incertitudes significatives.

La réglementation a mis en place des mesures transitoires pour faciliter la transition vers le nouveau dispositif prudentiel. S'agissant du calcul des engagements envers les assurés en normes Solvabilité 2, MAIF VIE a fait le choix de ne pas avoir recours aux mesures transitoires et répond d'ores et déjà aux exigences.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageNumber="9 / 107" -->

PRÉSENTATION& SYNTHÈSES
===


## Gestion du capital

MAIF VIE doit couvrir les exigences réglementaires (notamment Capital de solvabilité requis - SCR - et Minimum de capital requis - MCR) par des fonds propres qui remplissent les critères de qualité attendus pour être éligibles, et en quantité suffisante.

MAIF VIE détermine les exigences de capital à l'aide de la formule standard qui reflète le profil de risque moyen des entreprises d'assurance européennes. MAIF VIE considère que le calibrage de cette formule est adapté à la mesure de son profil de risque.

La réglementation a mis en place des mesures transitoires pour faciliter la transition vers le nouveau dispositif prudentiel. S'agissant du calcul des fonds propres et du SCR, MAIF VIE a fait le choix de ne pas avoir recours aux mesures transitoires et répond d'ores et déjà aux exigences.

Au 31 décembre 2021, MAIF VIE, disposant principalement de fonds propres de base de niveau 1 c'est-à-dire les plus disponibles pour absorber des pertes et de plus grande qualité, reconnue par la règlementation Solvabilité 2 (1 187 963 milliers d'euros) et avec ses titres subordonnés remboursables de fonds propres de niveau 3 (34 500 milliers d'euros), couvre de manière satisfaisante les exigences avec les ratios suivants :

› 2,5 fois le SCR - Capital de solvabilité requis pour un montant de 492 938 milliers d'euros (avec un taux de couverture de 248 %) ;

› 5,4 fois le MCR - Minimum de capital requis pour un montant de 221 822 milliers d'euros (avec un taux de couverture de 536 %).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->
<figure>

![](figures/2)

<!-- FigureContent="A" -->

</figure>


ACTIVITÉ & RÉSULTATS

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="11 / 107" -->


## A Activité & Résultats

Ce paragraphe est consacré à la présentation :

› de MAIF VIE et de son activité ;

› des résultats de souscription, des investissements et des autres activités.


### A.1 Activité


#### A.1.1 Présentation de la société

MAIF VIE est une société anonyme au capital de 122 000 000 euros constituée le 14 septembre 1984 , régie par le Code des assurances, située au 50 avenue Salvador Allende - « Le Pavois >> CS90000 79029 NIORT Cedex 9 (Téléphone : 05.49.04.49.04 - Site internet : www.maif.fr).

Son superviseur est l'Autorité de Contrôle Prudentiel et de Résolution (ACPR), 4 Place de Budapest CS 92459 75436 Paris Cedex 09 (Téléphone : 01.49.95.40.00 - Site internet : www.acpr.banque-france.fr).

Ses comptes en normes françaises sont audités par les cabinets de commissaires aux comptes suivants :

› PricewaterhouseCoopers Audit, 63 rue de Villiers 92208 Neuilly-sur-seine Cedex Téléphone : 01.56.57.58.59 - Site internet : www.pwc.fr Représenté par Frédéric Trouillard-Mignen.

› Mazars, Tour Exaltis 61 rue Henri Regnault 92400 Courbevoie Téléphone : 01.49.97.60.00 - Site internet : www.mazars.fr Représenté par Olivier Leclerc.

Créée en 1984, MAIF VIE compte aujourd'hui plus de 587 400 adhérents pour un encaissement brut de reassurance de 810 412 milliers d'euros réalisé sur le territoire français et un effectif en équivalent temps plein à la clôture de 212 (198 en 2020).

MAIF VIE assure le développement de l'offre d'assurance vie du groupe MAIF en apportant une réponse adaptée aux besoins des sociétaires MAIF :

› protéger sa famille ;

› assurer l'avenir de ses enfants ;

› se constituer, par l'épargne, un complément de retraite et de revenu ;

› favoriser la transmission d'un capital, ...

MAIF VIE est présente dans les domaines de :

› l'épargne et l'épargne retraite (contrat d'assurance vie en euros Nouveau Cap, contrats multisupports Assurance vie Responsable et Solidaire et PER Individuel MAIF) ;

› la prevoyance (contrats d'assurance deces toutes causes Rassurcap et Rassurcap Solutions, contrat de prevoyance complementaire des independants Assurance Prévoyance Professionnelle) ;

› la prise en charge des obsèques avec Sollicitudes ;

› l'assurance en couverture de prêts (Assurance Emprunteur MAIF, Avantage Emprunteur).

MAIF VIE gere ses produits principalement sous forme de contrats collectifs à adhésion facultative relevant des branches 20 et 22 du Code des assurances.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> ACTIVITÉ & RÉSULTATS

<!-- PageHeader="12 / 107" -->

La correspondance entre les lignes d'activité (segmentation Solvabilité 2 par groupe de risques homogènes) et les principales garanties des contrats est la suivante :

| Catégorie de produits | Produits | Ligne d'activité (LOB) |
| - | - | - |
| Épargne | Assurance vie monosupport - Cap'Epargne, Cap'Etudes - Nouveau Cap (y compris PEP), Capiplan | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance vie multisupports Compartiment euros - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DS Cap | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance vie multisupports Compartiments UC - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DS Cap | Assurance indexée ou en unités de comptes (UC) |
| Retraite ou assimilés | Épargne retraite - MAIF Régime à Cotisations Définies (y compris Retraite Solutions) - PER Responsable et Solidaire (depuis le 01/09/2020) Compartiment euros | Contrats d'assurance vie avec participation aux bénéfices |
|| Épargne retraite - PER Responsable et Solidaire (depuis le 01/09/2020) Compartiments UC | Assurance indexée ou en unités de comptes (UC) |
|| Épargne retraite MAIF Regime Prestations Assurance retraite - IFC | Contrats d'assurance vie avec participation aux bénéfices |
| Prévoyance | Obsèques vie entière - Sollicitudes | Contrats d'assurance vie avec participation aux bénéfices |
|| Temporaire décès - Rassurcap - Rassurcap Solutions - Assurance Prévoyance Professionnelle | Autres activités d'assurance vie |
|| Assurance en couverture de prêts - Assurance Emprunteur MAIF - Prêts immobiliers (y-compris Avantage Emprunteur) - Prêts à la consommation | Autres activités d'assurance vie |
|| Rentes | Contrats d'assurance vie avec participation aux bénéfices |

Au 31 décembre 2021, le capital social de MAIF VIE est réparti de la manière suivante :

› MAIF : 99,99 % ;

› 8 personnes physiques : 0,01 %.

MAIF VIE ne distribue pas de dividendes.

MAIF VIE entre dans le périmètre de consolidation de la société MAIF et est intégrée globalement.

En tant que mutuelle, la MAIF n'a ni capital social ni actionnaire à rémunérer. Elle est la société mère du Groupe dont elle consolide l'ensemble des sociétés d'importances significatives.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="13 / 107" -->

<!-- PageHeader="\> ACTIVITÉ & RÉSULTATS" -->

<figure>

![](figures/3)

<!-- FigureContent="MAIF Assurance non vie Intégration globale 100 % 100 % 100 % 71 % 67 % MAIF AVENIR MAIF VIE ALTIMA Assurances SMACL Holding Groupe Delta Assurance vie Assurances SA Investissement % C : 100 % Assurance non vie % 1 : 100 % % C: 100 % Assurance non vie Innovation Digital Immo % C: 71 % Immobilier %1:100 % %1:71% et Economie % C : 100 % Collaborative %1 : 100 % % C: 100 % 33 % % 1 : 100 % 0,4 % 29,4 % Groupe Delta Groupe Loisirs Evasion 44 % SOCRAM Banque Groupe IMA Assistance 56 % Immobilier 36 % 20 % Banque % C : 100 % % C : 20 % % C : 29 % %1 : 100 % Groupe %1:20 % %1:29 % Immo Pan Europe Immobilier % C : 100 % Entités Entités % 1 : 100 % 64 % consolidées consolidées par par mise en intégration équivalence globale" -->

</figure>


Groupe Delta Immo : Delta Immo, Delta Falguière, Pommerim, Delta Rue Monsieur, Paludate, Delta Galilée, Delta Réaumur et Delta Trévise

Groupe Immo Pan Europe : Immo Pan Europe, Immo Pan 1, IPE Fort Neipperg, Immo Pan 2, IPE Froissart Lanro SPV, Misor SPV

Groupe Delta Loisirs Evasion : OPCI Delta Loisirs Evasion et SAS Delta Loisirs Evasion

% C : Pourcentage de contrôle qui représente la fraction des droits de vote en AG que détient l'entreprise consolidante. Il correspond à l'addition des taux de participation tant que la chaîne de contrôle (cad %>50%) n'est pas rompue. Il sert à déterminer le périmètre et la méthode de consolidation.

% I : Pourcentage d'intérêt qui représente la fraction des droits aux bénéfices. Il correspond au produit des taux de participation et sert à déterminer la quote-part des sociétés mises en équivalence et à partager les capitaux propres entre la part du groupe et les intérêts minoritaires.


#### A.1.2 Objectifs généraux de l'entreprise et principales tendances


##### \> Objectifs généraux de l'entreprise

L'année 2021 marque la troisième année de mise en œuvre du plan stratégique Groupe « Engagés pour demain >> 2019-2022 au sein duquel MAIF VIE s'inscrit pleinement. Ce plan est articulé autour de 2 axes stratégiques majeurs que sont la singularité et le pivot :

› le premier axe precite vise à approfondir les caractéristiques de notre fonctionnement afin de proposer un modèle d'entreprise différent, créant plus de satisfaction chez nos sociétaires, plus d'épanouissement chez nos acteurs internes et un impact toujours plus positif sur notre environnement. L'adoption par l'Assemblée Générale du 23 juin 2021 du statut de Société à mission s'inscrit naturellement dans cette démarche ;

› le second, quant à lui, va nous permettre de continuer à évoluer, en intégrant au mieux les transformations de notre métier induites par le développement du numérique, dans l'expérience que l'on propose à nos sociétaires, dans le bon usage des données collectées. Il s'agit également de nous ouvrir progressivement à de nouvelles activités et/ou de nouveaux marchés, capables de compléter notre activité traditionnelle.

Clair dans nos objectifs et dans notre vision à 4 ans, ce plan stratégique garde une capacité d'adaptation dans sa déclinaison opérationnelle, indispensable pour répondre aux transformations et évolutions du marché, toujours plus profondes et rapides.


## Principales tendances ›

Au 31 décembre 2021, MAIF VIE compte 587 419 adhérents soit une augmentation de 5,2 % (+ 4,4 % en 2020).

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> ACTIVITÉ & RÉSULTATS
===

<!-- PageHeader="14 / 107" -->

Les taux de rémunération servis sur les principaux contrats d'épargne restent dans la bonne moyenne du marché (1,30 % nets servis en 2021).

Les encaissements totaux bruts de réassurance sont en progression par rapport à 2020 et s'élèvent à 810 412 milliers d'euros soit + 10 % (- 4 % en 2020).

Le montant des prestations augmente de 3,9 % (+ 1,3% en 2020), l'ensemble du secteur présente une hausse plus marquée avec + 7 % (+ 0,1 % en 2020). Hors arrivées à échéance sur les contrats Cap'Epargne, Cap'Etudes, Nouveau Cap et Nouveau Cap PEP, les prestations progressent de + 4,25 %.

En épargne, MAIF VIE enregistre une forte progression de la collecte nette (+ 32,2%), celle-ci s'établissant à + 218 M€. Le marché de l'assurance vie retrouve une situation de collecte nette positive (+ 23,7 milliards d'euros). Pour mémoire MAIF VIE avait conservé une collecte nette positive en 2020, là où le marché était en décollecte, marquant ainsi la confiance de ses adhérents.

Les provisions techniques brutes de réassurance progressent de 4,5 % par rapport à 2020 (+ 4,2 % en 2020) et portent principalement sur les contrats d'assurance en cas de vie.

L'exercice 2021 se traduit par un bénéfice de 50 332 milliers d'euros (5 042 milliers d'euros en 2020).

Les frais généraux, avec un montant total de 97 854 milliers d'euros augmentent de 12 913 milliers d'euros sous l'effet principalement de la hausse des refacturations intragroupes et des commissions versées à nos partenaires distributeurs, de la progression des effectifs en lien avec l'ambition de développement de l'assurance de personnes.

Néanmoins, cette augmentation s'accompagne d'une maîtrise de l'évolution des frais généraux (hors charges de distribution) qui représentent 0,46 % des provisions mathématiques.


## Principaux résultats de l'exercice ›

Pour l'exercice 2021, le résultat global de l'entreprise fait l'objet d'une présentation économique distinguant les résultats de gestion, technique, financier et exceptionnel hors décisions de gestion. Ces décisions de gestion recouvrent essentiellement les préconisations de la Direction générale validées par le conseil d'administration liées à la politique de taux de rendement servis, au niveau de la provision pour participation aux bénéfices et aux éventuelles incidences du contexte financier.

Il s'établit ainsi :

| En milliers d'euros | 2021 | 2020 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Résultat de gestion | - 3 784 | 2 956 | 6 740 - | NS |
| Résultat technique | 27 387 | 26 369 | 1 018 | + 3,9 % |
| Résultat financier | 37 263 | 35 038 | 2 224 | + 6,3 % |
| Résultat exceptionnel | - 459 | 1 591 - | 1 132 | - 71,1 % |
| RESULTAT COMPTABLE AVANT DECISIONS DE GESTION | 60 407 | 62 773 | - 2 366 | -3,8% |
| Incidences des taux servis | 70 454 | 63 065 | 7 388 | + 11,7 % |
| Dotation (-) ou Reprise (+) nette de la provision pour participation aux bénéfices | - 55 703 | -119 629 | 63 926 | - 53,4 % |
| Charges (-) et Produits (+) non techniques 1 | 953 | 710 | 243 | + 34,2 % |
| Participation et intéressement des salariés | - 5 361 | 518 - | 4 842 - | NS |
| Impôt sur les sociétés, participation et intéressement | - 20 417 | - 1 359 | 19 058 - | NS |
| RESULTAT NET COMPTABLE | 50 332 | 5 042 | 45 290 | NS |

NA : non applicable NS : non significatif

1 Produits non techniques d'un montant équivalent à l'impôt sur les sociétés relatif aux dotations nettes de reprises sur la réserve de capitalisation (Arrêté du 30 décembre 2010 relatif aux modalités de calcul des mouvements sur la réserve de capitalisation et modifiant le Code des assurances A 333-3).

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="15 / 107" -->

Le résultat avant décisions de gestion est en recul de 2 366 milliers d'euros, soit -3,8 % par rapport à 2020 sous les effets combinés de ses différentes composantes.

Le conseil d'administration de MAIF VIE, lors de sa réunion du 15 décembre 2021, a décidé de servir un taux de rémunération dans la bonne moyenne du marché tout en renforçant les fonds propres de l'entreprise et en sécurisant le taux de provision à la participation aux bénéfices sur provisions mathématiques ayant pour conséquences :

› un excédent global de produits financiers nets de 70 454 milliers d'euros ;

› une dotation nette de la provision pour participation aux bénéfices pour un montant de 55 703 milliers d'euros ;

› des différents taux servis nets à 1,30 % sur la majorité des fonds euros.

Les paragraphes suivants présentent les résultats de souscription, des investissements et des autres activités tels qu'ils apparaissent dans les états financiers établis en normes comptables françaises avec leur évolution par rapport à la période précédente. En complément, les résultats de souscription sont présentés par ligne d'activité Solvabilité 2 et les résultats des investissements par catégorie de placements Solvabilité 2.


## A.2 Résultats de souscription

Cette partie présente notamment les informations contenues dans l'état quantitatif "Primes, sinistres et dépenses par ligne d'activité". Il s'agit d'un état réglementaire en normes Solvabilité 2 qui a la particularité de présenter par ligne d'activité en normes Solvabilité 2 (cf. tableau de correspondance paragraphe A.1.1), des montants de produits et de charges valorisés selon les normes comptables françaises. Cet état n'intègre pas les revenus financiers issus de l'activité d'assurance.

Une ligne d'activité est un regroupement de garanties d'assurance similaires. Pour MAIF VIE, il existe 3 lignes d'activités (cf. paragraphe A.1.1. tableau de correspondance entre les lignes d'activité et les principales garanties des contrats MAIF VIE).


### A.2.1 Résultats au niveau agrégé et comparaison avec N-1

Le résultat global de souscription de MAIF VIE correspond aux résultats de gestion et technique générés par l'activité d'assurance vie.

Le résultat de gestion peut être défini comme la différence entre les ressources de gestion et les frais généraux.

| En milliers d'euros | 2021 | 2020 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Ressources de gestion | 94 070 | 87 897 | 6 173 | + 7,0 % |
| Frais sur versement | 20 933 | 19 508 | 1 425 | + 7,3 % |
| Frais sur épargne gérée | 61 177 | 58 091 | 3 086 | + 5,3 % |
| Autres ressources de gestion | 11 960 | 10 299 | 1 661 | + 16,1 % |
| Frais généraux | 97 854 - | 84 941 - | 12 913 | + 15,2 % |
| RESULTAT DE GESTION | - 3 784 | 2 956 | - 6 740 | NS |

Les frais sur versements sont en hausse en lien avec la croissance de la collecte brute épargne et la croissance du chiffre d'affaires prévoyance.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> ACTIVITÉ & RÉSULTATS
===

<!-- PageHeader="16 / 107" -->

Les frais sur épargne gérée augmentent de 3 086 milliers d'euros, soit +5,3 %, par rapport à l'année précédente, en cohérence avec la progression des provisions mathématiques.

Les autres produits de gestion sont également en hausse. Ils sont principalement composés des :

› frais de prorogation du contrat Cap'Épargne ;

›

› commissions reçues ou à recevoir des reassureurs relatives aux contrats de prévoyance (Rassurcap, Rassurcap Solutions et Sollicitudes) et à l'assurance en couverture de prêts ; produits de gestion perçus des partenaires rétrocessionnaires en lien direct avec la gestion des Unités de Comptes (UC). Ces rétrocessions sur encours progressent de près de moitié en lien notamment avec la hausse de l'encours UC et celle du taux de retrocessions.

› commissions d'apport et de distribution nettes de reassurance.

Les frais généraux, avec un montant total de 97 854 milliers d'euros (84 941 milliers d'euros en 2020), augmentent de 12 913 milliers d'euros (+ 15,2 %). Cette hausse traduit la volonté de soutenir le développement de l'assurance de personne avec, d'une part, la décision de revaloriser le niveau des commissions versées à MAIF sur l'activité épargne et, d'autre part, un renforcement des équipes et des outils au sein de MAIF VIE. Cette évolution concerne principalement :

› les commissions versées aux partenaires de distribution (groupe MAIF, MAIF Solutions Financières, MGEN, ... ) avec + 7 201 milliers d'euros principalement lies à la modification de l'accord de commissionnement épargne avec la MAIF ;

› les refacturations intragroupe avec + 3 356 milliers d'euros en lien notamment avec la revue du périmètre de refacturation pour y intégrer les charges de marketing et de communication institutionnelle ;

› les + 2 356 milliers d'euros restants s'analysent au regard des évolutions :

o des rémunérations des prestataires (+ 344 milliers d'euros) ;

o des charges de personnel (+ 913 milliers d'euros) ;
 :unselected:
o des impôts et taxes (+ 472 milliers d'euros) ;

o des charges de gestion courante (+ 191 milliers d'euros) ;

o des dotations aux amortissements et provisions (+ 436 milliers d'euros).

Le résultat technique correspond à l'excédent ou au déficit constaté lorsque l'on met en regard d'une part, les primes pures (cotisations après déduction des frais) et, d'autre part, les prestations versées et les provisions techniques constituées pour faire face aux engagements futurs.

Il s'établit, pour 2021, à 27 387 milliers d'euros, en progression de 3,9 % par rapport à 2020 (26 369 milliers d'euros) et se présente comme suit :

| En milliers d'euros | 2021 | 2020 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Rassurcap et Rassurcap Solutions | 23 690 | 20 762 | 2 928 | + 14,1 % |
| Assurance en couverture de prêts | 1 740 | 4 397 | - 2 658 | - 60,4 % |
| Sollicitudes | 1 218 | 624 | 594 | + 95,2 % |
| Rentes | 83 | 81 | 2 | + 2,5 % |
| Autres (Epargne, PGG (1)) | 657 | 505 | 152 | + 30,1 % |
| RESULTAT TECHNIQUE | 27 387 | 26 369 | 1 018 | 3,9% |

(1) Provision Globale de Gestion

Les résultats techniques de Rassurcap et Rassurcap Solutions s'élèvent globalement à 23 690 milliers d'euros contre 20 762 milliers d'euros en 2020 sous l'effet principalement de la hausse des cotisations.

L'assurance en couverture de prêts génère un résultat technique net de réassurance de 1 740 milliers d'euros (4 397 milliers en 2020), en retrait du fait principalement de la variation des provisions techniques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" --> :unselected: :unselected: :unselected: :unselected:

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="17/ 107" -->

Le résultat technique Sollicitudes s'établit à 1 218 milliers d'euros (624 milliers en 2020), en augmentation du fait de la hausse des cotisations conjuguée à une baisse de la charge de sinistres (en lien notamment avec la baisse de la provision de taux).

Les autres éléments du résultat technique atteignent 657 milliers d'euros (505 milliers en 2020), ils proviennent de l'activité Epargne et des variations de la Provision Globale de Gestion.


## A.2.2 Analyse de la performance de souscription par ligne d'activité (ou Lob «Line Of Business >>)

MAIF VIE est exclusivement concernée par 3 lignes d'activité au sens des normes Solvabilité 2 :

› Lob 1 : contrats d'assurance vie avec participation aux bénéfices ;

› Lob 2 : assurance indexée ou en unités de compte (UC) ;

› Lob 3 : autres activités d'assurance vie.

En 2021, les ressources de gestion sont en hausse de 7 % et principalement issues des contrats d'assurance vie avec participation aux bénéfices représentant plus de 72 % de l'encaissement total. Les contrats d'assurance en unités de compte et de prévoyance génèrent respectivement 11 % et 17 % des encaissements.

| En milliers d'euros | 2021 Lob 1 Lob 2 Lob 3 Total |||| 2020 ||||
|||||| Lob 1 | Lob 2 | Lob 3 | Total |
| - | - | - | - | - | - | - | - | - |
| Ressources de gestion | 67 787 | 10 014 | 16 271 | 94 070 | 65 081 | 7 380 | 15 437 | 87 898 |
| Frais sur versements | 9 991 | 561 | 10 381 | 20 933 | 9 011 | 411 | 10 086 | 19 508 |
| Frais sur épargne gérée | 56 648 | 4 529 | - | 61 177 | 54 463 | 3 628 | - | 58 091 |
| Autres produits de gestion | 1 148 | 4 924 | 5 890 | 11 960 | 1 607 | 3 341 | 5 351 | 10 299 |
| Frais généraux | 63 671 | 5 550 | 28 634 | 97 854 | 55 874 | 5 152 | 23 915 | 84 941 |
| RESULTAT DE GESTION | 4 116 | 4 464 | 12 363 - | 3 784 - | 9 207 | 2 228 | 8 478 - | 2 956 |
| Rassurcap et Rassurcap Solutions | | | 23 690 | 23 690 | | | 20 762 | 20 762 |
| Assurance en couverture de prêts | | | 1740 | 1 740 | | | 4 397 | 4 397 |
| Sollicitudes | 1 218 | | | 1 218 | 624 | | | 624 |
| Rentes | 83 | | | 83 | 81 | | | 81 |
| Autres (Epargne, PGG (1)) | 730 | 82 - | 8 | 656 | 612 | 114 - | 8 | 506 |
| RESULTAT TECHNIQUE | 2 031 | 82 - | 25 438 | 27 387 | 1 317 | 114 - | 25 167 | 26 369 |
| RESULTAT GLOBAL DE SOUSCRIPTION | 6 147 | 4 382 | 13 075 | 23 603 | 10 524 | 2 114 | 16 689 | 29 326 |

(1) Provision Globale de Gestion

Le résultat global de souscription de MAIF VIE s'établit à 23 603 milliers d'euros et diminue de 20 % entre 2020 et 2021. Les contrats d'assurance vie avec participation aux bénéfices et autres activités d'assurance vie représentent 81 % du résultat global de souscription 2021 de l'entreprise.


## Contrats d'assurance vie avec participation aux bénéfices ›

Cette ligne d'activité regroupe les contrats d'assurance vie monosupport, les compartiments euros des contrats multisupports, le contrat obsèques Sollicitudes et les produits de retraite hors compartiment UC du PERI.

Le résultat global de souscription est bénéficiaire et s'explique par des ressources de gestion supérieures aux frais généraux.


## Contrats d'assurance indexée ou en unités de compte (UC) ›

Cette activité concerne l'ensemble des compartiments UC des contrats d'assurance vie multisupports et du PERI.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="18 / 107" -->

<!-- PageHeader="\> ACTIVITÉ & RÉSULTATS" -->

› Autres activités d'assurance vie

Le résultat de souscription d'un montant de 13 075 milliers est constitué du résultat technique de Rassurcap et Rassurcap Solutions ainsi que celui de l'assurance en couverture de prêt.


## A.3 Résultats des investissements


### A.3.1 Les revenus et les dépenses générés par les activités d'investissement

Ils sont constitués par les produits financiers acquis à l'entreprise et qui sont, pour l'essentiel, ceux générés par les placements représentatifs des capitaux propres.

De manière simplifiée, le résultat financier peut être présenté de la manière suivante :

| En milliers d'euros | 2021 | 2020 | Variation ||
|||| (en montant) | (en %) |
| - | - | - | - | - |
| Produits financiers | 337 632 | 323 110 | 14 523 | + 4,5 % |
| Charges financières | - 46 693 | 46 408 - | - 285 | + 0,6 % |
| Produits financiers nets | 290 939 | 276 701 | 14 238 | + 5,1 % |
| Produits financiers affectés aux contrats au TRI (1) | 253 676 | 241 663 | 12 013 | + 5,0 % |
| RESULTAT FINANCIER | 37 263 | 35 038 | 2 224 | 6,3% |

(1) Taux de rendement interne des placements

Hors frais de gestion, les produits nets des placements s'élèvent à 290 939 milliers d'euros, en hausse de 5,1% % par rapport à 2020, soit 14 238 milliers d'euros. Les produits ont augmenté , en 2021, de 14 523 milliers d'euros et les charges de 285 milliers d'euros.

La comparaison avec l'exercice précédent fait apparaître les évolutions suivantes :

| Actif | 2021 || 2020 || Variation ||
|| Produits | Charges | Produits | Charges | Produits | Charges |
| - | - | - | - | - | - | - |
| Placements (autres que les actifs en | | | | | | |
| représentation de contrats en UC ou indexés) | 337 384 | 46 693 | 322 897 | 46 408 | 14 487 | 285 |
|||||||| 
| Immobilier (autre que pour usage propre) | | | | | | |
| Participations | 34 330 | 4 | 41 141 | | - 6 811 | 4 |
| Actions | 4 033 | 6 323 | 3 346 | | 687 | 6 323 |
| Obligations | 209 099 | 31 401 | 222 575 | 35 154 | - 13 476 | - 3 753 |
| Fonds d'investissement | 88 266 | 8 965 | 54 400 | 11 254 | 33 866 | - 2 289 |
| Produits dérivés | | | | | | - |
| Dépôts autres que ceux assimilables à de la trésorerie | 1 656 | | 1 435 | | 221 | |
| Autres placements | | | | | | |
| Prêts et prêts hypothécaires | 88 | | 129 | | 41 - | |
| Trésorerie et équivalent trésorerie | 160 | | 83 | | 77 | |
| TOTAL | 337 632 | 46 693 | 323 109 | 46 408 | 14 523 | 285 |
| Produits nets des placements | 290 939 || 276 701 || 14 238 ||

La hausse des produits de placements de 14 523 milliers d'euros provient notamment de :

› la réalisation de plus-values suite à la cession des fonds actions plus importantes qu'en 2020 ;

› la baisse des revenus sur les obligations liés à la baisse des taux d'intérêt ;

› en partie compensée par la baisse des revenus sur les fonds classes en participation notamment les fonds immobiliers ;

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> ACTIVITÉ & RÉSULTATS" -->

<!-- PageHeader="19 / 107" -->

La hausse des charges de placements de 285 milliers d'euros provient plus particulièrement de :

› la dépréciation sur une SCI classée en actions non cotées ;

› en partie compensée par des dépréciations moins importantes sur les OPCI ;

› une baisse des moins-values réalisées sur des cessions d'obligations

Les flux concernant les ajustements ACAV portant spécifiquement sur les placements en représentation des contrats en UC sont intégrés au niveau du résultat technique des UC.


## A.4 Résultats des autres activités


### A.4.1 Evolution des autres charges et produits

| Rubriques | 2021 | 2020 | Variation |
| - | - | - | - |
| Résultat exceptionnel | 459 - | - 1 591 | 1 132 |
| Incidences des taux servis | 70 454 | 63 065 | 7 389 |
| Dotation (-) ou Reprise (+) nette de la PPB | - 55 703 | -119 629 | 63 926 |
| Charges (-) et Produits (+) non techniques 1 | 953 | 710 | 243 |
| Participation et intéressement | 5 361 - | 518 - | 4 843 - |
| Participation | 4 854 | 33 | 4 821 |
| Intéressement | 507 | 486 | 21 |
| Supplément d'intéressement | - | - | - |
| Impôt sur les sociétés | 20 417 - | - 1 359 | - 19 058 |
| Autres charges et produits | - 10 534 | - 59 322 | 48 788 |


## › Résultat exceptionnel

Le résultat exceptionnel de l'exercice est principalement constitue de dotations pour provisions pour risques et charges et contentieux.


## › Décisions de gestion


### Incidence des taux servis

La politique des taux servis décidée par le conseil d'administration permet de dégager en 2021 un excédent net global de produits financiers de 70 454 milliers d'euros contre 63 065 milliers d'euros en 2020.


## Dotation ou reprise nette de la Provision pour participation aux bénéfices (PPB)

Au regard de la politique de taux servis décidés et de l'objectif cible du taux de PPB sur PM, la dotation nette de reprise de la Provision pour participation aux bénéfices 2021 s'élève à 55 703 milliers d'euros contre 119 629 milliers d'euros en 2020.


### ›


## Charges et produits non techniques

Des produits non techniques d'un montant équivalent à l'impôt sur les sociétés relatif aux dotations nettes de reprises sur la réserve de capitalisation ont été constatés pour un montant de 953 milliers d'euros (contre 710 milliers d'euros en 2020).

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> ACTIVITÉ & RÉSULTATS
===

<!-- PageHeader="20 / 107" -->


## › Participation et intéressement

Un accord de participation Groupe, signé le 11 juin 2015, repose conformément aux dispositions des articles L 3324-1 à L 3324-4 du Code du travail sur le montant le plus élevé déterminé entre la formule de droit commun et la formule dérogatoire.

Une charge de participation a été comptabilisée à hauteur de 4 854 milliers d'euros en 2021 contre 33 milliers d'euros en 2020.

L'intéressement des salariés, au titre de l'exercice 2021, sur la base de l'accord signé le 30 juin 2018 s'élève à 507 milliers d'euros (486 milliers d'euros en 2020). Aucun supplément d'intéressement n'a été versé sur l'année.

› Impôt sur les sociétés

La détermination de l'impôt sur les sociétés, conformément à la réglementation fiscale, s'élève à 20 417 milliers d'euros (contre 1 359 milliers d'euros en 2020).


## A.5 Autres informations

MAIF VIE n'identifie pas d'autre information importante à communiquer sur l'activité et les résultats en dehors de ce qui est décrit dans ce rapport.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->
<figure>

![](figures/4)

<!-- FigureContent="B" -->

</figure>


SYSTÈME DE GOUVERNANCE

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageHeader="22 / 107" -->


## B Système de Gouvernance

Cette partie est consacrée à la présentation d'informations qualitatives relatives :

› aux caractéristiques générales du système de gouvernance de MAIF VIE ;

› aux exigences de compétence et d'honorabilité en place, notamment pour ce qui concerne les dirigeants effectifs et les responsables des fonctions clés gestion des risques, vérification de la conformité, audit interne et actuarielle ;

› au système de gestion des risques, avec le rôle de la fonction clé gestion des risques ;

› au système de contrôle interne, avec le rôle de la fonction cle vérification de la conformité

AAA

; à la fonction clé d'audit interne ;

à la fonction clé actuarielle ;

à la sous-traitance.


## B.1 Informations générales sur le système de gouvernance


### B.1.1 Evaluation du système de gouvernance

MAIF VIE considère avoir mis en place un système de gouvernance permettant de garantir une gestion saine, prudente et efficace de l'activité, conforme aux exigences de la réglementation. Son système de gouvernance est proportionné à la nature, à l'ampleur et à la complexité de ses opérations.


### B.1.2 Structure de gouvernance et fonctions clés

La description ci-après vise à communiquer des informations relatives à l'organisation des fonctions de gouvernance au sein de MAIF VIE.


#### B.1.2.1 Conseil d'administration


##### ›


###### Composition du conseil d'administration

Les membres du conseil d'administration sont élus par l'assemblée générale des actionnaires pour six ans et rééligibles. Le conseil d'administration se renouvelle partiellement suivant le nombre de membres en fonction afin que le renouvellement soit complet dans chaque période de six ans. Le conseil d'administration de MAIF VIE au 31 décembre 2021 est composé de huit membres.


## Attributions du conseil d'administration ›

Le conseil d'administration détermine les orientations de l'activité de la société et veille à leur mise en œuvre. Sous réserve des pouvoirs expressément attribués par la loi aux assemblées d'actionnaires et dans la limite de l'objet social, il se saisit de toute question intéressant la bonne marche de la société et règle par ses délibérations les affaires la concernant. Il procède à tout moment aux contrôles et vérifications qu'il estime opportuns.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageNumber="23 / 107" -->

Le conseil d'administration valide les orientations en matière d'allocation stratégique d'actifs, de gestion actif/passif et de politique des investissements financiers et des placements. Il se prononce en particulier sur les modalités de choix des intermédiaires financiers et sur les opérations des instruments financiers à terme.

Le conseil d'administration dispose également depuis l'exercice 2015 du pouvoir de choisir, nommer et révoquer le directeur général, comme il en dispose pour le président.


## ›


## President du conseil d'administration, dirigeant effectif

Le président est désigné par le conseil d'administration.

Monsieur Dominique Mahe a été désigné président lors de la réunion du conseil d'administration du 17 décembre 2015.

Au titre de dirigeant effectif de l'entreprise, il dispose d'un domaine de compétences et de pouvoirs suffisamment large sur les activités et les risques de l'entreprise, et est impliqué dans les décisions ayant un impact important sur la société, notamment en matière de stratégie, de budget ou de questions financières. Il fait preuve d'une disponibilité suffisante pour exercer ce rôle.

En lien avec la stratégie (construction des orientations et suivi), il prend part aux travaux de construction et de suivi du Plan stratégique, à son adaptation/réorientation.

Le président est présent dans l'organisation, a accès aux informations et est un interlocuteur privilégié sur des sujets majeurs. Il dispose des compétences et de l'expérience requises sur les activités et les risques de l'entreprise, dispose d'une vue assez complète et approfondie de l'ensemble de l'activité, ainsi que de réels pouvoirs de décision et de contrôle.

Le président est également chargé d'assurer les missions règlementaires dévolues au président du conseil d'administration. A ce titre, il convoque et anime le conseil d'administration qu'il préside. Il fixe les ordres du jour et facilite l'exercice par le conseil d'administration de sa capacité d'initiative, de décision et de contrôle. Il s'assure que les administrateurs disposent des éléments nécessaires pour exercer leurs missions, s'assure du bon fonctionnement et de la cohésion du collectif. Il dirige la fonction clé d'audit interne.

Le président rend compte à l'assemblée générale et exécute ses décisions.

Il présente au conseil d'administration, avec le directeur général, les orientations de l'activité de la société (notamment en matière de stratégie et d'opérations financières) et veille après décision à leur mise en œuvre, il se saisit de toute question intéressant la bonne marche de la société pour la soumettre au conseil d'administration et lui présente tout contrôle et vérification qu'il a fait mettre en œuvre. Il assure le rayonnement de la société.


## Vice-président et administrateurs ›

Le président est secondé par le vice-président du conseil d'administration. Le vice-président est désigné par les membres du conseil d'administration.

Yves Pellicier est vice-président depuis le 15 juin 2017.

Le conseil d'administration comporte également 6 administrateurs : Eric Berthoux, Arnaud Courdesses, Pascal Demurger, Marie-Pierre Elie, Nadine Naas et Christian Ponsolle.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

24 / 107

›


## Activités du conseil d'administration en 2021

Au cours de l'exercice écoulé, le conseil d'administration s'est réuni 5 fois. Sur l'ensemble de ces réunions qui se sont déroulées au cours de l'exercice 2021, le taux de présence des administrateurs était en moyenne de 81 % et le taux de représentés de 3 %.

Les décisions rendues par le conseil d'administration et / ou les informations portées à sa connaissance ont notamment porté sur :

› l'arrêté des comptes en normes françaises et proposition d'affectation du résultat ;

› l'arrêté des comptes annuels prudentiels en normes Solvabilité 2 ;

› le suivi du budget 2021 et la présentation du budget 2022 ;

› les orientations relatives aux décisions de gestion et fixation des taux de rémunération des contrats et des taux garantis 2022 ;

› l'allocation stratégique d'actifs ;

› l'ensemble des politiques Solvabilité 2, le rapport ORSA et la politique de la surveillance et de la gouvernance produit ;

› la situation sanitaire ;

› la modification des statuts pour la transformation de MAIF VIE en Société à mission ;

› la présentation de la nouvelle directrice générale adjointe et responsable du pôle performance économique, solvabilité et gouvernance, et de la nouvelle responsable de l'unité actuariat et pilotage des risques ;

› la désignation des deux nouveaux responsables fonctions clés : gestion des risques et actuarielle ;

› le rapport annuel d'activité 2020 du comité des risques, d'audit et des comptes ;

› les rapports individuels RSR et SFCR ;

› le rapport de la fonction clé actuarielle ;

› le rapport de la fonction clé audit interne ;

› le rapport de la fonction cle vérification de la conformité ;

› le rapport de la fonction clé gestion des risques ;

› le rapport sur le contrôle interne du dispositif LCB/FT et le rapport sur les procédures d'élaboration et de vérification de l'information financière et comptable ;

› le suivi de l'activité conformité, contrôle interne et continuité ;

› le plan d'audit interne 2022 ;

› la politique de la société en matière d'égalité professionnelle et salariale ;

› la souscription par l'ASAP du contrat Sollicitudes et Assurance Prevoyance Professionnelle ;

› le renouvellement des mandats de Marie-Pierre Elie, Nadine Naas et Arnaud Courdesses ;

› le renouvellement du mandat du cabinet PricewaterhouseCoopers en qualité de commissaire aux comptes titulaire et la nomination de Bénédicte Vignon en qualité de commissaire aux comptes suppléant, en remplacement du Xavier Crépon ;

› la revue des conventions réglementées.


### B.1.2.2 Direction générale

Le directeur général est désigné par le conseil d'administration.

Hélène N'Diaye occupe cette fonction depuis la réunion du conseil d'administration du 24 mai 2018.

En cette qualité, elle assume la direction générale de l'entreprise, sous le contrôle du conseil d'administration et dans le cadre des orientations arrêtées par ce dernier.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTEME DE GOUVERNANCE
===

<!-- PageNumber="25 / 107" -->

Elle participe à l'élaboration de la stratégie, assure la mise en œuvre de la stratégie validée par le conseil d'administration, rend compte devant le conseil d'administration, préside les Comités de direction générale de la société et représente la société aux Comités financiers.

Elle est investie des pouvoirs les plus étendus pour agir en toutes circonstances au nom de la société dans la limite de l'objet social et sous réserve des pouvoirs que la loi attribue expressément à l'assemblée générale et au conseil d'administration.

Elle représente la société dans ses rapports avec les tiers et dirige les domaines d'activités et équipes opérationnelles de l'entreprise.

Trois des quatre fonctions clés (vérification de la conformité, actuarielle, gestion des risques) lui sont rattachées hiérarchiquement et lui rendent compte de leurs travaux.

Elle décide avec le président, des orientations budgétaires, de l'arrêté des comptes et du résultat de l'entreprise, de la fabrication ou de l'évolution d'un produit, de la politique de gestion tactique des placements et de l'allocation stratégique d'actifs et tous les sujets règlementaires relevant de Solvabilité 2, soumis à décision du conseil d'administration.

La directrice générale a désigné deux directeurs généraux adjoints. Au 31 décembre 2021, cette fonction est occupée par Marina Sellari, responsable du pôle performance économique, solvabilité et gouvernance, et Frédéric Lecanu, responsable du pôle transformations et opérations.


## B.1.2.3 Comités spécialisés du conseil d'administration

Le conseil d'administration de MAIF VIE ne dispose d'aucun comité spécialisé et n'en a pas constitué sur l'exercice écoulé. MAIF VIE figure, depuis 2009, dans le périmètre du comité des risques d'audit et des comptes de la MAIF. Ses principales missions sont décrites ci-après :

| Comités du conseil d'administration | Principales missions et activités en 2021 |
| - | - |
| Comité des risques d'audit et des comptes de MAIF Six administrateurs MAIF et deux personnes externes à l'entreprise. | Le Comité des risques, d'audit et des comptes est chargé, pour les sociétés incluses dans le périmètre de ses interventions précisé dans son règlement d'application pratique, à savoir le groupe MAIF, MAIF, et MAIF VIE, d'éclairer et d'alerter les conseils d'administration concernés dans le domaine des risques, des comptes, du contrôle interne et du contrôle externe, et des politiques écrites. |
|| Sans préjudice des compétences du conseil d'administration et de la direction générale, il est notamment chargé de : › suivre le processus d'élaboration de l'information financière et, le cas échéant, de formuler des recommandations pour en garantir l'intégrité ; › suivre l'efficacité du système de contrôle interne ; › suivre l'efficacité de l'audit interne, en ce qui concerne les procédures relatives à l'élaboration et au traitement de l'information comptable et financière, sans qu'il soit porté atteinte à l'indépendance de ce dernier ; › suivre la politique, les procédures et l'efficacité du système de gestion des risques ; › piloter la procédure de sélection des Commissaires aux comptes ; › émettre une recommandation sur les Commissaires aux comptes proposés à la désignation et au renouvellement par l'assemblée générale ; › veiller à l'indépendance des Commissaires aux comptes ; › suivre la réalisation du contrôle legal des comptes sociaux par les Commissaires aux comptes ; › auditionner les responsables des fonctions clés sur demande du conseil d'administration. |

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> SYSTÈME DE GOUVERNANCE
===

<!-- PageNumber="26 / 107" -->

|||
| - | - |
| | › établir son rapport annuel. En outre, le Comité des risques d'audit et des comptes approuve préalablement les services autres que la certification des comptes fournis par les Commissaires aux comptes ou les membres du réseau auquel ils appartiennent, après analyse des risques pesant sur l'indépendance des Commissaires aux comptes et des mesures de sauvegarde appliquées par ces derniers. Il doit rendre compte régulièrement au conseil d'administration de l'exercice de ses missions et l'informer sans délai de toute difficulté rencontrée. Il doit notamment rendre compte des résultats de la mission de certification des comptes, de la manière dont cette mission a contribué à l'intégrité de l'information financière et du rôle qu'il a joué dans ce processus. |
|| Par ailleurs, le règlement d'application pratique en vigueur définit les modalités de l'accès aux réunions du comité des fonctions clés Groupe et/ou solo. |

<figure>

![](figures/5)

</figure>


Ce comité exerce son activité sous la responsabilité du conseil d'administration, seul habilité à prendre les décisions et organise librement ses travaux dont il rend compte au moins annuellement dans un rapport remis au conseil d'administration.


## B.1.2.4 Fonctions clés

Le système de gouvernance de MAIF VIE comporte quatre fonctions clés règlementaires de contrôle, chacune dotée de responsables clairement identifiés :

› la fonction gestion des risques, confiée à un chef de projets Actuariat ;

› la fonction de vérification de la conformité, confiée au responsable de la conformité et des contrôles ;

› la fonction audit interne confiée au responsable de l'audit interne MAIF ;

› la fonction actuarielle, confiée à un chef de projets Actuariat.

Trois des quatre fonctions clés (gestion des risques, actuarielle et vérification de la conformité) et leurs travaux sont rattachées au directeur général de MAIF VIE.

Le responsable de la fonction d'audit interne a été désigné au sein du groupe MAIF (salarié MAIF), l'organisation de sa mission étant précisée dans une convention conclue entre MAIF VIE et MAIF.

Les rôles et missions de ces fonctions sont précisés en paragraphes B.3 pour la fonction gestion des risques, B.4.3 pour la fonction de vérification de la conformité, B.5 pour la fonction audit interne et B.6 pour la fonction actuarielle.

Les responsables des fonctions clés ont été formellement désignés par le directeur général et présentés au conseil d'administration.

Les fonctions clés disposent de l'autorité, des ressources et de l'indépendance opérationnelle nécessaires afin de mener leurs tâches à bien, de rendre des comptes au conseil d'administration et de le conseiller.

Les responsables des fonctions clés se tiennent en relation permanente avec les dirigeants effectifs, notamment dans le cadre des instances relatives à la gouvernance décisionnelle du groupe MAIF.

Lors de sa séance du 21 juin 2018, le conseil d'administration de Parnasse-MAIF (devenue depuis MAIF VIE) a validé une procédure d'accès au conseil d'administration des fonctions clés et un contrat de service relatif à la fonction d'audit interne de MAIF VIE.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="27 / 107" -->

<!-- PageHeader="\> SYSTEME DE GOUVERNANCE" -->

<figure>

![](figures/6)

<!-- FigureContent="Au 31 décembre 2021 Conseil d'administration de MAIF VIE DIRIGEANTS EFFECTIFS Président du Conseil d'administration Directrice générale Dominique Mahé Hélène N'Diaye Directrice générale Directeur général adjoint adjointe Marina Sellari Frédéric Lecanu Audit Interne MAIF Pôle Performance Economique, Solvabilité et Gouvernance Pôle Conformité et Contrôles Vianney Dumont Marina Sellari Domaine Pilotage Economique Financier et Prudentiel Fabrice Couturier Fonction audit interne Marina Sellari Fonction vérification de la conformité Chef de Projets Actuariat François Lair Christophe Lateulère Rattachement hiérarchique :unselected: Responsables des fonctions clés (ces responsables ont un lien fonctionnel avec les Dirigeants effectifs) Fonction gestion des risques Fonction actuarielle" -->

</figure>



### B.1.3 Changement important du système de gouvernance intervenu au cours de la période

Une nouvelle organisation relative à la désignation de François Lair en qualité de responsable fonction clé gestion des risques et de Christophe Lateulère en qualité de responsable fonction clé actuarielle a été mise en place.

Ces deux nouveaux responsables sont rattachés prudentiellement à la directrice générale, dirigeant effectif, pour tous les travaux afférents à leurs fonctions clés. Toutefois et en accord avec l'ACPR, François Lair et Christophe Lateulère effectuent également des missions relatives à leurs postes de chefs de projets actuariat et sont ainsi également rattachés à la responsable du pôle performance économique, solvabilité et gouvernance.


### B.1.4 Information sur la politique et les pratiques de rémunération et droits à rémunération sur la période

MAIF VIE considère que sa pratique de rémunération la préserve de toute prise de risques excessive par ses dirigeants et salariés afin de se centrer sur le seul intérêt qui prime, celui de l'adhérent.

La politique de rémunération est réexaminée chaque année dans le cadre des dispositifs prévus par la loi pour les classes 1 à 7 (NAO, participation, intéressement) et/ou en cas de changement significatif de la gouvernance ayant un impact sur les responsabilités qui seraient confiées.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="28 / 107" -->

La rémunération du président du conseil d'administration et celle du directeur général sont fixées par le conseil d'administration.


## B.1.4.1 Administrateurs

Les fonctions d'administrateurs ne sont pas rémunérées mais conformément à la réglementation, le conseil d'administration peut leur allouer des indemnités dans les limites fixées par l'assemblée générale, et leur rembourser leurs frais de déplacements dans le cadre de leurs missions.


### B.1.4.2 Dirigeants effectifs, membres de la Direction générale et cadres hors classe


## › Critères de rémunération

Le président ne perçoit ni variable, ni salaire, ni primes d'intéressement ou de participation au titre de cette fonction. Par ailleurs, il n'y a aucun salarié cadre hors classe au sein de l'entreprise. La directrice générale est salariée MAIF, ainsi que les deux directeurs généraux adjoints ; ces deux derniers sont détachés à 100% de leur temps à MAIF VIE.


## Avantages sociaux ›

Le président, la directrice générale et les directeurs généraux adjoints, relevant d'un statut de mandataire social ou de salarié MAIF hors classe, possèdent les avantages associés à ces statuts qui sont ceux accordés par MAIF. Ces avantages correspondent notamment, depuis le 01/01/2020, a un PER Obligatoire qui s'ajoute au régime à cotisations définies (1% de branche) dont bénéficie l'ensemble des salariés.

Le président et la directrice générale bénéficient d'un régime à cotisations définies (article 82 du Code Général des Impôts).

B.1.4.3 Salariés relevant des classes 1 à 7 de la convention collective des sociétés d'assurance


## Critères de rémunération ›

L'évolution du salaire de base des classes 1 à 7 est encadrée par un accord et par le dispositif des négociations annuelles obligatoires portant sur les salaires.

En 2021, la masse salariale a évolué de 1,85 % sous l'effet de l'ancienneté, des progressions et augmentations individuelles et des promotions.

La direction a souhaité le versement d'une prime exceptionnelle liée à l'implication des salariés sur le projet de rénovation des SI de MAIF VIE modulée en fonction de l'ancienneté à la date du 1er janvier 2021 correspondant à 1,72 % de la masse salariale.

Par principe, il n'y a pas de bonus individuel pour les salariés des classes 1 à 7. Les salariés bénéficient en outre d'un accord de participation qui vise à redistribuer une partie des résultats réalisés par le groupe ainsi que d'un accord d'intéressement spécifique conclu pour une période triennale.

L'intéressement et la participation versés en 2021 au titre de l'année 2020 ont représenté un avantage équivalent à 8,90 % de la masse salariale brute.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageHeader="29 / 107" -->

Cette rémunération variable collective est complétée d'un plan d'épargne entreprise et d'un plan d'épargne pour la retraite collective au bénéfice de l'ensemble des collaborateurs.

› Avantages sociaux

Les avantages sociaux constituent une part de rémunération supplémentaire qui s'ajoute au salaire de base. Ces avantages sociaux sont déconnectés de toute notion de performance individuelle et/ou collective. Ils sont attribués à tous les salariés bénéficiaires tels que prévus par la loi, la convention collective nationale ou les accords collectifs de MAIF VIE.

Sans être exhaustif, les avantages sociaux se retrouvent essentiellement dans les domaines suivants :

\-------

santé, Prévoyance ;

retraite ; transports ; aides familiales ;

aides au logement ;

restauration d'entreprise ; activités sociales et culturelles du comité social et économique.

En 2021, le poids des avantages sociaux représentait 7,87 % de la masse salariale brute pour les classes 1 à 7.


### B.1.4.4 Rémunération des fonctions-clés

La rémunération des fonctions clés est déterminée dans la politique de rémunération.

Pour la fonction clé Audit interne, le principe et la pratique de rémunération relèvent de la catégorie des cadres hors classe. La formalisation des objectifs et le taux du variable applicable pour cette fonction ne sont pas de nature à conduire à une prise de risque excessive.

Pour les autres fonctions : Actuarielle, Gestion des Risques et Vérification de la Conformité, le principe et la pratique de rémunération relèvent de la catégorie des salariés de la classe 1 à 7 de la Convention Collective des Sociétés d'Assurance. Il n'existe donc pas de rémunération variable pour ces fonctions clés.


#### B.1.5 Transactions importantes conclues avec des personnes exerçant une influence notable sur l'entreprise ou des membres du conseil d'administration ou de la Direction générale

Les contrats souscrits auprès de MAIF VIE par ses administrateurs, dirigeants salariés, leurs conjoints, ascendants et descendants ne comportent aucune condition préférentielle par rapport aux conditions pratiquées par les autres sociétaires.

Il n'a pas été accordé d'avance et de crédit à l'ensemble des membres des organes d'administration et de direction ni conclu de transactions importantes avec des personnes exerçant une influence notable sur l'entreprise ou des membres du conseil d'administration ou de la Direction générale.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="30 / 107" -->


## B.2 Exigences de compétences et d'honorabilité

MAIF VIE accorde une attention particulière aux exigences de compétence et d'honorabilité.


### B.2.1 Personnes concernées

Les membres du conseil d'administration, les dirigeants effectifs, les membres du Comité de direction générale ainsi que les responsables des fonctions clés répondent aux conditions d'honorabilité, d'expérience et de compétences qui leur sont applicables.

En outre, conformément à la directive sur la distribution d'assurances, les personnels MAIF prenant directement part à la distribution des produits d'assurance MAIF VIE et leurs responsables répondent, sous la responsabilité de la direction des richesses humaines groupe, à un profil de compétence et d'honorabilité en adéquation avec leurs missions et obligations.

B.2.2 Exigences spécifiques de compétences appliquées aux membres du conseil d'administration, aux dirigeants effectifs et aux responsables des fonctions clés


#### B.2.2.1 Membres du conseil d'administration

Le conseil d'administration veille à ce que ses membres disposent collectivement des connaissances et de l'expérience appropriée dans les marchés de l'assurance et les marchés financiers, la stratégie de la société et son modèle économique, le système de gouvernance de la société, l'analyse financière et actuarielle et les exigences législatives et règlementaires applicables à la société.

La compétence collective du conseil d'administration de MAIF VIE est assurée par les mandats alloués en son sein aux membres du conseil d'administration de MAIF VIE. Ces derniers répondent aux exigences de compétences et d'honorabilité fixées dans le cadre de la politique de compétences et d'honorabilité du groupe MAIF.

Par ailleurs, chaque administrateur est tenu de respecter les exigences d'honorabilité fixées par le Code des assurances et s'engage formellement à défendre et respecter les principes et obligations à caractère deontologique des administrateurs des mutuelles de l'association des assureurs mutualistes (AAM).


#### B.2.2.2 Dirigeants effectifs

Afin de garantir une gestion saine, prudente et avisée de la société, dans le respect des intérêts des assurés, les dirigeants effectifs nommés par le conseil d'administration répondent aux conditions d'honorabilité, de compétences et d'expérience nécessaires à leurs fonctions.

Les règles relatives au domaine de compétence et aux pouvoirs du président dirigeant effectif sont posées par le Règlement intérieur du conseil d'administration. Ce dernier précise également les parcours favorisant l'acquisition des compétences nécessaires à l'exercice de la présidence du conseil et de la direction effective.


##### B.2.2.3 Responsables des fonctions clés

Le directeur général arrête le profil individuel de compétences des responsables des fonctions clés. Ce profil est réévalué afin de tenir compte des évolutions législatives et règlementaires propres à la fonction.

Les responsables des fonctions clés à MAIF VIE (François Lair - fonction clé « Gestion des risques >>, Fabrice Couturier - fonction cle << Verification de la conformité >>, Christophe Lateulere - fonction cle << Actuarielle >> et Vianney Dumont - fonction cle << Audit interne > par

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTEME DE GOUVERNANCE
===

<!-- PageHeader="31/ 107" -->

convention conclue entre MAIF et MAIF VIE) possèdent l'honorabilité, les compétences (diplômes, qualifications professionnelles et connaissances) ainsi que l'expérience professionnelle nécessaires à leurs fonctions.

B.2.3 Processus d'appréciation des compétences et de l'honorabilité des membres du conseil d'administration, des dirigeants effectifs et des responsables des fonctions clés à MAIF VIE


## B.2.3.1 Membres du conseil d'administration

L'assemblée générale des actionnaires élit les administrateurs en s'appuyant sur l'information délivrée préalablement par le conseil d'administration.

Le dossier de candidature des postulants à un mandat d'administrateur élu par l'assemblée générale comprend une copie d'une pièce d'identité, leur curriculum vitae afin d'évaluer leurs diplômes, qualifications professionnelles, connaissances et expériences pertinentes, ainsi qu'un extrait de casier judiciaire national datant de moins de trois mois (non conservé) et une attestation sur l'honneur manuscrite de non-incapacité à exercer un mandat d'administrateur conformément aux articles L. 322-2, R. 322-11-6 et R. 322-167 du Code des assurances et 258 et 273 du Règlement délégué du 10 octobre 2014.

Les membres du conseil d'administration, à titre personnel ou en qualité de représentants permanents des actionnaires personnes morales proposés par la MAIF, bénéficient du dispositif de formation prévu dans le cadre des politiques respectives de compétences et d'honorabilité de ces sociétés. Le conseil d'administration consacre une fois par an un point de son ordre du jour à la formation des administrateurs.


## B.2.3.2 Dirigeants effectifs

En tant que de besoin, les dirigeants effectifs bénéficient de formations spécifiques, appropriées à l'exercice de leurs responsabilités. Avant leur nomination, ils produisent un extrait de leur casier judiciaire datant de moins de trois mois (non conservé) ainsi qu'une attestation sur l'honneur de non-incapacité à exercer la fonction conforme à l'art. L. 322-2 du Code des assurances.


## B.2.3.3 Responsables des fonctions clés

Les responsables des fonctions clés bénéficient d'une formation ad hoc, en particulier sur les évolutions législatives et règlementaires impactant leur activité.

Avant leur nomination, ils produisent un extrait de leur casier judiciaire datant de moins de trois mois (non conservé) ainsi qu' une attestation sur l'honneur de non-incapacité à exercer la fonction conforme à l'art. L. 322-2 du Code des assurances

Le directeur général s'assure par tout moyen de l'adéquation des fonctions clés, de leur indépendance et de la qualité de leur fonctionnement.


## B.2.4 Evaluation sur une base continue

Le conseil d'administration procède régulièrement, par tout moyen à sa convenance, à sa propre évaluation. Souhaitant incarner la vision de l'entreprise « Engagés pour demain », il a mené en son sein des travaux visant à renforcer sa cohésion, sa diversité et son efficacité afin de favoriser le plein exercice des responsabilités d'administrateur, l'efficacité du processus de décision et la valorisation du collectif.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> SYSTÈME DE GOUVERNANCE" -->

<!-- PageNumber="32 / 107" -->

Le conseil d'administration s'assure en outre que les dirigeants effectifs continuent à satisfaire aux exigences mentionnées précédemment. Ces derniers sont également tenus de porter sans délai à la connaissance du conseil d'administration tout élément nouveau susceptible d'avoir une influence sur leur situation d'honorabilité.

Enfin, le directeur général s'assure en ce qui le concerne que les responsables des fonctions clés continuent à satisfaire auxdites exigences de compétences et d'honorabilité. Les responsables des fonctions clés sont tenus de porter sans délai à la connaissance du directeur général tout élément nouveau pouvant avoir une influence sur leur situation d'honorabilité professionnelle.


## B.3 Système de gestion des risques y compris l'évaluation interne des risques et de la solvabilité

La gestion des risques s'attache à identifier, évaluer et hiérarchiser les risques relatifs aux activités d'une organisation, quelles que soient la nature ou l'origine de ces risques, pour les traiter méthodiquement de manière coordonnée et économique, afin de diminuer et contrôler la probabilité des événements redoutés, et réduire l'impact éventuel de ces événements. Le risque est l'effet de l'incertitude sur les objectifs (normes ISO31000), il peut être vu comme une menace ou une opportunité.

À ce titre, il s'agit d'une composante de la stratégie d'entreprise qui vise à connaitre et réduire la probabilité d'échec ou d'incertitude de tous les facteurs pouvant affecter les ambitions de l'entreprise. Le système de gestion des risques repose sur une approche systémique contenant de nombreux livrables. Une démarche globale est engagée pour disposer d'une vision holistique des risques.


### B.3.1 Description du système de gestion des risques


#### B.3.1.1 Organisation et gouvernance du système de gestion des risques

Le système de gestion des risques repose sur le partage d'informations et la coordination de nombreux acteurs. Les premiers intervenants sont les propriétaires de risques (directions métiers) qui gèrent couramment leurs risques liés à leurs activités. La coordination d'ensemble, soit par famille de risque, soit dans une vision globale entreprise, est réalisée à travers des comités décisionnels de l'entreprise.

Cette gouvernance décisionnelle a pour objectif de répondre aux exigences de la réglementation prudentielle Solvabilité 2 en termes de gestion par familles de risques.

Les principaux comités sont présentés sur le schéma ci-dessous et les domaines couverts et missions des comités sont décrits tels qu'ils figurent dans les politiques associées au 31 décembre en vigueur.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTEME DE GOUVERNANCE
===

<figure>

![](figures/7)

<!-- FigureContent="Gouvernance décisionnelle Solvabilité 2 Groupe\\* MAIF Groupe et sociétés : . Toutes les fonctions clés solos et groupes, sauf pour la gestion des placements d'Altimg et SMACL Assurances SA qui ont leurs propres gouvernances, articulées avec les équipes de MAIF :selected: Instance avec les dirigeants effectifs et le directeur général adjoint - DAFS A NOTED: . l'absence d'un dirigenht effectif doit rester neestionnelir, à défaut ife pouvoir participer, il doit communiquer pot fert san accord sur les décisions proposées pes \\* · seuls les DE et FC Groupe sont requis pour les séances traitant des sujets spécifiques Groupe (notamment le validation des comptes consolidés), les DE des filoles sant invités mais peuvent se faire reprisantar, Fonction Gestion des Risques Conseil d'administration et/ou comité spécialisé :unselected: Fonction Actuarielle :selected: Fonction Vérification de la Conformité :selected: Fonction Audit Interne :unselected: Membre permanent 1 Membre invité Souscription et produits Pilotage économique et prudentiel Gestion des placements Pilotage des risques \\* Réunion Produits et Politique Souscription Sessions Souscription et produits Réunion Pilotage Economique et Prudentiel \\* :unselected: Sessions Pilotage économique et prudentiel (intégrant les investissements stratégiques) Comité politiques d'investissements Comité actif/passif Comité pilotage risques audit et conformité :unselected: :unselected: - Sessions reassurance + et gestion d'actifs" -->

</figure>

 :unselected: :selected:
SMACL ASSURANCES SA intègrera les instances décisionnelles de gouvernance du groupe à compter du 1er janvier 2022.

Dans le cadre du système de gestion des risques, le conseil d'administration s'assure de la mise en place d'un système de gestion des risques efficace. Il approuve notamment la stratégie de gestion des risques, définit l'appétence et la préférence au risque.

L'appétence au risque est le niveau de risque agrégé maximum qu'une entité est prête à prendre en vue de la poursuite de son activité et afin d'atteindre ses objectifs stratégiques à l'horizon de son Plan stratégique. La préférence aux risques définit des lignes directrices qui fixent des acceptations ou des refus en termes de prise de risque pour l'entreprise.

Quant au Comité des risques d'audit et des comptes (comité spécialisé du conseil d'administration MAIF), sa mission dans le cadre du système de gestion des risques consiste plus particulièrement à examiner et suivre le dispositif global de maîtrise des risques et son application.


## › Domaine Souscription et produits

La réunion produits et politique souscription :

Cette réunion est une instance de validation par les deux dirigeants effectifs de livrables à soumettre au conseil d'administration, relevant notamment du domaine souscription et produits. A titre d'illustration : la politique globale de souscription et le suivi des business plans.


## › Domaine pilotage économique et prudentiel

La réunion pilotage économique et prudentiel (RPEP) :

Cette réunion est une instance de validation par les deux dirigeants effectifs de livrables à soumettre au conseil d'administration relevant notamment du domaine pilotage économique et prudentiel comme les reporting annuels (Normes Françaises et Solvabilité 2) et les rapports Solvabilité 2 (SFCR, RSR).


## › Domaine gestion des placements

Le Comité actif/passif :

› est une instance decisionnaire sur les resultats de l'allocation stratégique a moyen-long terme et la politique de gestion actif/passif, à soumettre au conseil d'administration ou en cas d'alerte du responsable de la fonction clé gestion des risques ;

› est une instance de reporting pour les autres sujets : évolution et paramétrage des modèles.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="33 / 107" --> :selected: :selected: :selected: :selected: :selected: :unselected: :selected:

> SYSTÈME DE GOUVERNANCE
===

34 / 107

Le Comité politiques d'investissements et gestion d'actifs :

› est une instance decisionnaire pour les points à présenter au conseil d'administration et les alertes du responsable de la fonction clé gestion des risques : politique de gestion des risques des investissements (y compris aménagements en cours d'exercice), politique gestion des risques de liquidité et concentration, politique d'allocation et de gestion tactique des placements ;

› est une instance de reporting pour les autres sujets : perspectives économiques à court et moyen terme, exécution de la politique de gestion des risques des investissements (respect, performance, etc), anticipations et perspectives en matière de gestion financière, contrôle interne et maîtrise des risques, expertises externes (en cas de besoin).

› Domaine pilotage des risques

Le Comité pilotage des risques, audit et conformité :

\> évalue le profil de risque ;

propose le niveau d'appétence et une déclinaison en tolérances par type de risques ;

\----- > fournit un cadre de gestion des risques ;

définit les scénarii de stress contre lesquels la société entend se prémunir ;

supervise les travaux ORSA (régulier et ponctuel) et soumet au conseil d'administration des préconisations issues de l'analyse des résultats ;

› prépare notamment la mise à jour des politiques globales de gestion des risques, ORSA, de conformité, de contrôle interne, de sous-traitance, de continuité d'activité et d'audit interne.

› étudie le tableau de bord des risques ;

› est informé de l'actualisation de la charte de gouvernance de la sécurité des SI et de la politique protection des données

B.3.1.2 Missions de la fonction clé gestion des risques, principaux travaux, compétences

La fonction gestion des risques, au travers du système de gestion des risques qu'elle anime, vise à déceler, mesurer, contrôler, gérer et reporter en permanence les risques auxquels est soumis l'organisme conformément au cadre arrêté par le conseil d'administration MAIF et la Direction générale MAIF.

Les principales missions de la fonction clé gestion des risques groupe sont :

› animer et veiller au système de gestion des risques sur l'ensemble des risques, y-compris émergents, et le processus ORSA ;

› rendre compte des expositions aux risques de manière détaillée et conseiller le conseil d'administration et la Direction générale MAIF sur les questions de gestion des risques, y compris celles d'ordre stratégique ;

› assurer la relation avec les autorites de tutelle sur son domaine. Elle est l'interlocuteur privilégié des organisations professionnelles sur les thèmes relevant de sa mission.

Le titulaire de la fonction clé gestion des risques, actuaire membre certifié de l'Institut des Actuaires, s'appuie sur les dispositifs clés suivants :

› une politique globale de gestion des risques qui établit annuellement la stratégie en matière de gestion des risques qui est ensuite déclinée dans des politiques « métiers » ;

› une « cartographie tous risques >> intégrant une section spécifique pour les risques opérationnels y compris ceux de non-conformité ;

› un outil de modélisation prospective permettant d'évaluer des risques, notamment dans le cadre des travaux de l'ORSA ;

› un rapport ORSA et les reportings de suivi des risques (appétence, tolérances et indicateurs).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTEME DE GOUVERNANCE
===

<!-- PageNumber="35 / 107" -->

Il s'appuie pour l'exécution de ses missions :

› sur les équipes du Pôle Performance Economique Solvabilité et Gouvernance. Notamment l'Unité Actuariat et Pilotage des Risques de MAIF VIE, unité composée de 9 collaborateurs dont 5 actuaires membres associés, qualifiés ou certifiés de l'Institut des actuaires,

› sur les équipes du Pôle Conformité et Contrôles de MAIF VIE pour la gestion des risques opérationnels (9 personnes).

B.3.1.3 Identification, stratégies, prises de décisions et communication sur les risques


## \> Cartographie tous risques

La cartographie tous risques regroupe toute l'information sur les risques auxquels est exposée MAIF VIE. Elle porte aussi bien sur les risques supportés (acceptés, garantis, ... ) dans le cadre du métier d'assureur que sur ceux découlant de leur mise en œuvre opérationnelle.

Elle a plusieurs objectifs :

› décrire les risques et leur évaluation à un instant donné en vue d'alimenter l'ORSA ;

› hiérarchiser les risques à l'aide de différentes mesures et permettre de se concentrer sur certains risques ;

› comparer la perception des risques, à un instant donne, entre le SCR et ses composantes par risque à l'évaluation ORSA ;

› identifier et valider/faire évoluer la stratégie de gestion des risques et les politiques « métiers » ;

› définir les responsabilités et les rôles/contributions des propriétaires de risques et des contributeurs à la maîtrise des risques ;

› piloter et suivre le système de gestion des risques ;

AAA > organiser un reporting interne aux instances décisionnelles ;

revoir annuellement les risques identifiés, compléter en fonction des évolutions internes ou externes.

L'approche retenue par MAIF VIE, en cohérence avec le groupe MAIF, propose de définir les natures des risques selon trois niveaux de risques :

› niveau 1 : les familles de risques (souscription, de marché, opérationnels, ... ) ;

› niveau 2 : les catégories de risques au sein d'une même famille (fraude interne par exemple) ;

› niveau 3 : les risques. Ce niveau correspond aux risques identifiés dans chacune des catégories de risques, pour lesquels une description détaillée et un scénario d'évaluation sont précisés. Il s'agit d'une déclinaison opérationnelle du risque (compréhension et maîtrise du risque à un niveau opérationnel).

Les familles et catégories de risques ont été construites en cohérence avec celles définies dans le cadre de la formule standard réglementaire utilisée pour calculer le Capital de solvabilité requis. Le schéma synoptique présentant la structure de la cartographie tous risques du groupe MAIF (grandes familles et catégories de risques) est présenté en partie C.


## Stratégie de gestion des risques ›

La stratégie en matière de gestion des risques est formalisée dans la politique globale de gestion des risques, et ensuite déclinée opérationnellement au niveau de chacune des politiques « métiers ».

Les principales orientations de la gestion des risques sont définies chaque fin année, elles s'inscrivent dans la stratégie de l'entreprise afin de conserver le profil de risque à un niveau compatible avec l'appétence au risque et sa préférence au risque.

Toutes les décisions prises dans la gestion quotidienne s'inscrivent dans cette stratégie de risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="36 / 107" -->

›


## Considération des risques dans la prise de décisions

La gouvernance décisionnelle mise en place permet de structurer un reporting adéquat à la fois sur l'activité, la performance ou la vision risque.

La gestion des risques relève de la responsabilité de tous à travers une coordination de toutes les actions quotidiennes. L'approche coordonnée et intégrée de la gestion des risques est animée par le responsable de la fonction. Le responsable de la fonction clé gestion des risques s'assure du partage de l'information entre les acteurs, à la fois sur les activités récurrentes et stratégiques de l'entreprise.

Le responsable de la fonction clé gestion des risques de l'entreprise est membre permanent ou invité de ces comités permettant ainsi la mise sous tension et l'efficience du système de gestion des risques. Ce fonctionnement permet également de s'assurer d'une bonne prise en compte des risques dans la chaîne décisionnelle et d'une bonne information des responsables des fonctions clés, essentielles pour la réalisation de leurs missions dans le système de gestion des risques.

Le responsable de la fonction vérification de la conformité vérifie la conformité des informations produites aux exigences réglementaires en s'appuyant sur le dispositif de contrôle interne selon le périmètre et les modalités définis dans les politiques de conformité et de contrôle interne.

La fonction actuarielle collabore notamment sur la qualité des données et les méthodes d'évaluation des risques (mesure quantitative et modèle prospectif).

L'audit interne réalise des missions d'audit du système de gestion des risques.

Des analyses de sensibilité sur le taux SCR sont réalisées dans certains dossiers pour aider à la construction des tests de résistance pour l'ORSA régulier ou ponctuel.


## › Communication sur les risques

Afin d'assurer en permanence l'efficacité du dispositif de maîtrise des risques et permettre, le cas échéant, l'adoption de mesures correctrices, la fonction clé gestion des risques rend compte au Comité pilotage des risques, audit et conformité, au Comité des risques d'audit et des comptes MAIF et au conseil d'administration du déroulement de ses missions.

Elle informe a minima annuellement ces instances sur le degré du risque et des mesures correctrices mises en œuvre selon les modalités prévues par ces instances.


### B.3.2 Evaluation interne des risques et de la Solvabilité (ORSA)

L'ORSA (Own Risk and Solvency Assessment) est l'un des outils du système de gestion des risques, qui formalise et accompagne la prise de décision.

Pour la réalisation de l'ORSA régulier, MAIF VIE a retenu une fréquence annuelle.

Certaines circonstances pourraient rendre l'ORSA nécessaire en dehors de l'échéance régulière annuelle. Cet ORSA dit « ponctuel >> sera réalisé chaque fois que le profil de risque de l'entreprise évolue ou serait amené à évoluer de manière significative, sur décision des dirigeants effectifs ou bien encore pour répondre à une demande émanant de l'autorité de contrôle (ACPR). Le processus de réflexion pour le lancement d'un ORSA ponctuel a été enrichi de critères quantitatifs au niveau du groupe MAIF.

L'évaluation du Besoin Global de Solvabilité (BGS) est réalisée :

› sur la base d'une évaluation à moyen/long terme des mesures de risques fondées sur les critères d'appétence ;

› en étudiant la résistance de l'entreprise, sur ses critères d'appétence, face à des situations dégradées ;

› sur l'analyse des risques matériels ou jugés les plus importants pour l'entreprise ou de la déviation aux hypothèses sous-jacentes du SCR ;

› et enfin, en décrivant les actions du management envisageables et les autres moyens mis en place pour surveiller et agir sur les risques.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

<!-- PageNumber="37 / 107" -->

<!-- PageHeader="\> SYSTÈME DE GOUVERNANCE" -->


#### B.3.2.1 Processus ORSA régulier

Le responsable de la fonction clé gestion des risques supervise la réalisation des évaluations internes des risques et de la solvabilité dites « ORSA >> (Own Risk and Solvency Assessment). Une politique ORSA a été arrêtée par le conseil d'administration. L'ORSA est réalisé via un travail collaboratif des salariés, des dirigeants effectifs et des administrateurs autour d'une vision intégrée des risques. Cette vision rendement/risque se déploie progressivement au sein de l'entreprise.

Les résultats du processus ORSA permettent au conseil d'administration de formaliser :

› le profil de risque de l'entreprise et les principaux facteurs de risque ;

› le respect de l'appétence au risque sur l'horizon temporel défini ainsi que la disponibilité et la suffisance des fonds propres ;

› les changements significatifs du profil de risque sur la période écoulée et les principales sensibilités sur les indicateurs financiers (sur le bilan en particulier) ;

› le périmètre des risques non couverts par le SCR (ainsi que la portée de ces risques) et ceux devant faire l'objet d'un contrôle renforcé dit risque jugé important1 qu'il soit quantifiable ou non.

Les éléments ci-après décrivent les principales étapes clés du processus ORSA régulier telles que mises en place au sein de MAIF VIE, en lien fonctionnel avec le groupe MAIF, à savoir :

› la construction des tests de résistances ;

› les chiffrages Solvabilité 2 dans plusieurs dossiers décisionnels ;

\> l'évaluation des tests de résistance ORSA ;

AAA > la détermination d'éventuelles actions de management ; > la rédaction du rapport ORSA ;

› et la validation du rapport.

La fonction cle gestion des risques coordonne l'exécution de l'ORSA. Ces travaux sont supervisés par le Comité pilotage des risques audit et conformité qui soumet le rapport et ses conclusions au conseil d'administration. Après avis du Comité des risques d'audit et des comptes de la MAIF, le conseil d'administration valide le rapport ORSA.

Le rapport ORSA est examiné au moins une fois par an par le conseil d'administration et à chaque fois que nécessaire en cohérence avec les éléments déclenchants énumérés dans la politique ORSA.


#### B.3.2.2 Lien entre la gestion du capital et le profil de risques

MAIF VIE a identifié ses principaux risques. Elle immobilise des capitaux pour couvrir les pertes potentielles qui pourraient subvenir. À ce jour, le profil de risque de MAIF VIE est en adéquation avec les hypothèses sous-jacentes au calcul du Capital de solvabilité requis (SCR) selon la formule standard prévue par la réglementation. Par conséquent, MAIF VIE évalue ses risques « quantifiables >> par la formule standard. Ces mêmes risques peuvent faire l'objet de scénarios de stress permettant d'évaluer l'évolution des taux de solvabilité en cas de réalisation de tels événements. Des risques « non quantifiables >> sont évalués à dire d'expert de manière qualitative.

<!-- Footnote="1 Dans Solvabilité 2, un élément « important » est, par définition, susceptible d'influencer le jugement et la décision. C'est ainsi qu'il faut comprendre la notion de risque important (source ACPR). Pour MAIF VIE, les risques jugés importants peuvent être définis comme des risques pouvant nuire de manière brutale et significative à la société. MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="38 / 107" -->


## B.4 Système de contrôle interne


### B.4.1 Description du système de contrôle interne

L'organisation du contrôle interne est structurée selon trois lignes de maîtrise et repose sur deux notions : le contrôle permanent regroupant les contrôles de premier et deuxième niveaux (les deux premières lignes de maîtrise) et le contrôle périodique avec le contrôle de troisième niveau (la troisième et dernière ligne de maîtrise).

Le contrôle permanent de premier niveau encadre la première prise de risque qui se situe au niveau des opérationnels et de la hiérarchie sur le terrain.

Le contrôle permanent de deuxième niveau anime et coordonne le système de contrôle interne.

Le contrôle périodique est indépendant des deux premiers niveaux et a pour vocation d'aider l'entreprise à atteindre ses objectifs en évaluant ses processus de gestion des risques, de contrôle, et plus globalement de gouvernance d'entreprise. Il est assuré par l'audit interne (voir § B.5).

Le pôle conformité et contrôles (PCC) pilote par subdélégation de la directrice générale la mise en œuvre du système de contrôle interne et rend compte de ses travaux au Comité de pilotage des risques, d'audit et de conformité, instance regroupant les dirigeants effectifs de MAIF, de ses principales filiales et les responsables des fonctions clés. Il informe le Comité des risques d'audit et des comptes et le conseil d'administration. Il rend compte de la mise en œuvre des pratiques définies au sein du groupe MAIF et pour ce faire, participe aux liens fonctionnels réalisés par la Direction Maîtrise des Risques et Contrôle Interne de la MAIF.

Le pôle conformité et contrôles pilote la mise en œuvre de la gestion des risques opérationnels et anime le système de contrôle interne avec le concours d'un réseau de correspondants métiers. Les procédures clés du système de contrôle interne reposent sur :

› la cartographie des risques opérationnels ;

› un dispositif de contrôle permanent de 2ème niveau organisé autour d'autoévaluations de vérifications de déclaration, de vérifications de procédure et de vérification d'efficacité. Il est traduit dans un plan annuel de vérifications.

› un dispositif de suivi des incidents opérationnels ;

› un reporting du dispositif.

Le pôle conformité et contrôles assure également la coordination du plan de continuité des activités.


#### B.4.2 Procédures clés du système de contrôle interne


##### › Lien entre le système de gestion des risques et le système de contrôle interne

Le système de gestion des risques et le système de contrôle interne interagissent de manière complémentaire.

Le système de gestion des risques pilote tous les risques en fixant des limites et un cadre de gestion des risques. Le système de contrôle interne veille à son application et au respect des normes de la société. Le système de contrôle interne s'appuie donc sur le système de gestion des risques pour positionner les contrôles à effectuer. Ainsi, les travaux des deux systèmes interagissent et assurent une séparation des rôles et responsabilités.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="39 / 107" -->


## B.4.3 Fonction de vérification de la conformité


### B.4.3.1 Missions

Le pôle conformité et contrôles s'assure, sur son périmètre d'activité, de la sécurisation des opérations conformément aux dispositions legales, règlementaires et aux normes professionnelles ou déontologiques. Il contribue à limiter les risques de sanctions, de pertes financières et/ou d'image qui seraient liés à une non-conformité.

Son périmètre d'activité concerne principalement :

› la protection des sociétaires, assurés, adhérents, clients, y compris dans la gestion des capitaux non réclamés ;

› la lutte contre le blanchiment d'argent et le financement du terrorisme (LCB-FT) et la fraude interne et externe associée ;

› la conformité des nouveaux produits et services ainsi que la conformité des modifications substantielles de produits et services existants ;

› la protection des données personnelles ;

› la déontologie ;

› la sécurisation des sous-traitances importantes ou Critiques (STIC).


#### B.4.3.2 Organisation de la fonction de vérification de la conformité et principaux travaux

La fonction de vérification de la conformité reste indépendante dans l'exercice de sa mission et n'effectue pas d'acte de gestion opérationnelle.

Le pôle conformité et contrôles réalise également les contrôles de vérification de la conformité à partir du Plan de contrôle conformité.


## B.5 Fonction d'audit interne


### B.5.1 Missions

La finalité première de l'audit interne est d'évaluer l'adéquation et l'efficacité du système de gouvernance de l'entreprise.

Au travers de cette évaluation, l'audit interne est la dernière « ligne de maîtrise >>, qui donne à l'entreprise une assurance sur le degré de maîtrise de ses opérations.

Au-delà de l'assurance donnée, l'audit interne apporte aussi, par ses recommandations, des conseils d'amélioration.

Une valeur ajoutée complémentaire de l'audit interne est de réaliser, grâce à son indépendance et sa qualification, des investigations ou états des lieux sur des sujets pouvant être sensibles, pour le compte de la Direction générale.


### B.5.2 Organisation de la fonction audit interne et principaux travaux

La Direction de l'Audit interne de la MAIF, qui assure la fonction d'audit interne du Groupe MAIF et de ses entités solo dont MAIF VIE, est rattachée hiérarchiquement au Directeur général de la MAIF qui supervise le management de l'entité.

En outre, une fois par an, le Directeur de l'audit interne rencontre le Président de la MAIF. Pour ce qui relève de sa responsabilité de fonction clé d'audit interne MAIF VIE solo, elle rend compte fonctionnellement :

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="40 / 107" -->

› aux dirigeants effectifs de MAIF VIE sur les domaines de co-décision des dirigeants effectifs (politique d'audit interne, plan d'audit) ;

› au Comité des Risques d'Audit et des Comptes de la MAIF.

Ce rattachement et ce reporting au plus haut niveau, contribuent à l'indépendance de l'Audit interne.

La Direction Audit interne MAIF est composée à fin décembre 2021 d'une équipe pluridisciplinaire de 16 postes, dimensionnée pour mener à bien ses missions. Les acteurs sont formes à la méthodologie de l'audit interne directement issue de celle enseignee par l'IFACI. En fonction du thème des missions confiées, des formations spécialisées complémentaires peuvent être assurées.

La Direction de l'Audit interne MAIF conduit ses activités en conformité aux Normes internationales de la pratique professionnelle de l'audit interne émises par l'IIA (Institute of Internal Auditors). Elle a obtenu le 10 mai 2021 le renouvellement de son label de certification 'The IIA / IFACI', label qu'elle détient depuis le 14 février 2018.

L'audit interne réalise principalement des missions d'assurance portant sur un processus, un thème ou une entité. En complément, il mène aussi ponctuellement des missions de conseil. Les conclusions d'audit font l'objet d'un rapport final remis notamment aux dirigeants effectifs de MAIF VIE, aux membres du Comité des risques d'audit et des comptes, aux responsables des fonctions clés MAIF VIE et GROUPE. Il contient les constats, les risques identifiés, les recommandations et le plan d'action des structures auditées (le cas échéant, celui-ci a pu faire l'objet au préalable d'un arbitrage par la Direction Générale).

Annuellement, l'audit interne communique à la Direction générale, au conseil d'administration et au Comité des risques d'audit et des comptes un rapport comprenant les résultats des activités d'audit interne de l'année.


## B.6 Fonction actuarielle


### B.6.1 Missions

Les missions de la fonction actuarielle peuvent être regroupées selon quatre thématiques :

› les provisions techniques :

o coordonner le calcul des provisions techniques voire le superviser pour certains cas prévus par la directive Solvabilité 2 ;

o garantir le caractère approprié des méthodologies, des modèles sous-jacents et des hypothèses utilisés pour le calcul des provisions techniques ;

o apprécier la suffisance et la qualité des données utilisées dans le calcul des provisions techniques ;

o comparer les meilleures estimations aux observations empiriques ;

o informer le conseil d'administration de la fiabilité et du caractère adéquat du calcul des provisions techniques.

› la souscription : émettre un avis sur la politique globale de souscription ;

› la reassurance : émettre un avis sur l'adéquation des dispositions prises en matière de réassurance ;

› le système de gestion des risques : contribuer à la mise en œuvre effective du système de gestion des risques, en particulier pour ce qui concerne la modélisation des risques sous- tendant le calcul du Capital de solvabilité requis (SCR) et du Minimum de capital requis (MCR) et pour ce qui concerne l'évaluation interne des risques et de la solvabilité (ORSA).


### B.6.2 Organisation de la fonction actuarielle et principaux travaux

Le système de gouvernance de MAIF VIE comporte quatre fonctions clés de contrôle dont la fonction actuarielle, confiée à un chef de projet actuariat issu de l'unité Actuariat et Pilotage des Risques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" --> :unselected: :unselected: :unselected: :unselected: :unselected:

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="41/ 107" -->

Les missions attribuées à l'unité Actuariat et Pilotage des Risques regroupent les activités suivantes :

› piloter et réaliser les travaux de gestion prévisionnelle des activités de l'entreprise tant en Normes Françaises que sous Solvabilité 2 ;

› contribuer aux travaux relevant de la responsabilité des fonctions clés << Gestion des Risques >> et « Actuarielle >> ;

› réaliser les travaux techniques et économiques liés à la conception et à l'évolution des produits ;

› évaluer les engagements techniques de la société notamment au moment de l'arrêté des comptes ;

› s'assurer en permanence du respect des normes techniques et actuarielles en vigueur ainsi que des contraintes réglementaires afférentes ;

› analyser les éléments techniques relatifs à la gestion des traités de reassurance.

La fonction actuarielle rédige notamment un rapport annuel permettant de motiver les différentes opinions exprimées par le responsable de la fonction actuarielle sur les provisions techniques, la politique de souscription et la politique de reassurance.

Dans ce rapport, la fonction actuarielle documente les travaux réalisés et les résultats, identifie les éventuelles lacunes et émet des recommandations.

Ce rapport, ainsi que les opinions, sont notamment partagés en amont de la présentation aux instances au sein du Comité groupe de la fonction actuarielle qui associe les différentes directions impliquées dans les travaux.

Cependant, si l'opinion de la fonction actuarielle faisait émerger une problématique significative sur un des sujets évoqués précédemment, l'opinion serait alors transmise au conseil d'administration au moment de sa validation.

Les productions de la fonction actuarielle, qui a en charge également le pilotage et la réalisation de l'évaluation quantitative des risques dont l'ORSA, sont notamment présentées au responsable de la fonction gestion des risques et aux différentes instances ..


## B.7 Sous-traitance

MAIF VIE a conclu des accords de sous-traitance avec des prestataires de services tiers en vue d'externaliser diverses activités. Ce recours à la sous-traitance est justifié par la nature des activités externalisées, des décisions stratégiques de l'entreprise et par la volonté de recourir à des niveaux d'expertise, de savoir-faire ou de connaissances adaptés.

La Directive Européenne 2009/138/CE du 25 novembre 2009 sur l'accès aux activités de l'assurance et de la réassurance et leur exercice dite « Directive Solvabilité Il >> impose aux entreprises d'assurance la mise en place d'un dispositif de maîtrise de sa sous-traitance et plus particulièrement de ses sous-traitants dits << Importants ou Critiques >> (STIC) selon des critères definis par les articles 49 et 50 de la Directive.

MAIF VIE restant responsable des activités sous traitées, son dispositif de maîtrise de la sous- traitance a pour objectif principal la protection des adhérents et la maîtrise des risques inhérents à l'intervention d'un acteur tiers (risques opérationnels avec impacts financier et/ou de réputation dans une hypothèse de défaillance desdits prestataires de services).

Dans le cadre de la définition et de la mise en œuvre de son dispositif STIC, MAIF VIE a rédigé sa Politique de sous-traitance, laquelle est validée par le conseil d'administration de l'entreprise et révisée annuellement. Le 17/06/2021, le CA de MAIF VIE a ainsi validé la mise à jour 2021 de la Politique de sous-traitance de MAIF VIE. Cette politique précise le cadre et les principales règles en matière de sous-traitance à MAIF VIE, dont les critères de qualification d'une sous-traitance importante ou critique, le dispositif de suivi des activités externalisées, ainsi que les responsabilités des différents acteurs. Elle vient également en déclinaison de la Politique de Sous-traitance Groupe MAIF.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> SYSTÈME DE GOUVERNANCE
===

<!-- PageNumber="42 / 107" -->

En 2021, en concertation avec le groupe MAIF, MAIF VIE a poursuivi les travaux d'animation et de renforcement de son dispositif de supervision des sous-traitances importantes ou critiques.

Le Comité STIC de MAIF VIE a pour mission de suivre les risques liés aux STIC avec l'ensemble des acteurs concernés, et notamment de valider la mise à jour de la cartographie STIC, l'entrée et la poursuite de la relation d'affaires avec les partenaires identifiés comme étant critiques ou importants. Il a vocation à se réunir, à minima, une fois par an.

›


## La sécurisation de la sous-traitance de MAIF VIE

En 2021, MAIF VIE s'est attaché à optimiser la sécurisation de ces relations partenariales au travers de deux processus :

\- le processus de selection et de sécurisation du partenariat avec le sous-traitant important ou critique ;

\- le processus de suivi du sous-traitant important ou critique.

› La selection du sous-traitant important ou critique

MAIF VIE qualifie ses sous-traitances importantes ou critiques (STIC) en s'appuyant sur un certain nombre de critères combinés dans un arbre de décision et qui permettent de définir la criticité de la sous-traitance.

Ces critères sont :

› la qualification de l'activité :

o une activité standard au sens de l'article R354-7 Il a du Code des assurances, o une activité

o :unselected: relevant de la sous-traitance et/ou de l'externalisation au sens de l'article L310-3 13 du Code des assurances,
 :unselected:
o une fonction clé au sens de Solvabilité 2,
 :unselected:
o une activité cœur de métier,

o une fonction/activité support sensible,

o une sous-traitance d'un risque opérationnel dont l'arrêt brutal engendrerait une perte significative ou le non-respect des engagements envers ses sociétaires,

› des seuils économiques définis en interne,

› le caractère substituable ou non de l'activité dans un délai limité,

› le caractère durable ou non de l'activité.

A cela s'ajoute une étude du sous-traitant sur un ensemble d'éléments qui peuvent s'ajuster selon le type d'activité sous-traitée :

› leur dispositif de continuité d'activité,

› leur sécurité du système d'information,

› leur dispositif de contrôle interne,

› leur viabilité financière,

› leurs elements contractuels : attestation d'assurance, attestation RC ...

Enfin, la sécurisation de la relation s'effectue également au travers d'une convention écrite signée entre le représentant de l'entité délégante et le représentant du sous-traitant qui intégre des clauses spécifiques telles que la continuité des activités, la protection et la qualité des données utilisées pour l'activité ou la fonction confiée par le délégant, le respect de la réglementation, l'honorabilité du prestataire, ....


## Le suivi regulier du sous-traitant important ou critique ›

Pour maîtriser les risques liés aux sous-traitants importants ou critiques, MAIF VIE a mis en place un certain nombre de contrôles :

› Des vérifications annuelles : questionnaire annuel de suivi à destination des sous-traitants importants ou critiques, renseigné sur une base déclarative ((suivi du Plan de continuité et MAIF VIE - Rapport sur la solvabilité et la situation financière 2021 :unselected: :unselected: :unselected:

> SYSTÈME DE GOUVERNANCE
===

<!-- PageHeader="43 / 107" -->

de reprise d'activité, suivi du dispositif de contrôle interne, évaluation du système d'information ... ),

Un suivi régulier dans le cadre des comités de suivi du partenariat. ›

Parallèlement, le plan de contrôle interne de MAIF VIE intègre la vérification de l'effectivité, de la qualité et de l'exhaustivité des vérifications annuelles réalisées dans le cadre du dispositif STIC.


## B.8 Autres informations

MAIF VIE n'a pas identifié d'autre information significative à mentionner dans ce paragraphe.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->
<figure>

![](figures/8)

<!-- FigureContent="C" -->

</figure>


PROFIL DE RISQUE

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> PROFIL DE RISQUE" -->

C Profil de Risque
===

Le profil de risque regroupe l'ensemble des risques auxquels la société d'assurance est exposée, la quantification de ces expositions et l'ensemble des mesures de protection face à ces risques.

Ces éléments sont présentés de manière détaillée dans les sections ci-après.

MAIF VIE est une société d'assurance dont l'activité principale est de couvrir les besoins des adhérents en matière d'assurance vie : protéger sa famille, assurer l'avenir de ses enfants, se constituer par l'épargne un complément de retraite, favoriser la transmission d'un capital ... A ce titre, elle n'est pas exposée à des risques spécifiques par rapport au marché et ne couvre pas de risques de nature exceptionnelle.

Les principaux risques auxquels MAIF VIE est exposée résultent de son activité d'assureur vie, de son organisation et de son environnement, notamment économique. Le profil de risque reflète les conclusions de la cartographie tous risques, outil essentiel du système de gestion des risques dont la structure est présentée en § B.3 de ce rapport.

Le schéma synoptique suivant présente la cartographie tous risques qui s'applique aux principales entités du groupe MAIF :

C.1. Risque de souscription

C.2. Risque de marché

C.3. Risque de crédit

C.5. Risque opérationnel C.6. Autres risques importants

<figure>

![](figures/9)

<!-- FigureContent="Risque de Risque de Risque opérationnel souscription non Risque de Risque de défaut y compris de non-conformité Risque d'entreprise souscription vie vie et santé marché Risque émergent Mortalité Prime et Frais Action Créance du passif Stratégique Economique Longévité Catastrophe par scénario Immobilier Créance de l'actif Business Environnemental Invalidité / Catastrophe Taux Morbidité engendrée par Image et réputation et Sociétal l'homme engagement Catastrophe Change Réserve sociétal Technologique Spread Cessation Résiliation Politique Concentration Dépense Règlementaire Liquidité Révision C.4. Risque de liquidité Tarification" -->

</figure>


Risque quantifié dans le SCR

Risque non quantifié dans le SCR

Le profil de risque est notamment évalué de manière quantitative selon le calcul du SCR déterminé par la formule standard, pour la plupart des risques quantifiables (SCR pour Solvency Capital Requirement, représentant le capital cible requis dans le cadre de la réglementation européenne Solvabilité 2). Le SCR correspond au montant de fonds propres estimé comme nécessaire pour absorber des chocs de forte intensité susceptibles de se produire environ une fois tous les 200 ans. Les résultats de ce calcul sont présentés dans la suite de ce rapport au § E.2.

Ainsi, les risques les plus importants sont compris dans le calcul du SCR. Pour ces risques, la formule standard définie dans la directive Solvabilité 2 prévoit une méthode d'évaluation normalisée qui facilite la comparaison des sociétés d'assurance sur leur exposition aux risques de natures différentes. Le calibrage de la formule standard est adapté pour la mesure du profil de risque MAIF VIE. Ces risques sont couverts par la mobilisation de fonds propres en quantité et en qualité suffisante (présentés dans la partie E) et par divers dispositifs de gestion des risques.

Dans le calcul du SCR, une réduction du besoin en capital est prévue pour tenir compte des effets de diversification entre les risques. Les principales catégories qui contribuent au SCR et MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

<!-- PageHeader="45 / 107" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="46 / 107" -->

génèrent de la diversification entre elles sont le risque de souscription vie et le risque de marché. Les autres catégories de risques, ont des expositions unitaires plus faibles.

D'autres risques significatifs pour MAIF VIE - risques d'entreprise, émergents (décrits au § C.6.6), ou de liquidité (§ C.4) - sont également détaillés. Ces derniers ne sont pas intégrés dans le calcul du SCR selon la formule standard, et sont couverts par des dispositifs de gestion des risques spécifiques.

Les traits saillants de l'exposition MAIF VIE à chacun de ces principaux risques sont présentés ci-dessous, après un rappel de la nature du risque.

Les principales techniques de gestion ou d'atténuation de ces risques mises en œuvre par MAIF VIE sont également décrites ci-après. Ces techniques propres à chaque risque s'inscrivent dans le dispositif général de gestion des risques détaillé auparavant au § B.3.

Afin de faciliter la lecture, les principaux risques auxquels MAIF VIE est exposée sont regroupés au sein des modules suivants :

AAAAAA

Risque de souscription, Risque de marché,

Risque de crédit,

Risque de liquidité, Risque opérationnel,

Autres risques importants, incluant les risques d'entreprise et les risques émergents.

Ces modules rappellent la structure de la cartographie tous risques. Les risques qui n'entrent pas dans le champ du SCR sont distingués, permettant ainsi de faire le lien avec la structure de la cartographie tous risques de MAIF VIE.

Les expositions au risque ont augmenté en raison de la crise sanitaire qui a perturbé toutes les activités économiques. Les impacts de la crise sanitaire sont doubles en affectant négativement les assureurs tant du côté de l'actif que du passif.

Le profil de risque de MAIF VIE est peu exposé concernant les branches d'activité prévoyance, chômage, ou encore assurance-crédit, les plus impactées par la crise liée à la pandémie COVID- 19. Ainsi, MAIF VIE est suffisamment solide pour faire face à ses engagements. La diversification des risques, le niveau et la qualité des fonds propres, la stratégie d'investissement prudente sont autant d'éléments qui assurent à MAIF VIE, de fortes capacités de résistance dans des scénarios adverses.


## C.1 Risque de souscription

Les risques auxquels est exposée MAIF VIE diffèrent selon la nature et l'objet du produit :

› les produits d'épargne, pour leur composante libellée en euros, induisent principalement des risques de marché, des risques de rachats et des risques de dérive des frais (dépenses). La composante en unités de compte des produits d'épargne, sur laquelle les risques de marché sont supportés par l'assuré, est exposée à une diminution de l'encours (décès, rachats, arbitrages avec le compartiment libellé en euros ... ) ayant pour incidence une dégradation du résultat sur ce type de produit ;

› les produits d'épargne retraite (non assimilés à des produits d'épargne) et les rentes viagères (issues des options de règlements d'autres produits d'épargne ou prévoyance de MAIF VIE) engendrent essentiellement des risques financiers, mais aussi des risques techniques de souscription (longévité, ... ) ;

› les produits de prévoyance génèrent des risques techniques de souscription (décès, invalidité, anti-sélection, ... ) pouvant entraîner, en cas de dérive des indicateurs techniques, une dégradation du résultat sur ces produits.

Au regard de son profil d'activité, majoritairement concentrée sur les produits d'épargne, MAIF VIE est intrinsèquement exposée à l'environnement des aléas assurantiels, engendrant des risques de souscription, et plus particulièrement :

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> PROFIL DE RISQUE
===

<!-- PageHeader="47 / 107" -->

› risque de distribution et de tarification ;

› risque d'une forte hausse des rachats en épargne.

Au regard de la nature des produits, MAIF VIE n'est pas exposée aux risques de révision.

Le SCR souscription vie de MAIF VIE se décompose en risque de mortalité, de longévité, d'invalidité, de cessation, de dépenses, de révision et catastrophe.


## C.1.1 Risque de distribution et de tarification

En tant que filiale assurance vie de la MAIF, MAIF VIE s'attache à fournir une réponse adaptée aux besoins des sociétaires MAIF en matière d'assurance vie et prévoyance ; ainsi, MAIF VIE jouit du réseau de distribution MAIF pour diffuser son offre produit, mais également de l'expertise des conseillers-vendeurs de MAIF Solutions Financières (filiale détenue à 100 % par MAIF) pour la souscription de contrats d'assurance vie avec des versements élevés.

La non-atteinte des objectifs de distribution sur un ou plusieurs produits peut impliquer une dérive de la performance économique attendue sur ce ou ces produits. L'élaboration et le suivi d'un plan de développement annuel (définition des objectifs de développement) permet de piloter la stratégie commerciale. Le CPAD suit mensuellement l'atteinte des objectifs de distribution.

Plusieurs travaux permettent de vérifier l'exposition aux risques de distribution et de tarification :

› l'analyse annuelle du positionnement de l'offre permet de comparer par produit l'offre MAIF par rapport au marché ;

› l'adéquation des tarifs, et notamment le suivi annuel de la mortalité, le suivi annuel de la tarification des contrats, l'analyse empirique de la sinistralite et des rachats, ...

Par ailleurs, l'existence d'une politique Gouvernance et Surveillance des Produits (GSP) et d'un Comité souscription et produits renforce le pilotage de la stratégie produits de MAIF VIE.

Le risque potentiel sur de nouveaux produits est encadré par un processus spécifique dédié à la conception ou la modification d'un produit.


## C.1.2 Risque de cessation

Au regard de son profil d'activité, majoritairement concentrée sur les produits d'épargne, MAIF VIE est exposée à un risque de forte hausse des rachats en épargne.

Pour la plupart, des contrats d'assurance vie d'épargne peuvent être rachetés à tout moment par les assurés. Les rachats peuvent être motivés par des besoins de liquidité, des facteurs liés à la fiscalité avantageuse des produits d'assurance, des repositionnements vers des concurrents offrant des produits plus rémunérations/performants ...

Notamment, dans certaines situations financières :

› des rachats importants de contrats d'épargne, pour leur composante libellée en euros, pourraient avoir des impacts significatifs sur la situation financière et la solvabilité de l'entreprise ; ce serait notamment le cas dans une situation de forte et rapide hausse des taux d'intérêt, occasionnant la vente de titres en moins-values latentes et affectant directement le résultat financier et donc les fonds propres de l'entreprise (voir section suivante C.2 sur l'exposition au risque de taux d'intérêt).

› dans une situation persistante de taux d'intérêt long terme à des niveaux extrêmement bas, des rachats importants pourraient améliorer ponctuellement la solvabilité de l'entreprise mais nuiraient à son équilibre économique et à sa pérennité.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> PROFIL DE RISQUE" -->

<!-- PageHeader="48 / 107" -->

Les études actif/passif, le ratio de PPB/PM (provision pour participation aux bénéfices sur provision mathématique) et la situation de richesse latente sont des dispositifs existants de maîtrise du risque de rachat. Voir également les « facultés contractuelles et règlementaires face au risque de liquidité >> (ci-après, partie C.4) qui permettent également de faire face au risque de cessation.


## C.1.3 Risque de dépenses

Le risque de dépense peut être d'origine endogène, comme une mauvaise maîtrise des charges internes (masse salariale, dérive des coûts des projets, ... ) ou d'origine exogène comme l'inflation, des cotisations sociales supplémentaires, des nouvelles règlementations engendrant des coûts projets complémentaires ...

Le système de commissionnement mis en place auprès des réseaux de distribution a été conçu de manière à ce que les ressources de gestion prélevées sur les contrats d'épargne soient toujours supérieures aux commissions versées.

Les frais et leur dérive potentielle sont par ailleurs régulièrement mesurés de manière prospective permettant ainsi d'anticiper et de piloter les coûts en adéquation avec les ressources de gestion.


## C.1.4 Risque de mortalité, de longévité, d'invalidité et catastrophe

Les risques de mortalité, de longévité et d'invalidité correspondent respectivement à une dérive de la mortalité, de la longévité ou de l'invalidité par rapport aux lois comportementales construites et utilisées par MAIF VIE. Le risque de catastrophe résulte d'évènements extrêmes ou irréguliers ayant comme conséquence un effet de surmortalité ponctuelle et immédiate. A ce titre, l'éventualité de la survenance d'une épidémie ou d'un phénomène climatique (canicule ... ) est un risque de sursinistralité (décès / incapacité / invalidité) impactant l'équilibre économique de nos produits. La structure du portefeuille conjuguée à la couverture de reassurance actuelle permet une bonne maitrise de ce risque.

Des méthodes statistiques et actuarielles sont utilisées pour concevoir les produits et suivre leur adéquation et équilibre technique. La mesure des engagements et des flux futurs des produits d'assurance vie et de prévoyance (et des risques de souscription sous-jacents) s'appuie sur des hypothèses d'évolution prospective de facteurs d'origine économique, démographique, ou liés à la mortalité, l'invalidité ou le comportement des assurés. MAIF VIE utilise son expérience, ses données et celles de la profession afin d'élaborer des estimations au plus juste.

Des risques émergents, tels qu'un nouveau cadre fiscal, pourraient entraîner des pertes incohérentes par rapport aux hypothèses retenues ; avec un éventuel impact significatif de la situation financière.

Les lois comportementales relatives aux risques de mortalité, de longévité, d'invalidité/morbidité et de rachat sont mises à jour annuellement et permettent d'appréhender d'éventuelles modifications de tendances, qui pourraient entraîner une modification de la structure et de la cession des risques. Il s'agit également de s'assurer qu'aucune déformation du portefeuille dans ses caractéristiques ne nécessite une modification de la tarification. Par ailleurs, le système de suivi interne construit autour de comparaisons mensuelles et annuelles permet d'alerter en cas de modification importante et durable de la sinistralité.

À noter que MAIF VIE utilise une table de mortalité d'expérience, certifiée par un actuaire agréé à cet effet, conformément à l'article A132-18 du Code des assurances, pour la tarification de garanties en cas de décès des contrats de prévoyance individuelle temporaire et de prêts qu'elle assure.

De plus, les échanges entretenus au cours de l'année avec nos réassureurs nous permettent de comparer le coût de nos traités à ceux du marché. Les propositions tarifaires pour l'année suivante sont également évaluées en regard de l'évolution des portefeuilles.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> PROFIL DE RISQUE
===

<!-- PageHeader="49 / 107" -->

Programme de reassurance pour limiter les risques techniques des produits de prévoyance

Les produits de prévoyance génèrent des risques techniques de souscription (décès, invalidité, antisélection, ... ).

La politique de réassurance de MAIF VIE s'inscrit en cohérence avec celle des autres entités du groupe.


## C.2 Risque de marché

Le risque de marché est le risque de perte qui peut résulter des fluctuations des prix des actifs financiers qui composent un portefeuille d'actifs.

Le risque de marché de MAIF VIE se décompose en risques actions, immobilier, de taux d'intérêt, de spread, de change et de concentration.

Le risque de spread de crédit est décrit au § C.3 consacré au risque de crédit.

MAIF VIE a délégué à MAIF la mise en œuvre de sa gestion financière et de sa gestion actif/passif ainsi que la comptabilité de ses placements

MAIF VIE applique l'approche par transparence, qui consiste à calculer le capital de solvabilité requis sur la base de chaque actif sous-jacent des organismes de placement collectif et autres investissements sous forme de fonds.

Les principaux risques identifiés sont les risques financiers (risques de marché, risque de concentration, risque de liquidité et risque de défaut) et les risques de durabilité (mauvaises pratiques ESG des entités financées et risques de non-adaptation à la transition énergétique et écologique).

A noter que pour le risque de marché (§ C.2), le risque crédit (§ C.3), le risque de liquidité (§ C.4) et les risques de durabilité (§ C.6.4), le principe de la personne prudente s'applique.

Les investissements dans des instruments financiers sont effectués après une analyse pour identifier, mesurer, suivre, gérer, contrôler et déclarer les risques de manière adéquate et selon le « principe de la personne prudente ».

MAIF VIE détermine, en collaboration avec les équipes Investissements et placements et gestion actif-passif de MAIF, chaque année, son allocation d'actifs et ses règles de gestion présentées dans la politique de Gestion Tactique des Placements.

La politique de gestion des risques des investissements, la politique de gestion tactique des placements et la politique de gestion des risques de liquidité et concentration de MAIF VIE sont fixées de façon à garantir la sécurité, la qualité, la liquidité et la rentabilité de l'ensemble du portefeuille. De même, les actifs sont majoritairement domiciliés dans les pays membres de L'OCDE. Cela permet de garantir leur disponibilité. Les investissements directs sont effectués selon un processus de sélection rigoureux des émetteurs et de limites portant sur le respect d'un haut niveau de qualité de portefeuille (exclusions notations financières et extra-financières) et sur des stratégies de diversification par pays, secteur, et émetteur

Par ailleurs, l'investissement dans une nouvelle classe d'actifs est conditionné à l'approbation du comité politiques d'investissements et gestion d'actifs avant d'être validé par le conseil d'administration.

Le respect de l'ensemble des allocations et règles de gestion définies dans [les][la] politique[s] annuelle[s] de gestion tactique des placements ou validées lors des différents comités est vérifié quotidiennement.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> PROFIL DE RISQUE
===

<!-- PageHeader="50 /107" -->

Les équipes de gestion spécialisées par classes d'actifs effectuent quotidiennement un suivi des risques de marché. Elles réalisent leurs propres analyses des actifs financiers dans lesquels MAIF VIE investit, en amont de l'investissement et durant la période de détention.

Ces principes s'appliquent également aux actifs détenus en représentation des contrats d'assurance vie multisupports pour leur composante en unités de compte. Les supports en unités de compte sont sélectionnés dans le meilleur intérêt des assurés (et leurs bénéficiaires) et en cohérence avec la politique d'investissements du groupe MAIF.


## C.2.1 Risque actions

Le risque actions correspond au risque de variation du cours des actions (cotées ou non cotées).

Après une contraction de plus de 3% en 2020, la croissance mondiale a nettement rebondi en 2021 et devrait s'établir proche de 6%. La vaccination et les mesures de soutiens publiques ont permis de renforcer ce rebond. En revanche, les goulets d'étranglement ont favorisé une accélération rapide de l'inflation poussant les banques centrales à réduire leur soutien.

Aux Etats-unis, la croissance annuelle 2021 devrait approcher les 6%. L'inflation en forte hausse depuis mai a atteint 7% en décembre, un record depuis 1982. Avec la réduction des excès de capacités sur le marché du travail, cela a poussé la Fed à annoncer une accélération de sa normalisation monétaire.

En Zone euro, des restrictions de mouvements ont été réintroduites à chaque vague pandémique, pénalisant ainsi la mobilité et les services. A cela s'ajoute la persistance de perturbations sur les chaines de valeur mondiales qui affectent particulièrement l'Allemagne. Au total, la dynamique a ralenti au T4 et après -6,4% en 2020 la croissance annuelle devrait s'établir proche de 5% en 2021. Comme aux Etats-unis, les pressions inflationnistes sont prégnantes, 4,9% en novembre, et la BCE a déjà commencé à réduire ses achats.

Dans ce contexte, les actifs risqués ont été privilégiés par les investisseurs. Ainsi, les indices actions affichent des gains importants sur l'année, dans un contexte de volatilité relativement contenue. Le CAC 40 s'est apprécié de 29% contre 21% pour l'Eurostoxx 50 et 27% pour le SP500. Seul l'indice des pays émergents, impacté par le recul des marchés chinois, s'affiche en territoire négatif avec un recul de 4,6%.

L'exposition de MAIF VIE aux marchés actions est modérée et a fait l'objet d'un pilotage renforcé durant cette période

MAIF VIE est exposée à un fort reajustement à la baisse des marchés actions (pas de retour à la situation initiale avant plusieurs années) engendrant, si elle se produisait :

› une chute de la valeur du portefeuille d'actifs et des produits financiers futurs sur les fonds propres en normes Solvabilité 2 ;

› et un risque de provisionnement pour risque d'exigibilité (PRE) et pour dépréciation durable (PDD) impactant directement le résultat de l'entreprise en normes comptables françaises.

Dans des conditions normales de marché et de volatilité, le risque actions est maîtrisé.

Ce risque est évalué selon la formule standard.


## C.2.2 Risque immobilier

Le risque immobilier correspond au risque de baisse du prix des actifs immobiliers ou de la valeur des participations détenues dans des fonds/sociétés du fait de la baisse du prix des actifs immobiliers sous-jacents.

Grâce aux politiques publiques accommodantes avec notamment la politique du « quoi qu'il en coûte >> et à la solidarité des différents acteurs comme les actions de solidarité de France

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="51 / 107" -->

Assureurs (ex FFA) vis-à-vis des locataires en difficultés, des commerces et hôtels fermés administrativement, les marchés immobiliers n'ont pas connu en 2021 les baisses annoncées. Par ailleurs, dans un contexte de taux bas, d'accès facile aux liquidités et de prime de risque immobilière importante, les investisseurs ont pour la plupart maintenu leurs expositions en immobilier de placement et globalement, les valorisations des immeubles se sont bien tenues avec des niveaux de perception des flux locatifs satisfaisants.

Dans ce contexte, le portefeuille immobilier de MAIF VIE s'est montré très resilient, conséquence de plusieurs choix forts :

› MAIF VIE s'est depuis longtemps attachée à diversifier son allocation en immobilier de placement sur un plan géographique mais aussi un sur un plan sectoriel en investissant entre autres en santé et en résidentiel,

› le recentrage opéré sur le bureau « prime > concourt également à la bonne robustesse du portefeuille immobilier,

› enfin, la prise en compte des enjeux Environnementaux Sociaux et de Gouvernance (ESG) depuis plus de 10 ans dans la conduite de la politique d'investissement en immobilier de placements de MAIF VIE, permet aujourd'hui d'avoir globalement des actifs de qualité, en phase avec les attentes des utilisateurs ce qui sous-tend leurs prises à bail et leurs bonnes appréciations par les experts immobiliers.

Ce risque est évalué selon la formule standard.


## C.2.3 Risque de taux d'intérêt

Le risque de taux d'intérêt est le risque de variation du prix ou de la valorisation d'un actif ou d'un passif financier résultant d'une variation défavorable des taux d'intérêt sans risque. Ceci inclut l'asymétrie entre l'actif et le passif.

En période de baisse des taux d'intérêt :

› la valorisation au bilan des indemnisations ou engagements sous forme de rentes augmente,

› la valorisation du portefeuille d'obligations à taux fixe augmente,

› en revanche, les nouveaux investissements réalisés sur des taux d'intérêt plus faibles font baisser le rendement du portefeuille.

À l'inverse, en période de hausse des taux :

› la valorisation au bilan des indemnisations ou engagements sous forme de rentes diminue,

› la valeur du portefeuille d'obligations à taux fixe diminue, ce qui peut avoir un impact significatif sur la solvabilité,

› en revanche, les nouveaux investissements réalisés sur des taux d'intérêt plus élevés augmenteront le rendement du portefeuille obligataire.

De par son activité d'épargne (domaine d'activité prédominante de l'entreprise) et la structure de son portefeuille d'actifs (majoritairement positionné sur des placements obligataires), MAIF VIE est exposée aux risques financiers (dégradation des marchés financiers) et plus particulièrement à une situation de hausse brutale et pérenne des taux d'intérêts, ce qui conduirait à une chute de la valeur de marché du portefeuille d'actifs et exposerait MAIF VIE à un risque de rachats.

Plusieurs dispositifs de gestion de ce risque sont mis en œuvre :

› la duration du portefeuille obligataire gere en direct est strictement encadrée par une fourchette,

› la constitution de réserve (provision pour participation aux bénéfices) offrant la capacité d'augmenter la rémunération des contrats d'épargne. La provision pour participation aux bénéfices s'élève à 6,04 % des encours des produits d'épargne (supports en euros) au 31 décembre 2021.

Une situation durable de taux d'intérêt négatifs pousserait à repenser le business model de l'activité assurance vie de MAIF VIE et exposerait plus fortement MAIF VIE aux autres risques financiers (chute des marchés actions, immobiliers, ... ).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="52 / 107" -->

La provision pour participation aux bénéfices (réserve) offre la capacité, sur quelques années, de fournir un niveau de rémunération compétitif aux contrats d'épargne dans une situation de taux bas ou négatifs.

Après une année 2020 marquée par la multiplicité et la gravité des chocs survenus, l'année 2021 aura permis d'entrevoir le début d'un certain retour à la normale. Cette année a été caractérisé par la fin du fort rebond macroéconomique et financier (entamé dès le 3ème trimestre 2020 dans les pays développés), par le retour de l'inflation sur des niveaux élevés, par la persistance de la pandémie (atténuée par la vaccination) et par l'accélération de la transition énergétique. Malgré l'amorce de la normalisation des banques centrales, accentuée par le risque inflationniste, le marché est resté resilient en 2021.

Ce risque est évalué selon la formule standard.


## C.2.4 Risque de change

Le risque de change correspond au risque de variation du prix ou de la valorisation d'un actif ou d'un passif financier libellé en devises étrangères résultant d'une évolution défavorable du cours de conversion avec la devise de référence (monnaie nationale).

Les investissements sont réalisés exclusivement sur des actifs libellés en euro. Néanmoins, de manière limitée, MAIF VIE pourrait être exposée aux risques de change dans le cadre de fonds d'investissements en gestion déléguée.

Ce risque est évalué selon la formule standard.


## C.3 Risque de crédit

Le risque de crédit est le risque de perte du principal (somme investie ou prêtée au départ) ou perte de rémunération financière résultant de fluctuations affectant la qualité de crédit d'émetteurs de valeurs mobilières, de contreparties ou de tout autre débiteur et auquel les sociétés d'assurance sont exposées sous forme de risque de spread, risque de concentration et risque de défaut de contrepartie.

Trois types de risque de crédit font l'objet d'un suivi spécifique à MAIF VIE :

› les placements financiers, et plus particulièrement le portefeuille de produits de taux : risque de spread et risque de concentration ;

› les créances sur les réassureurs, nées des opérations de cession en reassurance : risque de défaut ;

› les créances sur les banques, nées des dépôts de trésorerie : risque de défaut.

Le principe de la personne prudente décrit au §C.2 (risque de marché) s'applique au risque de crédit.


### C.3.1 Risque de spread

Le risque de spread de crédit est le risque de variation du prix ou de la valorisation d'un actif résultant d'une évolution défavorable de l'écart entre le taux de rentabilité actuariel d'une obligation et le taux de rentabilité actuariel d'un emprunt sans risque de durée identique. L'origine du risque est une dégradation de la qualité de crédit de l'émetteur obligataire (hors obligations souveraines zone euro pour le calcul réglementaire).

Le spread est également couramment appelé prime de risque.

Le risque de spread est maîtrisé par la mise en place d'un processus de sélection rigoureux des émetteurs et de limites portant sur le respect d'un haut niveau de qualité de portefeuille (limites par notation) et sur des stratégies de diversification par pays, secteur, et émetteurs.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="53 / 107" -->

À fin 2021, le portefeuille obligataire, ventilé par notation, est présenté ci-dessous.

Portefeuille obligataire par notation en valeur de marché - 2021

<figure>

![](figures/10)

<!-- FigureContent="Non noté 2% BBB AAA 12% 10% A 22% AA 54% AA A BBB" -->

</figure>


AAA

Non noté

En 2021, les spreads de crédit ont poursuivi leur resserrement entamé dès avril 2020 dans un climat globalement peu volatile. Différents facteurs peuvent expliquer ce phénomène : soutien budgétaire (différents dispositifs mis en place par les gouvernements), soutien monétaire (politiques des banques centrales très accommodantes), adaptation à la situation sanitaire (vaccination, tests, assouplissement des restrictions, ... ), des liquidités importantes sur les marchés, ... En parallèle, les fondamentaux des émetteurs corporates se sont nettement améliorés en 2021, illustrés par des taux de défaut sur des plus bas historiques (1,7% à fin 2021 contre 6,9% fin 2020 pour les émetteurs à haut rendement au niveau global, plus bas depuis 10 ans). Les portefeuilles crédit sont essentiellement exposés à des émetteurs de bonne qualité (catégorie investissement), les secteurs les plus cycliques sont structurellement sous-pondérés et les émetteurs les plus impactés par la crise ont été allégés ou cédés en 2020. Ainsi, ce positionnement prudent devrait limiter le risque de spread face au retrait progressif attendu du soutien des Etats et de celui des banques centrales.

Ce risque est évalué selon la formule standard.


## C.3.2 Risque de concentration

Le risque de concentration se définit comme le risque découlant de toutes les expositions au risque qui sont assorties d'un potentiel de perte suffisamment important pour menacer la solvabilité ou la situation financière de MAIF VIE. Ce risque découle ainsi, soit d'un manque de diversification du portefeuille d'actifs, soit d'une exposition importante à un seul et unique émetteur de valeurs mobilières ou d'un groupe d'émetteurs liés.

Le risque de concentration de marché est limité à MAIF VIE par la mise en place de différents niveaux de diversification dans le cadre de la politique de gestion tactique des placements :

› le premier niveau de diversification est établi à partir d'une matrice d'allocation des actifs qui fixe des proportions minimales et maximales pour chacune des grandes classes d'actifs de l'univers d'investissements,

› de plus, des règles de dispersion existent pour la gestion directe,

› enfin, des ratios d'emprise sont imposés par la politique de gestion tactique des placements pour la gestion déléguée, hors fonds dédiés aux sociétés du groupe MAIF.

Le respect de ces limites est vérifié à chaque souscription sur un émetteur.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" --> :unselected: :unselected: :selected:

<!-- PageNumber="> PROFIL DE RISQUE" -->

<!-- PageHeader="54 / 107" -->

À fin 2021, le portefeuille obligataire en direct, ventilé par secteur, est présenté ci-dessous. Le secteur Agences & Supranationales regroupe les institutions gouvernementales supranationales comme l'Union Européenne ou la Banque Internationale pour la Reconstruction et le Développement (BIRD).

<figure>

<figcaption>

Portefeuille obligataire par secteur - 2021

</figcaption>

![](figures/11)

<!-- FigureContent="Non financières 23% Etats 42% Financières Agences & 31% Supranationales 4%" -->

</figure>


L'exposition de MAIF VIE aux titres souverains émis par les Etats et agences & supranationales est présentée ci-après.

Exposition de MAIF VIE aux titres souverains - 2021

<figure>

![](figures/12)

<!-- FigureContent="Allemagne Autriche Espagne 2% Finlande 3% 2% 1% Irlande 4% Belgique 6% Supranational 9% France 73%" -->

</figure>


Dans le cadre de la gestion déléguée, MAIF VIE a défini plusieurs critères de sélection des sociétés de gestion (ou de toute autre entité assurant ou conseillant la gestion).

Le risque de concentration est évalué selon la formule standard (SCR Marché).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="55 / 107" -->


## C.3.3 Risque de défaut de contrepartie pour les réassureurs

Le risque de défaut de contrepartie est le risque qu'un débiteur ou qu'une contrepartie de MAIF VIE n'honore pas ses engagements dans les conditions initialement prévues (livraison, paiement, remboursement, etc.).

Le risque de défaut de contrepartie pour les réassureurs est géré en amont de la souscription des traités de réassurance, notamment en définissant des critères de sélection des réassureurs, essentiellement sur la notation des réassureurs et sur la dispersion des réassureurs, de manière à éviter une exposition excessive à un reassureur en particulier.

Au cours de l'année, un suivi régulier est réalisé sur les traités de reassurance. Ce suivi comprend notamment :

› la constitution des comptes de trésorerie,

› la réalisation des comptes techniques annuels, par traite et par type de risques couverts,

› l'évolution des notations des réassureurs,

› la communication des informations aux réassureurs,

› le recouvrement et le suivi des soldes à régler.

Le risque de défaut de contrepartie est évalué selon la formule standard.


## C.4 Risque de liquidité

Le risque de liquidité se décompose en deux risques :

› le risque d'illiquidité, qui se définit comme le risque de ne pas pouvoir céder des actifs en vue d'honorer les engagements financiers de l'assureur au moment où ceux-ci deviennent exigibles. MAIF VIE considère également comme risque d'illiquidité, le risque de céder nécessairement des actifs dans des conditions dégradées pour honorer ses engagements financiers au moment où ceux-ci deviennent exigibles (risque de devoir vendre des actifs en moins-values et donc de générer des pertes pour faire face aux engagements : décaissements pour régler les sinistres, les frais généraux ... ) ;

› le risque de trésorerie (gap de trésorerie), qui correspond à une impossibilité de faire face aux engagements de court terme (décaissements pour régler les prestations, les frais généraux ... ).

Le principe de la personne prudente décrit au § C.2 (risque de marché) s'applique au risque de liquidité.

MAIF VIE investit majoritairement sur des supports considérés comme liquides (placements monétaires, emprunts d'État de la zone euro, obligations dites « Investment grade » émises par des agences, des entreprises et des banques, fonds d'actifs cotés ... ).

La part importante de titres liquides dans le portefeuille MAIF VIE devrait permettre de faire face à un besoin imprévu de trésorerie sans avoir à céder des titres difficilement cessibles. Les revenus récurrents générés par les obligations, les dividendes et les loyers assurent des entrées régulières en trésorerie.


## Processus de gestion de la liquidité ›

Des travaux actif/passif sont réalisés chaque année, en amont de la rédaction des politiques de gestion des risques des investissements et de gestion tactique des placements. Ces travaux intègrent notamment la définition de l'allocation stratégique d'actifs et l'analyse du risque de liquidité avec une vision moyen/long terme, ainsi il en découle la détermination de limites opérationnelles sur les investissements.

Quotidiennement, sont réalisés l'actualisation du prévisionnel de trésorerie sur l'année en cours et le suivi des règles de gestion (limites).

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> PROFIL DE RISQUE
===

<!-- PageHeader="56 / 107" -->

Des calculs de projections de flux sont réalisés mensuellement permettant ainsi de déceler les éventuelles insuffisances de trésorerie a court/moyen terme. Ces projections permettent chaque mois de définir quelles sont les enveloppes d'investissement à réaliser par la Direction Investissements et Placements sur le mois.

Des calculs de projections de flux sont réalisés trimestriellement par MAIF VIE, permettant ainsi de déceler les éventuelles insuffisances de trésorerie à court/moyen terme.

Au moins une fois par an, une analyse des risques de liquidité est réalisée pour le portefeuille de chaque entité du groupe MAIF.

Depuis 2020, un suivi du risque de liquidité des obligations au quotidien est réalisé à partir de BarraOne, l'outil de suivi du risque de marché utilisé par la Direction Investissements et Placements. Ce suivi repose sur un indicateur de marché très courant : le bid ask spread qui correspond à l'écart entre le prix maximum qu'un acheteur est prêt à payer pour acheter un titre et le prix minimum auquel un vendeur est prêt à vendre.

Logiquement, plus cet écart est important, plus les acheteurs auront du mal à s'entendre pour faire des transactions et donc plus la liquidité sera dégradée.

C'est un indicateur suivi globalement sur le portefeuille pour donner une « tendance >> de l'évolution de la liquidité du marché.

Par ailleurs, comme évoqué en partie D du présent rapport, MAIF VIE a recours à la correction pour volatilité dans l'évaluation de sa solvabilité. A ce titre, conformément à l'article R354-2 du Code des assurances, elle doit établir un plan de liquidité comportant une prévision des flux de trésorerie entrants et sortants au regard des actifs et des passifs faisant l'objet de cette correction. Le prévisionnel de trésorerie contrôlé quotidiennement par la DIP s'inscrit dans ce cadre pour l'année en cours, les travaux actifs passifs menes annuellement par la GAPEF couvrant la partie prospective.


## › Facultés contractuelles et règlementaires face au risque de liquidité en assurance vie

La survenance d'une impasse de trésorerie liée à un risque non anticipé au passif deviendrait problématique en cas d'absence de liquidité des actifs.

Or, le portefeuille d'actifs de MAIF VIE est principalement positionné sur des actifs dont la cession dans des conditions normales s'effectue dans un délai inférieur ou égal à 3 jours.

Conformément aux conditions contractuelles, MAIF VIE, dispose, pour la grande majorité des produits, d'un délai d'un mois pour régler les prestations décès et rachats, après réception de l'ensemble des pièces exigées.

Par ailleurs, l'article L132-21 du Code des assurances permet à l'entreprise d'assurance de verser la valeur de rachat dans un délai n'excédant pas deux mois à compter de la date de réception de l'ensemble des pièces exigées. Au-delà de ce délai, les sommes non versées produisent de plein droit intérêt au taux légal majoré de moitié durant deux mois, puis, à l'expiration de ce délai de deux mois, au double du taux légal.

En dernier recours, dans un contexte financier dégradé, trois facultés règlementaires pourraient être activées :

› L'article L.131-4 du code des assurance, dans le cas où la valeur liquidative d'une UC ne serait pas disponible, permet à MAIF VIE de (i) proposer de remettre à l'adhérent ou bénéficiaires les parts de l'OPC ou (ii) suspendre les rachats, arbitrages, versements, prestations, transferts et conversions en rentes sur ces UC.

› L'article L.612-33 du Code monétaire et financier, permet à l'ACPR de suspendre, restreindre ou interdire temporairement la libre disposition de tout ou partie des actifs et de suspendre ou limiter le paiement des valeurs de rachat, la faculté d'arbitrages, le versement d'avances sur contrat ou la faculté de renonciation.

› La loi Sapin 2 permet, depuis 2016, au haut conseil de stabilité financière (HCSF) de suspendre ou limiter le rachat des contrats d'assurance vie, si le secteur de l'assurance vie se trouvait, dans son ensemble, en difficulté.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageNumber="> PROFIL DE RISQUE" -->

<!-- PageHeader="57 / 107" -->

MAIF VIE n'a pas eu recours à des cessions de titres pour faire face à ses engagements et n'a pas été confronté à des besoins exceptionnels de liquidité. Les indicateurs de liquidité sont restés stables en 2021, sur des niveaux n'entrainant ni d'alertes sur la liquidité des portefeuilles ni de problèmes pour que les gérants réalisent leurs opérations habituelles.


## C.5 Risque opérationnel

La définition du risque opérationnel telle que rédigée dans les textes Solvabilité 2, est celle retenue par MAIF VIE et le groupe MAIF : « Le risque opérationnel est un risque de perte provenant de processus internes inadéquats ou défaillants, de personnes, de systèmes ou d'événements externes >>. Cette définition recouvre notamment les erreurs humaines, les fraudes et malveillances, les défaillances des systèmes d'information, les problèmes liés à la gestion du personnel, les litiges commerciaux, les accidents-incendies-inondations sur le bâtiment occupé par MAIF VIE.

Ce risque est évalué selon la formule standard, via une approche forfaitaire globale et indépendante des risques sous-jacents.

Les principaux risques opérationnels de MAIF VIE identifiés sont décrits ci-après.


### C.5.1 Risque de sous-traitances importantes ou critiques (STIC)

Le risque et le dispositif de MAIF VIE sont présentés au § B.7.


#### C.5.2 Risque de non continuité ou indisponibilité des données ou applicatifs

Le système d'information de MAIF VIE n'est pas exposé directement aux intrusions d'origine externe dans la mesure où il est protégé par le système d'information de la MAIF, et donc dépendant du niveau de sécurité du système d'information de cette dernière.

MAIF dispose d'une équipe dédiée à la sécurité des systèmes d'information (RSSI Groupe). Au sein de MAIF VIE, un collaborateur a désormais en charge la mission de RSSI, incluant la déclinaison de la politique SI groupe.

Les équipes informatiques ont élaboré et tiennent à jour un plan de reprise des activités SI. Concernant le risque d'indisponibilité du système d'information, un processus spécifique existe sur la réalisation de tests et de validation des applicatifs.


#### C.5.3 Risque de non-conformité aux obligations réglementaires

Afin de sécuriser la mise en conformité aux nombreuses obligations règlementaires qui s'imposent aux sociétés d'assurance vie, MAIF VIE s'inscrit dans une démarche d'amélioration continue tant sur la réglementation existante que sur la déclinaison des règlementations nouvelles.

En amont, les mises en conformité au regard des évolutions législatives et réglementaires impliquent tous les métiers concernés et sont accompagnées par les différentes entités juridiques et la Conformité.

En aval, la maîtrise des risques de non-conformité, et le suivi des dispositifs mis en œuvre, sont sécurisés par le contrôle permanent et périodique.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="> PROFIL DE RISQUE" -->

<!-- PageHeader="58 / 107" -->


## C.6 Autres risques importants

En complément des risques décrits ci-avant, MAIF VIE a identifié plusieurs autres risques d'entreprise ou risques émergents significatifs.

Pour ces risques, l'impact n'est pas quantifiable avec précision. Cependant, des processus de gestion des risques et de contrôle interne ont été déployés de manière à fournir une maîtrise attentive des risques et une sécurité raisonnable de l'activité, de la solvabilité et de la situation financière de MAIF.


### C.6.1 Risque de construction, maintenance ou usages inadaptés ou inadéquats des modélisations

MAIF VIE utilise des modèles pour mesurer ses engagements (calcul de provisions), évaluer sa solvabilité (mesure des SCR et MCR), concevoir ses tarifications, estimer les impacts potentiels face à des situations extrêmes futures, réaliser les études la gestion actif/passif et definir l'allocation stratégique d'actif.

La complexité et la précision des modèles développés par MAIF VIE peuvent engendrer un risque, dit de modèle, se manifestant par l'écart entre les phénomènes que l'on cherche à représenter et leur réalisation.

L'encadrement des modèles mis en place par MAIF VIE limite la portée de ce risque : existence d'une gouvernance stricte concernant l'utilisation (une charte d'utilisation et recueil de bonnes pratiques) et les évolutions des modèles (environnement dédié de développement et processus de validation). Une documentation robuste et structurée vient compléter ces dispositions pour assurer une accessibilité et une compréhension du fonctionnement des modèles.

L'interprétation des résultats issus des modélisations est réalisée par confrontation avec le passé (back testing), partagée avec différents intervenants en dehors des développeurs et complétée par des analyses de sensibilités (sur les hypothèses).


### C.6.2 Risque de réputation et gestion de crise

Le groupe MAIF procède à une veille régulière sur son image et sa réputation. Cette veille est à la fois vis-à-vis de la presse, internet par exemple blog, réseaux sociaux ou publications judiciaires.

La gestion de crise peut conduire à activer notamment le Plan de continuité des activités de MAIF VIE qui est présenté au §B4.


### C.6.3 Risque d'évolution économique ou juridique

MAIF VIE, de par son secteur d'activité, est exposée à un risque d'évolution économique, réglementaire ou juridique qui engendrerait une perte de compétitivité des produits d'assurance vie par rapport à d'autres produits d'épargne (bancaires ou autres), et ainsi conduirait, non seulement à des rachats massifs par arbitrage, mais aussi à une baisse significative du chiffre d'affaires. Cette situation conduirait nécessairement MAIF VIE à repenser son offre produits ainsi que son business model.

MAIF VIE est attentive à toute évolution juridique, règlementaire et économique au travers des différents processus de veille auxquels elle participe soit directement, soit sous couvert du groupe et réalise des chiffrages le cas échéant.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="59 / 107" -->


## C.6.4 Risque de durabilité

Le principe de la personne prudente décrit au § C.2 (risque de marché) s'applique en matière de gestion d'actifs, au risque de durabilité.

C.6.4.1 Risque ESG (environnement, social et gouvernance) en matière de gestion d'actifs

En cohérence avec sa raison d'être et ses valeurs, le groupe MAIF s'engage dans la prise en compte des risques extra-financiers dans ses politiques d'investissements (politique de gestion des risques des investissements et politique de gestion tactique des placements), et dans une démarche d'investisseur 100 % responsable.

Cette démarche d'investisseur 100 % responsable consiste à évaluer systématiquement et en préalable à tout investissement les caractéristiques environnementales, sociales et de gouvernance des entreprises et des états dans lesquels elle investit directement, ainsi que la démarche de prise en compte des sujets extra-financiers par les fonds d'investissement qu'elle sélectionne.

Ainsi, les gérants d'actifs financiers du groupe MAIF ne peuvent investir directement que dans des Etats qui ont aboli la peine de mort et présentent des niveaux de sécurité, de développement humain retraité des inégalités et de maîtrise de la corruption satisfaisants. Dans le même sens, les entreprises qui ne répondraient pas aux attentes du groupe MAIF en matière d'orientation de leur activité ou de bonnes pratiques de responsabilité sociétale des entreprises (RSE) ne peuvent pas être financées directement (grâce à l'analyse ESG).

Le groupe MAIF s'engage ainsi à ne pas financer directement les entreprises qui ne respecteraient pas les 10 principes du Pacte Mondial des Nations Unies, les entreprises qui commercialisent des armes, les producteurs de tabac, les entreprises liées au charbon et à l'industrie pétrolière. En gestion déléguée, les gérants font leurs meilleurs efforts pour intégrer l'ISR et le financement de la transition énergétique et écologique dans chaque classe d'actifs. En complément, un suivi quotidien des actualités concernant les entreprises dans le portefeuille géré en direct est effectué. En cas de controverse sur une société, une analyse des expositions sur l'ensemble du portefeuille (gestion directe et gestion déléguée) via la transparisation est réalisée.

Pour 2021, l'objectif d'allocation ISR pour le groupe MAIF était d'investir au moins 85 % des actifs financiers (hors participations stratégiques et immobilier d'exploitation) dans des supports qui prennent en compte les sujets environnementaux, sociaux et de gouvernance. A fin 2021, le groupe MAIF a atteint une part ISR de 89 %.


## C.6.4.2 Risque lié à la transition énergétique et écologique

Le groupe MAIF s'inscrit dans la transition énergétique et écologique (risque physique, risque de transition et risque de responsabilité) en proposant notamment une politique d'investissements en adéquation avec les enjeux liés au changement climatique.

Une mauvaise prise en compte de la transition climatique et énergétique dans les investissements constituerait un double risque pour le groupe MAIF :

› une perte de contrôle sur les externalités négatives provoquées par les entreprises, Etats et projets finances par le groupe,

› une perte de valeur ou de capital associée aux conséquences physiques du changement climatique, à une mauvaise adaptation des entreprises ou politiques publiques à la transition énergétique, et aux risques d'image ou juridiques induits par la lutte contre les émissions de substances polluantes et de gaz à effet de serre.

En 2020, le groupe a franchi une étape supplémentaire en adoptant une « Stratégie Climat » ambitieuse pour la gestion de ses investissements, au service d'une transition climatique inclusive. Cette stratégie Climat volontariste en phase avec l'objectif de l'accord de Paris vise l'alignement, d'ici 2030, de son portefeuille d'investissements sur une trajectoire au plus proche d'1,5°C. Pour atteindre cet objectif, le groupe s'engage à sortir totalement du charbon MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

› PROFIL DE RISQUE
===

<!-- PageHeader="60 /107" -->

thermique d'ici 2030. Cette politique a été renforcée en 2021 par la définition d'une stratégie de sortie des énergies fossiles, le Groupe MAIF se fixant pour objectif de sortir totalement des énergies fossiles non conventionnelles d'ici 2030 et du pétrole d'ici 2040.

L'adoption de cette stratégie est également l'opportunité pour le groupe MAIF de démontrer sa singularité et de contribuer à une transition climatique inclusive, c'est-à-dire attentive aux enjeux sociétaux liés à la transition tels que son impact sur l'emploi, les territoires ou l'accès à l'énergie.

La mise en œuvre de cette stratégie se traduit par un renforcement de notre politique d'exclusion ainsi que par des actions d'engagement auprès de nos parties prenantes.


## Politique d'exclusion du charbon

Le Groupe MAIF ne finance plus en direct et via ses fonds dédiés :

› les entreprises minières qui produisent du charbon thermique ;

› les entreprises qui développent de nouveaux projets de mines, centrales et infrastructures charbon ;

› les entreprises qui réalisent plus de 10 % de leur chiffre d'affaires grâce au charbon ;

› les entreprises qui produisent plus de 10 % de leur électricité à partir du charbon ;

› les entreprises dont la capacité de production d'électricité à partir de charbon est supérieure à 5 GW.


## Politique d'exclusion des autres énergies fossiles

Sont exclus des investissements en direct et des fonds dédiés :

› les entreprises développant de nouveaux projets d'exploration ou de production de pétrole ou de gaz naturel ;

› les entreprises dont plus de 5 % de la production d'hydrocarbures est issue d'énergies fossiles non conventionnelles (pétrole et gaz de schiste, sables bitumineux, forage en Arctique, forage en eaux très profondes) ;

› les entreprises de l'industrie pétrolière ou produisant de l'électricité à partir de pétrole pour plus de 5 % de leur chiffre d'affaires.

En complément de ces exclusions, la mutuelle entend mener une démarche d'engagement auprès des entreprises et faire évoluer favorablement les sociétés de gestion en s'engageant auprès de celles à travers lesquelles elle investit.

Pour 2021, le groupe MAIF s'était par ailleurs engage à investir au moins 7% des actifs financiers (hors participations stratégiques et immobilier d'exploitation) en faveur de la transition énergétique et écologique (« part verte »).

En ligne avec sa raison d'être, qui vise à garantir un mieux commun, MAIF VIE cherche à contribuer par ses investissements à une transition climatique inclusive, créatrice d'emplois durables, attentive aux inégalités sociales et au développement des territoires et promotrice de modes de vie plus durables.

L'entreprise rendra compte régulièrement de la politique mise en œuvre et de l'exposition résiduelle de ses investissements liés au charbon thermique et aux énergies fossiles.

En parallèle, le groupe MAIF évalue les émissions de gaz à effet de serre induites par ses placements financiers, ainsi que l'alignement du mix énergétique des producteurs d'électricité que le Groupe MAIF finance avec un scénario de réchauffement climatique à 2ºC.

Depuis 2020, le Groupe réalise également une évaluation de l'alignement de ses portefeuilles d'investissements avec la trajectoire des accords de Paris, ainsi qu'une mesure des risques physiques et des risques de transition liés à ses investissements.

Par ailleurs, une veille active est menée sur les évolutions règlementaires sur le climat, les nouveaux outils et les nouvelles méthodologies d'analyse des risques climatiques des portefeuilles d'investissements financiers.

Le groupe MAIF procède également à une veille sur les impacts physiques du réchauffement climatique en particulier pour des évènements extrêmes ou l'offre d'assurance.

Fin 2020, le groupe MAIF a participé au premier exercice de stress test climatique proposé par L'ACPR.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> PROFIL DE RISQUE
===

<!-- PageHeader="61/ 107" -->

L'approche RSE tâche de sensibiliser les sociétaires sur l'urgence climatique au regard des usages quotidiens. Plusieurs solutions éducatives sur le thème du développement durable sont proposées par le groupe MAIF et mis à la disposition de tous.

Le groupe MAIF mène des actions pour réduire l'impact environnemental lié à ses immeubles d'exploitation et moyens généraux. Sa stratégie repose sur les objectifs de diminution des sources de pollution et la préservation des ressources. Elle se traduit par des actions concrètes : la réduction de la consommation énergétique, l'intégration des principes d'économie circulaire dans ses achats de mobilier ou encore le respect des référentiels de Haute Qualité Environnementale (NF HQE). Ses démarches sont certifiées et, en particulier, MAIF est ISO50001 pour son management de l'énergie.


## C.6.5 Risques émergents

MAIF VIE est en veille sur les risques émergents. Ce sont les risques non encore connus, qui peuvent se développer ou qui existent déjà et évoluent constamment .. Les risques émergents sont ainsi caractérisés par un haut degré d'incertitude, puisque certains ne se manifesteront jamais. Ces risques font l'objet d'un processus spécifique d'identification et d'analyse. La société réalise une veille de marché (publications scientifiques, règlementation, décisions de justice, économie ... ) pour identifier et élaborer des réponses de manière à anticiper la survenance de ces risques.


## › Risque de montée en puissance des robot-advisors (« robots-conseillers »)

Le risque concerne les produits d'épargne, il se matérialiserait ainsi : les sociétaires confient à des agrégateurs, la gestion de leur multi-bancarisation.

Ces autres acteurs se positionnent ainsi en intermédiaire sur les offres épargne dont l'assurance vie et font développer un comportement opportuniste des sociétaires.

Une conséquence serait une chute du portefeuille d'assurance : les assurés se repositionnant sur des offres digitales, plus conformes à leurs attentes.

Ce risque émergent est suivi notamment à travers les risques de dépense et de hausse des rachats.


## C.7 Autres informations

MAIF VIE n'a pas eu recours à des véhicules de titrisation. La titrisation est une technique financière qui vise à transférer à des investisseurs des actifs financiers en les transformant en titres financiers émis sur le marché des capitaux, le tout via une société ad hoc.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->
<figure>

![](figures/13)

<!-- FigureContent="D" -->

</figure>


VALORISATION À DES FINS DE SOLVABILITÉ

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="63 / 107" -->

D Valorisation à des fins de Solvabilité
===

La valorisation des actifs et des passifs de MAIF VIE concourt à la détermination du SCR, du MCR et des fonds propres éligibles pour les couvrir, présentés dans la suite de ce rapport au paragraphe E.

Le bilan repris ci-après présente le bilan prudentiel, établi selon les règles définies dans la directive Solvabilité 2 et ses textes d'application. Le montant des actifs et passifs valorisé selon les normes comptables françaises y figure également à titre de comparaison. L'actif net correspond à la différence entre le total de l'actif et le total du passif. Le bilan est arrêté au 31 décembre 2021.

Les chiffres sont présentés en milliers d'euros.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="64 / 107" -->

| ACTIF - 31/12/2021 | Solvabilité 2 | Normes comptables francaises | Ecart |
| - | - | - | - |
| Écarts d'acquisitions | - | - | - |
| Frais d'acquisition reportes | - | - | - |
| Actifs incorporels | - | 22 942 | -22 942 |
| Impôts différés actifs | - | - | - |
| Excédent de régime de retraite | - | - | - |
| Immobilisations corporelles pour usage propre | 1 563 | 4 566 | -3 003 |
| Placements (autres que les actifs en représentation de contrats en UC ou indexés) | 12 263 743 | 11 040 419 | 1 223 324 |
| Immobilier (autre que pour usage propre) | - | - | - |
| Participations | 849 000 | 670 933 | 178 067 |
| Actions | 88 991 | 75 622 | 13 369 |
| Actions cotées | - | - | - |
| Actions non cotées | 88 991 | 75 622 | 13 369 |
| Obligations | 8 842 050 | 8 119 624 | 722 425 |
| Obligations d'Etat | 4 112 039 | 3 655 073 | 456 966 |
| Obligations de sociétés | 4 730 011 | 4 464 551 | 265 460 |
| Obligations structurées | - | - | - |
| Titres garantis | - | - | - |
| Fonds d'investissement | 2 054 415 | 1 744 953 | 309 462 |
| Produits dérivés | - | - | - |
| Dépôts autres que ceux assimilables à de la trésorerie | 429 286 | 429 286 | - |
| Autres placements | - | - | - |
| Placements en représentation de contrats en UC ou indexé: | 708 868 | 708 868 | - |
| Prêts et prêts hypothécaires | 2 341 | 2 341 | - |
| Avances sur polices | 2 341 | 2 341 | - |
| Prêts et prêts hypothécaires aux particuliers | - | - | - |
| Autres prêts et prêts hypothécaires | - | - | - |
| Provisions techniques cédées | 80 181 | 77 730 | 2 451 |
| Non vie et santé similaire à la non-vie | - | - | - |
| Non vie hors santé | - | - | - |
| Santé similaire à la non vie | - | - | - |
| Vie et santé similaire à la vie, hors UC ou indexés | 80 181 | 77 730 | 2 451 |
| Santé similaire à la vie | - | - | - |
| Vie hors santé, UC ou indexés | 80 181 | 77 730 | 2 451 |
| UC ou indexés | - | - | - |
| Dépôts auprès des cédantes | - | - | - |
| Créances nées d'opérations d'assurance | 13 588 | 13 588 | - |
| Créances nées d'opérations de reassurance | 515 | 515 | - |
| Autres créances (hors assurance) | 13 282 | 13 282 | - |
| Actions auto-détenues | - | - | - |
| Instruments de fonds propres appelés et non payés | - | - | - |
| Trésorerie et équivalent trésorerie | 145 082 | 145 082 | - |
| Autres actifs non mentionnés dans les postes ci-dessus | - | - | - |
| TOTAL DE L'ACTIF | 13 229 164 | 12 029 334 | 1 199 830 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

65 / 107

| PASSIF - 31/12/2021 | Solvabilité 2 | Normes comptables francaises | Ecart |
| - | - | - | - |
| Provisions techniques - non-vie | - | - | - |
| Provisions techniques non-vie (hors santé) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques santé (similaire à la non-vie) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques - vie (hors UC ou indexes) | 11 124 548 | 10 560 291 | 564 257 |
| Provisions techniques santé (similaire à la vie) | - | - | - |
| Provisions techniques calculées comme un tout | - | | - |
| Meilleure estimation | - | | - |
| Marge de risque | - | | - |
| Provisions techniques vie (hors santé, UC ou indexés) | 11 124 548 | 10 560 291 | 564 257 |
| Provisions techniques calculées comme un tout | - | - | - |
| Meilleure estimation | 11 003 450 | | 11 003 450 |
| Marge de risque | 121 098 | | 121 098 |
| Provisions techniques UC ou indexes | 711 218 | 708 868 | 2 350 |
| Provisions techniques calculées comme un tout | - | - | - |
| Meilleure estimation | 703 795 | | 703 795 |
| Marge de risque | 7 422 | | 7 423 |
| Autres provisions techniques | - | - | - |
| Passifs éventuels | 2 519 | - | 2 519 |
| Provisions autres que les provisions techniques | 716 | 716 | - |
| Provision pour retraite et autres avantages | 1 585 | 1 567 | 19 |
| Dettes pour dépôts espèces des réassureurs | 76 937 | 76 937 | - |
| Impôts différés passifs | 34 207 | - | 34 207 |
| Produits dérivés | - | - | - |
| Dettes envers les établissements de crédit | - | - | - |
| Dettes fin. autres que celles envers les établis. de crédit | - | - | - |
| Dettes nées d'opérations d'assurance | 1 230 | 1 230 | - |
| Dettes nées d'opérations de réassurance | 1 926 | 1 926 | - |
| Autres dettes (non liées aux opérations d'assurance) | 51 815 | 51 815 | - |
| Dettes subordonnées | 34 500 | 34 500 | - |
| Dettes subordonnées exclues des fonds propres de base | - | - | - |
| Dettes subordonnées incluses dans les fonds propres de base | 34 500 | 34 500 | - |
| Autres dettes non mentionnées dans les postes ci-dessus | - | - | - |
| TOTAL DU PASSIF | 12 041 201 | 11 437 849 | 603 352 |
| | | | |
| ACTIF NET | 1 187 963 | 591 485 | 596 478 |

L'établissement du bilan, selon les règles en normes Solvabilité 2 et selon les normes comptables françaises, s'appuie sur certaines hypothèses et estimations.

› Principes generaux de valorisation des actifs et passifs dans le bilan en normes Solvabilité 2

Conformément à l'article 75 de la directive 2009/138/CE :

› les actifs sont valorisés au montant pour lequel ils pourraient être échanges dans le cadre d'une transaction conclue, dans des conditions de concurrence normale, entre des parties informées et consentantes ;

› les passifs sont valorisés au montant pour lequel ils pourraient être transférés ou réglés dans le cadre d'une transaction conclue, dans des conditions de concurrence normales, entre des parties informées et consentantes. Aucun ajustement visant à tenir compte de la qualité de crédit propre à l'entreprise d'assurance n'est effectué.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

66 / 107

Ces valorisations doivent être réalisées en se fondant sur l'hypothèse d'une continuité d'exploitation de l'entreprise et respecter les principes et la hiérarchie de valorisation définis dans les articles 9, 10 et suivants du règlement délégué 2015/35 de la commission européenne du 10 octobre 2014.

› Principales différences avec les normes comptables françaises

Les principales différences de valorisation entre le bilan en normes Solvabilité 2 et le bilan établi selon les normes comptables françaises portent sur les postes suivants :

› valorisation nulle des actifs incorporels ;

› revalorisation des placements à la juste valeur, intégrant les plus et moins-values latentes à la valeur historique corrigée des amortissements et dépréciations retenue dans le bilan normes comptables françaises ;

› revalorisation en valeur économique des provisions techniques ;

› comptabilisation d'impôts differes, non reconnus dans le bilan en normes comptables françaises ;

› reclassement de 85,4% de la Provision pour Participation aux Bénéfices (PPB) en fonds excédentaires.

Par ailleurs, l'évaluation des actifs et des passifs a été réalisée sur la base de l'hypothèse de la continuité de l'exploitation.

Les paragraphes D1, D2 et D3 ci-après développent le contenu des postes les plus significatifs du bilan et détaillent les principes généraux de valorisation décrits ci-dessus pour le bilan en normes Solvabilité 2 en les comparant aux normes comptables françaises, respectivement pour les actifs, les provisions techniques et les autres passifs (hors provisions techniques). Le paragraphe D4 revient sur les principales natures d'actifs pour lesquelles des méthodes de valorisation alternatives à l'utilisation directe ou indirecte d'un marché actif sont utilisées.

MAIF VIE n'a pas identifié d'autre information significative à mentionner dans ce paragraphe en dehors de ce qui est décrit ci-dessous.


## D.1 Actifs

Les différents postes du bilan actif de MAIF VIE sont présentés et détaillés dans cette section.


### D.1.1 Actifs incorporels

Ce poste comprend notamment, en normes comptables françaises, des progiciels et des immobilisations incorporelles liées à des livraisons à soi-même de solutions informatiques qui sont amortis linéairement sur une durée de 3 à 8 ans.

En normes Solvabilité 2, faute de marché actif permettant de déterminer leur juste valeur, ces actifs ont une valeur nulle.


### D.1.2 Immobilisations corporelles pour usage propre

Ce poste regroupe l'ensemble du mobilier, matériel et les aménagements s'y rapportant.

Les actifs corporels d'exploitation sont inscrits, en normes comptables françaises, à l'actif pour leur coût d'acquisition, sous déduction, le cas échéant, d'une valeur résiduelle à la condition qu'elle soit mesurable de manière fiable et significative et font l'objet d'amortissements annuels selon les modalités suivantes :

› Agencements, aménagements, installations : Linéaire sur 5 ou 10 ans

› Matériel de transport : Linéaire sur 5 ans

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="67/ 107" -->

› Matériel informatique et bureautique :

› Mobilier et matériel de bureau :

› Autres matériels :

Linéaire sur 3 ou 4 ans

Linéaire sur 5 ou 10 ans

Linéaire sur 5 ans

En normes Solvabilité 2, la juste valeur retenue pour ces actifs autres que les agencements et aménagements est égale à leur valeur nette comptable établie pour les comptes en normes comptables françaises.

En l'absence de marché permettant de déterminer leur juste valeur, les agencements et aménagements ont une valeur nulle.


## D.1.3 Placements

Ce paragraphe concerne les placements autres que les actifs en représentation de contrats en unités de compte ou indexés.

La grande majorité des titres détenus par MAIF VIE est valorisée par référence au prix de transactions sur un marché actif pour le même actif ou pour des actifs similaires. Compte tenu de la qualité de son portefeuille d'instruments financiers, MAIF VIE considère que la valorisation de l'essentiel de ses placements ne présente ainsi pas d'incertitudes significatives.

La juste valeur des instruments financiers négociés sur des marchés actifs correspond au prix de transactions constatées sur ce marché, lorsqu'ils sont disponibles. Le marché est considéré comme actif si des cours présentant une dispersion limitée peuvent être obtenus aisément et régulièrement auprès d'une bourse, d'un courtier, d'un service de cotation ou d'autres sources externes et qu'ils reflètent des transactions réelles. Comme le nombre et le volume de transactions peuvent parfois difficilement être appréciés, MAIF VIE prend également en compte la fréquence d'alimentation des cotations comme critère d'activité du marché. En pratique, le dernier cours coté est retenu.

En l'absence de transactions constatées sur un marché actif, l'évaluation est confiée à des prestataires externes et indépendants ou repose sur des techniques de valorisation. Ces techniques intègrent une part importante de jugement, liée au choix des méthodes de valorisation, des hypothèses retenues, des paramètres et des données utilisés. Elles s'appuient autant que possible sur des données de marché observables. Pour les participations, des méthodes de valorisation spécifiques sont prévues dans les textes d'application de la directive Solvabilité 2. Ces méthodes sont précisées ci-après.


### D.1.3.1 Participations

Les participations correspondent aux entreprises liées à MAIF VIE dans lesquelles MAIF VIE ou le groupe MAIF détient au moins 20 % du capital.

MAIF VIE ne détient pas de titres d'entreprises liées qui sont cotés sur un marché actif ou à valoriser à zéro.

Les principales participations de MAIF VIE sont des fonds d'investissements valorisés au dernier prix de rachat calculé et publié par la société de gestion.

En normes comptables françaises, les participations sont comptabilisées sur la base du coût historique, diminué d'éventuelles dépréciations.


### D.1.3.2 Actions non cotées

Sont classées dans la rubrique « actions non cotées >> les parts des sociétés civiles immobilières qui sont valorisées au dernier prix de rachat calculé et publié par la société de gestion.

En normes comptables françaises, les parts de SCI sont comptabilisées sur la base du coût historique, diminué d'éventuelles dépréciations.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="68 / 107" -->

Les actions sont valorisées selon la méthode retenue dans les comptes annuels publiés en normes comptables françaises correspondant au montant pour lequel elles pourraient être échangees dans le cadre d'une transaction conclue, dans des conditions de concurrence normales, entre des parties informées et consentantes.

Ainsi, ces actions sont valorisées selon la méthode retenue en normes comptables françaises.


## D.1.3.3 Obligations

Les obligations d'Etat regroupent les obligations et titres de créances négociables émis ou garantis par :

› des autorités publiques ;

› des administrations centrales ;

› des institutions publiques supranationales ;

› des administrations régionales ou municipales.

Les obligations de sociétés sont constituées des obligations et titres de créances négociables ainsi que des titres participatifs et subordonnés émis par des entreprises.

Les obligations sont valorisées à la juste valeur dans le bilan en normes Solvabilité 2. Toutefois, pour un nombre limité d'obligations ou autres titres de sociétés, le prix de revient est conservé.

En normes comptables françaises, les obligations sont comptabilisées sur la base du prix d'achat, ajusté des surcotes et décotes (différence entre le coût d'acquisition et le prix de remboursement) amorties sur la durée du titre et d'éventuelles dépréciations en cas de difficultés financières de l'émetteur.


### D.1.3.4 Fonds d'investissement

Les fonds d'investissement détenus à moins de 20 % par le groupe regroupent l'ensemble des organismes dont l'objectif est le placement collectif en valeurs mobilières et/ou dans d'autres actifs.

Le graphique ci-dessous présente la répartition de la valeur des parts de fonds acquises par MAIF VIE selon la nature des actifs détenus par les fonds :

Répartition des fonds d'investissements détenus en fonction de la nature des actifs détenus par les fonds - 2021

<figure>

![](figures/14)

<!-- FigureContent="2% 36% 39% 23%" -->

</figure>


<!-- PageFooter="Fonds Actions" -->

<!-- PageFooter="Fonds Immobiliers" -->
 :unselected:
<!-- PageFooter="Fonds Obligataires :unselected: :unselected: Fonds Monétaires" -->

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="69 / 107" -->

La valeur des fonds retenue dans le bilan en normes Solvabilité 2 correspond au dernier prix de rachat calculé et publié par la société de gestion ou aux valeurs d'expertise pour les sociétés immobilières.

En normes comptables françaises, les parts de fonds sont comptabilisées au prix d'achat, ajusté des dépréciations éventuelles.


## D.1.3.5 Dépôts autres que ceux assimilables à de la trésorerie

Il s'agit des dépôts non transferables c'est-à-dire des dépôts non utilisables à tout moment pour effectuer des paiements, non échangeables contre des liquidités et non transferables sans restriction ou pénalité significative.

MAIF VIE classe sur cette ligne les dépôts et comptes à terme et les valorise en normes Solvabilité 2 comme en normes comptables françaises, à leur prix de revient, compte tenu de leur montant non significatif.


## D.1.4 Placements en représentation de contrats en unités de compte ou indexés

Cette ligne correspond aux placements représentant les provisions techniques afférentes aux compartiments en UC des produits d'assurance vie multisupports.

Elle est composée de fonds d'investissement valorisés au dernier prix de rachat calculé et publié par la société de gestion comme dans les comptes en normes comptables françaises.


### D.1.5 Avances sur polices

Cette ligne correspond aux prêts accordés aux assurés et garantis par les contrats.

Le montant des avances sur polices en normes comptables françaises est comptabilisé sur la base des montants prêtés et intérêts à recevoir.

En normes Solvabilité 2, le montant des avances est identique à celui en normes comptables françaises, compte tenu de leur montant non significatif.


## D.1.6 Provisions techniques cédées

Le montant des provisions techniques cédées aux réassureurs en normes comptables françaises est comptabilisé sur la base des résultats de l'année et correspond aux données communiquées aux réassureurs.

En normes Solvabilité 2, le montant des provisions techniques cédées correspond à la meilleure estimation des provisions techniques de réassurance, ajustée du risque de défaut des contreparties. Les précisions sur leur détermination sont apportées dans le paragraphe consacré aux provisions techniques.


### D.1.7 Autres actifs

Les autres actifs comprennent les créances nées d'opérations d'assurance directe, les créances nées d'opérations de réassurance, les autres créances (hors assurance) et la trésorerie. Ces postes sont enregistrés à leur valeur nominale en normes comptables françaises.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

70 / 107

En normes Solvabilité 2, la juste valeur retenue pour ces actifs est égale à leur valeur nette comptable établie pour les comptes en normes comptables françaises.

MAIF VIE considère en effet que la différence entre le prix de revient et la valeur économique n'est pas significative pour ces actifs.


## D.2 Provisions techniques

Les provisions techniques constatent au passif du bilan les engagements de MAIF VIE vis-à-vis de ses assurés ou des bénéficiaires des contrats d'assurance.


### D.2.1 Généralités


#### D.2.1.1 Principes de valorisation des provisions techniques

En normes comptables françaises, l'évaluation des provisions techniques suit une méthodologie qui a pour objectif de respecter l'article R 343-3 du Code des assurances.

En normes Solvabilité 2, l'évaluation des provisions techniques suit les principes définis par la règlementation Solvabilité 2 pour l'évaluation des provisions techniques qui diffèrent en fonction de la nature réplicable ou non des passifs d'assurance. Est considéré comme réplicable, tout contrat d'assurance dont les flux peuvent être parfaitement répliques par un portefeuille d'instruments financiers, à condition que le marché soit suffisamment actif, liquide et transparent.

Les passifs d'assurance de MAIF VIE étant non réplicables, les provisions techniques brutes sont égales à la somme d'une provision de meilleure estimation dite « Best Estimate >> et d'une marge de risque :

› la provision de meilleure estimation correspond à la meilleure estimation de la valeur actualisée probable des flux futurs de trésorerie estimés de façon la plus exacte possible et projetés jusqu'à l'extinction des contrats. L'actualisation des flux consiste à déterminer la valeur d'aujourd'hui de flux qui se produiront dans le futur.

› la marge de risque est calculée de manière à garantir que la valeur des provisions techniques soit équivalente au montant dont les entreprises d'assurance et de reassurance auraient besoin pour reprendre et honorer les engagements de la compagnie. Elle est calculée en déterminant le coût que représente la mobilisation d'un montant de fonds propres éligibles égal au SCR nécessaire pour faire face aux engagements d'assurance et de réassurance sur toute leur durée.


#### D.2.1.2 Mesures transitoires

MAIF VIE a fait le choix de ne pas appliquer de mesures transitoires pour le calcul des provisions techniques.


##### D.2.1.3 Courbe de taux

Les flux futurs sont actualises avec la courbe des taux sans risque fournie par l'Autorite Européenne des Assurances et des Pensions Professionnelles (EIOPA, selon son sigle anglais).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

71/ 107

<figure>

![](figures/15)

<!-- FigureContent="Courbes des taux 3,5% 3,0% 2,5% 2,0% 1,5% 1,0% 0,5% 0,0% |1 5 9 .13 17 21 25 29 33 37 41 45 49 53 57 61 65 69 73 77 81 85 89 93 97 101 105 109 113 117 121 125 129 133 -0,5% -1,0% · · · · courbe des taux 2020 (avec VA) - courbe des taux 2021 (avec VA)" -->

</figure>



## D.2.1.4 Classification

Les provisions techniques sont regroupées dans le bilan en normes Solvabilité 2 en cinq catégories.

<figure>

![](figures/16)

<!-- FigureContent="Non-vie Vie hors UC Non-vie hors santé Vie hors santé Unités de comptes Santé similaire Santé similaire à la non-vie à la vie" -->

</figure>


Elles sont calculées au niveau des lignes d'activités (« lines of business »> - LoB) ou au niveau de portefeuilles homogènes de risques - par contrat/garantie - au sein d'une même ligne d'activité.


## D.2.2 Provisions techniques vie


### D.2.2.1 Périmètre et outils

D.2.2.1.1 Périmètre produits

› Classification par ligne d'activité

En normes Solvabilité 2, la maille retenue pour compléter les états réglementaires est la «ligne d'activité». Conformément à la segmentation imposée par les normes Solvabilité 2, les engagements de MAIF VIE sont affectés aux lignes d'activité selon le tableau présenté en § A1.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

72 / 107

| Catégorie de bilan | Produits | (LOB)Ligne d'activité |
| - | - | - |
| Provisions techniques vie (hors santé, UC ou indexés) | Assurance vie monosupport | Contrats d'assurance vie avec participation aux bénéfices |
|| - Cap'Epargne, Cap'Etudes - Nouveau Cap (y compris PEP), Capiplan ||
|| Assurance vie multisupports Compartiment euros - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DSCap | Contrats d'assurance vie avec participation aux bénéfices |
|| Épargne retraite - PER Responsable et Solidaire Compartiment euros | Contrats d'assurance vie avec participation aux bénéfices |
|| Assurance retraite - IFC | Contrats d'assurance vie avec participation aux bénéfices |
|| Obsèques vie entière - Sollicitudes | Contrats d'assurance vie avec participation aux bénéfices |
|| Temporaire décès - Rassurcap - Rassurcap Solutions - Assurance Prévoyance Professionnelle | Autres activités d'assurance vie |
|| Assurance en couverture de prêts - Assurance Emprunteur MAIF, Avantage Emprunteur - Autres Prêts immobiliers - Prêts à la consommation | Autres activités d'assurance vie |
|| Rentes | Contrats d'assurance vie avec participation aux bénéfices |
| Provisions techniques UC ou indexés | Assurance vie multisupports Compartiments UC - Assurance vie Responsable et Solidaire, Epargne Vie Simplicité - Libre Cap, DSCap | Assurance indexée ou en unités de comptes (UC) |
|| Épargne retraite - PER Responsable et Solidaire Compartiments UC | Assurance indexée ou en unités de comptes (UC) |

L'assignation d'un engagement d'assurance à une ligne d'activité reflète la nature des risques relatifs à cet engagement. Un produit est susceptible de couvrir plusieurs natures de risques et peut ainsi relever de plusieurs lignes d'activité. La forme juridique de l'engagement n'est pas nécessairement déterminante pour caractériser la nature du risque.


## › Affaires nouvelles et primes futures

La valorisation des actifs et des passifs de l'entreprise doit être réalisée sous l'hypothèse d'une continuité d'exploitation, conformément à l'article 7 du Règlement délégué (UE) 2015/35.

Au regard de la frontière des contrats, aucune prime future pouvant provenir d'affaires nouvelles ou contrats en cours n'est intégrée aux calculs, sauf pour les produits suivants :


## › Obsèques vie entière

Le paiement sous forme de primes temporaires est contractuellement prévu sous peine de réduction du contrat.


## › Temporaire décès

Les primes dues au titre de l'année en cours doivent être prises en considération en cohérence avec le caractère annuel des produits Rassurcap et Rassurcap Solutions qui implique que les engagements de MAIF VIE cessent la veille de la prochaine échéance annuelle de chaque contrat. Les adhésions Rassurcap et Rassurcap Solutions à échéance en janvier seront prises en compte dans la meilleure estimation pour une année supplémentaire dans la limite du taux de reconduction observé.


### › Assurance en couverture de prêts

Les cotisations futures sont prises en compte, conformément aux conditions contractuelles et notamment aux caractéristiques de l'amortissement du prêt correspondant.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="73 / 107" -->


## D.2.2.1.2 Outils de modélisation utilisés

›


### Diffusion des trajectoires économiques

L'outil édité par Moody's Analytics est utilisé pour générer les trajectoires économiques en univers risque neutre. Cet éditeur est reconnu et est un leader du marché. Cet outil est également utilisé pour la génération des trajectoires économiques en univers réel utilisées dans le cadre des travaux actif passif et d'allocation stratégique d'actifs.

Cet outil permet le paramétrage et la diffusion de trajectoires économiques nécessaires à la projection de toutes les classes génériques d'actifs définies par MAIF VIE.

Moody's Analytics met à disposition des utilisateurs les éléments nécessaires à la bonne compréhension des modèles et la bonne utilisation de l'outil, à savoir :

› documentation de l'outil ;

› documentation des modèles utilisés dans l'outil ;

› fichiers de calibrage standard ;

› outils de (re)calibrage.

Par ailleurs, l'outil réalise en sortie des tests nécessaires à la validation des trajectoires risque neutre générées (test de martingalité notamment).

› Projection des flux de trésorerie

Des outils de modélisation spécifiques aux calculs Solvabilité 2 ont été développés pour répondre aux calculs règlementaires de la meilleure estimation des provisions techniques et des SCR .

Les évaluations de la meilleure estimation des provisions techniques des produits de prévoyance et d'épargne sont réalisées sur Prophet et Excel.


## › Modélisation des actions futures du management (produits d'épargne en euros)

Certaines actions futures du management sont implémentées dans les outils de modélisation Solvabilité 2 pour les produits d'épargne en euros. Il s'agit des hypothèses suivantes :

› la decision des taux a servir en fin d'année sur les compartiments euros des contrats d'assurance vie est implémentée dans les outils de projection comme une action du management : le taux de rémunération des contrats d'assurance vie à servir correspond à la stratégie de rémunération détaillée ci-après ;

› la gestion d'actifs ou stratégie financière (retranscription de la politique annuelle de gestion des risques d'investissements) définit les règles appliquées sur les actifs en cas d'investissement ou de désinvestissement. Elle précise notamment :

o l'Allocation Stratégique d'Actifs cible (ASA et ASA LT) : cette cible est figée sur la base d'etudes actif-passif (realisees par la MAIF), validée par le Conseil d'administration de MAIF VIE et utilisée pour toutes les simulations stochastiques;

o la manière d'atteindre l'ASA.

› Modélisation de la reassurance

La réassurance est modélisée sur la base du programme de réassurance en place.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" --> :unselected: :unselected:

> VALORISATION À DES FINS DE SOLVABILITÉ
===

74 / 107

› Détermination de la meilleure estimation des provisions techniques

Le montant de la meilleure estimation des provisions techniques est déterminé par la somme du montant de la meilleure estimation des provisions techniques issu de l'outil de projection des flux de trésorerie, de 85 % des plus-values latentes résiduelles actualisées issues de l'outil de projection des flux de trésorerie et du montant des provisions techniques complémentaires constaté en normes comptables françaises (provisions pour sinistres à payer réelles, provision pour sinistres en cours, ... ) ainsi qu'éventuellement du montant de l'écart de convergence. Ce montant est diminué du montant de provision pour participation aux bénéfices mis en fonds excédentaires conformément à l'arrêté du 24 décembre 2019 (application de la méthode 2 proposée par l'ACPR)


## D.2.2.2 Données, hypothèses et paramètres

D.2.2.2.1

Données

› Données contrats

Pour les besoins de l'inventaire, les données contrats sont mises à jour en date de clôture et contrôlées dans les bases du suivi du portefeuille vie. Ces données sont utilisées pour les calculs sous forme de models-points (agrégation par regroupement de contrats homogènes, c'est-à-dire présentant des caractéristiques identiques).


## › Données de placement

Les données du portefeuille d'actifs (type d'actif, valeur comptable, valeur de marché, coupon couru, ... ) en date d'inventaire sont utilisées pour les calculs des produits d'épargne. Les actifs sont regroupés en 3 catégories :

› obligations (souveraines, d'entreprise, sécurisées) ;

› actions et assimilés : fonds ou sociétés de capital investissement, fonds actions et d'obligations convertibles, fonds obligataires, fonds immobiliers et d'infrastructures ; › trésorerie : fonds monétaires, ...

Les obligations sont prises en compte ligne à ligne et les autres actifs sont agrégés en différentes sous-catégories d'actifs.

Les nominaux et coupons des obligations sont recalculés selon l'approche dite de neutralisation du risque de crédit avec comme références la courbe de taux d'actualisation et les valeurs de marché.

D.2.2.2.2

Hypothèses

› Hypothèses financières


### › Actualisation

MAIF VIE a retenu la courbe de taux publiée par EIOPA au 31 décembre 2021, conforme aux principes des actes délégués de la directive Solvabilité 2 (articles 43 à 48) et avec application de la correction pour volatilité visée à l'article 77 quinquies de la directive 2009/138/CE.

Dans la mesure où les contrats de MAIF VIE relèvent de la réglementation française et sont vendus en France, seule la courbe en monnaie euros est utilisée.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="75 / 107" -->


## › Scénarii économiques

Les hypothèses retenues sont fournies par Moody's Analytics en calibrage standard réalisé sur la base d'une courbe de taux swap de marché. Un nouveau calibrage est effectué pour les taux nominaux afin de prendre en compte la courbe officielle fournie par l'EIOPA avec et sans correction pour volatilité et sur les taux réels afin de calibrer sur les OATi françaises (données Reuters).


### › Stratégie financière

La stratégie financière utilisée pour les calculs stochastiques sur les produits d'épargne est fondée sur une atteinte progressive sur une période définie de l'Allocation Stratégique d'Actifs (ASA) validée par le conseil d'administration de MAIF VIE en fin d'année.

Elle définit également les règles appliquées sur les actifs en cas d'investissement ou de désinvestissement et précise notamment les ordres de priorité de cessions par catégorie de titres.


## › Hypothèses comportementales


### › Table de mortalité

Les tables de mortalité utilisées pour traduire la mortalité réelle observée sur le portefeuille sont les tables d'expérience MAIF VIE et les tables réglementaires.


### › Table de chute

Les rachats structurels concernent les produits d'épargne, le produit retraite PER et les contrats de prévoyance obsèques et assurance en couverture de prêt.

Les hypothèses retenues correspondent à l'observation des portefeuilles en date d'inventaire de l'exercice précédent. Ces rachats sont appliqués en pourcentage de la provision mathématique en épargne (rachats partiels) et en pourcentage des adhésions en stock en épargne (rachats totaux) et prévoyance.

› Table de rachats conjoncturels

Les rachats conjoncturels ne concernent que les produits d'épargne en euros (monosupports et compartiment euros des multisupports) et sont liés à la concurrence sur le marché de l'assurance vie.


## › Tables d'incapacité et d'invalidité

Les tables d'incapacité et d'invalidité utilisées pour traduire l'entrée et le maintien en incapacité et en invalidité réelle observés sur le portefeuille sont des tables d'expérience fournies par des prestataires externes.

› Taux de turn-over

Le taux de turn-over utilisé pour les engagements sociaux est basé sur des données réelles communiquées par chaque employeur.


## › Hypothèses de gestion

Les hypothèses ci-dessous correspondent uniquement à celles utilisées dans le cadre du calcul des provisions techniques.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="76 / 107" -->

› Stratégie de rémunération

La stratégie de rémunération concerne les produits « Epargne euros >> et le produit « obsèques vie entière » (Sollicitudes).

Pour l'épargne euros, elle a pour objectif d'optimiser l'utilisation de la production financière nette des frais financiers et tenant compte des variations de la PRE, de la PPB et de la richesse de l'assureur sous différentes contraintes :

o réglementaires : Participation aux Bénéfices minimum, Ecoulement de la PPB en 8 ans ;

o contractuelles : Taux technique, Taux Minimum Garanti ;
 :unselected:
o de revalorisation : définition d'un taux cible, qui est fonction du taux de la concurrence.

La contrainte de TMG s'applique uniquement sur la première année de projection, puisque l'engagement contractuel de MAIF VIE au 31/12/21 ne porte que sur l'année suivante (2022).


## › Charges de gestion

Les données utilisées relatives aux coûts administratifs, financiers et de sinistres reposent sur :

o le calcul moyen sur trois exercices des coûts ABC (Activity Based Costing = coûts analytiques basés sur la cartographie d'activités de l'entreprise à la date d'inventaire de l'exercice) exprimés par famille de produits hors commissions de gestion sur l'assurance en couverture de prêts et hors coûts des projets exceptionnels significatifs et déclinés en charges fixes / charges variables et par destination (acquisition, administration des contrats, gestion des sinistres, gestion des placements et autres charges techniques) ;

o et sur une masse complémentaire de frais de gestion correspondant aux coûts exceptionnels de projets significatifs non compris dans le calcul des coûts de base ABC (cycles d'investissement SI), exprimés par famille (épargne, prévoyance, obsèques et autres).


### D.2.2.2.3 Paramètres

Les paramètres concernent des éléments structurels des produits (tarification, ... ) ou de contexte (impôts, ... ). Ils sont distingués des hypothèses dans la mesure où leur utilisation pour les calculs ne génère pas de risque de modélisation (application d'une règle contractuelle, juridique, réglementaire, ... ).


## Tarification ›

Les ressources de gestion prises en compte dans la modélisation correspondent aux frais sur versements des cotisations contractuellement prévues (Sollicitudes, Assurance Emprunteur MAIF, Rassurcap et Rassurcap Solutions) ainsi qu'aux chargements effectués sur les provisions mathématiques des contrats en cours. Les valeurs retenues sont celles utilisées dans la tarification en vigueur à la date d'inventaire sans aucune simplification.

Commissions et frais de distribution et de gestion ›

Seules les commissions relatives aux cotisations contractuellement prévues ainsi que celles correspondant au commissionnement sur les frais sur épargne gérée sont à considérer.

Les modes et les niveaux de commissionnement sont ceux en vigueur à la date d'inventaire.


## Impôts et prélèvements sociaux ›

Le taux d'impôt sur les sociétés retenu pour le calcul de la reserve de capitalisation tient compte des évolutions attendues.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021 :unselected: :unselected: :unselected:

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="77 / 107" -->

Le taux de prélèvements sociaux est fixé à 17,2 %, conformément à la réglementation en vigueur au 31 décembre 2021 mais sans tenir compte des spécificités type épargne handicap et domiciliation à l'étranger.


## D.2.2.2.4 Informations sur les niveaux d'incertitude

L'incertitude la plus élevée se concentre sur la ligne d'activité «contrats d'assurance vie avec participation aux bénéfices».


## D.2.2.3 Valeur des provisions techniques


### D.2.2.3.1 Résultats au 31 décembre 2021

Une ventilation exprimée en milliers d'euros par ligne d'activité de la meilleure estimation des provisions techniques vie brutes de réassurance montre une prépondérance des contrats d'assurance vie avec participation aux bénéfices à hauteur de 94 %.

| Lignes d'activité (LoB) | Meilleure estimation des provisions techniques vie brute 31/12/2021 | Meilleure estimation des provisions techniques vie cédées 31/12/2021 |
| - | - | - |
| Contrats d'assurance vie avec participation aux bénéfices | 11 027 726 | - |
| Assurance indexée ou en UC | 703 795 | 80 181 |
| Autres activités d'assurance vie | -24 276 | - |
| TOTAL PROVISIONS TECHNIQUES | 11 707 245 | 80 181 |

L'annulation des effets de la correction pour volatilité représente un impact de + 9 447 milliers d'euros sur la meilleure estimation brute de reassurance.

| 31/12/2021 | Meilleure estimation des provisions techniques vie brute | Meilleure estimation des provisions techniques vie cédées |
| - | - | - |
| Valeur retenue | 11 707 245 | 80 181 |
| Valeur sans l'effet de correction pour volatilité | 11 716 692 | 80 734 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="78 / 107" -->

D.2.2.3.2 Focus entre la meilleure estimation des provisions et les provisions en normes comptables françaises

La meilleure estimation, égale à la valeur actuelle probable des potentiels flux futurs de trésorerie, ne doit comprendre que les flux de trésorerie futurs associés aux contrats d'assurance en vigueur à la date d'évaluation, sous l'hypothèse d'une continuité d'exploitation (run-off avec maintien d'activité). La projection de flux de trésorerie intervenant dans ce calcul doit tenir compte de toutes les entrées (ressources) et sorties (prestations et dépenses) de fonds nécessaires pour honorer les engagements d'assurance sur leur durée de vie.

La meilleure estimation, évaluée en brut de réassurance, doit reposer sur des informations actuelles et crédibles et à partir d'hypothèses réalistes et spécifiques à l'entreprise en faisant appel à des méthodes actuarielles et techniques statistiques adéquates.

Compte tenu des éléments précédents, on observe de nombreuses différences dans les hypothèses et méthodes utilisées entre les états financiers et les évaluations à des fins de solvabilité, comme le montre le tableau ci-dessous :

| Hypothèses principales || Normes Solvabilité 2 | Normes comptables françaises |
| - | - | - | - |
| Actif | Trajectoires économiques | Barrie-Hibbert | Sans objet |
|| Stratégie financière | Allocation Stratégique d'actifs (ASA) | Sans objet |
| Passif | Mortalité | Table d'expérience ou table réglementaire | Table d'expérience certifiée ou table réglementaire |
|| Rachats structurels | Table d'expérience | Sans objet |
|| Rachats conjoncturels | Formule ONC (épargne uniquement) | Sans objet |
| Dépenses | Frais | Coûts par activités (approche ABC) | Coûts (approche Provision Globale de Gestion) |
| Actualisation | Taux | Courbe de taux EIOPA avec correction pour volatilité | Taux technique ou courbes de taux (Provision Globale de Gestion et Provisions pour Aléas Financiers) |


## D.2.3 Provisions techniques non-vie

MAIF VIE n'est pas concernée par les provisions techniques non-vie.


### D.2.4 Marge de risque


#### D.2.4.1 Méthodologie

La marge de risque est une composante des provisions techniques qui complète la meilleure estimation des provisions.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="79 / 107" -->

La marge de risque doit être calculée en déterminant le coût d'immobilisation d'un montant de fonds propres nécessaires pour atteindre le SCR afin d'honorer les engagements d'assurance et de réassurance sur leur durée de vie.

Le calcul de la marge de risque repose en effet sur un scénario de transfert du portefeuille vers une autre société d'assurance qui, n'ayant aucun fond propre avant le transfert, doit lever les fonds propres nécessaires pour couvrir le SCR.

Pour des raisons de proportionnalité, MAIF VIE a fait le choix de retenir une méthode simplifiée pour le calcul de la marge de risque. Cette méthode consiste à déterminer les SCR futurs proportionnellement aux meilleures estimations des provisions futures.


## D.2.4.2 Valeur de la marge de risque

D.2.4.2.1 Résultat au 31 décembre 2021

Le montant de la marge de risque par ligne d'activités au 31 décembre 2021 exprimé en milliers d'euros est décomposé ci-dessous :

| Lignes d'activité (LoB) | Marge de risque 31/12/2021 |
| - | - |
| Contrats d'assurance vie avec participation aux bénéfices | 121 354 |
| Assurance indexée ou en UC | 7 423 |
| Autres activités d'assurance vie | -256 |
| TOTAL MARGE DE RISQUE | 128 520 |


## D.2.5 Niveau de confiance sur le niveau des provisions techniques

Des analyses de mouvements ont été effectuées de manière à expliquer les variations de la meilleure estimation des provisions techniques entre la clôture de l'année 2020 et celle de l'année 2021, ainsi qu'entre les provisions techniques calculées en normes comptables françaises et en normes Solvabilité 2.

L'ensemble des écarts peut être justifié. Dans le cadre de l'analyse expliquant le passage des provisions techniques normes comptables françaises aux provisions techniques en normes Solvabilité 2 sur le périmètre épargne, un écart résiduel subsiste. Celui-ci s'explique principalement par l'écart de convergence du modèle et est jugé non significatif : il est de l'ordre de 0,08 % de la meilleure estimation des provisions techniques au 31 décembre 2021.


## D.3 Autres passifs

Les principaux postes du passif, en dehors des provisions techniques, sont décrits ci-après :

› provisions, autres que les provisions techniques ;

› impôts différés passifs ;

› dettes autres que les dettes subordonnées ;

› dettes subordonnées incluses dans les fonds propres de base.


### D.3.1 Provisions autres que les provisions techniques

La provision pour risques et charges correspond à l'estimation des coûts liés à la conclusion d'un accord de gestion des fins de carrière au sein de l'entreprise, à l'indemnité de fin de MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> VALORISATION À DES FINS DE SOLVABILITÉ
===

<!-- PageHeader="80 / 107" -->

contrat due aux titulaires de Contrats à Durée Déterminée au terme de ceux-ci et à une provision pour contentieux. En normes Solvabilité 2, l'évaluation est identique à celle comptabilisée en normes comptables françaises.


## D.3.2 Impôts différés

En normes comptables françaises, seuls les impôts exigibles sont comptabilisés dans les comptes sociaux. L'impôt exigible est le montant des impôts sur le bénéfice payable (ou récupérable) au titre du bénéfice imposable (ou perte fiscale) d'un exercice.

En normes Solvabilité 2, les entreprises d'assurance valorisent et comptabilisent des impôts différés actifs et passifs qui répondent aux définitions et principes des normes comptables internationales, notamment :

› les impôts differes sont les montants d'impôts payables ou recuperables au cours de périodes futures afférents :
 :unselected:
o sauf exceptions, aux différences temporelles (déductibles et imposables) entre la valeur des actifs et passifs au bilan et leur base fiscale ;

o le cas échéant, au report en avant de pertes fiscales non utilisées et de crédits d'impôts non utilisés.

› les actifs d'impôts differes au titre des différences temporelles déductibles ne sont comptabilisés que s'il est probable qu'ils puissent être imputés sur des bénéfices imposables futurs.

› les passifs (actifs) d'impôts differes sont evalues au montant que l'on s'attend à payer aux (recouvrer auprès des) administrations fiscales en utilisant les taux d'impôt qui ont été adoptés ou quasi-adoptés à la date de clôture. Ils ne font pas l'objet d'une actualisation.

MAIF VIE retient, pour les impôts différés passifs ou actifs, la valeur égale à la somme :

› des impôts différés issus de la fiscalité différée (décalages temporaires entre résultat fiscal et résultat comptable) et latente (taxation éventuelle) ;

› des impôts différés liés à la revalorisation des actifs et des passifs. La valorisation des impôts differes au bilan a été réalisée poste à poste, en appliquant un taux d'imposition de 25,83 % à l'écart entre la valeur en normes comptables françaises et la valeur en normes Solvabilité 2. Toutefois, un taux d'imposition de 15 % a été appliqué pour certains actifs détenus a long terme.

Le montant inscrit au bilan correspond au solde entre les impôts différés passifs et les impôts différés actifs compensables.

Le tableau suivant détaille les impôts différés générés, d'une part par la différence d'évaluation des actifs et passifs entre le bilan en normes Solvabilité 2 et le bilan en normes comptables françaises et, d'autre part, entre l'évaluation des actifs et des passifs entre le bilan en normes françaises et le bilan fiscal.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

81 / 107
===

\> VALORISATION À DES FINS DE SOLVABILITÉ
===

| | 2021 |||
|| Différence d'évaluation | IDA | IDP |
| - | - | - | - |
| Différence d'évaluation des placements (Bilan S2 - Bilan Normes Françaises) | 1 223 324 | | 315 116 |
| Différence d'évaluation des autres actifs (Bilan S2 - Bilan Normes Françaises) | -23 494 | 6 702 | 633 |
| Actifs incorporels | -22 942 | 5 926 | |
| Immobilisations corporelles pour usage propre | -3 003 | 776 | |
| Provisions techniques cédées | 2 451 | | 633 |
| Autres actifs | | | |
| Différence d'évaluation des provisions techniques (Bilan S2 - Bilan Normes Françaises) | 1 045 373 | 270 020 | |
| Différence d'évaluation des autres postes du passif (Bilan S2 - Bilan Normes Françaises) | 2 538 | 655 | 0 |
| Provisions autres que les provisions techniques | | | |
| Provisions pour retraite et autres avantages | 19 | 5 | |
| Autres passifs | 2 519 | 651 | |
| Total des différences entre le Bilan S2 et le Bilan normes françaises | 2 247 740 | 277 377 | 315 749 |
| Différence d'évaluation entre le Bilan normes françaises et le Bilan fiscal | 16 131 | 4 196 | 30 |
| Total des Impôts Différés | | 281 572 | 315 779 |
| Impôts Différés - Situation Nette | | | 34 207 |


## D.3.3 Dettes autres que les dettes subordonnées

Les autres dettes recouvrent les dettes nées d'opérations d'assurance directe, les dettes nées d'opérations de réassurance, les dettes envers des établissements de crédit, les dettes sociales et fiscales et les autres dettes (charges à payer comptabilisées au profit des sociétés du groupe dans le cadre des accords de distribution et de prestations intra-groupe). Ces postes comptables sont enregistrés, en normes comptables françaises, à leur valeur nominale.

En normes Solvabilité 2, ces dettes sont évaluées à leur valeur comptable sans retraitement, MAIF VIE considérant que cette valeur ne présente pas de différences significatives avec une évaluation économique.


## D.3.4 Dettes subordonnées incluses dans les fonds propres de base

Lors de ses réunions des 7 octobre et 14 décembre 2004, le conseil d'administration de MAIF VIE, conformément aux orientations prises pour le renforcement des fonds propres de la société, a décidé une émission de titres subordonnés remboursables (TSR) pour un montant nominal total de 34 500 milliers d'euros, représenté par 345 000 titres de 100 euros de nominal.

L'emprunt a été émis le 12 janvier 2005, date correspondant également à la souscription et au règlement des titres.

Les titres subordonnés remboursables ont été émis au nominal.

Les droits des titulaires sont représentés par une inscription en compte à leur nom dans le registre de l'émetteur.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> VALORISATION À DES FINS DE SOLVABILITÉ
===

82 / 107
===

Les titres sont amortissables in fine et seront amortis en totalité le 11 janvier 2025 par remboursement au pair.

Au 31 décembre 2021, l'endettement s'élève à 34 500 milliers d'euros. Les dettes subordonnées sont incluses, en normes comptables françaises, sur la ligne « passifs subordonnés >> proche des fonds propres.

En normes Solvabilité 2, la valorisation est réalisée à la hauteur du montant nominal des titres.


## D.4 Méthodes de valorisation alternatives

Les méthodes de valorisation alternatives correspondent à des méthodes autres que celles qui utilisent uniquement des prix cotés sur un marché actif, pour des actifs ou des passifs identiques ou analogues. Elles ne concernent pas les provisions techniques.

Les méthodes et modalités de valorisation retenues par MAIF VIE sont précisées dans les paragraphes précédents consacrés à chaque catégorie d'actifs et d'autres passifs.

Compte tenu de la structure de son portefeuille, MAIF VIE a recours à des méthodes alternatives de valorisation pour une part limitée de ses actifs à l'image des fonds d'investissement « non cotés >> pour lesquels la valeur liquidative publiée par la société de gestion n'est pas issue de cotations sur un marché réglementé.


## D.5 Autres informations

MAIF VIE n'identifie pas d'autres informations importantes à communiquer sur ces méthodes, compte tenu du caractère non significatif de son exposition.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->
<figure>

![](figures/17)

<!-- FigureContent="E" -->

</figure>


GESTION DU CAPITAL

> GESTION DU CAPITAL
===

<!-- PageHeader="84 / 107" -->

E Gestion du capital
===

Ce paragraphe est consacré à la présentation des informations qualitatives et quantitatives relatives :

› aux fonds propres et à leur gestion ;

› aux exigences de capital et à leur couverture.

Les fonds propres en normes Solvabilité 2 sont :

› essentiellement constitues par l'excédent des actifs sur les passifs du Bilan en normes Solvabilité 2 (presente dans la partie D) incluant les fonds propres en normes comptables françaises ;

› classés par niveau de qualité en fonction de leur capacité à absorber des pertes.

MAIF VIE dispose principalement de fonds propres de base de niveau 1 (les plus disponibles pour absorber des pertes et de plus grande qualité, reconnus par la réglementation Solvabilité 2).

L'exigence de capital se décline sur deux niveaux :

› Minimum de capital requis - MCR : niveau en deçà duquel l'entreprise d'assurance présente un risque inacceptable de ne pas pouvoir faire face à ses engagements ;

› Capital de solvabilité requis - SCR : niveau permettant à l'entreprise d'assurance de faire face à des pertes imprévues.

MAIF VIE utilise la formule standard définie par la réglementation pour déterminer l'exigence en capital, son calibrage étant adapté pour la mesure de son profil de risque.

MAIF VIE n'a pas recouru aux mesures temporaires optionnelles proposées par la réglementation pour faciliter la transition vers le nouveau dispositif prudentiel.

Le calcul de la couverture du besoin en capital consiste à comparer les fonds propres éligibles à l'exigence de capital.

MAIF VIE couvre de manière satisfaisante le Minimum de capital requis et le Capital de solvabilité requis, avec les ratios suivants :

› 2,5 fois le SCR - Capital de solvabilité requis pour un montant de 492 938 milliers d'euros (avec un taux de couverture de 248 %) ;

› 5,4 fois le MCR - Minimum de capital requis pour un montant de 221 822 milliers d'euros (avec un taux de couverture de 536 %).


## E.1 Fonds propres

Les fonds propres en normes Solvabilité 2 sont composés de fonds propres de base et de fonds propres auxiliaires :

› les fonds propres de base sont constitués de l'excédent des actifs sur les passifs (issus du bilan en normes Solvabilité 2) auquel est soustrait le montant des actions détenues en propre et est ajouté le montant des passifs subordonnés ;

› les fonds propres auxiliaires, qui doivent être approuvés par l'ACPR, comprennent des éléments pouvant être appelés et utilisés pour absorber des pertes (fraction de capital non appelée et/ou non versée, ... ).

Les fonds propres doivent également être classés selon leur niveau de qualité. Ce classement se fait en fonction de leur disponibilité pour absorber les pertes, de leur degré de subordination

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> GESTION DU CAPITAL
===

<!-- PageHeader="85 /107" -->

et de leur durée ou permanence. Il existe trois niveaux et les fonds propres de niveau 1 sont ceux qui présentent la plus grande capacité d'absorption des pertes et donc la meilleure qualité.

Enfin, parmi l'ensemble de ces fonds propres disponibles, les éléments éligibles à la couverture du Capital de solvabilité requis (SCR) et du Capital minimum requis (MCR) doivent respecter des limites quantitatives assurant que les exigences de solvabilité soient couvertes majoritairement par des fonds propres de la meilleure qualité. En particulier, les fonds propres de niveau 1 doivent couvrir au moins 50 % du SCR et 80 % du MCR.

Les paragraphes suivants présentent, pour MAIF VIE :

› les objectifs, politiques et procédures appliqués pour la gestion des fonds propres ;

› la composition, les montants et la qualité des fonds propres en normes Solvabilité 2 à la fin de l'exercice ainsi qu'un comparatif avec l'exercice précédent ;

› les explications des différences entre les fonds propres en normes Solvabilité 2 et les fonds propres issus des états financiers établis en normes comptables françaises. Ces derniers font l'objet d'une présentation préalable.


## E.1.1 Informations sur les objectifs, politiques et procédures appliques par l'entreprise pour la gestion de ses fonds propres

MAIF VIE doit couvrir a minima les exigences réglementaires en fonds propres (notamment Capital de solvabilité requis - SCR - et Minimum de capital requis - MCR dans l'environnement Solvabilité 2) par des fonds propres :

› qui remplissent les critères de qualité attendus pour être éligibles ;

› en quantité suffisante.

La solidité financière et prudentielle de MAIF VIE se mesure à travers ses fonds propres. MAIF VIE gère ses ressources pour disposer en permanence d'un niveau de fonds propres qui garantisse la pérennité de l'entreprise.


### E.1.1.1 Objectifs de gestion du capital

Dans le cadre de son système de gestion des risques, MAIF VIE a défini deux indicateurs d'appétence :

› un niveau de fonds propres en normes comptables françaises, hors dettes subordonnées dont elle doit disposer à l'horizon de son Plan stratégique ;

› un taux minimum de couverture du SCR par les fonds propres éligibles du Bilan en normes Solvabilité 2.

Ces fonds propres comptables sont constitués principalement de :

› capitaux propres ;

› réserves dont la réserve de capitalisation ;

› résultat de l'exercice et report à nouveau.

L'évolution des fonds propres comptables resultera principalement de la réalisation de bénéfices (ou pertes) dans les années à venir et des mouvements sur la réserve de capitalisation suite à des cessions d'obligations.

Dans le bilan en normes Solvabilité 2, les fonds propres intègrent, en plus des fonds propres comptables :

› les fonds excédentaires dont la fraction de Provision pour Participation aux Bénéfices (décret du 24 décembre 2019) ;

›

les plus ou moins-values sur les actifs et les passifs comprises dans la réserve de

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

<!-- PageHeader="86 / 107" -->

\> GESTION DU CAPITAL
===

réconciliation ;

› les titres subordonnés remboursables.

Aucun changement important dans la gestion des fonds propres n'est survenu au cours de la période de référence.


## E.1.1.2 Processus et procédures de gestion du capital

Une présentation annuelle du niveau de fonds propres et de la nature des fonds propres disponibles, au sens des normes Solvabilité 2, est dispensée au Comité d'audit, des risques et des comptes MAIF et au conseil d'administration de MAIF VIE.

Les fonds propres font l'objet d'une classification selon les critères définis par la réglementation Solvabilité 2 (caractéristiques, qualités, disponibilités, durées, ... ).


### E.1.2 Capitaux propres en normes comptables françaises

Les fonds propres issus des états financiers établis en normes comptables françaises font l'objet d'une présentation préalable à celle des fonds propres en normes Solvabilité 2.

| Rubriques | 2021 | 2020 | Variation ||
|||| (en mantan+1 | (en %) |
| - | - | - | - | - |
| Capital social | 122 000 | 122 000 | - | |
| Autres réserves | 319 153 | 311 710 | 7 443 | 2,4% |
| Réserve de capitalisation | 154 852 | 152 451 | 2 401 | 1,6% |
| Réserve pour le développement des activités | 161 782 | 156 876 | 4 906 | 3,1% |
| Réserve pour fonds de garantie | 2 519 | 2 383 | 136 | 5,7% |
| Report à nouveau | 100 000 | 100 000 | - | 0,0% |
| Resultat de l'exercice | 50 332 | 5 042 | 45 290 | NS |
| TOTAL | 591 485 | 538 752 | 52 733 | 9,8% |

Le montant des fonds propres au passif du bilan en normes comptables françaises s'élève, au 31 décembre 2021, à 591 485 milliers d'euros, en progression de 52 733 milliers d'euros sous l'effet exclusivement du résultat net comptable de l'exercice 2021 et des mouvements annuels de la réserve de capitalisation.

Le conseil d'administration propose à l'assemblée générale des actionnaires :

› d'approuver les comptes sociaux et le résultat net comptable (+ 50 332 milliers d'euros), tels qu'ils ont été arrêtés par le conseil d'administration ;

› de prélever sur le résultat la somme de 100 milliers d'euros à titre de dotation de la réserve pour fonds de garantie, en application de la loi du 25 juin 1999 ;

› d'affecter le solde du résultat bénéficiaire de 50 232 milliers d'euros en réserve pour développement des activités.

La réserve pour fonds de garantie s'élèvera ainsi à 2 619 milliers d'euros et la réserve pour développement des activités à 212 014 milliers d'euros.


#### E.1.3 Fonds propres en normes Solvabilité 2

Le montant des fonds propres en normes Solvabilité 2 de MAIF VIE a augmenté sur l'exercice 2021 de 115 885 milliers d'euros pour atteindre ainsi 1 222 463 milliers d'euros, dont :

› 478 766 milliers d'euros de fonds excédentaires,

› 709 197 milliers d'euros d'excédent de l'actif sur le passif hors fonds excédentaires,

› et 34 500 milliers d'euros de dettes subordonnées.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

<!-- PageHeader="87 / 107" -->


## \> GESTION DU CAPITAL


### E.1.3.1 Les fonds propres de base

Le tableau ci-dessous présente la méthodologie et les méthodes de classification des fonds propres de base de MAIF VIE :

| Eléments de Fonds propres | Bilan Normes françaises | Bilan Prudentiel (Solvabilité 2) | Fonds propres de base |||
|||| Niveau 1 | Niveau 2 | Niveau 3 |
| - | - | - | - | - | - |
| Capital social | Capital social | Méthode retenue Valorisation au montant du capital versé | X :selected: | :unselected: | :unselected: |
| Fonds excédentaires | | Méthode retenue (méthode 2) 85,35% du montant de la PPB diminué du montant de la PPB de 8 ans d'ancienneté | X :selected: | :unselected: | :unselected: |
| | Réserve de capitalisation Autres réserves Report à nouveau Résultats de l'exercice | Méthode retenue Valorisation au montant comptabilisé en NF | :unselected: | :unselected: | :unselected: |
| Réserve de réconciliation | Ajustements | Somme des différences d'évaluation entre NF et normes Solvabilité 2 suivantes : 1- les actifs 2- les provisions techniques 3- les autres passifs | X :selected: | :unselected: | :unselected: |
| Passifs subordonnés | Passifs subordonnés | Evaluation à la valeur de transfert (sans prise en compte de la qualité de crédit des prêteurs). Simplification retenue : Valorisation à la hauteur du montant nominal des titres. | :unselected: | :unselected: | X :selected: |

A l'exception des TSR, l'ensemble des fonds propres de base de MAIF VIE est de niveau 1.


### E.1.3.2 Les fonds propres auxiliaires

MAIF VIE n'a pas d'éléments qui sont ou pourraient être admis comme fonds propres auxiliaires.


### E.1.3.3 Information sur toute restriction importante affectant la disponibilité des fonds propres de l'entreprise

Après analyse décrite ci-après, les fonds propres de MAIF VIE ne sont pas soumis à des restrictions affectant leurs disponibilités ou leurs durées.

Par ailleurs, MAIF VIE ne dépend pas d'éléments de fonds propres faisant l'objet de mesures transitoires au sens des normes Solvabilité 2.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="88 / 107" -->

E.1.3.3.1 Réserve de capitalisation
===

Le montant de la réserve de capitalisation de 154 852 milliers d'euros, fonds propre de base classé en niveau 1, ne peut être utilisé qu'en cas de réalisation de moins-values suite à des cessions d'obligations.

Toutefois, au regard de l'article R. 351-2-1 du Code des assurances, la valeur actuelle attendue du montant de la réserve de capitalisation subsistant à l'issue de la durée des engagements (57 593 milliers d'euros au 31 décembre 2021) est intégrée à la réserve de réconciliation et fait partie des éléments de fonds propres de base, classée au niveau 1. Elle ne fait donc ainsi l'objet d'aucune restriction importante.

E.1.3.3.2 Titres subordonnés remboursables

MAIF VIE a emis le 12 janvier 2005 des titres subordonnés remboursables à hauteur de 34 500 milliers d'euros, exclusivement réservés aux personnes morales actionnaires, la MAIF et la CASDEN Banque Populaire.

Ces TSR portent intérêt calculé sur leur montant nominal non remboursé au taux d'intérêt. Le taux d'intérêt applicable pour chacune des périodes d'intérêts comprises entre le 1er janvier 2015 et le 11 janvier 2025 est variable et est égal à l'EURIBOR 3 mois auquel est ajoutée une marge de 280 points. Ils seront amortis en totalité le 11 janvier 2025 par remboursement au pair.

MAIF VIE peut rembourser, par anticipation, tout ou partie de ces TSR à tout moment conformément à l'article A 334-3 du Code des assurances, sous réserve de l'accord préalable de l'ACPR, à chaque échéance de période d'intérêts à partir du 31 décembre 2014. Il convient de noter que les souscripteurs ne peuvent pas demander de remboursement par anticipation.

Dans l'hypothèse d'une liquidation de MAIF VIE avant le terme convenu, ces titres seront remboursables à un prix égal au pair et uniquement après le désintéressement complet de tous les créanciers privilégiés et chirographaires et au même rang que tous les autres emprunts subordonnés qui pourraient être emis ou contractes par MAIF VIE.

MAIF VIE se réserve également la possibilité d'émettre de la dette subordonnée auprès de tiers.

E.1.3.3.3 Fonds cantonnés

MAIF VIE réalise chaque année un examen de l'importance des fonds cantonnes. Cet examen porte sur l'ensemble des points d'analyse décrits dans le § 5.6 de la notice << Solvabilité II >> de l'ACPR relative aux fonds propres (traduction de l'orientation EIOPA-BoS-14/169 FR sur les fonds cantonnés).

Au 31 décembre 2021, suite à la transformation du Perp-MAIF en PER Responsable et Solidaire en septembre 2020, et en l'absence de textes encadrant la tenue des comptabilités auxiliaires d'affectation des nouveaux PER (qui ne deviendra obligatoire qu'à compter de la mise en place du cantonnement légal, soit au plus tard au 1er janvier 2023), il n'existe pas de canton réglementaire dans les comptes de MAIF VIE. Toutefois, l'entreprise a fait le choix de réaliser une comptabilité auxiliaire d'affectation des opérations du PER à des fins de gestion.

MAIF VIE considère que le PER Responsable et Solidaire n'est pas important au regard de cet examen, dont les résultats sont résumés ci-après :

› la non-complexité des risques découlant du fonds cantonné ;

› l'approche sécurisée et la liquidité des actifs inclus dans le fonds cantonné ;

› l'absence de capitaux propres dans le Bilan du fonds cantonne (et donc pas de restriction sur les fonds propres disponibles en normes Solvabilité 2) ;

› la non-significativité du fonds cantonné au regard de l'activité globale de MAIF VIE ;

› et la très faible différence sur la mesure des exigences réglementaires (calcul du SCR).

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="89 / 107" -->


## \> GESTION DU CAPITAL

E.1.3.3.4 Politique de dividende

MAIF VIE a vocation à autofinancer son développement, ce que permettent les résultats issus de son Plan stratégique sans versement de dividendes à l'actionnaire.

Dans le cadre d'un dépassement significatif de ces résultats prévisionnels, l'actionnaire étudierait toutefois l'opportunité d'un versement de dividendes.


### E.1.4 Tableau de passage des fonds propres

Le tableau de passage des fonds propres en normes comptables françaises aux fonds propres en normes Solvabilité 2, présente une explication quantitative des différences importantes entre les capitaux propres selon les états financiers de l'entreprise et l'excédent de l'actif sur le passif du Bilan en normes Solvabilité 2 :

| | Passage de Bilan NF à S2 ||||| Fonds propres S2 |
|| Fonds propres des comptes individuels | Evaluation en valeur de marché des placements | Evaluation des provisions techniques en valeur actuelle plus marge de risque | Autres ajustements de valeurs | Reconnaissance des impôts différés ||
| - | - | - | - | - | - | - |
| Postes de fonds propres communs NF - S2 | | | | | | |
| Capital social ou Fonds d'établissement | 122 000 | | | | | 122 000 |
| Postes de fonds propres NF | | | | | | |
| Autres réserves | 316 634 | | | | | 316 634 |
| Report à nouveau | 100 000 | | | | | 100 000 |
| Resultat de l'exercice | 50 332 | | | | | 50 332 |
| Fonds de garantie | 2 519 | | | | | 2 519 |
| Postes de fonds propres S2 | | | | | | |
| Fonds excédentaires (art 91) | | | 478 765 | | | 478 765 |
| Reserve de reconciliation, hors elements provenant des comptes sociaux | | 1 223 324 | -1 045 373 | -23 494 | -36 744 | 117 713 |
| Total des fonds propres hors dettes subordonnées | 591 485 | 1 223 324 | -566 608 | -23 494 | -36 744 | 1 187 963 |
| Dettes subordonnées | 35 000 | | | | | 35 000 |
| Total fonds propres | 626 485 | 1 223 324 | -566 608 | -23 494 | -36 744 | 1 222 963 |


### E.1.5 Analyse des variations significatives de fonds propres sur la période

Le montant des fonds propres de base niveau 1 s'élève au 31 décembre 2021 à 1 187 963 milliers d'euros, en hausse de 115 885 milliers d'euros par rapport au 31 décembre 2020.

| | 2021 | 2020 | Variation ||
| - | - | - | - | - |
| Fonds propres de base - Niveau 1 | 1 187 963 | 1 072 078 | 115 885 | 11% |
| Capitaux propres | 122 000 | 122 000 | - | 0% |
| Fonds excédentaires | 478 766 | 487 306 | -8 540 | -2% |
| Réserve de réconciliation | 587 197 | 462 773 | 124 424 | 27% |
| - Eléments provenant des comptes sociaux | 469 485 | 416 753 | 52 732 | 13% |
| Réserves | 319 153 | 311 711 | 7 442 | 2% |
| Report à nouveau et resultat de l'exercice | 150 332 | 105 042 | 45 290 | 43% |
| - Ajustements liés aux retraitements S2 | 117 712 | 46 020 | 71 692 | NS |
| Différence d'évaluation des placements | 1 223 324 | 1 596 377 | -373 053 | -23% |
| Différence d'évaluation des autres actifs | -23 494 | 2 490 | -25 984 | NS |
| Différence d'évaluation des provisions techniques | -1 045 373 | -1 538 375 | 493 002 | -32% |
| Différence d'évaluation des autres postes du passif | -36 744 | -14 473 | -22 271 | NS |

Cette évolution s'explique principalement par :

› une augmentation du montant des fonds propres établis en normes comptables françaises de 52 733 milliers d'euros sur la période, en raison notamment du résultat net 2021 de 50 332 milliers d'euros et des mouvements sur la réserve de capitalisation réalisés en

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="90 / 107" -->

2021 pour un montant de 2 4017 milliers d'euros ;

› une diminution des fonds excédentaires, liée au reclassement d'une partie du montant de la provision pour participation aux bénéfices (PPB) conformément à l'arrêté du 24 décembre 2019 et en application des préconisations de l'ACPR portant sur les modalités de calcul à retenir par les entreprises d'assurance, de 8 540 milliers d'euros malgré la dotation à la provision pour participation aux bénéfices de 55 703 milliers d'euros . Cette baisse s'explique par le passage du taux de reclassement de 96,5% en 2020 à 85,4% en 2021, en raison notamment de l'évolution de la courbe de taux ;

› une progression du montant de la réserve de réconciliation, hors éléments provenant des comptes en normes comptables françaises, de 124 424 milliers d'euros, conséquence :

o d'une différence d'évaluation des placements baissant de 373 053 milliers d'euros en raison d'un stock de plus-values latentes en diminution ;

o des différences d'évaluation des autres actifs en baisse de 25 984 milliers d'euros, en raison principalement d'une diminution de la différence d'évaluation des provisions techniques cédées ;

o des écarts de provisionnements techniques entre les normes comptables françaises et les normes Solvabilité 2 en évolution de 493 002 milliers d'euros ; o d'une variation de l'évaluation des autres postes de passif de 22 271 milliers d'euros, en raison notamment de l'évolution de la reconnaissance des impôts différés passifs.

Le montant des fonds propres niveau 3 concernant exclusivement les TSR est stable à 34 500 milliers d'euros.

L'annulation des effets de la correction pour volatilité représente un impact de - 5 886 milliers d'euros sur les fonds propres de niveau 1.

| | Fonds propres de base - Niveau | Fonds propres de base - Niveau |
|| 1 | 3 |
| - | - | - |
| Valeur retenue | 1 187 963 | 34 500 |
| Valeur sans l'effet de correction pour volatilité | 1 182 077 | 34 500 |


## E.2 Capital de solvabilité requis et Minimum de capital requis

L'exigence de capital se décline sur deux niveaux :

› Minimum de capital requis - MCR ;

› Capital de solvabilité requis - SCR.


### E.2.1 Capital de solvabilité requis (SCR)

Le SCR correspond au montant de fonds propres estimé comme nécessaire pour absorber des chocs de forte intensité susceptibles de se produire environ 1 fois tous les 200 ans.

Il se fonde sur l'hypothèse d'une continuité de l'exploitation de l'entreprise et est calibré de manière à garantir que tous les risques quantifiables auxquels l'entreprise est exposée soient pris en considération. Il couvre le portefeuille en cours, ainsi que le nouveau portefeuille dont la souscription est attendue dans les douze mois à venir.

Les principaux risques auxquels est exposée MAIF VIE sont décrits dans le paragraphe C de ce rapport.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" --> :unselected: :unselected: :unselected: :unselected:

> GESTION DU CAPITAL
===

<!-- PageHeader="91/107" -->

Le SCR peut être calculé soit par la formule standard qui reflète le profil de risque moyen des entreprises d'assurances sur le marché européen, soit par un modèle interne après autorisation de l'ACPR, soit par une combinaison de ces deux méthodes.

MAIF VIE determine le Capital de solvabilité requis (SCR) à l'aide de la formule standard qui reflète le profil de risque moyen des entreprises d'assurance et de son activité.

Le SCR formule standard est calculé selon une approche modulaire. L'assureur doit calculer la perte potentielle subie en cas d'événement défavorable lié à une trentaine de facteurs de risque (regroupés en six modules). Les méthodes, hypothèses et paramètres à utiliser pour chacun des modules de risque ou sous-modules sont définis par la réglementation.

Pour tenir compte de la probabilité faible de réalisation simultanée de tous ces événements, la formule standard comprend des correlations entre ces facteurs de risques et permet ainsi de calculer l'effet positif de la diversification.

Enfin, le SCR est encore atténué par des ajustements pour obtenir le SCR final. Les entités vie sont notamment concernées par l'ajustement « capacité d'absorption des pertes par les impôts différés >> (capacité d'imputer des pertes au résultat fiscal permettant de payer moins d'impôts dans le futur) et par l'ajustement « capacité d'absorption des pertes par les provisions techniques >>.

Au 31 décembre 2021, le montant du SCR est de 492 938 milliers d'euros, en hausse de 48 032 milliers d'euros par rapport au 31 décembre 2020. Hors capacité d'absorption des pertes par les provisions techniques, le montant du SCR de base s'élève à 1 384 190 milliers d'euros.

Le tableau présente le détail du SCR par module :

| En milliers d'euros | 2021 || 2020 | Variation SCR net ||
|| SCR net (y compris capacité d'absorption des pertes PT / ID) | SCR brut (hors capacité d'absorption des pertes PT / ID) | SCR net (y compris capacité d'absorption des pertes PT / ID) | En valeur | En % |
| - | - | - | - | - | - |
| Risque de marché | 431 706 | 1 267 398 | 367 038 | 64 668 | 18% |
| Risque de défaut de contrepartie | 18 562 | 18 562 | 20 340 | 1 778 - | -9% |
| Risque de souscription vie | 105 767 | 311 701 | 87 301 | 18 466 | 21% |
| Risque de souscription santé | - | - | - | | |
| Risque de souscription non-vie | - | - | - | | |
| Diversification entre modules | 80 935 - | 213 471 - | 70 459 - | 10 476 - | 15% |
| Risque lié aux actifs incorporels | - | - | - | | |
| SCR de base | 475 100 | 1 384 190 | 404 220 | 70 880 | 18% |
| Capacité d'absorption des pertes par les provisions techniques | | 909 090 - | | | |
| Capacité d'absorption des pertes par les impôts différés | - 34 207 | 34 207 - | 11 889 - | - 22 318 | NS |
| Risque opérationnel | 52 045 | 52 045 | 52 575 | 530 - | -1% |
| SCR notionnel pour les fonds cantonnes | - | - | - | | |
| Diversification entre fonds cantonnés | - | - | - | | |
| SCR (hors exigences de capital supplémentaires) | 492 938 | 492 938 | 444 906 | 48 032 | 11% |
| Exigences de capital supplémentaires déjà imposées | | | | | |
| SCR | 492 938 | 492 938 | 444 906 | 48 032 | 11% |

Les principaux éléments explicatifs du montant du Capital de solvabilité requis sont :

› le SCR relatif au risque de marché, net de capacité d'absorption par les provisions techniques à 431 706 milliers d'euros (367 038 milliers d'euros au 31 décembre 2020), qui représente 91 % du SCR de base transitoire net de capacité d'absorption par les provisions techniques ;

› le SCR relatif au risque de souscription vie, net de capacité d'absorption par les provisions techniques à 105 767 milliers d'euros (87 301 milliers d'euros au 31 décembre 2020), qui pèse 22 % ;

› le SCR relatif au risque de défaut de contrepartie, net de capacité d'absorption par les provisions techniques, à 18 562 milliers d'euros (20 340 milliers d'euros au 31 décembre 2020), qui représente 4 % ;

› le SCR net relatif au risque opérationnel qui tient compte de la composante « provisions techniques >> pour atteindre 52 045 milliers d'euros.

MAIF VIE - Rapport sur la solvabilité et la situation financière 2021

> GESTION DU CAPITAL
===

<!-- PageHeader="92 / 107" -->

L'annulation des effets de la correction pour volatilité représente un impact de + 3 303 milliers d'euros sur le SCR.

| | SCR |
| - | - |
| Valeur retenue | 492 938 |
| Valeur sans l'effet de correction pour volatilité | 496 241 |


## E.2.2 Minimum de capital requis

Le MCR est le niveau minimal de fonds propres que l'organisme doit détenir en permanence. Le calcul du MCR s'effectue en trois étapes :

› application d'une formule linéaire simple en fonction des primes et des provisions techniques pour obtenir le MCR linéaire ;

› retraitement éventuel du MCR linéaire pour qu'il soit compris entre 25 % et 45 % du SCR afin d'aboutir au MCR combiné ;

› application d'un plancher absolu dépendant de l'activité pour obtenir le MCR final.

Au 31 décembre 2021, le montant du MCR s'élève à 221 822 milliers d'euros (200 208 milliers d'euros au 31 décembre 2020). Ce montant correspond au plafond égal à 45 % du SCR.

La composante linéaire du MCR, basée notamment sur les provisions techniques et les capitaux sous-risques, s'élève à 270 463 milliers d'euros, en baisse de 6 402 milliers d'euros par rapport à son niveau à fin 2020.

| | 2021 | 2020 | Variation ||
| - | - | - | - | - |
| MCR | 221 822 | 200 208 | 21 614 | 10,8% |
| MCR - Formule linéaire | 270 463 | 276 865 | -6 402 | -2,3% |
| Plancher (25% SCR) | 123 235 | 111 227 | 12 008 | 10,8% |
| Plafond (45% SCR) | 221 822 | 200 208 | 21 614 | 10,8% |
| SCR | 492 938 | 444 906 | 48 032 | 10,8% |

L'annulation des effets de la correction pour volatilité représente un impact de + 1 486 milliers d'euros sur le MCR.

| | MCR |
| - | - |
| Valeur retenue | 221 822 |
| Valeur sans l'effet de correction pour volatilité | 223 308 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> GESTION DU CAPITAL
===


## E.2.3 Informations sur la couverture des exigences de marge

Au 31 décembre 2021, MAIF VIE couvre de manière satisfaisante le Minimum de capital de solvabilité requis et le Capital de solvabilité requis.

| En milliers d'euros | 31/12/2021 | 31/12/2020 | Variation ||
|||| En valeur | En % |
| Fonds propres de base de niveau 1 | 1 187 963 | 1 072 078 | 115 885 | 10,8% |
| - | - | - | - | - |
| Capitaux propres | 122 000 | 122 000 | - | 0,0% |
| Fonds excédentaires | 478 766 | 487 306 | 8 540 - | -1,8% |
| Réserve de réconciliation | 587 197 | 462 772 | 124 425 | 26,9% |
| - Eléments provenant des comptes sociaux | 469 485 | 416 753 | 52 732 | 12,7% |
| - Ajustements liés aux retraitements S2 | 117 712 | 46 019 | 71 693 | NS |
| Fonds propres de base de niveau 3 | 34 500 | 34 500 | - | 0,0% |
| Dettes subordonnées Impôts différés actifs | 34 500 - | 34 500 - | - - | 0,0% |
| Eléments disponibles pour le SCR | 1 222 463 | 1 106 578 | 115 885 | 10,5% |
| Eléments disponibles pour le MCR | 1 187 963 | 1 072 078 | 115 885 | 10,8% |
| | ||||
| Capital de solvabilité requis | 492 938 | 444 906 | 48 032 | 10,8% |
| Eléments éligibles en couverture du SCR | 1 222 463 | 1 106 578 | 115 885 | 10,5% |
| Couverture par l'ensemble des éléments éligibles sous respect des limites quantitatives | 248,0% | 248,7% | -1 pts | |
| Minimum de capital requis | 221 822 | 200 208 | 21 614 | 10,8% |
| Eléments éligibles en couverture du MCR | 1 187 963 | 1 072 078 | 115 885 | 10,8% |
| Couverture par l'ensemble des éléments éligibles sous respect des limites quantitatives | 535,6% | 535,5% | 0 pts | |


## › Capital de solvabilité requis

Au 31 décembre 2021, le ratio de couverture du SCR est de 248,0 %, en baisse de 1 point par rapport au 31 décembre 2020, malgré la hausse des éléments éligibles en couverture du SCR de 115 885 milliers d'euros.

L'annulation des effets de la correction pour volatilité représente un impact de - 3 points sur le ratio de couverture du SCR.

| | Ratio de couverture du |
|| SCR |
| - | - |
| Valeur retenue | 248,0% |
| Valeur sans l'effet de correction pour volatilité | 245,2% |


### ›


## Minimum de capital de solvabilité requis

Au 31 décembre 2021, le ratio de couverture du MCR est de 535,6 %, en hausse de moins de 1 point par rapport au 31 décembre 2020.

A noter : parmi l'ensemble des fonds propres « disponibles », les éléments dits « éligibles » à la couverture du Capital de solvabilité requis (SCR) et du Capital minimum requis (MCR) doivent respecter des limites quantitatives assurant que les exigences de solvabilité sont couvertes majoritairement par des fonds propres de la meilleure qualité.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="93 / 107" -->

> GESTION DU CAPITAL
===

<!-- PageHeader="94/ 107" -->


## E.3 Utilisation du sous-module "risque sur actions" fondé sur la durée dans le calcul du Capital de solvabilité requis

MAIF VIE n'utilise pas cette option.

Cette option concerne le SCR des assureurs vie qui peuvent, sous certaines conditions et après autorisation de l'ACPR, appliquer aux actifs cantonnes de certaines activités de retraite un choc sur actions réduit par rapport au niveau imposé par la formule standard.


## E.4 Différences entre la formule standard et tout modèle interne utilisé

MAIF VIE n'utilise pas de modèle interne et de paramètres spécifiques.


## E.5 Non-respect du Minimum de capital requis et non-respect du Capital de solvabilité requis

MAIF VIE respecte les exigences réglementaires.


## E.6 Autres informations

MAIF VIE n'identifie pas d'autre information notable à communiquer sur la gestion du capital.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> ANNEXES
===

<!-- PageHeader="95 / 107" -->

Les états quantitatifs Solvabilité 2 suivants sont annexés au rapport sur la solvabilité et la situation financière :

• S.02.01.02 Bilan

• S.05.01.02 Primes, sinistres et frais par ligne d'activité

· S.12.01.02 Provisions techniques vie et santé similaire à la vie

• S.22.01.21 Impact des mesures relatives aux garanties de long terme et des mesures transitoires

•

S.23.01.01

Fonds propres

S.25.01.21 • SCR Formule standard Minimum de capital requis (MCR)

A noter, l'état S.05.02.01 - Primes, sinistres et frais par pays n'est pas publié dans la mesure où il n'est pas obligatoire compte tenu du seuil de chiffre d'affaires réalisé en France.

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

· S.28.01.01

<!-- PageNumber="96 / 107" -->


## \> ANNEXES


## S.02.01.02 - BILAN

| ACTIFS | Valeur Solvabilité II |
| - | - |
| Goodwill | |
| Frais d'acquisition différés | |
| Immobilisations incorporelles | - |
| Actifs d'impôts différés | - |
| Excédent du régime de retraite | - |
| Immobilisations corporelles détenues pour usage propre | 1 563 |
| Investissements (autres qu'actifs en représentation de contrats en unités de compte et indexés) | 12 263 743 |
| Biens immobiliers (autres que détenus pour usage propre) | - |
| Détentions dans des entreprises liées, y compris participations | 849 000 |
| Actions | 88 991 |
| Actions - cotées | - |
| Actions - non cotées | 88 991 |
| Obligations | 8 842 050 |
| Obligations d'État | 4 112 039 |
| Obligations d'entreprise | 4 730 011 |
| Titres structures | - |
| Titres garantis | - |
| Organismes de placement collectif | 2 054 415 |
| Produits dérivés | - |
| Dépôts autres que les équivalents de trésorerie | 429 286 |
| Autres investissements | - |
| Actifs en représentation de contrats en unites de compte et indexés | 708 868 |
| Prêts et prêts hypothécaires | 2 341 |
| Avances sur police | 2 341 |
| Prêts et prêts hypothécaires aux particuliers | - |
| Autres prêts et prêts hypothécaires | - |
| Montants recouvrables au titre des contrats de reassurance | 80 181 |
| Non-vie et santé similaire à la non-vie | - |
| Non-vie hors santé | - |
| Santé similaire à la non-vie | - |
| Vie et santé similaire à la vie, hors santé, UC et indexés | 80 181 |
| Santé similaire à la vie | - |
| Vie hors santé, UC et indexés | 80 181 |
| Vie UC et indexés | - |
| Dépôts auprès des cédantes | - |
| Créances nées d'opérations d'assurance et montants à recevoir d'intermédiaires | 13 588 |
| Créances nées d'opérations de reassurance | 515 |
| Autres créances (hors assurance) | 13 282 |
| Actions propres auto-détenues (directement) | - |
| Éléments de fonds propres ou fonds initial appelé(s), mais non encore payé(s) | - |
| Trésorerie et équivalents de trésorerie | 145 082 |
| Autres actifs non mentionnés dans les postes ci-dessus | - |
| TOTAL DE L'ACTIF | 13 229 164 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageNumber="97/ 107" -->


## \> ANNEXES

| PASSIFS | Valeur Solvabilité II |
| - | - |
| Provisions techniques non-vie | - |
| Provisions techniques non-vie (hors santé) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | - |
| Marge de risque | - |
| Provisions techniques santé (similaire à la non-vie) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation :selected: | - |
| Marge de risque | - |
| Provisions techniques vie (hors UC et indexés) | 11 124 548 |
| Provisions techniques santé (similaire à la vie) | - |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | - |
| Marge de risque | - |
| Provisions techniques vie (hors santé, UC et indexés) | 11 124 548 |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | 11 003 450 |
| Marge de risque | 121 098 |
| Provisions techniques UC et indexés | 711 218 |
| Provisions techniques calculées comme un tout | - |
| Meilleure estimation | 703 795 |
| Marge de risque | 7 422 |
| Autres provisions techniques | |
| Passifs éventuels | 2 519 |
| Provisions autres que les provisions techniques | 716 |
| Provisions pour retraite | 1 585 |
| Dépôts des réassureurs | 76 937 |
| Passifs d'impôts différés | 34 207 |
| Produits dérivés | - |
| Dettes envers des établissements de crédit | - |
| Dettes financières autres que celles envers les établissements de crédit | - |
| Dettes nees d'opérations d'assurance et montants dus aux intermédiaires | 1 230 |
| Dettes nées d'opérations de reassurance | 1926 |
| Autres dettes (hors assurance) | 51 815 |
| Passifs subordonnés | 34 500 |
| Passifs subordonnés non inclus dans les fonds propres de base | - |
| Passifs subordonnés inclus dans les fonds propres de base | 34 500 |
| Autres dettes non mentionnées dans les postes ci-dessus | - |
| TOTAL DU PASSIF | 12 041 201 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageNumber=">" -->


## ANNEXES

<!-- PageHeader="98 / 107" -->


## S.05.01.02 - PRIMES, SINISTRES ET DEPENSES PAR LIGNE D'ACTIVITE

| | Ligne d'activité pour: engagements d'assurance et de réassurance non-vie (assurance directe et réassurance proportionnelle acceptée) |||||||||
|| médicaux Assurance des frais | Assurance de protection du revenu | Assurance travailleurs d'indemnisation des | automobile Assurance de responsabilité civile | véhicules à moteur Autre assurance des | Assurance maritime, aérienne et transport | aux biens Assurance incendie et autres dommages | générale Assurance de responsabilité civile | cautionnement Assurance crédit et |
| - | - | - | - | - | - | - | - | - | - |
| Primes émises | | | | | | | | | |
| Brut - assurance directe | | | | | | | | | |
| Brut - Reassurance proportionnelle acceptée | | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Primes acquises | | | | | | | | | |
| Brut - assurance directe Brut - Reassurance proportionnelle acceptée 
|| | | | | | | | | || :unselected: 
||| | || || || | || || |||| | 
| Brut - Reassurance non proportionnelle acceptée | | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Charge des sinistres | | | | | | | | | |
| Brut - assurance directe Brut - Reassurance proportionnelle acceptée 
|| | | | | | | | | || :unselected: | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | | |
| Net | | | | | | | | | |
| Variation des autres provisions techniques | | | | | | | | | |
| Brut - assurance directe | | | | | | | | | |
| Brut - Reassurance proportionnelle acceptée | :unselected: 
||| | || || | | | || |||| | | | 
| Brut - Reassurance non proportionnelle acceptée | | | | | | | | | |
| Part des réassureurs | | | | | | | | | |
| Net | | | | | | | | | |
| Dépenses engagées | :unselected: | | | | | | | | |
| Autres dépenses | | | | | | | | | |
| Total des dépenses | | | | | | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageNumber=">" -->


## ANNEXES

<!-- PageHeader="99 / 107" -->

| | Ligne d'activité pour: engagements d'assurance et de réassurance non-vie (assurance directe et réassurance proportionnelle acceptée) ||| Ligne d'activité pour: réassurance non proportionnelle acceptée ||| | Total |
|| juridique protection Assurance de | Assistance | Pertes diverses pécuniaires | Santé | Accidents | transport maritime, Assurance aérienne et | Biens | |
| - | - | - | - | - | - | - | - | - |
| Primes émises | | | | | | | | |
| Brut - assurance directe | :unselected: | | | | | | | |
| Brut - Réassurance proportionnelle | | | | | | | | |
| acceptée | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | |
| Net | | | | | | | | |
| Primes acquises | :unselected: | | | | | | | |
| Brut - assurance directe | :unselected: | | | | | | | |
| Brut - Reassurance proportionnelle | | | | | | | | |
| acceptée | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | |
| Part des réassureurs | | | | | | | | |
| Net | :unselected: | | | | | | | |
| Charge des sinistres | | | | | | | | |
| Brut - assurance directe | :unselected: | | | | | | | |
| Brut - Reassurance proportionnelle | | | | | | | | |
| acceptée | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | |
| Net | | | | | | | | |
| Variation des autres provisions techniques | :unselected: | | | | | | | |
| Brut - assurance directe Brut - Reassurance proportionnelle | :unselected: 
|| || | || || || || || | | |
| acceptée | | | | | | | | |
| Brut - Reassurance non proportionnelle acceptée | :unselected: | | | | | | | |
| Part des réassureurs | :unselected: | | | | | | | |
| Net | | | | | | | | |
| Dépenses engagées | | | | | | | | |
| Autres dépenses | | | | | | | | |
| Total des dépenses | | | | | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<figure>

![](figures/18)

</figure>


<!-- PageNumber="> ANNEXES" -->

| | || Ligne d'activité pour: engagements d'assurance vie | ||| | ||| Engagements de réassurance vie |||| | |
| | | | ||| | contrats d'assurance non-vie   |||| d'assurance autres que les    ||| | Rentes découlant des   | santé| | | | | Total | |
| || ||
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| | Assurance maladie  || aux bénéfices || unités de compte| | | | | | ||| | || Assurance indexée et en    ||| Autres assurances vie   || | d'assurance santé ||| Rentes découlant des  et liées aux engagements | | contrats d'assurance non-vie  || | Réassurance maladie  | | || | | Réassurance vie  |||| | | | | | | ||
| Primes émises | | | | | | | | | | | | | | | ||
| Brut | | 646 607 | 103 693 | 62 593 | | | | | | | | | | | 812 893 ||
| Part des réassureurs | | 10 867 | | 14 598 | | | | | | | | | | | 25 465 ||
| Net | | 635 740 | 103 693 | 47 995 | | | | | | | | | | | 787 428 ||
| Primes acquises | | | | | | | | | | | | | | | ||
| Brut | | 644 260 | 103 693 | 62 459 | | | | | | | | | | | 810 412 ||
| Part des réassureurs | | 10 109 | | 15 008 | | | | | | | | | | | 25 117 ||
| Net | | 634 150 | 103 693 | 47 451 | | | | | | | | | | | 785 294 ||
| Charge des sinistres | | | | | | | | | | | | | | | ||
| Brut | | 507 086 | 20 652 | 15 540 | | | | | | | | | | | 543 278 ||
| Part des réassureurs | | 2 079 | | 3 405 | | | | | | | | | | | 5 484 ||
| Net | | 505 006 | 20 652 | 12 136 | | | | | | | | | | | 537 794 ||
| Variation des autres provisions techniques | | | | | | | | | | | | | | | ||
| Brut | | 65 518 | 78 029 | 1 928 | | | | | | | | | | | 145 475 ||
| Part des réassureurs | | 5 168 | | 1 137 | | | | | | | | | | | 6 305 ||
| Net | | 60 350 | 78 029 | 791 | | | | | | | | | | | 139 170 ||
| Dépenses engagées | - | 62 647 | 630 | 21 450 | | | - | | | - | | - | | - | 84 727 ||
| Autres dépenses | | | | | | | | | | | | | | | ||
| Total des dépenses | | | | | | | | | | | | | | | 84 727 ||| | | | | | | | | 
|||||||||

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="100 / 107" -->
| ||
| | | Assurance avec participation  
||| | et liées aux engagements   | engagements d'assurance  
<!-- PageNumber=">" -->

ANNEXES
===

<!-- PageHeader="101 / 107" -->

S.12.01.02 - PROVISIONS TECHNIQUES ASSIMILABLES A LA VIE PAR CONTRAT

<figure>

![](figures/19)

</figure>


| | | Assurance indexée et en unités de compte ||| Autres assurances vie ||| Rentes découlant des | | | Assurance santé (assurance directe) || Rentes découlant des contrats d'assurance non- vie et liées aux engagements | Réassurance santé (réassurance | Total (santé similaire à la vie) |
|| Assurance avec participation aux bénéfices | | Contrats sans options ni | Contrats avec options ou | | Contrats sans options ni | Contrats avec options ||||
|||||||||||
|| ou garanties | contrats d'assurance non-vie et liées aux engagements d'assurance autres que les engagements | Réassurance acceptée | Total (vie hors santé, y compris UC) | Contrats sans options ni | Contrats avec options ou |||||| acceptée) | | | garanties | garanties | | garanties | d'assurance santé ||| garanties | garanties | d'assurance santé | | |
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| Provisions techniques calculées comme un tout | | | | | | | | | | | | | | | |
| Total des montants recouvrables au titre de la | | | | | | | | | | | | | | | |
| reassurance/des véhicules de titrisation et de la | | | | | | | | | | | | | | | |
| reassurance finite, après l'ajustement pour pertes probables pour défaut de la contrepartie, | | | | | | | | | | | | | | | |
| correspondant aux provisions techniques calculées comme un tout | | | | | | | | | | | | | | | |
| Provisions techniques calculées comme la somme de la meilleure estimation et de la marge de risque | | | | | | | | | | | | | | | |
| Meilleure estimation | | | | | | | | | | | | | | | |
| Meilleure estimation brute | 11 027 726 | | 703 795 | | | 24 276 - | | | | 11 707 245 | | | | | |
| Total des montants recouvrables au titre de la | | | | | | 20 603 - 3 673 - | | | | | | | | | |
| réassurance/des véhicules de titrisation et de la réassurance finite, après ajustement pour pertes probables pour défaut de la contrepartie Meilleure estimation nette des montants recouvrables au titre de la réassurance/des véhicules de titrisation et de la réassurance finite - total | 100 784 10 926 942 | | - 703 795 ||| | | | | 80 181 11 627 064 || | | | |
| Marge de risque | 121 354 | 7 423 | | | 256 - | | | | | 128 520 | | | | | |
| Montant de la déduction transitoire sur les provisions techniques | | | | | | | | | | | | | | | |
| Provisions techniques calculées comme un tout Meilleure estimation 
|| | | | | | | | | | | | | | | || | | | | | | | | | | | | | | |
| Marge de risque | | | | | | | | | | | | | | | |
| Provisions techniques - Total | 11 149 080 | 711 218 | | | 24 532 - | | | | | 11 835 766 | | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="102 / 107" -->

\> ANNEXES
===

S.22.01.21 - IMPACT DES MESURES RELATIVES AUX GARANTIES DE LONG TERME ET DES MESURES TRANSITOIRES

| | Montant avec mesures relatives aux garanties de long terme et mesures transitoires | Impact des mesures transitoires sur les provisions techniques | Impact des mesures transitoires sur les taux d'intérêt | Impact d'une correction pour volatilité fixée à zéro | Impact d'un ajustement égalisateur fixé à zéro |
| - | - | - | - | - | - |
| Provisions techniques | 11 835 766 | | | 9 447 | |
| Fonds propres de base | 1 222 463 | | | 5 886 - | |
| Fonds propres éligibles pour couvrir le SCR | 1 222 463 | | | 5 886 - | |
| Capital de solvabilité requis | 492 938 | | | 3 303 | |
| Fonds propres éligibles pour couvrir le minimum de capital requis | 1 187 963 | | | 5 886 - | |
| Minimum de capital requis | 221 822 | | | 1 486 | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="103 / 107" -->

\> ANNEXES
===

S.23.01.01 - FONDS PROPRES

| | Total | Niveau 1 - non restreint | Niveau 1 - restreint | Niveau 2 | Niveau 3 |
| - | - | - | - | - | - |
| Fonds propres de base avant déduction pour participations dans d'autres secteurs financiers, comme prévu à l'article 68 du règlement délégué 2015/35 | | | | | |
| Capital en actions ordinaires (brut des actions propres) | 122 000 | 122 000 | | - | |
| Compte de primes d'émission lié au capital en actions ordinaires | - | - | | - | |
| Fonds initial, cotisations des membres ou élément de fonds propres de base équivalent pour les mutuelles et les entreprises de type mutuel | - | - | ! | - | |
| Comptes mutualistes subordonnés | - | | - | - | - |
| Fonds excédentaires | 478 766 | 478 766 | | | |
| Actions de préférence | - | | - | - | - |
| Compte de primes d'émission lié aux actions de préférence | - | | - | - | - |
| Réserve de réconciliation | 587 197 | 587 197 | ! | | |
| Passifs subordonnés | 34 500 | | - | - | 34 500 |
| Montant égal à la valeur des actifs d'impôts différés nets | - | | | -- | - |
| Autres éléments de fonds propres approuvés par l'autorité de contrôle en tant que fonds propres de base non spécifiés supra | - | - | - | - | - |
| Fonds propres issus des états financiers qui ne devraient pas être inclus dans la réserve de reconciliation et qui ne respectent pas les critères de fonds propres de Solvabilité II Fonds propres issus des états financiers qui ne devraient pas être inclus dans la réserve de réconciliation et qui ne respectent pas les critères de fonds propres de Solvabilité II | ! | - | : ! ! ! | | ! |
| Deductions | | ! | ! | ! | |
| Déductions pour participations dans des établissements de crédit et des établissements financiers | | | | | |
| Total fonds propres de base après déductions | 1 222 463 | 1 187 963 | - | - | 34 500 |
| Fonds propres auxiliaires | | | -- | -- | |
| Capital en actions ordinaires non libéré et non appelé, appelable sur demande Fonds initial, cotisations des membres ou élément de fonds propres de base équivalents, non libérés, non appelés et appelables sur demande, pour les mutuelles et les entreprises de type mutuel Actions de préférence non libérées et non appelées, appelables sur demande | | | -- ! | | |
| Engagements juridiquement contraignants de souscrire et de payer des passifs subordonnés sur demande Lettres de crédit et garanties relevant de l'article 96, paragraphe 2, de la directive 2009/138/CE Lettres de crédit et garanties ne relevant pas de l'article 96, paragraphe 2, de la directive 2009/138/CE Rappels de cotisations en vertu de l'article 96, point 3, de la directive 2009/138/CE | | | -- -- | ! ! ! | |
| Rappels de cotisations ne relevant pas de l'article 96, paragraphe 3, de la directive 2009/138/CE Autres fonds propres auxiliaires | |||||

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

<!-- PageHeader="104 / 107" -->

\> ANNEXES
===

| | Total | Niveau 1 - non restreint | Niveau 1 - restreint | Niveau 2 | Niveau 3 |
| - | - | - | - | - | - |
| Total fonds propres auxiliaires | | | | | |
| Fonds propres éligibles et disponibles | | | | | |
| Total des fonds propres disponibles pour couvrir le capital de solvabilité requis | 1 222 463 | 1 187 963 | - | - | 34 500 |
| Total des fonds propres disponibles pour couvrir le minimum de capital requis | 1 187 963 | 1 187 963 | - | - | |
| Total des fonds propres éligibles pour couvrir le capital de solvabilité requis | 1 222 463 | 1 187 963 | - | | 34 500 |
| Total des fonds propres éligibles pour couvrir le minimum de capital requis | 1 187 963 | 1 187 963 | - | | |
| Capital de solvabilité requis | 492 938 | | | | |
| Minimum de capital requis | 221 822 | | | | |
| Ratio fonds propres éligibles sur capital de solvabilité requis | 248,00% | | | | |
| Ratio fonds propres éligibles sur minimum de capital requis | 535,55% | | | | |
| Réserve de réconciliation | | | | | |
| Excédent d'actif sur passif | 1 187 963 | | | | |
| Actions propres (détenues directement et indirectement) | | | | | |
| Dividendes, distributions et charges prévisibles | | | | | |
| Autres éléments de fonds propres de base | 600 766 || | | |
| Ajustement pour les éléments de fonds propres restreints relatifs aux portefeuilles sous ajustement | - | | | | |
| égalisateur et aux fonds cantonnes Reserve de réconciliation | 587 197 | | | | |
| Bénéfices attendus | | | | | |
| Bénéfices attendus inclus dans les primes futures (EPIFP) - activités vie | 59 725 | | | | |
| Bénéfices attendus inclus dans les primes futures (EPIFP) - activités non-vie | | | | | |
| Total bénéfices attendus inclus dans les primes futures (EPIFP) | 59 725 | | | | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

> ANNEXES
===

<!-- PageHeader="105 / 107" -->

S.25.01.21 - CAPITAL DE SOLVABILITÉ REQUIS - POUR LES ENTREPRISES QUI UTILISENT LA FORMULE STANDARD

Article 112

Capital de solvabilité PPE

requis brut

Simplifications

1 267 398

Risque de marché

Risque de défaut de la contrepartie

18 562

Risque de souscription en vie

311 701

Risque de souscription en santé

\-

Risque de souscription en non-vie

\-

Diversification

\-213 471

Risque lié aux immobilisations incorporelles

\-

Capital de solvabilité requis de base

1 384 189

Calcul du capital de solvabilité requis Risque opérationnel 52 045 Capacité d'absorption des pertes des provisions techniques -909 090

Capacité d'absorption de pertes des impôts différés Capital requis pour les activités exercées conformément à l'article 4 de la directive 2003/41/CE

Capital de solvabilité requis à l'exclusion des exigences de capital supplémentaire 492 938

Exigences de capital supplémentaire déjà définies - Capital de solvabilité requis 492 938

Autres informations sur le SCR Capital requis pour le sous-module risque sur actions fondé sur la durée Total du capital de solvabilité requis notionnel pour la part restante

Total du capital de solvabilité requis notionnel pour les fonds cantonnés Total du capital de solvabilité requis notionnel pour les portefeuilles sous ajustement égalisateur Effets de diversification dus à l'agrégation des nSCR des FC selon l'article 304

Approche basée sur le taux d'imposition

Approche basée sur le taux d'imposition moyen Non

| Calcul de la capacité d'absorption de pertes des impôts différés ||
| LAC DT | -34 207 |
| - | - |
| LAC DT justifiée par la reprise de passifs d'impôts différés | -34 207 |
| LAC DT justifiée au regard de probables bénéfices économiques imposables futurs | |
| LAC DT justifiée par le report en arrière, exercice en cours | |
| LAC DT justifiée par le report en arrière, exercices futurs 
|| || |
| LAC DT maximale | |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

\-34 207

> ANNEXES

<!-- PageHeader="106 / 107" -->

S.28.01.01 - MINIMUM DE CAPITAL REQUIS (MCR) - ACTIVITE D'ASSURANCE OU DE REASSURANCE VIE UNIQUEMENT OU ACTIVITE D'ASSURANCE OU DE REASSURANCE NON-VIE UNIQUEMENT


## Terme de la formule linéaire pour les engagements d'assurance et de reassurance non-vie

ResultatMCRNL

| | Meilleure estimation et PT calculées comme un tout, nettes (de la réassurance / des véhicules de titrisation) | Primes émises au cours des 12 derniers mois, nettes (de la réassurance) |
| - | - | - |
| Assurance frais médicaux et reassurance proportionnelle y afférente | | |
| Assurance de protection du revenu, y compris réassurance proportionnelle y afférente | | |
| Assurance indemnisation des travailleurs et reassurance proportionnelle y afférente | | |
| Assurance de responsabilité civile automobile et reassurance proportionnelle y afférente | | |
| Autre assurance des véhicules à moteur et reassurance proportionnelle y afférente | | |
| Assurance maritime, aérienne et transport et réassurance proportionnelle y afférente | | |
| Assurance incendie et autres dommages aux biens et réassurance proportionnelle y afférente | | |
| Assurance de responsabilité civile générale et réassurance proportionnelle y afférente | | |
| Assurance crédit et cautionnement et reassurance proportionnelle y afférente | | |
| Assurance de protection juridique et reassurance proportionnelle y afférente | | |
| Assurance assistance et reassurance proportionnelle y afférente | | |
| Assurance pertes pécuniaires diverses et réassurance proportionnelle y afférente | | |
| Réassurance santé non proportionnelle | | |
| Réassurance accidents non proportionnelle | | |
| Reassurance maritime, aérienne et transport non proportionnelle | | |
| Reassurance dommages non proportionnelle | | |


## Terme de la formule linéaire pour les engagements d'assurance et de réassurance vie

RésultatMCRL

270 463

| | Meilleure estimation et PT calculées comme un tout, nettes (de la réassurance / des véhicules | Montant total du capital sous risque net (de la réassurance/ des véhicules de titrisation) |
| - | - | - |
| Engagements avec participation aux bénéfices - Prestations garanties | 9 286 293 | |
| Engagements avec participation aux bénéfices - Prestations discrétionnaires futures | 1 640 649 | |
| Engagements d'assurance avec prestations indexées et en unités de compte | 703 795 | |
| Autres engagements de (ré)assurance vie et de (ré)assurance santé | - | |
| Montant total du capital sous risque pour tous les engagements de (ré)assurance vie | | 10 368 018 |


### Calcul du MCR global

|||
| - | - |
| MCR lineaire | 270 463 |
| Capital de solvabilité requis | 492 938 |
| Plafond du MCR | 221 822 |
| Plancher du MCR | 123 234 |
| MCR combiné | 221 822 |
| Seuil plancher absolu du MCR | 3 700 |
| Minimum de capital requis | 221 822 |

<!-- PageFooter="MAIF VIE - Rapport sur la solvabilité et la situation financière 2021" -->

MAIF.FR Retrouvez-nous aussi sur

MAIF VIE - société anonyme au capital de 122 000 000 € - RCS Niort 330 432 782 - CS 90000 - 79029 Niort cedex 9. Entreprise régie par le Code des assurances.

<figure>

![](figures/20)

<!-- FigureContent="MAIF" -->

</figure>

}'''




response = get_openai_response(prompt)    
print(response) 

1) Présentation de l'organisme.
MAIF VIE est une société anonyme d'assurance vie, filiale de la MAIF, créée le 14 septembre 1984. Elle opère principalement dans le domaine des risques vie et a adopté le statut de société à mission depuis 2021. Elle est intégralement détenue par la MAIF et s'engage dans une mission d'intérêt collectif.

2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres, Résultat technique, Résultat financier, Résultat net après impôts)
En 2021, MAIF VIE a enregistré des encaissements bruts de réassurance de 810 412 milliers d'euros, en hausse de 10% par rapport à 2020. Les taux de rémunération servis sont de 1,30 % net. La provision pour participation aux bénéfices est de 6,04 % des provisions mathématiques en euros, en augmentation de 0,43 point par rapport à fin 2020. Le résultat net comptable est de 50 332 milliers d'euros.

3) Provisionnement
Les provisions techniques brutes de réassurance progressent de 4,5 % par rapport à 2

In [8]:
prompt = '''Tu es un assistant expert dans le domaine financier et en assurance. Tu sais bien extraire les informations à partir d'un
passage de texte.

Ces informations sont généralement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
-Placements autres que les actifs en represenation de contrats en UC ou indexe.
-Changement de la gouvernance.
-Perspectives vision de l'organisme et projets futurs.

#Règles:
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
- N'invente pas des informations qui ni fugrent pas dans le texte.
- Et le resume de fin devrait sous forme de six paragraphes comme suit:
1) Présentation de l'organisme.
2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts)
3) Provisionnement
4) Placements autres que les actifs en represenation de contrats en UC ou indexe.
5) Changement de la gouvernance.
6) Perspectives vision de l'organisme et projets futurs.

Le but estd e combiner deux textes et d'en faire un seul

Le premier texte est le suivant:
''{1) Présentation de l'organisme.
MAIF VIE est une société anonyme d'assurance vie, filiale du groupe MAIF, créée le 14 septembre 1984 et régie par le Code des assurances. Elle est également une société à mission depuis 2021, inscrivant sa mission d'intérêt collectif dans ses statuts. MAIF VIE est supervisée par l'Autorité de Contrôle Prudentiel et de Résolution (ACPR) et ses comptes sont audités par PricewaterhouseCoopers Audit et Mazars.

2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres, Résultat technique, Résultat financier, Résultat net après impôts)
En 2022, MAIF VIE a enregistré des encaissements bruts de réassurance de 834 368 milliers d'euros, en hausse de 3% par rapport à 2021. La société a servi des taux de rémunération dans la moyenne du marché (2,10 % net) et a porté sa provision pour participation aux bénéfices à 5,7 % des provisions mathématiques en euros de ses contrats d'épargne. Le résultat net comptable s'élève à 51 214 milliers d'euros.

3) Provisionnement
Les provisions techniques vie s'élèvent à 9 451 048 milliers d'euros et les provisions techniques UC ou indexés à 777 106 milliers d'euros. La meilleure estimation des provisions techniques vie brute est de 9 932 764 milliers d'euros, et la marge de risque associée est de 295 391 milliers d'euros.

4) Placements autres que les actifs en représentation de contrats en UC ou indexés.
Les placements de MAIF VIE, hors actifs en UC ou indexés, totalisent 10 560 376 milliers d'euros. Les participations s'élèvent à 939 457 milliers d'euros, les actions non cotées à 64 828 milliers d'euros, et les obligations à 7 506 141 milliers d'euros.

5) Changement de la gouvernance.
Yves Pellicier a été nommé président du conseil d'administration en juin 2022. Hélène N'Diaye est la directrice générale et Marina Sellari la directrice générale déléguée. Le conseil d'administration distingue les fonctions de président et de directeur général depuis 2015.

6) Perspectives vision de l'organisme et projets futurs.
MAIF VIE s'inscrit dans le Plan stratégique Groupe 2019-2022 « Engagés pour demain » et a élaboré son propre plan stratégique aligné sur les ambitions du Groupe MAIF mais adapté à ses enjeux propres. Elle poursuit une démarche d'investisseur responsable, intégrant les enjeux environnementaux, sociaux et de gouvernance (ESG) dans ses politiques d'investissement. MAIF VIE vise également à s'inscrire dans la trajectoire de l'Accord de Paris et à contribuer à une transition climatique inclusive.}''

Le second est le suivant:
''{1) Présentation de l'organisme.
MAIF VIE est une société anonyme d'assurance vie, filiale de la MAIF, créée le 14 septembre 1984. Elle opère principalement dans le domaine des risques vie et a adopté le statut de société à mission depuis 2021. Elle est intégralement détenue par la MAIF et s'engage dans une mission d'intérêt collectif.

2) Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres, Résultat technique, Résultat financier, Résultat net après impôts)
En 2021, MAIF VIE a enregistré des encaissements bruts de réassurance de 810 412 milliers d'euros, en hausse de 10% par rapport à 2020. Les taux de rémunération servis sont de 1,30 % net. La provision pour participation aux bénéfices est de 6,04 % des provisions mathématiques en euros, en augmentation de 0,43 point par rapport à fin 2020. Le résultat net comptable est de 50 332 milliers d'euros.

3) Provisionnement
Les provisions techniques brutes de réassurance progressent de 4,5 % par rapport à 2020 (+ 4,2 % en 2020) et portent principalement sur les contrats d'assurance en cas de vie.

4) Placements autres que les actifs en represenation de contrats en UC ou indexe.
Les placements financiers sont gérés en adéquation avec les règles de prudence, avec une allocation stratégique d'actifs et des règles de gestion visant la sécurité, la qualité, la liquidité et la rentabilité du portefeuille.

5) Changement de la gouvernance.
Depuis le conseil d'administration du 17 décembre 2015, les fonctions de président du conseil d'administration et de directeur général sont distinctes. Dominique Mahé est le président et Hélène N'Diaye est la directrice générale.

6) Perspectives vision de l'organisme et projets futurs.
MAIF VIE s'inscrit dans le Plan stratégique Groupe 2019-2022 « Engagés pour demain » et poursuit son développement en restant fidèle à ses valeurs fondatrices. Elle veille à l'efficacité et à l'adéquation de son système de gouvernance avec la stratégie et les risques inhérents à ses activités. Elle est attentive à toute évolution de son profil de risque et procède à une veille active sur les domaines financier et réglementaire.}''
''' 

response = get_openai_response(prompt)    
print(response) 

1) Présentation de l'organisme.
MAIF VIE est une société anonyme d'assurance vie, filiale du groupe MAIF, créée le 14 septembre 1984. Elle est à la fois une société à mission depuis 2021, inscrivant sa mission d'intérêt collectif dans ses statuts, et est supervisée par l'Autorité de Contrôle Prudentiel et de Résolution (ACPR). Ses comptes sont audités par PricewaterhouseCoopers Audit et Mazars.

2) Activité de l'organisme
En 2022, MAIF VIE a enregistré des encaissements bruts de réassurance de 834 368 milliers d'euros, marquant une hausse de 3% par rapport à 2021. La société a servi des taux de rémunération de 2,10 % net et a augmenté sa provision pour participation aux bénéfices à 5,7 % des provisions mathématiques en euros. Le résultat net comptable s'est élevé à 51 214 milliers d'euros.

3) Provisionnement
Les provisions techniques vie de MAIF VIE s'élèvent à 9 451 048 milliers d'euros, avec des provisions techniques UC ou indexés à 777 106 milliers d'euros. La meilleure estimation 